In [1]:
# Importing Libraries 

In [2]:
import pandas as pd 
import numpy as np
import sklearn
import copy
import seaborn as sn
import os
import warnings
import datetime as dt 
import statsmodels.api as sm
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from pandas.tseries.offsets import MonthEnd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, precision_recall_curve, auc
from sklearn.feature_selection import f_classif
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from scipy.stats import chi2_contingency
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 500)
%matplotlib inline

In [3]:
## Going in the directory where data is present 

In [4]:
cd "C:\Users\ydhav\OneDrive - Auronova Consulting Solutions Private Limited\Auronova_Training_Material\B_Score_Training"

C:\Users\ydhav\OneDrive - Auronova Consulting Solutions Private Limited\Auronova_Training_Material\B_Score_Training


In [5]:
# Card Transaction data 

In [6]:
i1_card_transaction = pd.read_excel('i1_card_transaction.xlsx')

In [7]:
i1_card_transaction_raw = i1_card_transaction

In [8]:
i1_card_transaction.head(10)

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,MERCHANT_CATEGORY,REVERSAL_FLAG,AMOUNT_IN_LCY
0,CUKTNSPPNCCCNNNC,Card Fee,2012-11-09,6012,N,250
1,CUKTNSPPNCCCNNNC,Unique Transaction,2012-11-12,6051,N,15000
2,CUKTNSPPNCCCNNNC,Unique Transaction,2012-11-12,6051,N,10000
3,CUKTNSPPNCCCNNNC,Unique Transaction,2012-11-18,6051,N,7216
4,CUKTNSPPNCCCNNNC,Purchase REVERSAL,2012-11-18,5411,N,1925
5,CUKTNSPPNCCCNNNC,WITHDRAWAL,2012-11-19,6011,N,5000
6,CUKTNSPPNCCCNNNC,Cash Fee,2012-11-19,6012,N,150
7,CUKTNSPPNCCCNNNC,Purchase REVERSAL,2012-11-22,9399,N,500
8,CUKTNSPPNCCCNNNC,Cash Fee,2012-11-25,6012,N,150
9,CUKTNSPPNCCCNNNC,Cash Fee,2012-11-25,6012,N,150


In [9]:
list(i1_card_transaction.columns)

['MASK_CREDITCARD',
 'TRANS_DESCRIPTION',
 'POST_DATE_AND_TIME',
 'MERCHANT_CATEGORY',
 'REVERSAL_FLAG',
 'AMOUNT_IN_LCY']

In [10]:
i1_card_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 809755 entries, 0 to 809754
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   MASK_CREDITCARD     809755 non-null  object        
 1   TRANS_DESCRIPTION   809755 non-null  object        
 2   POST_DATE_AND_TIME  809755 non-null  datetime64[ns]
 3   MERCHANT_CATEGORY   809755 non-null  int64         
 4   REVERSAL_FLAG       809755 non-null  object        
 5   AMOUNT_IN_LCY       809755 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 37.1+ MB


In [11]:
i1_card_transaction.describe()

,MERCHANT_CATEGORY,AMOUNT_IN_LCY
count,809755.000000,809755.000000
mean,5923.739694,1029.692545
std,728.747004,3743.681922
min,742.000000,0.000000
25%,5651.000000,99.000000
50%,6012.000000,200.000000
75%,6012.000000,874.000000
max,9950.000000,697651.000000


In [12]:
i1_card_transaction.shape

(809755, 6)

In [13]:
i1_card_transaction.isnull().sum()

MASK_CREDITCARD       0
TRANS_DESCRIPTION     0
POST_DATE_AND_TIME    0
MERCHANT_CATEGORY     0
REVERSAL_FLAG         0
AMOUNT_IN_LCY         0
dtype: int64

In [14]:
i1_card_transaction.duplicated().sum()

19788

In [15]:
# dropping duplicate values
#i1_card_transaction.drop_duplicates(keep=False, inplace=True)
i1_card_transaction = i1_card_transaction.drop_duplicates()

In [16]:
i1_card_transaction.shape

(789967, 6)

In [17]:
i1_card_transaction.MASK_CREDITCARD.value_counts(dropna=False)

FTKUGFNNWGWFPWWF    996
FTKUGFNNGKTNKNWK    767
FTKUGFNNNUUPUGUC    748
FTKUGGWPGPKWNNNP    723
FTKUGFNNNWKSNNNP    720
                   ... 
FTKUGGWPCFGPCGWN      1
CUKTNSPPNFNWNNNC      1
CUKTNSPPNPCTFGPP      1
FCPFSTNTSNKKTGCF      1
FTKUGFNNCSUKKTKT      1
Name: MASK_CREDITCARD, Length: 8337, dtype: int64

In [18]:
i1_card_transaction.TRANS_DESCRIPTION.value_counts(dropna=False)

Purchase                           294406
Card Payment                        95699
Periodic Fixed Fee                  94523
Purchase REVERSAL                   83943
Cash Fee                            37725
Late Payment Charges                36064
Withdrawal                          33588
Profit Earned(Special Sav A\c)      23105
Rebate Contributed By Bank          20288
E-Channel Payments                  14858
Monthly Murabaha Instalment         11283
Monthly Murabaha Installment         7453
WITHDRAWAL                           7451
Unique Transaction                   6017
Montlhly Murabaha Installment        5099
Utility Bill Payment                 4421
Migration Transaction Debit          4315
Payment                              2428
Credit Voucher                       1611
Migration Transaction Credit         1370
Debit Adjustments                    1093
Card Replacement Fee                  722
Card Fee                              670
CREDIT VOUCHER                    

In [19]:
i1_card_transaction.MERCHANT_CATEGORY.value_counts()

6012    362412
5411     69556
6011     41002
5541     38272
5812     22425
5651     18737
7011     15884
5311     14361
4814     10590
5968      8793
5691      8489
5977      8270
5733      7477
4812      7426
5912      7300
5814      6535
6051      6211
5735      5894
9399      5433
5661      5304
5944      4853
5712      4584
8999      3899
7832      3672
5942      3547
5309      3233
7523      3198
5734      3175
5732      3133
7230      2911
8062      2809
5941      2724
5699      2649
4511      2626
8099      2560
5441      2505
5945      2267
5999      2022
5511      1946
5641      1936
4722      1907
3034      1899
8043      1751
7399      1722
5621      1623
7994      1468
5499      1346
4900      1321
5947      1275
3026      1274
5399      1261
5967      1232
5631      1202
7298      1197
5697      1180
5533      1080
5611      1010
7997       996
4816       950
4215       893
7512       891
8011       873
5964       848
7311       756
8299       750
3236       726
5948      

In [20]:
i1_card_transaction.REVERSAL_FLAG.value_counts(dropna=False)

N    788087
R      1880
Name: REVERSAL_FLAG, dtype: int64

In [21]:
i1_card_transaction['POST_DATE_AND_TIME'].agg(['min', 'max'])

min   2009-01-08
max   2013-07-01
Name: POST_DATE_AND_TIME, dtype: datetime64[ns]

In [22]:
i1_card_transaction.MASK_CREDITCARD.nunique()

8337

In [23]:
i1_card_transaction.TRANS_DESCRIPTION.nunique()

42

In [24]:
i1_card_transaction.MERCHANT_CATEGORY.nunique()

431

In [25]:
# Card Reporting data 

In [26]:
i8_cards = pd.read_excel('i8_cards.xlsx')

In [27]:
i8_cards.head(10)

,PERIOD,MASKED_CUSTOMER_ID,MASK_CREDITCARD,a_issue_date2,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,...,A_Gender,NATIONALITY,A_Marital_Status,A_Industry,A_Emp_Sts,A_DOB,A_Emp_Start,A_No_Depend,A_salary,A_Sector
0,2012-11-30,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,50341.34,0.00,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
1,2012-12-31,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,56926.71,50341.34,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
2,2013-01-31,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,60332.69,56926.71,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
3,2013-02-28,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,59646.63,60332.69,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
4,2013-03-31,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,56928.16,59646.63,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
5,2013-04-30,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,53251.81,56928.16,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
6,2013-05-31,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,54727.52,53251.81,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
7,2013-06-30,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,58237.75,54727.52,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
8,2013-07-31,DHSSDDG,CUKTNSPPNCCCNNNC,2012-11-07,2013-07-31,2012-02-28,60000,36000,60860.17,58237.75,...,MALE,784,MARRIED,PERSONAL,EMPLOYED,1962-03-25,2003-09-01,2.0,NaN,Government
9,2013-03-31,QTDFQGQ,CUKTNSPPNCCFNNNS,2013-03-25,2013-07-31,2012-10-08,65700,39420,62750.00,0.00,...,MALE,356,MARRIED,PERSONAL,EMPLOYED,1975-04-22,2004-01-01,3.0,NaN,Semi Government


In [28]:
list(i8_cards.columns)

['PERIOD',
 'MASKED_CUSTOMER_ID',
 'MASK_CREDITCARD',
 'a_issue_date2',
 'A_card_expiry_date',
 'Customer_Join_Date',
 'CARD_LIMIT',
 'CASH_LIMIT',
 'OS_BALANCE',
 'OPENING_BALANCE',
 'CLOSING_BALANCE',
 'CLOSING_BALANCE_CASH',
 'CLOSING_BALANCE_FEES',
 'NB_DAYS_DELINQ',
 'Customer_Occupation',
 'REGIONS',
 'A_Gender',
 'NATIONALITY',
 'A_Marital_Status',
 'A_Industry',
 'A_Emp_Sts',
 'A_DOB',
 'A_Emp_Start',
 'A_No_Depend',
 'A_salary',
 'A_Sector']

In [29]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 142417 entries, 0 to 142416
Data columns (total 26 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PERIOD                142417 non-null  datetime64[ns]
 1   MASKED_CUSTOMER_ID    142417 non-null  object        
 2   MASK_CREDITCARD       142417 non-null  object        
 3   a_issue_date2         141314 non-null  datetime64[ns]
 4   A_card_expiry_date    142417 non-null  datetime64[ns]
 5   Customer_Join_Date    116899 non-null  datetime64[ns]
 6   CARD_LIMIT            142417 non-null  int64         
 7   CASH_LIMIT            142417 non-null  int64         
 8   OS_BALANCE            139290 non-null  float64       
 9   OPENING_BALANCE       139290 non-null  float64       
 10  CLOSING_BALANCE       139290 non-null  float64       
 11  CLOSING_BALANCE_CASH  139290 non-null  float64       
 12  CLOSING_BALANCE_FEES  139290 non-null  float64       
 13 

In [30]:
i8_cards.describe()

,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,REGIONS,NATIONALITY,A_No_Depend,A_salary
count,1.424170e+05,1.424170e+05,139290.000000,139290.000000,139290.000000,139290.000000,139290.000000,135178.000000,142417.000000,142417.000000,52581.000000,63908.000000
mean,3.726311e+04,2.236829e+04,21450.655267,20859.169440,21450.655267,3979.872093,2269.252436,1.371887,1.708722,683.037116,2.946064,33953.835970
std,1.393787e+05,8.363279e+04,22423.369856,22383.297147,22423.369856,7236.730040,2542.059456,2.846676,1.294964,182.602470,1.898775,26769.094869
min,0.000000e+00,0.000000e+00,0.000000,-385110.990000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,53.640000
25%,1.700000e+04,1.020000e+04,8911.652500,8169.655000,8911.652500,0.000000,590.000000,0.000000,1.000000,586.000000,1.000000,18745.000000
50%,2.500000e+04,1.500000e+04,18609.655000,18279.015000,18609.655000,0.000000,1540.000000,0.000000,1.000000,784.000000,2.000000,25282.000000
75%,3.500000e+04,2.100000e+04,31286.377500,30872.642500,31286.377500,5900.000000,3300.000000,1.000000,2.000000,784.000000,4.000000,40520.000000
max,1.000000e+07,6.000000e+06,997521.870000,997521.870000,997521.870000,138000.000000,249830.720000,9.000000,7.000000,901.000000,23.000000,390000.000000


In [31]:
i8_cards.shape

(142417, 26)

In [32]:
i8_cards.isnull().sum()

PERIOD                      0
MASKED_CUSTOMER_ID          0
MASK_CREDITCARD             0
a_issue_date2            1103
A_card_expiry_date          0
Customer_Join_Date      25518
CARD_LIMIT                  0
CASH_LIMIT                  0
OS_BALANCE               3127
OPENING_BALANCE          3127
CLOSING_BALANCE          3127
CLOSING_BALANCE_CASH     3127
CLOSING_BALANCE_FEES     3127
NB_DAYS_DELINQ           7239
Customer_Occupation     57534
REGIONS                     0
A_Gender                30926
NATIONALITY                 0
A_Marital_Status        39171
A_Industry              32021
A_Emp_Sts               45143
A_DOB                   25858
A_Emp_Start             83579
A_No_Depend             89836
A_salary                78509
A_Sector                76269
dtype: int64

In [33]:
i8_cards.duplicated().sum()


3261

In [34]:
# dropping duplicate values
i8_cards = i8_cards.drop_duplicates()

In [35]:
i8_cards.shape

(139156, 26)

In [36]:
i8_cards.MASKED_CUSTOMER_ID.nunique()

5625

In [37]:
i8_cards.MASK_CREDITCARD.nunique()

6243

In [38]:
i8_cards['PERIOD'].agg(['min', 'max'])

min   2008-10-08
max   2013-07-31
Name: PERIOD, dtype: datetime64[ns]

In [39]:
i8_cards['a_issue_date2'].agg(['min', 'max'])

min   2008-06-01
max   2013-07-29
Name: a_issue_date2, dtype: datetime64[ns]

In [40]:
i8_cards['A_card_expiry_date'].agg(['min', 'max'])

min   2009-06-08
max   2013-07-31
Name: A_card_expiry_date, dtype: datetime64[ns]

In [41]:
check=i8_cards[i8_cards['MASK_CREDITCARD'].isin(list(i1_card_transaction.MASK_CREDITCARD))]

In [42]:
check.MASK_CREDITCARD.nunique()

6112

In [43]:
check.shape

(138800, 26)

In [44]:
check1=i8_cards[~i8_cards['MASK_CREDITCARD'].isin(list(i1_card_transaction.MASK_CREDITCARD))]

In [45]:
check1.MASK_CREDITCARD.nunique()

131

In [46]:
#i8_cards['issue_date_yr_mon'] = pd.to_datetime(i8_cards['a_issue_date2']).dt.strftime('%Y-%m')

In [47]:
#i8_cards.issue_date_yr_mon.value_counts()

In [48]:
i8_cards_org = i8_cards

In [49]:
# Removing cases with missing issue date 

In [50]:
#i8_cards = i8_cards[~i8_cards.a_issue_date2.isnull()]

In [51]:
i8_cards.shape

(139156, 26)

In [52]:
i8_cards['min_period'] = i8_cards.groupby('MASK_CREDITCARD')['PERIOD'].transform(min)

In [53]:
# Replace missing issue date with minimum reporting date 

In [54]:
i8_cards.loc[i8_cards.a_issue_date2.isnull() ,'a_issue_date2']= i8_cards['min_period']

In [55]:
i8_cards['report_date']=i8_cards['PERIOD']+pd.offsets.MonthEnd(0)

In [56]:
#i8_cards['issue_date']=i8_cards['a_issue_date2']+pd.offsets.MonthEnd(0)

In [57]:
i8_cards['min_report_date'] = i8_cards.groupby('MASK_CREDITCARD')['report_date'].transform(min)
i8_cards['max_report_date'] = i8_cards.groupby('MASK_CREDITCARD')['report_date'].transform(max)

In [58]:
i8_cards['issue_date'] = i8_cards['min_report_date']

In [59]:
i8_cards.shape

(139156, 31)

In [60]:
i8_cards.loc[i8_cards.NB_DAYS_DELINQ.isnull() ,'NB_DAYS_DELINQ']=0

In [61]:
i8_cards.NB_DAYS_DELINQ.value_counts()

0.0    95021
1.0    20007
9.0    14308
2.0     4305
3.0     1617
4.0      998
5.0      866
6.0      717
8.0      662
7.0      655
Name: NB_DAYS_DELINQ, dtype: int64

In [62]:
i8_cards['NB_DAYS_DELINQ'] =i8_cards.groupby(['MASK_CREDITCARD','report_date'])['NB_DAYS_DELINQ'].transform(max)

In [63]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139156 entries, 0 to 142416
Data columns (total 31 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   PERIOD                139156 non-null  datetime64[ns]
 1   MASKED_CUSTOMER_ID    139156 non-null  object        
 2   MASK_CREDITCARD       139156 non-null  object        
 3   a_issue_date2         139156 non-null  datetime64[ns]
 4   A_card_expiry_date    139156 non-null  datetime64[ns]
 5   Customer_Join_Date    114661 non-null  datetime64[ns]
 6   CARD_LIMIT            139156 non-null  int64         
 7   CASH_LIMIT            139156 non-null  int64         
 8   OS_BALANCE            136038 non-null  float64       
 9   OPENING_BALANCE       136038 non-null  float64       
 10  CLOSING_BALANCE       136038 non-null  float64       
 11  CLOSING_BALANCE_CASH  136038 non-null  float64       
 12  CLOSING_BALANCE_FEES  136038 non-null  float64       
 13 

In [64]:
i8_cards=i8_cards.drop(['PERIOD','a_issue_date2'],axis=1)

In [65]:
i8_cards.drop_duplicates(subset=['MASK_CREDITCARD','report_date'],inplace= True)

In [66]:
i8_cards.shape

(139155, 29)

In [67]:
#i8_cards['ref_date']=pd.to_datetime("2013-07-31")

In [68]:
i8_cards['mob']=((i8_cards['report_date']-i8_cards['issue_date'])/np.timedelta64(1,'M'))
#factor_0['mob']=factor_0['mob'].astype('int')
i8_cards['mob']=i8_cards['mob'].round()


In [69]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139155 entries, 0 to 142416
Data columns (total 30 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    139155 non-null  object        
 1   MASK_CREDITCARD       139155 non-null  object        
 2   A_card_expiry_date    139155 non-null  datetime64[ns]
 3   Customer_Join_Date    114660 non-null  datetime64[ns]
 4   CARD_LIMIT            139155 non-null  int64         
 5   CASH_LIMIT            139155 non-null  int64         
 6   OS_BALANCE            136037 non-null  float64       
 7   OPENING_BALANCE       136037 non-null  float64       
 8   CLOSING_BALANCE       136037 non-null  float64       
 9   CLOSING_BALANCE_CASH  136037 non-null  float64       
 10  CLOSING_BALANCE_FEES  136037 non-null  float64       
 11  NB_DAYS_DELINQ        139155 non-null  float64       
 12  Customer_Occupation   82800 non-null   object        
 13 

In [70]:
i8_cards['mob']=i8_cards['mob'].astype(int)

In [71]:
i8_cards

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,A_Emp_Start,A_No_Depend,A_salary,A_Sector,min_period,report_date,min_report_date,max_report_date,issue_date,mob
0,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,50341.34,0.00,50341.34,15000.0,...,2003-09-01,2.0,NaN,Government,2012-11-30,2012-11-30,2012-11-30,2013-07-31,2012-11-30,0
1,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56926.71,50341.34,56926.71,26000.0,...,2003-09-01,2.0,NaN,Government,2012-11-30,2012-12-31,2012-11-30,2013-07-31,2012-11-30,1
2,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,60332.69,56926.71,60332.69,26000.0,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-01-31,2012-11-30,2013-07-31,2012-11-30,2
3,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,59646.63,60332.69,59646.63,26000.0,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-02-28,2012-11-30,2013-07-31,2012-11-30,3
4,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56928.16,59646.63,56928.16,26000.0,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-03-31,2012-11-30,2013-07-31,2012-11-30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142412,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,7379.02,1381.36,7379.02,0.0,...,2004-02-09,2.0,NaN,NaN,2012-03-31,2013-03-31,2012-03-31,2013-07-31,2012-03-31,12
142413,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,2829.02,7379.02,2829.02,0.0,...,2004-02-09,2.0,NaN,NaN,2012-03-31,2013-04-30,2012-03-31,2013-07-31,2012-03-31,13
142414,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,7819.52,2829.02,7819.52,0.0,...,2004-02-09,2.0,NaN,NaN,2012-03-31,2013-05-31,2012-03-31,2013-07-31,2012-03-31,14
142415,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,10970.93,7819.52,10970.93,0.0,...,2004-02-09,2.0,NaN,NaN,2012-03-31,2013-06-30,2012-03-31,2013-07-31,2012-03-31,15


In [72]:
i8_cards.mob.value_counts()

0     6243
1     6027
2     5888
3     5708
4     5525
5     5351
6     5162
7     4959
8     4673
9     4366
10    4133
11    3896
12    3718
13    3533
14    3395
15    3225
16    3091
17    2995
18    2901
19    2794
20    2630
21    2512
22    2408
23    2321
24    2262
25    2209
26    2174
27    2120
28    2071
29    2021
30    1983
31    1929
32    1865
33    1799
34    1723
35    1655
36    1570
37    1478
38    1426
39    1378
40    1332
41    1282
42    1223
43    1174
44    1112
45    1065
46    1015
47     974
48     920
49     853
50     301
51     194
52     158
53     135
54     116
55      92
56      67
57      25
Name: mob, dtype: int64

In [73]:
i8_cards.head(20)

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,A_Emp_Start,A_No_Depend,A_salary,A_Sector,min_period,report_date,min_report_date,max_report_date,issue_date,mob
0,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,50341.34,0.00,50341.34,15000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2012-11-30,2012-11-30,2013-07-31,2012-11-30,0
1,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56926.71,50341.34,56926.71,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2012-12-31,2012-11-30,2013-07-31,2012-11-30,1
2,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,60332.69,56926.71,60332.69,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-01-31,2012-11-30,2013-07-31,2012-11-30,2
3,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,59646.63,60332.69,59646.63,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-02-28,2012-11-30,2013-07-31,2012-11-30,3
4,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56928.16,59646.63,56928.16,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-03-31,2012-11-30,2013-07-31,2012-11-30,4
5,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,53251.81,56928.16,53251.81,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-04-30,2012-11-30,2013-07-31,2012-11-30,5
6,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,54727.52,53251.81,54727.52,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-05-31,2012-11-30,2013-07-31,2012-11-30,6
7,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,58237.75,54727.52,58237.75,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-06-30,2012-11-30,2013-07-31,2012-11-30,7
8,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,60860.17,58237.75,60860.17,26000.00,...,2003-09-01,2.0,NaN,Government,2012-11-30,2013-07-31,2012-11-30,2013-07-31,2012-11-30,8
9,QTDFQGQ,CUKTNSPPNCCFNNNS,2013-07-31,2012-10-08,65700,39420,62750.00,0.00,62750.00,0.00,...,2004-01-01,3.0,NaN,Semi Government,2013-03-31,2013-03-31,2013-03-31,2013-07-31,2013-03-31,0


In [74]:
i8_cards.isnull().sum()

MASKED_CUSTOMER_ID          0
MASK_CREDITCARD             0
A_card_expiry_date          0
Customer_Join_Date      24495
CARD_LIMIT                  0
CASH_LIMIT                  0
OS_BALANCE               3118
OPENING_BALANCE          3118
CLOSING_BALANCE          3118
CLOSING_BALANCE_CASH     3118
CLOSING_BALANCE_FEES     3118
NB_DAYS_DELINQ              0
Customer_Occupation     56355
REGIONS                     0
A_Gender                29889
NATIONALITY                 0
A_Marital_Status        38128
A_Industry              30817
A_Emp_Sts               44100
A_DOB                   24835
A_Emp_Start             82271
A_No_Depend             88002
A_salary                77100
A_Sector                74846
min_period                  0
report_date                 0
min_report_date             0
max_report_date             0
issue_date                  0
mob                         0
dtype: int64

In [75]:
i8_cards['min_mob'] = i8_cards.groupby('MASK_CREDITCARD')['mob'].transform(min)
i8_cards['max_mob'] = i8_cards.groupby('MASK_CREDITCARD')['mob'].transform(max)

In [76]:
i8_cards['cum_count']=i8_cards.groupby(by='MASK_CREDITCARD').cumcount()

In [77]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139155 entries, 0 to 142416
Data columns (total 33 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    139155 non-null  object        
 1   MASK_CREDITCARD       139155 non-null  object        
 2   A_card_expiry_date    139155 non-null  datetime64[ns]
 3   Customer_Join_Date    114660 non-null  datetime64[ns]
 4   CARD_LIMIT            139155 non-null  int64         
 5   CASH_LIMIT            139155 non-null  int64         
 6   OS_BALANCE            136037 non-null  float64       
 7   OPENING_BALANCE       136037 non-null  float64       
 8   CLOSING_BALANCE       136037 non-null  float64       
 9   CLOSING_BALANCE_CASH  136037 non-null  float64       
 10  CLOSING_BALANCE_FEES  136037 non-null  float64       
 11  NB_DAYS_DELINQ        139155 non-null  float64       
 12  Customer_Occupation   82800 non-null   object        
 13 

In [78]:
# i8_cards['min_mob_report']=((i8_cards['min_report_date']-i8_cards['issue_date'])/np.timedelta64(1,'M'))
# #factor_0['mob']=factor_0['mob'].astype('int')
# i8_cards['min_mob_report']=i8_cards['min_mob_report'].round()
# i8_cards['min_mob_report']=i8_cards['min_mob_report'].astype(int)

In [79]:
# i8_cards['max_mob_report']=((i8_cards['max_report_date']-i8_cards['issue_date'])/np.timedelta64(1,'M'))
# #factor_0['mob']=factor_0['mob'].astype('int')
# i8_cards['max_mob_report']=i8_cards['max_mob_report'].round()
# i8_cards['max_mob_report']=i8_cards['max_mob_report'].astype(int)

In [80]:
i8_cards['min_cum_count'] = i8_cards.groupby('MASK_CREDITCARD')['cum_count'].transform(min)
i8_cards['max_cum_count'] = i8_cards.groupby('MASK_CREDITCARD')['cum_count'].transform(max)

In [81]:
i8_cards

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,report_date,min_report_date,max_report_date,issue_date,mob,min_mob,max_mob,cum_count,min_cum_count,max_cum_count
0,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,50341.34,0.00,50341.34,15000.0,...,2012-11-30,2012-11-30,2013-07-31,2012-11-30,0,0,8,0,0,8
1,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56926.71,50341.34,56926.71,26000.0,...,2012-12-31,2012-11-30,2013-07-31,2012-11-30,1,0,8,1,0,8
2,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,60332.69,56926.71,60332.69,26000.0,...,2013-01-31,2012-11-30,2013-07-31,2012-11-30,2,0,8,2,0,8
3,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,59646.63,60332.69,59646.63,26000.0,...,2013-02-28,2012-11-30,2013-07-31,2012-11-30,3,0,8,3,0,8
4,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56928.16,59646.63,56928.16,26000.0,...,2013-03-31,2012-11-30,2013-07-31,2012-11-30,4,0,8,4,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142412,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,7379.02,1381.36,7379.02,0.0,...,2013-03-31,2012-03-31,2013-07-31,2012-03-31,12,0,16,12,0,16
142413,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,2829.02,7379.02,2829.02,0.0,...,2013-04-30,2012-03-31,2013-07-31,2012-03-31,13,0,16,13,0,16
142414,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,7819.52,2829.02,7819.52,0.0,...,2013-05-31,2012-03-31,2013-07-31,2012-03-31,14,0,16,14,0,16
142415,SRDFPFR,FTKUGGWWUNGPNNNT,2013-07-31,2012-01-08,12500,7500,10970.93,7819.52,10970.93,0.0,...,2013-06-30,2012-03-31,2013-07-31,2012-03-31,15,0,16,15,0,16


In [82]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 139155 entries, 0 to 142416
Data columns (total 35 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    139155 non-null  object        
 1   MASK_CREDITCARD       139155 non-null  object        
 2   A_card_expiry_date    139155 non-null  datetime64[ns]
 3   Customer_Join_Date    114660 non-null  datetime64[ns]
 4   CARD_LIMIT            139155 non-null  int64         
 5   CASH_LIMIT            139155 non-null  int64         
 6   OS_BALANCE            136037 non-null  float64       
 7   OPENING_BALANCE       136037 non-null  float64       
 8   CLOSING_BALANCE       136037 non-null  float64       
 9   CLOSING_BALANCE_CASH  136037 non-null  float64       
 10  CLOSING_BALANCE_FEES  136037 non-null  float64       
 11  NB_DAYS_DELINQ        139155 non-null  float64       
 12  Customer_Occupation   82800 non-null   object        
 13 

In [83]:
# i8_cards['valid']=np.where((i8_cards['min_mob']==i8_cards['min_mob_report']) & (i8_cards['min_mob']==i8_cards['min_cum_count']) & (i8_cards['min_mob_report']==i8_cards['min_cum_count']) &
#                            (i8_cards['max_mob']==i8_cards['max_mob_report']) & (i8_cards['max_mob']==i8_cards['max_cum_count']) & (i8_cards['max_mob_report']==i8_cards['max_cum_count'])
#                             ,1,0) 

In [84]:
#i8_cards.valid.value_counts()

In [85]:
i8_cards.shape

(139155, 35)

In [86]:
i8_cards['valid_vintage']=np.where((i8_cards['min_mob']==i8_cards['min_cum_count']) &
                                  (i8_cards['max_mob']==i8_cards['max_cum_count']),1,0)

In [87]:
i8_cards.valid_vintage.value_counts()

1    138818
0       337
Name: valid_vintage, dtype: int64

In [88]:
i8_cards.head()

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,min_report_date,max_report_date,issue_date,mob,min_mob,max_mob,cum_count,min_cum_count,max_cum_count,valid_vintage
0,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,50341.34,0.00,50341.34,15000.0,...,2012-11-30,2013-07-31,2012-11-30,0,0,8,0,0,8,1
1,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56926.71,50341.34,56926.71,26000.0,...,2012-11-30,2013-07-31,2012-11-30,1,0,8,1,0,8,1
2,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,60332.69,56926.71,60332.69,26000.0,...,2012-11-30,2013-07-31,2012-11-30,2,0,8,2,0,8,1
3,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,59646.63,60332.69,59646.63,26000.0,...,2012-11-30,2013-07-31,2012-11-30,3,0,8,3,0,8,1
4,DHSSDDG,CUKTNSPPNCCCNNNC,2013-07-31,2012-02-28,60000,36000,56928.16,59646.63,56928.16,26000.0,...,2012-11-30,2013-07-31,2012-11-30,4,0,8,4,0,8,1


In [89]:
i8_cards_invalid_vintage = i8_cards[(i8_cards['valid_vintage'] ==0)]

In [90]:
i8_cards_invalid_vintage

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,min_report_date,max_report_date,issue_date,mob,min_mob,max_mob,cum_count,min_cum_count,max_cum_count,valid_vintage
21741,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,25234.20,28465.80,25234.20,11258.40,...,2009-11-30,2013-07-31,2009-11-30,0,0,44,0,0,43,0
21742,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,28387.40,25234.20,28387.40,11558.40,...,2009-11-30,2013-07-31,2009-11-30,1,0,44,1,0,43,0
21743,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,28687.40,28387.40,28687.40,11558.40,...,2009-11-30,2013-07-31,2009-11-30,2,0,44,2,0,43,0
21744,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,26349.04,28687.40,26349.04,10500.00,...,2009-11-30,2013-07-31,2009-11-30,3,0,44,3,0,43,0
21745,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,26922.15,26349.04,26922.15,11868.43,...,2009-11-30,2013-07-31,2009-11-30,4,0,44,4,0,43,0
21746,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,26974.39,26922.15,26974.39,11520.67,...,2009-11-30,2013-07-31,2009-11-30,5,0,44,5,0,43,0
21747,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,28125.28,26974.39,28125.28,11500.00,...,2009-11-30,2013-07-31,2009-11-30,6,0,44,6,0,43,0
21748,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,26318.90,28125.28,26318.90,10796.93,...,2009-11-30,2013-07-31,2009-11-30,7,0,44,7,0,43,0
21749,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,25202.95,26318.90,25202.95,9480.98,...,2009-11-30,2013-07-31,2009-11-30,8,0,44,8,0,43,0
21750,GPQGPQH,FTKUGFNNCFNGGSKT,2013-07-08,2009-04-28,28000,16800,26836.75,25202.95,26836.75,8442.83,...,2009-11-30,2013-07-31,2009-11-30,9,0,44,9,0,43,0


In [91]:
i8_cards_invalid_vintage.shape

(337, 36)

In [92]:
i8_cards_invalid_vintage["MASK_CREDITCARD"].nunique()

9

In [93]:
i8_cards_valid_vintage = i8_cards[(i8_cards['valid_vintage'] ==1)]

In [94]:
i8_cards_valid_vintage.shape

(138818, 36)

In [95]:
i8_cards_valid_vintage["MASK_CREDITCARD"].nunique()

6234

In [96]:
#valid_vintage_check

In [97]:
# Filtering i8_cards by considering only valid_vintage and mob>=18

In [98]:
i8_cards= i8_cards[(i8_cards['valid_vintage'] ==1) & (i8_cards['max_mob'] >=18)]

In [99]:
i8_cards.shape

(109142, 36)

In [100]:
i8_cards.head()

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,min_report_date,max_report_date,issue_date,mob,min_mob,max_mob,cum_count,min_cum_count,max_cum_count,valid_vintage
6056,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,33513.00,0.00,33513.00,23600.0,...,2011-09-30,2013-07-31,2011-09-30,0,0,22,0,0,22,1
6057,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,39713.55,33513.00,39713.55,23900.0,...,2011-09-30,2013-07-31,2011-09-30,1,0,22,1,0,22,1
6058,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,40754.38,39713.55,40754.38,23900.0,...,2011-09-30,2013-07-31,2011-09-30,2,0,22,2,0,22,1
6059,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,39827.63,40754.38,39827.63,23900.0,...,2011-09-30,2013-07-31,2011-09-30,3,0,22,3,0,22,1
6060,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,40625.53,39827.63,40625.53,23900.0,...,2011-09-30,2013-07-31,2011-09-30,4,0,22,4,0,22,1


In [101]:
i8_cards["MASK_CREDITCARD"].nunique()

2892

In [102]:
#run_25thoct23

In [103]:
#i8_cards.to_csv('i8_cards_check.csv',index= False)

In [104]:
# Taking cases for max_mob >=12

In [105]:
#i8_cards_12 = i8_cards[(i8_cards['max_mob'] >=12)]

In [106]:
#i8_cards.shape

In [107]:
#i8_cards_12.shape

In [108]:
#i8_cards_18 = i8_cards[(i8_cards['max_mob'] >=18)]

In [109]:
#i8_cards_18.shape

In [110]:
i8_cards_6_12 =i8_cards[(i8_cards['mob'] >0) & (i8_cards['mob'] <=6)]

In [111]:
i8_cards_6_12['max_dq_6_12'] = i8_cards_6_12.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(max)

In [112]:
i8_cards_12_12 =i8_cards[(i8_cards['mob'] >6) & (i8_cards['mob'] <=12)]

In [113]:
i8_cards_12_12['max_dq_12_12'] = i8_cards_12_12.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(max)

In [114]:
i8_cards_6_12_sel = i8_cards_6_12[['MASK_CREDITCARD','max_dq_6_12']]

In [115]:
i8_cards_12_12_sel = i8_cards_12_12[['MASK_CREDITCARD','max_dq_12_12']]

In [116]:
i8_cards_6_12_sel = i8_cards_6_12_sel.drop_duplicates()

In [117]:
i8_cards_12_12_sel = i8_cards_12_12_sel.drop_duplicates()

In [118]:
i8_cards_6_12_sel.shape

(2892, 2)

In [119]:
i8_cards_12_12_sel.shape

(2892, 2)

In [120]:
roll_rate_6_12 =pd.merge(i8_cards_6_12_sel,i8_cards_12_12_sel,how='left',on='MASK_CREDITCARD')

In [121]:
roll_rate_6_12 = roll_rate_6_12[~roll_rate_6_12.max_dq_12_12.isnull()]

In [122]:
roll_rate_6_12.shape

(2892, 3)

In [123]:
roll_rate_6_12.to_csv('roll_rate_6_12.csv',index= False)

In [124]:
i8_cards_12_24 =i8_cards[(i8_cards['mob'] >0) & (i8_cards['mob'] <=12)]

In [125]:
i8_cards_12_24['max_dq_12_24'] = i8_cards_12_24.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(max)

In [126]:
i8_cards_24_24 =i8_cards[(i8_cards['mob'] >12) &(i8_cards['mob'] <=24)]

In [127]:
i8_cards_24_24['max_dq_24_24'] = i8_cards_24_24.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(max)

In [128]:
i8_cards_12_24_sel = i8_cards_12_24[['MASK_CREDITCARD','max_dq_12_24']]

In [129]:
i8_cards_24_24_sel = i8_cards_24_24[['MASK_CREDITCARD','max_dq_24_24']]

In [130]:
i8_cards_12_24_sel = i8_cards_12_24_sel.drop_duplicates()

In [131]:
i8_cards_24_24_sel = i8_cards_24_24_sel.drop_duplicates()

In [132]:
i8_cards_12_24_sel.shape

(2892, 2)

In [133]:
i8_cards_24_24_sel.shape

(2892, 2)

In [134]:
roll_rate_12_24 =pd.merge(i8_cards_12_24_sel,i8_cards_24_24_sel,how='left',on='MASK_CREDITCARD')

In [135]:
roll_rate_12_24.shape

(2892, 3)

In [136]:
roll_rate_12_24 = roll_rate_12_24[~roll_rate_12_24.max_dq_24_24.isnull()]

In [137]:
roll_rate_12_24.shape

(2892, 3)

In [138]:
roll_rate_12_24.to_csv('roll_rate_12_24.csv',index= False)

In [139]:
i8_cards_9_18 =i8_cards[(i8_cards['mob'] >0) & (i8_cards['mob'] <=9)]

In [140]:
i8_cards_9_18['max_dq_9_18'] = i8_cards_9_18.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(max)

In [141]:
i8_cards_18_18 =i8_cards[(i8_cards['mob'] >9) & (i8_cards['mob'] <=18)]

In [142]:
i8_cards_18_18['max_dq_18_18'] = i8_cards_18_18.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].transform(max)

In [143]:
i8_cards_9_18_sel = i8_cards_9_18[['MASK_CREDITCARD','max_dq_9_18']]

In [144]:
i8_cards_18_18_sel = i8_cards_18_18[['MASK_CREDITCARD','max_dq_18_18']]

In [145]:
i8_cards_9_18_sel = i8_cards_9_18_sel.drop_duplicates()

In [146]:
i8_cards_18_18_sel = i8_cards_18_18_sel.drop_duplicates()

In [147]:
i8_cards_9_18_sel.shape

(2892, 2)

In [148]:
i8_cards_18_18_sel.shape

(2892, 2)

In [149]:
roll_rate_9_18 =pd.merge(i8_cards_9_18_sel,i8_cards_18_18_sel,how='left',on='MASK_CREDITCARD')

In [150]:
roll_rate_9_18.shape

(2892, 3)

In [151]:
roll_rate_9_18 = roll_rate_9_18[~roll_rate_9_18.max_dq_18_18.isnull()]

In [152]:
roll_rate_9_18.shape

(2892, 3)

In [153]:
roll_rate_9_18.to_csv('roll_rate_9_18.csv',index= False)

In [154]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 36 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [155]:
# i8_cards['mob_report']=((i8_cards['report_date']-i8_cards['min_report_date'])/np.timedelta64(1,'M'))
# #factor_0['mob']=factor_0['mob'].astype('int')
# i8_cards['mob_report']=i8_cards['mob_report'].round()
# i8_cards['mob_report']=i8_cards['mob_report'].astype(int)

In [156]:
# i8_cards['mob_report_min'] = i8_cards.groupby('MASK_CREDITCARD')['mob_report'].transform(min)
# i8_cards['mob_report_max'] = i8_cards.groupby('MASK_CREDITCARD')['mob_report'].transform(max)

In [157]:
i8_cards.shape

(109142, 36)

In [158]:
i8_cards['bad_def']=np.where(i8_cards['NB_DAYS_DELINQ']>=4.0,1,0)

In [159]:
i8_cards.bad_def.value_counts()

0    91758
1    17384
Name: bad_def, dtype: int64

In [160]:
#i8_cards_vintage = i8_cards[(i8_cards['valid_vintage'] ==1) & (i8_cards['mob_report_max'] >=18)]

In [161]:
i8_cards.shape

(109142, 37)

In [162]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 37 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [163]:
i8_cards['issue_date_yr'] = pd.to_datetime(i8_cards['issue_date']).dt.strftime('%Y')

In [164]:
default_flag = i8_cards.groupby(['MASK_CREDITCARD','issue_date_yr'])['bad_def'].aggregate('max')

In [165]:
default_flag

MASK_CREDITCARD   issue_date_yr
FCPFSTNTFNUSNNNG  2011             1
FCPFSTNTNCSKNNNF  2011             0
FCPFSTNTNKUTNNNW  2011             0
FCPFSTNTNTGWNNNW  2011             0
FCPFSTNTSCCKNNNG  2011             0
                                  ..
FTKUGGWPWGFGNNNK  2011             1
FTKUGGWPWGPNNNNG  2011             0
FTKUGGWPWNTKNNNU  2011             0
FTKUGGWPWUGKNNNS  2011             0
FTKUGGWWUFFSNNNW  2011             0
Name: bad_def, Length: 2892, dtype: int32

In [166]:
type(default_flag)

pandas.core.series.Series

In [167]:
default_flag1 =default_flag.to_frame() 

In [168]:
default_flag1

,,bad_def
MASK_CREDITCARD,issue_date_yr,
FCPFSTNTFNUSNNNG,2011,1
FCPFSTNTNCSKNNNF,2011,0
FCPFSTNTNKUTNNNW,2011,0
FCPFSTNTNTGWNNNW,2011,0
FCPFSTNTSCCKNNNG,2011,0
...,...,...
FTKUGGWPWGFGNNNK,2011,1
FTKUGGWPWGPNNNNG,2011,0
FTKUGGWPWNTKNNNU,2011,0


In [169]:
type(default_flag1)

pandas.core.frame.DataFrame

In [170]:
i8_cards_default = i8_cards[i8_cards['bad_def'] ==1]

In [171]:
i8_cards_default.shape

(17384, 38)

In [172]:
default_cases = i8_cards_default.groupby(['MASK_CREDITCARD','issue_date_yr'])['mob'].aggregate('min')

In [173]:
default_cases

MASK_CREDITCARD   issue_date_yr
FCPFSTNTFNUSNNNG  2011             11
FCPFSTNTSGFNNNNW  2011              9
FCPFSTNTSGGFNNNC  2011             15
FCPFSTNTSGGUNNNW  2011             16
FCPFSTNTSGWWNNNG  2011              5
                                   ..
FTKUGGNNWUFSCPUW  2009             17
FTKUGGNNWWSFGTUT  2010             24
FTKUGGWPKWFKNNNC  2011             12
FTKUGGWPNNTWNNNK  2011              8
FTKUGGWPWGFGNNNK  2011             10
Name: mob, Length: 665, dtype: int32

In [174]:
default_cases1 =default_cases.to_frame() 

In [175]:
default_cases1

,,mob
MASK_CREDITCARD,issue_date_yr,
FCPFSTNTFNUSNNNG,2011,11
FCPFSTNTSGFNNNNW,2011,9
FCPFSTNTSGGFNNNC,2011,15
FCPFSTNTSGGUNNNW,2011,16
FCPFSTNTSGWWNNNG,2011,5
...,...,...
FTKUGGNNWUFSCPUW,2009,17
FTKUGGNNWWSFGTUT,2010,24
FTKUGGWPKWFKNNNC,2011,12


In [176]:
default_flag1.shape

(2892, 1)

In [177]:
default_cases1.shape

(665, 1)

In [178]:
default_flag1

,,bad_def
MASK_CREDITCARD,issue_date_yr,
FCPFSTNTFNUSNNNG,2011,1
FCPFSTNTNCSKNNNF,2011,0
FCPFSTNTNKUTNNNW,2011,0
FCPFSTNTNTGWNNNW,2011,0
FCPFSTNTSCCKNNNG,2011,0
...,...,...
FTKUGGWPWGFGNNNK,2011,1
FTKUGGWPWGPNNNNG,2011,0
FTKUGGWPWNTKNNNU,2011,0


In [179]:
default_cases1

,,mob
MASK_CREDITCARD,issue_date_yr,
FCPFSTNTFNUSNNNG,2011,11
FCPFSTNTSGFNNNNW,2011,9
FCPFSTNTSGGFNNNC,2011,15
FCPFSTNTSGGUNNNW,2011,16
FCPFSTNTSGWWNNNG,2011,5
...,...,...
FTKUGGNNWUFSCPUW,2009,17
FTKUGGNNWWSFGTUT,2010,24
FTKUGGWPKWFKNNNC,2011,12


In [180]:
vintage_dt =pd.merge(default_flag1,default_cases1,how='left',on=['MASK_CREDITCARD','issue_date_yr'])

In [181]:
vintage_dt.shape

(2892, 2)

In [182]:
vintage_dt

,,bad_def,mob
MASK_CREDITCARD,issue_date_yr,,
FCPFSTNTFNUSNNNG,2011,1,11.0
FCPFSTNTNCSKNNNF,2011,0,NaN
FCPFSTNTNKUTNNNW,2011,0,NaN
FCPFSTNTNTGWNNNW,2011,0,NaN
FCPFSTNTSCCKNNNG,2011,0,NaN
...,...,...,...
FTKUGGWPWGFGNNNK,2011,1,10.0
FTKUGGWPWGPNNNNG,2011,0,NaN
FTKUGGWPWNTKNNNU,2011,0,NaN


In [183]:
vintage_dt.fillna(999,inplace= True)

In [184]:
vintage_dt

,,bad_def,mob
MASK_CREDITCARD,issue_date_yr,,
FCPFSTNTFNUSNNNG,2011,1,11.0
FCPFSTNTNCSKNNNF,2011,0,999.0
FCPFSTNTNKUTNNNW,2011,0,999.0
FCPFSTNTNTGWNNNW,2011,0,999.0
FCPFSTNTSCCKNNNG,2011,0,999.0
...,...,...,...
FTKUGGWPWGFGNNNK,2011,1,10.0
FTKUGGWPWGPNNNNG,2011,0,999.0
FTKUGGWPWNTKNNNU,2011,0,999.0


In [185]:
vintage_dt.to_csv('vintage_analysis.csv')

In [186]:
#i8_cards_mom = i8_cards[(i8_cards['valid_vintage'] ==1) & (i8_cards['mob_report_max'] >=24)]

In [187]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 38 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [188]:
i8_cards['issue_date_yr_mon'] = pd.to_datetime(i8_cards['issue_date']).dt.strftime('%Y-%m')

In [189]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 39 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [190]:
MOM_Analysis = i8_cards.groupby(['issue_date_yr_mon'])['MASK_CREDITCARD'].aggregate('nunique')

In [191]:
MOM_Analysis

issue_date_yr_mon
2008-10     27
2008-11     46
2008-12     25
2009-01     29
2009-02     24
2009-03     25
2009-04     54
2009-05    154
2009-06    808
2009-07     94
2009-08     75
2009-09     53
2009-10     57
2009-11     53
2009-12     58
2010-01     47
2010-02     67
2010-03     38
2010-04     56
2010-05     37
2010-06     54
2010-07     89
2010-08     68
2010-09     60
2010-10     66
2010-11     59
2010-12     52
2011-01     33
2011-02     24
2011-03     18
2011-04     29
2011-05     38
2011-06     18
2011-07     43
2011-08     24
2011-09     67
2011-10     75
2011-11     77
2011-12    111
2012-01     60
Name: MASK_CREDITCARD, dtype: int64

In [192]:
i8_cards.MASK_CREDITCARD.nunique()

2892

Preparing Target Definition

In [193]:
### We will start preparing data on basis on the target variable With the definition of bad as 90+ in next 12 months

In [194]:
# Selecting cases with max reporting >=18

In [195]:
#i8_cards_vintage = i8_cards[(i8_cards['valid_vintage'] ==1) & (i8_cards['mob_report_max'] >=18)]

In [196]:
# Using i8_cards_vintage

In [197]:
#i8_cards_vintage.to_csv('i8_cards_vintage.csv')

In [198]:
#i8_cards_vintage1 = i8_cards_vintage

In [199]:
i8_cards = i8_cards.assign(
    lag1 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-1),
    lag2 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-2),
    lag3 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-3),
    lag4 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-4),
    lag5 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-5),
    lag6 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-6),
    lag7 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-7),
    lag8 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-8),
    lag9 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-9),
    lag10 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-10),
    lag11 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-11),
    lag12 = i8_cards.groupby('MASK_CREDITCARD')['bad_def'].shift(-12)
    #lag2 = i8_cards_vintage1['bad_def'].shift(2)
)

In [200]:
i8_cards["bad_flag"] = i8_cards[["lag1", "lag2","lag3","lag4","lag5","lag6","lag7","lag8","lag9","lag10","lag11","lag12"]].max(axis=1)

In [201]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 52 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [202]:
i8_cards['bad_flag'].fillna(0,inplace= True)

In [203]:
i8_cards=i8_cards.drop(["lag1","lag2","lag3","lag4","lag5","lag6","lag7","lag8","lag9","lag10","lag11","lag12"],axis=1)

In [204]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 40 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [205]:
i8_cards['report_date_yr_mon'] = pd.to_datetime(i8_cards['report_date']).dt.strftime('%Y-%m')

In [206]:
MOM_Analysis1 = i8_cards.groupby(['report_date_yr_mon'])['MASK_CREDITCARD'].aggregate('count')

In [207]:
MOM_Analysis1

report_date_yr_mon
2008-10      27
2008-11      73
2008-12      98
2009-01     127
2009-02     151
2009-03     176
2009-04     230
2009-05     384
2009-06    1192
2009-07    1286
2009-08    1361
2009-09    1414
2009-10    1471
2009-11    1524
2009-12    1582
2010-01    1629
2010-02    1696
2010-03    1734
2010-04    1790
2010-05    1826
2010-06    1880
2010-07    1967
2010-08    2034
2010-09    2094
2010-10    2157
2010-11    2215
2010-12    2264
2011-01    2272
2011-02    2270
2011-03    2267
2011-04    2264
2011-05    2285
2011-06    2284
2011-07    2317
2011-08    2329
2011-09    2388
2011-10    2446
2011-11    2498
2011-12    2598
2012-01    2637
2012-02    2617
2012-03    2599
2012-04    2589
2012-05    2572
2012-06    2525
2012-07    2515
2012-08    2495
2012-09    2479
2012-10    2460
2012-11    2445
2012-12    2434
2013-01    2414
2013-02    2371
2013-03    2349
2013-04    2311
2013-05    2261
2013-06    2242
2013-07    2227
Name: MASK_CREDITCARD, dtype: int64

In [208]:
MOM_Analysis2 = i8_cards.groupby(['report_date_yr_mon'])['bad_flag'].aggregate('sum')

In [209]:
MOM_Analysis2

report_date_yr_mon
2008-10     27.0
2008-11     73.0
2008-12     98.0
2009-01    127.0
2009-02    151.0
2009-03    175.0
2009-04    226.0
2009-05    298.0
2009-06    375.0
2009-07    384.0
2009-08    391.0
2009-09    407.0
2009-10    411.0
2009-11    411.0
2009-12    409.0
2010-01    412.0
2010-02    416.0
2010-03    417.0
2010-04    419.0
2010-05    421.0
2010-06    423.0
2010-07    424.0
2010-08    428.0
2010-09    426.0
2010-10    428.0
2010-11    430.0
2010-12    418.0
2011-01    416.0
2011-02    406.0
2011-03    398.0
2011-04    396.0
2011-05    394.0
2011-06    401.0
2011-07    411.0
2011-08    429.0
2011-09    445.0
2011-10    463.0
2011-11    475.0
2011-12    489.0
2012-01    500.0
2012-02    507.0
2012-03    506.0
2012-04    514.0
2012-05    517.0
2012-06    520.0
2012-07    522.0
2012-08    519.0
2012-09    514.0
2012-10    503.0
2012-11    491.0
2012-12    485.0
2013-01    479.0
2013-02    474.0
2013-03    470.0
2013-04    465.0
2013-05    457.0
2013-06    451.0
2013-07     

In [210]:
#i8_cards_vintage1 = i8_cards_vintage1.shift(1)

In [211]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 41 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [212]:
#i8_cards['Modeling_Data']=np.where((i8_cards['mob']>=6) | (i8_cards['bad_def']==0),1,0) 

In [213]:
#i8_cards.shape

In [214]:
#i8_cards.Modeling_Data.value_counts()

In [215]:
#i8_cards['Exclusion_MOB_LT6']=np.where((i8_cards['mob']<6),1,0) 

In [216]:
#i8_cards['Exclusion_DPD_GE4']=np.where((i8_cards['bad_def']==1),1,0) 

In [217]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 41 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [218]:
#i8_cards=i8_cards.drop(["Modeling_Data","Exclusion_MOB_LT6","Exclusion_DPD_GE4"],axis=1)

In [219]:
i8_cards_snap =i8_cards.assign(
                               Accounts =i8_cards['MASK_CREDITCARD'],
                               Exclusion_MOB_LT6 =np.where((i8_cards['mob']<6),1,0) ,
                               Exclusion_DPD_GE4 =np.where((i8_cards['mob']>=6) & (i8_cards['bad_def']==1),1,0) ,
                               Modeling_Data =np.where((i8_cards['mob']>=6) & (i8_cards['bad_def']==0),1,0)
                              ).groupby('report_date_yr_mon').agg({'Accounts' : 'count',
                                                                   'Exclusion_MOB_LT6' : 'sum',
                                                                   'Exclusion_DPD_GE4': 'sum',
                                                                   'Modeling_Data' :'sum'
                                                                  })

In [220]:
i8_cards_snap

,Accounts,Exclusion_MOB_LT6,Exclusion_DPD_GE4,Modeling_Data
report_date_yr_mon,,,,
2008-10,27,27,0,0
2008-11,73,73,0,0
2008-12,98,98,0,0
2009-01,127,127,0,0
2009-02,151,151,0,0
2009-03,176,176,0,0
2009-04,230,203,0,27
2009-05,384,311,0,73
2009-06,1192,1094,0,98


In [221]:
#excl_check

In [222]:
i8_cards['Exclusion_MOB_LT6']=np.where((i8_cards['mob']<6),1,0) 

In [223]:
i8_cards['Exclusion_DPD_GE4']=np.where((i8_cards['mob']>=6) & (i8_cards['bad_def']==1),1,0) 

In [224]:
i8_cards['Inclusion_MOB_GE6']=np.where((i8_cards['mob']>=6),1,0) 

In [225]:
i8_cards['Inclusion_DPD_LT4']=np.where((i8_cards['bad_def']==0),1,0) 

In [226]:
i8_cards['Modeling_Data']=np.where((i8_cards['mob']>=6) & (i8_cards['bad_def']==0),1,0) 

In [227]:
i8_cards['Bad']=np.where((i8_cards['Modeling_Data']==1) & (i8_cards['bad_flag']==1),1,0) 

In [228]:
i8_cards['Good']=np.where((i8_cards['Modeling_Data']==1) & (i8_cards['bad_flag']==0),1,0) 

In [229]:
#i8_cards['Bad_Rate']=np.where(factor_4.modified_limit==0,0,(factor_4.curr_bal_total/factor_4.modified_limit))

In [230]:
i8_cards.head()

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,A_card_expiry_date,Customer_Join_Date,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,...,issue_date_yr_mon,bad_flag,report_date_yr_mon,Exclusion_MOB_LT6,Exclusion_DPD_GE4,Inclusion_MOB_GE6,Inclusion_DPD_LT4,Modeling_Data,Bad,Good
6056,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,33513.00,0.00,33513.00,23600.0,...,2011-09,1.0,2011-09,1,0,0,1,0,0,0
6057,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,39713.55,33513.00,39713.55,23900.0,...,2011-09,1.0,2011-10,1,0,0,1,0,0,0
6058,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,40754.38,39713.55,40754.38,23900.0,...,2011-09,1.0,2011-11,1,0,0,1,0,0,0
6059,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,39827.63,40754.38,39827.63,23900.0,...,2011-09,1.0,2011-12,1,0,0,1,0,0,0
6060,FTTGSTT,FCPFSTNTFNUSNNNG,2013-07-31,2011-09-06,40000,24000,40625.53,39827.63,40625.53,23900.0,...,2011-09,1.0,2012-01,1,0,0,1,0,0,0


In [231]:
list(i8_cards.columns)

['MASKED_CUSTOMER_ID',
 'MASK_CREDITCARD',
 'A_card_expiry_date',
 'Customer_Join_Date',
 'CARD_LIMIT',
 'CASH_LIMIT',
 'OS_BALANCE',
 'OPENING_BALANCE',
 'CLOSING_BALANCE',
 'CLOSING_BALANCE_CASH',
 'CLOSING_BALANCE_FEES',
 'NB_DAYS_DELINQ',
 'Customer_Occupation',
 'REGIONS',
 'A_Gender',
 'NATIONALITY',
 'A_Marital_Status',
 'A_Industry',
 'A_Emp_Sts',
 'A_DOB',
 'A_Emp_Start',
 'A_No_Depend',
 'A_salary',
 'A_Sector',
 'min_period',
 'report_date',
 'min_report_date',
 'max_report_date',
 'issue_date',
 'mob',
 'min_mob',
 'max_mob',
 'cum_count',
 'min_cum_count',
 'max_cum_count',
 'valid_vintage',
 'bad_def',
 'issue_date_yr',
 'issue_date_yr_mon',
 'bad_flag',
 'report_date_yr_mon',
 'Exclusion_MOB_LT6',
 'Exclusion_DPD_GE4',
 'Inclusion_MOB_GE6',
 'Inclusion_DPD_LT4',
 'Modeling_Data',
 'Bad',
 'Good']

In [232]:
i8_cards['NB_DAYS_DELINQ'].value_counts(dropna=False)

0.0    71462
1.0    15809
9.0    14141
2.0     3267
3.0     1220
4.0      763
5.0      717
6.0      602
8.0      597
7.0      564
Name: NB_DAYS_DELINQ, dtype: int64

In [233]:
i8_cards_snap1 =i8_cards.assign(
                               Total_Count =i8_cards['MASK_CREDITCARD'],
                               DPD_0_Count = np.where((i8_cards['NB_DAYS_DELINQ'].isnull())|(i8_cards['NB_DAYS_DELINQ']==0),1,0),
                               DPD_1_Count = np.where((i8_cards['NB_DAYS_DELINQ']==1),1,0),
                               DPD_2_Count = np.where((i8_cards['NB_DAYS_DELINQ']==2),1,0),
                               DPD_3_Count = np.where((i8_cards['NB_DAYS_DELINQ']==3),1,0),
                               DPD_4_Count = np.where((i8_cards['NB_DAYS_DELINQ']>=4),1,0),
                               Total_Exposure = i8_cards['OS_BALANCE'],
                               Exclusion_MOB_LT6 =i8_cards['Exclusion_MOB_LT6'] ,
                               Exclusion_DPD_GE4 =i8_cards['Exclusion_DPD_GE4'] ,
                               Inclusion_MOB_GE6 = i8_cards['Inclusion_MOB_GE6'],
                               Inclusion_DPD_LT4 = i8_cards['Inclusion_DPD_LT4'],
                               Modeling_Data = i8_cards['Modeling_Data'],
                               Bad           = i8_cards['Bad'],
                               Good          = i8_cards['Good']
                              ).groupby('report_date_yr_mon').agg({'Total_Count' : 'count',
                                                                   'DPD_0_Count' : 'sum',
                                                                   'DPD_1_Count' : 'sum',
                                                                   'DPD_2_Count' : 'sum',
                                                                   'DPD_3_Count' : 'sum',
                                                                   'DPD_4_Count' : 'sum',
                                                                   'Total_Exposure' : 'sum',
                                                                   'Exclusion_MOB_LT6' : 'sum',
                                                                   'Exclusion_DPD_GE4': 'sum',
                                                                   'Inclusion_MOB_GE6':'sum',
                                                                   'Inclusion_DPD_LT4':'sum',
                                                                   'Modeling_Data' :'sum',
                                                                   'Bad':'sum',
                                                                   'Good':'sum',
                                                                  })

In [234]:
i8_cards_snap1

,Total_Count,DPD_0_Count,DPD_1_Count,DPD_2_Count,DPD_3_Count,DPD_4_Count,Total_Exposure,Exclusion_MOB_LT6,Exclusion_DPD_GE4,Inclusion_MOB_GE6,Inclusion_DPD_LT4,Modeling_Data,Bad,Good
report_date_yr_mon,,,,,,,,,,,,,,
2008-10,27,27,0,0,0,0,0.000000e+00,27,0,0,27,0,0,0
2008-11,73,73,0,0,0,0,0.000000e+00,73,0,0,73,0,0,0
2008-12,98,98,0,0,0,0,0.000000e+00,98,0,0,98,0,0,0
2009-01,127,127,0,0,0,0,0.000000e+00,127,0,0,127,0,0,0
2009-02,151,151,0,0,0,0,0.000000e+00,151,0,0,151,0,0,0
2009-03,176,176,0,0,0,0,0.000000e+00,176,0,0,176,0,0,0
2009-04,230,230,0,0,0,0,0.000000e+00,203,0,27,230,27,27,0
2009-05,384,384,0,0,0,0,0.000000e+00,311,0,73,384,73,73,0
2009-06,1192,1192,0,0,0,0,0.000000e+00,1094,0,98,1192,98,98,0


In [235]:
i8_cards_snap1.dtypes

Total_Count            int64
DPD_0_Count            int32
DPD_1_Count            int32
DPD_2_Count            int32
DPD_3_Count            int32
DPD_4_Count            int32
Total_Exposure       float64
Exclusion_MOB_LT6      int32
Exclusion_DPD_GE4      int32
Inclusion_MOB_GE6      int32
Inclusion_DPD_LT4      int32
Modeling_Data          int32
Bad                    int32
Good                   int32
dtype: object

In [236]:
i8_cards_snap1.to_csv('Data_waterfall.csv',index= True)

In [237]:
#stop_check_exclusion

In [238]:
#i8_cards.to_csv('i8_cards_check1.csv',index= False)

In [239]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 48 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   A_card_expiry_date    109142 non-null  datetime64[ns]
 3   Customer_Join_Date    87833 non-null   datetime64[ns]
 4   CARD_LIMIT            109142 non-null  int64         
 5   CASH_LIMIT            109142 non-null  int64         
 6   OS_BALANCE            106684 non-null  float64       
 7   OPENING_BALANCE       106684 non-null  float64       
 8   CLOSING_BALANCE       106684 non-null  float64       
 9   CLOSING_BALANCE_CASH  106684 non-null  float64       
 10  CLOSING_BALANCE_FEES  106684 non-null  float64       
 11  NB_DAYS_DELINQ        109142 non-null  float64       
 12  Customer_Occupation   60635 non-null   object        
 

In [240]:
# Changing Card Expiry Date and Customer Joining Date to Month End 

In [241]:
i8_cards['card_expiry_date']=i8_cards['A_card_expiry_date']+pd.offsets.MonthEnd(0)

In [242]:
i8_cards['customer_join_date']=i8_cards['Customer_Join_Date']+pd.offsets.MonthEnd(0)

In [243]:
i8_cards=i8_cards.drop(["A_card_expiry_date","Customer_Join_Date"],axis=1)

In [244]:
i8_cards.A_Sector.value_counts()

Government         35771
Semi Government     6402
Private             3707
Free Zone             47
Name: A_Sector, dtype: int64

In [245]:
# Replacing REGIONS value in i8_cards 

In [246]:
i8_cards.REGIONS.value_counts()

1    70609
2    22781
3     8418
6     4924
4     1586
7      571
5      253
Name: REGIONS, dtype: int64

In [247]:
i8_cards.loc[i8_cards['REGIONS']==1,'REGION']="Abu_Dhabi"
i8_cards.loc[i8_cards['REGIONS']==2,'REGION']="Dubai"
i8_cards.loc[i8_cards['REGIONS']>2,'REGION']="Others"

In [248]:
i8_cards.REGION.value_counts()

Abu_Dhabi    70609
Dubai        22781
Others       15752
Name: REGION, dtype: int64

In [249]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 49 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   CARD_LIMIT            109142 non-null  int64         
 3   CASH_LIMIT            109142 non-null  int64         
 4   OS_BALANCE            106684 non-null  float64       
 5   OPENING_BALANCE       106684 non-null  float64       
 6   CLOSING_BALANCE       106684 non-null  float64       
 7   CLOSING_BALANCE_CASH  106684 non-null  float64       
 8   CLOSING_BALANCE_FEES  106684 non-null  float64       
 9   NB_DAYS_DELINQ        109142 non-null  float64       
 10  Customer_Occupation   60635 non-null   object        
 11  REGIONS               109142 non-null  int64         
 12  A_Gender              82870 non-null   object        
 

In [250]:
# Reading mapping file for Nationality and Merchant Category 

In [251]:
Nationality_Mapping = pd.read_excel('Mapping.xlsx','Sheet1')

In [252]:
Nationality_Mapping

,NATION_CODE,NATIONALITY
0,784,United Arab Emirates
1,400,Jordan
2,586,Pakistan
3,356,India
4,818,Egypt
5,760,Syria
6,275,Palestine
7,422,Lebanon
8,608,Philippines
9,364,Iran


In [253]:
# # renaming the column "A"
# df.rename(columns = {"Name": "Names"}, 
#           inplace = True)

In [254]:
#i8_cards=i8_cards.drop(["NATION_CODE","NATIONALITY_y"],axis=1)

In [255]:
# i8_cards.rename(columns = {"NATIONALITY_x": "NATIONALITY"}, 
#           inplace = True)

In [256]:
i8_cards.columns

Index(['MASKED_CUSTOMER_ID', 'MASK_CREDITCARD', 'CARD_LIMIT', 'CASH_LIMIT',
       'OS_BALANCE', 'OPENING_BALANCE', 'CLOSING_BALANCE',
       'CLOSING_BALANCE_CASH', 'CLOSING_BALANCE_FEES', 'NB_DAYS_DELINQ',
       'Customer_Occupation', 'REGIONS', 'A_Gender', 'NATIONALITY',
       'A_Marital_Status', 'A_Industry', 'A_Emp_Sts', 'A_DOB', 'A_Emp_Start',
       'A_No_Depend', 'A_salary', 'A_Sector', 'min_period', 'report_date',
       'min_report_date', 'max_report_date', 'issue_date', 'mob', 'min_mob',
       'max_mob', 'cum_count', 'min_cum_count', 'max_cum_count',
       'valid_vintage', 'bad_def', 'issue_date_yr', 'issue_date_yr_mon',
       'bad_flag', 'report_date_yr_mon', 'Exclusion_MOB_LT6',
       'Exclusion_DPD_GE4', 'Inclusion_MOB_GE6', 'Inclusion_DPD_LT4',
       'Modeling_Data', 'Bad', 'Good', 'card_expiry_date',
       'customer_join_date', 'REGION'],
      dtype='object')

In [257]:
# new_df = pd.merge(A_df, B_df,  how='left', left_on='[A_c1,c2]', right_on = '[B_c1,c2]')

In [258]:
#i8_cards =pd.merge(i8_cards,Nationality_Mapping,how='left',left_on='NATIONALITY', right_on = 'NATION_CODE')

In [259]:
i8_cards.rename(columns = {"NATIONALITY": "NATION_CODE"}, 
          inplace = True)

In [260]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 6056 to 142399
Data columns (total 49 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   CARD_LIMIT            109142 non-null  int64         
 3   CASH_LIMIT            109142 non-null  int64         
 4   OS_BALANCE            106684 non-null  float64       
 5   OPENING_BALANCE       106684 non-null  float64       
 6   CLOSING_BALANCE       106684 non-null  float64       
 7   CLOSING_BALANCE_CASH  106684 non-null  float64       
 8   CLOSING_BALANCE_FEES  106684 non-null  float64       
 9   NB_DAYS_DELINQ        109142 non-null  float64       
 10  Customer_Occupation   60635 non-null   object        
 11  REGIONS               109142 non-null  int64         
 12  A_Gender              82870 non-null   object        
 

In [261]:
i8_cards =pd.merge(i8_cards,Nationality_Mapping,how='left', on ='NATION_CODE')

In [262]:
i8_cards=i8_cards.drop(["NATION_CODE","REGIONS"],axis=1)

In [263]:
i8_cards['cust_mob']=((i8_cards['report_date']-i8_cards['customer_join_date'])/np.timedelta64(1,'M'))
i8_cards['cust_mob']=i8_cards['cust_mob'].round()
#i8_cards['cust_mob']=i8_cards['cust_mob'].astype(int)

In [264]:
i8_cards.cust_mob.value_counts()

 16.0    2004
 20.0    1986
 19.0    1984
 21.0    1974
 15.0    1968
 22.0    1967
 18.0    1966
 17.0    1940
 14.0    1937
 23.0    1934
 24.0    1933
 25.0    1928
 26.0    1920
 27.0    1918
 13.0    1904
 28.0    1896
 29.0    1866
 12.0    1861
 30.0    1833
 31.0    1808
 32.0    1792
 33.0    1772
 11.0    1758
 34.0    1743
 35.0    1709
 36.0    1674
 10.0    1661
 37.0    1657
 38.0    1625
 39.0    1584
 9.0     1567
 40.0    1542
 41.0    1506
 42.0    1468
 8.0     1461
 43.0    1409
 7.0     1372
 44.0    1370
 45.0    1332
 46.0    1310
 6.0     1278
 47.0    1271
 48.0    1223
 49.0    1184
 5.0     1169
 50.0    1107
 4.0     1084
 51.0    1000
 3.0      971
 2.0      854
 52.0     598
 1.0      508
 53.0     500
 54.0     435
 55.0     392
 56.0     364
 0.0      336
 57.0     309
 58.0     259
-1.0      219
-2.0      208
-3.0      184
 59.0     182
-4.0      163
-5.0      145
 60.0     117
-6.0      114
-7.0      103
-8.0       97
-9.0       94
-10.0      89
-11.0 

In [265]:
# Age calculation 

In [266]:
i8_cards['Age']=((i8_cards['report_date']-i8_cards['A_DOB'])/np.timedelta64(1,'Y'))
i8_cards['Age']=i8_cards['Age'].round()

In [267]:
i8_cards.Age.value_counts()

 33.0    4697
 32.0    4566
 34.0    4541
 31.0    4126
 35.0    4115
 36.0    3780
 30.0    3714
 37.0    3336
 29.0    3334
 28.0    3244
 38.0    2997
 27.0    2986
 39.0    2946
 40.0    2773
 41.0    2762
 26.0    2701
 42.0    2521
 43.0    2314
 44.0    2128
 25.0    2099
 45.0    1982
 46.0    1800
 47.0    1746
 48.0    1519
 24.0    1508
 49.0    1452
 50.0    1319
 51.0    1146
 52.0    1042
 53.0     968
 54.0     869
 23.0     860
 55.0     798
 56.0     672
 57.0     572
 58.0     455
 22.0     402
 59.0     347
 60.0     265
 5.0      265
 61.0     247
 62.0     192
 6.0      185
 4.0      137
 7.0      132
 8.0      115
 3.0      106
 21.0      99
 63.0      93
 2.0       91
 67.0      50
 20.0      39
 68.0      38
 64.0      37
 69.0      36
 66.0      36
 65.0      33
 18.0      24
 1.0       22
 19.0      19
 70.0      17
 17.0      13
 74.0      12
 75.0      12
 76.0      12
 77.0      12
 15.0      12
 16.0      12
-0.0       10
 73.0       9
 9.0        8
 14.0 

In [268]:
i8_cards[["MASKED_CUSTOMER_ID","report_date","A_DOB","Age"]].sample(100)

,MASKED_CUSTOMER_ID,report_date,A_DOB,Age
35532,RGPHPRR,2011-09-30,1979-10-15,32.0
26089,REEEESD,2011-09-30,1970-07-15,41.0
90078,REEERQR,2010-12-31,1978-12-29,32.0
15191,GQGFRFR,2010-04-30,1986-07-13,24.0
40163,GGEGGHG,2011-12-31,NaT,NaN
43479,GHDEEEP,2012-01-31,NaT,NaN
6099,SSSSSSS,2011-05-31,1960-02-15,51.0
17006,THRRRQR,2011-07-31,1980-12-18,31.0
55689,RQFFRQF,2011-04-30,1976-04-02,35.0
17643,SRFRSSE,2012-04-30,1975-11-02,36.0


In [269]:
## Transaction data 

In [270]:
i1_card_transaction.shape

(789967, 6)

In [271]:
i1_card_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 789967 entries, 0 to 809754
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   MASK_CREDITCARD     789967 non-null  object        
 1   TRANS_DESCRIPTION   789967 non-null  object        
 2   POST_DATE_AND_TIME  789967 non-null  datetime64[ns]
 3   MERCHANT_CATEGORY   789967 non-null  int64         
 4   REVERSAL_FLAG       789967 non-null  object        
 5   AMOUNT_IN_LCY       789967 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 42.2+ MB


In [272]:
i1_card_transaction

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,MERCHANT_CATEGORY,REVERSAL_FLAG,AMOUNT_IN_LCY
0,CUKTNSPPNCCCNNNC,Card Fee,2012-11-09,6012,N,250
1,CUKTNSPPNCCCNNNC,Unique Transaction,2012-11-12,6051,N,15000
2,CUKTNSPPNCCCNNNC,Unique Transaction,2012-11-12,6051,N,10000
3,CUKTNSPPNCCCNNNC,Unique Transaction,2012-11-18,6051,N,7216
4,CUKTNSPPNCCCNNNC,Purchase REVERSAL,2012-11-18,5411,N,1925
...,...,...,...,...,...,...
809750,FTKUGGWWUNGPNNNT,Purchase REVERSAL,2013-06-26,9399,N,510
809751,FTKUGGWWUNGPNNNT,Payment,2013-06-27,6012,N,391
809752,FTKUGGWWUNGPNNNT,Monthly Murabaha Installment,2013-06-30,6012,N,296
809753,FTKUGGWWUNGPNNNT,Profit Earned (Special Sav A\c),2013-06-30,6012,N,1


In [273]:
# Coverting Trans Description to uppercase i1_card_transactions

In [274]:
i1_card_transaction['TRANS_DESCRIPTION'] = i1_card_transaction['TRANS_DESCRIPTION'].str.upper()

In [275]:
i1_card_transaction['TRANS_DESCRIPTION'] = i1_card_transaction['TRANS_DESCRIPTION'].str.strip()

In [276]:
i1_card_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 789967 entries, 0 to 809754
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   MASK_CREDITCARD     789967 non-null  object        
 1   TRANS_DESCRIPTION   789967 non-null  object        
 2   POST_DATE_AND_TIME  789967 non-null  datetime64[ns]
 3   MERCHANT_CATEGORY   789967 non-null  int64         
 4   REVERSAL_FLAG       789967 non-null  object        
 5   AMOUNT_IN_LCY       789967 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 42.2+ MB


In [277]:
i1_card_transaction

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,MERCHANT_CATEGORY,REVERSAL_FLAG,AMOUNT_IN_LCY
0,CUKTNSPPNCCCNNNC,CARD FEE,2012-11-09,6012,N,250
1,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-12,6051,N,15000
2,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-12,6051,N,10000
3,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-18,6051,N,7216
4,CUKTNSPPNCCCNNNC,PURCHASE REVERSAL,2012-11-18,5411,N,1925
...,...,...,...,...,...,...
809750,FTKUGGWWUNGPNNNT,PURCHASE REVERSAL,2013-06-26,9399,N,510
809751,FTKUGGWWUNGPNNNT,PAYMENT,2013-06-27,6012,N,391
809752,FTKUGGWWUNGPNNNT,MONTHLY MURABAHA INSTALLMENT,2013-06-30,6012,N,296
809753,FTKUGGWWUNGPNNNT,PROFIT EARNED (SPECIAL SAV A\C),2013-06-30,6012,N,1


In [278]:
# Filtering out values with REVERSAL_FLAG as R and keeping only N values 

In [279]:
#i1_card_transaction= i1_card_transaction[(df.origin == "JFK") & (df.carrier == "B6")]

In [280]:
i1_card_transaction = i1_card_transaction[i1_card_transaction['REVERSAL_FLAG'] =="N"]

In [281]:
i1_card_transaction.shape

(788087, 6)

In [282]:
i1_card_transaction['TRANSACTION_DATE']=i1_card_transaction['POST_DATE_AND_TIME']+pd.offsets.MonthEnd(0)

In [283]:
i1_card_transaction['PREV_TRANSACTION_DATE']  = i1_card_transaction['TRANSACTION_DATE'] - pd.DateOffset(months=1)

In [284]:
i1_card_transaction['PREV_TRANSACTION_DATE']  = i1_card_transaction['TRANSACTION_DATE'] + pd.offsets.MonthEnd(0)

In [285]:
i1_card_transaction

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,MERCHANT_CATEGORY,REVERSAL_FLAG,AMOUNT_IN_LCY,TRANSACTION_DATE,PREV_TRANSACTION_DATE
0,CUKTNSPPNCCCNNNC,CARD FEE,2012-11-09,6012,N,250,2012-11-30,2012-11-30
1,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-12,6051,N,15000,2012-11-30,2012-11-30
2,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-12,6051,N,10000,2012-11-30,2012-11-30
3,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-18,6051,N,7216,2012-11-30,2012-11-30
4,CUKTNSPPNCCCNNNC,PURCHASE REVERSAL,2012-11-18,5411,N,1925,2012-11-30,2012-11-30
...,...,...,...,...,...,...,...,...
809750,FTKUGGWWUNGPNNNT,PURCHASE REVERSAL,2013-06-26,9399,N,510,2013-06-30,2013-06-30
809751,FTKUGGWWUNGPNNNT,PAYMENT,2013-06-27,6012,N,391,2013-06-30,2013-06-30
809752,FTKUGGWWUNGPNNNT,MONTHLY MURABAHA INSTALLMENT,2013-06-30,6012,N,296,2013-06-30,2013-06-30
809753,FTKUGGWWUNGPNNNT,PROFIT EARNED (SPECIAL SAV A\C),2013-06-30,6012,N,1,2013-06-30,2013-06-30


In [286]:
Merchant_Mapping = pd.read_excel('Mapping.xlsx','Sheet2')

In [287]:
Merchant_Mapping

,Merchant_Category_Code,Merchant_Category,Merchant_Category_Description
0,742,Veterinary Services,Veterinary Services
1,763,Agricultural Cooperative,Agricultural Co-operatives
2,780,Landscaping Services,"Horticultural Services, Landscaping Services"
3,1520,General Contractors,General Contractors-Residential and Commercial
4,1711,"Heating, Plumbing, A/C",Air Conditioning Contractors – Sales and Insta...
5,1740,"Masonry, Stonework, and Plaster","Insulation – Contractors, Masonry, Stonework C..."
6,1750,Carpentry Contractors,Carpentry Contractors
7,1761,"Roofing/Siding, Sheet Metal","Roofing – Contractors, Sheet Metal Work – Cont..."
8,1771,Concrete Work Contractors,Contractors – Concrete Work
9,1799,Special Trade Contractors,"Contractors – Special Trade, Not Elsewhere Cla..."


In [288]:
Merchant_Mapping['Merchant_Category']=Merchant_Mapping['Merchant_Category'].str.upper()

In [289]:
Merchant_Mapping['Merchant_Category']= Merchant_Mapping['Merchant_Category'].str.strip()

In [290]:
Merchant_Mapping['Merchant_Category_Description']=Merchant_Mapping['Merchant_Category_Description'].str.upper()

In [291]:
Merchant_Mapping['Merchant_Category_Description']=Merchant_Mapping['Merchant_Category_Description'].str.strip()

In [292]:
i1_card_transaction =pd.merge(i1_card_transaction,Merchant_Mapping,how='left',left_on='MERCHANT_CATEGORY', right_on = 'Merchant_Category_Code')

In [293]:
i1_card_transaction.shape

(788087, 11)

In [294]:
i1_card_transaction

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,MERCHANT_CATEGORY,REVERSAL_FLAG,AMOUNT_IN_LCY,TRANSACTION_DATE,PREV_TRANSACTION_DATE,Merchant_Category_Code,Merchant_Category,Merchant_Category_Description
0,CUKTNSPPNCCCNNNC,CARD FEE,2012-11-09,6012,N,250,2012-11-30,2012-11-30,6012,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
1,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-12,6051,N,15000,2012-11-30,2012-11-30,6051,"NON-FI, MONEY ORDERS","NON-FINANCIAL INSTITUTIONS – FOREIGN CURRENCY,..."
2,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-12,6051,N,10000,2012-11-30,2012-11-30,6051,"NON-FI, MONEY ORDERS","NON-FINANCIAL INSTITUTIONS – FOREIGN CURRENCY,..."
3,CUKTNSPPNCCCNNNC,UNIQUE TRANSACTION,2012-11-18,6051,N,7216,2012-11-30,2012-11-30,6051,"NON-FI, MONEY ORDERS","NON-FINANCIAL INSTITUTIONS – FOREIGN CURRENCY,..."
4,CUKTNSPPNCCCNNNC,PURCHASE REVERSAL,2012-11-18,5411,N,1925,2012-11-30,2012-11-30,5411,"GROCERY STORES, SUPERMARKETS","GROCERY STORES, SUPERMARKETS"
...,...,...,...,...,...,...,...,...,...,...,...
788082,FTKUGGWWUNGPNNNT,PURCHASE REVERSAL,2013-06-26,9399,N,510,2013-06-30,2013-06-30,9399,GOVERNMENT SERVICES (NOT ELSEWHERE CLASSIFIED),GOVERNMENT SERVICES ( NOT ELSEWHERE CLASSIFIED)
788083,FTKUGGWWUNGPNNNT,PAYMENT,2013-06-27,6012,N,391,2013-06-30,2013-06-30,6012,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
788084,FTKUGGWWUNGPNNNT,MONTHLY MURABAHA INSTALLMENT,2013-06-30,6012,N,296,2013-06-30,2013-06-30,6012,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
788085,FTKUGGWWUNGPNNNT,PROFIT EARNED (SPECIAL SAV A\C),2013-06-30,6012,N,1,2013-06-30,2013-06-30,6012,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES


In [295]:
i1_card_transaction.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788087 entries, 0 to 788086
Data columns (total 11 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   MASK_CREDITCARD                788087 non-null  object        
 1   TRANS_DESCRIPTION              788087 non-null  object        
 2   POST_DATE_AND_TIME             788087 non-null  datetime64[ns]
 3   MERCHANT_CATEGORY              788087 non-null  int64         
 4   REVERSAL_FLAG                  788087 non-null  object        
 5   AMOUNT_IN_LCY                  788087 non-null  int64         
 6   TRANSACTION_DATE               788087 non-null  datetime64[ns]
 7   PREV_TRANSACTION_DATE          788087 non-null  datetime64[ns]
 8   Merchant_Category_Code         788087 non-null  int64         
 9   Merchant_Category              788017 non-null  object        
 10  Merchant_Category_Description  788017 non-null  object        
dtype

In [296]:
i1_card_transaction=i1_card_transaction.drop(["MERCHANT_CATEGORY","Merchant_Category_Code"],axis=1)

In [297]:
i1_card_transaction.Merchant_Category.value_counts()

FINANCIAL INSTITUTIONS                                                                                                      360764
GROCERY STORES, SUPERMARKETS                                                                                                 69529
AUTOMATED CASH DISBURSE                                                                                                      40924
SERVICE STATIONS                                                                                                             38238
EATING PLACES, RESTAURANTS                                                                                                   22425
FAMILY CLOTHING STORES                                                                                                       18734
HOTELS, MOTELS, AND RESORTS                                                                                                  15878
DEPARTMENT STORES                                                                  

In [298]:
#check1_break

In [299]:
payment_data = i1_card_transaction[(i1_card_transaction['TRANS_DESCRIPTION'].isin(['CARD PAYMENT','PAYMENT']))]

In [300]:
payment_data.shape

(97857, 9)

In [301]:
grp_payment_data  = payment_data.groupby(['MASK_CREDITCARD','PREV_TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_payment_data['Primary_Key'] = grp_payment_data['MASK_CREDITCARD'].astype(str) + "_" + grp_payment_data['PREV_TRANSACTION_DATE'].astype(str)
grp_payment_data
print(len(grp_payment_data))

83356


In [302]:
grp_payment_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83356 entries, 0 to 83355
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   MASK_CREDITCARD        83356 non-null  object        
 1   PREV_TRANSACTION_DATE  83356 non-null  datetime64[ns]
 2   AMOUNT_IN_LCY          83356 non-null  int64         
 3   Primary_Key            83356 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 2.5+ MB


In [303]:
late_payment_data = i1_card_transaction[(i1_card_transaction['TRANS_DESCRIPTION'].isin(['LATE PAYMENT CHARGES']))]

In [304]:
late_payment_data

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,REVERSAL_FLAG,AMOUNT_IN_LCY,TRANSACTION_DATE,PREV_TRANSACTION_DATE,Merchant_Category,Merchant_Category_Description
28,CUKTNSPPNCCCNNNC,LATE PAYMENT CHARGES,2013-01-29,N,100,2013-01-31,2013-01-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
34,CUKTNSPPNCCCNNNC,LATE PAYMENT CHARGES,2013-02-28,N,100,2013-02-28,2013-02-28,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
40,CUKTNSPPNCCCNNNC,LATE PAYMENT CHARGES,2013-03-30,N,100,2013-03-31,2013-03-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
47,CUKTNSPPNCCCNNNC,LATE PAYMENT CHARGES,2013-05-29,N,100,2013-05-31,2013-05-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
62,CUKTNSPPNCCCNNNC,LATE PAYMENT CHARGES,2013-06-29,N,100,2013-06-30,2013-06-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
...,...,...,...,...,...,...,...,...,...
787895,FTKUGGWWUFFSNNNW,LATE PAYMENT CHARGES,2012-07-29,N,100,2012-07-31,2012-07-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
787903,FTKUGGWWUFFSNNNW,LATE PAYMENT CHARGES,2012-09-29,N,100,2012-09-30,2012-09-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
787930,FTKUGGWWUFFSNNNW,LATE PAYMENT CHARGES,2013-01-29,N,100,2013-01-31,2013-01-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES
787945,FTKUGGWWUFFSNNNW,LATE PAYMENT CHARGES,2013-04-29,N,100,2013-04-30,2013-04-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES


In [305]:
late_payment_data['Count_Late_Payment'] = 1


In [306]:
late_payment_data.shape

(35562, 10)

In [307]:
grp_late_payment_data  = late_payment_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('Count_Late_Payment').reset_index()
grp_late_payment_data['Primary_Key'] = grp_late_payment_data['MASK_CREDITCARD'].astype(str) + "_" + grp_late_payment_data['TRANSACTION_DATE'].astype(str)
grp_late_payment_data
print(len(grp_late_payment_data))

35561


In [308]:
grp_late_payment_data

,MASK_CREDITCARD,TRANSACTION_DATE,AMOUNT_IN_LCY,Count_Late_Payment,Primary_Key
0,CUKTNSPPNCCCNNNC,2013-01-31,100,1,CUKTNSPPNCCCNNNC_2013-01-31
1,CUKTNSPPNCCCNNNC,2013-02-28,100,1,CUKTNSPPNCCCNNNC_2013-02-28
2,CUKTNSPPNCCCNNNC,2013-03-31,100,1,CUKTNSPPNCCCNNNC_2013-03-31
3,CUKTNSPPNCCCNNNC,2013-05-31,100,1,CUKTNSPPNCCCNNNC_2013-05-31
4,CUKTNSPPNCCCNNNC,2013-06-30,100,1,CUKTNSPPNCCCNNNC_2013-06-30
...,...,...,...,...,...
35556,FTKUGGWWUFFSNNNW,2012-07-31,100,1,FTKUGGWWUFFSNNNW_2012-07-31
35557,FTKUGGWWUFFSNNNW,2012-09-30,100,1,FTKUGGWWUFFSNNNW_2012-09-30
35558,FTKUGGWWUFFSNNNW,2013-01-31,100,1,FTKUGGWWUFFSNNNW_2013-01-31
35559,FTKUGGWWUFFSNNNW,2013-04-30,100,1,FTKUGGWWUFFSNNNW_2013-04-30


In [309]:
grp_late_payment_data=grp_late_payment_data.drop(['AMOUNT_IN_LCY'],axis=1)

In [310]:
grp_late_payment_data.Count_Late_Payment.value_counts()

1    35560
2        1
Name: Count_Late_Payment, dtype: int64

In [311]:
cash_trans_data = i1_card_transaction[(i1_card_transaction['TRANS_DESCRIPTION'].isin(['WITHDRAWAL','CASH ADVANCE']))]

In [312]:
cash_trans_data.shape

(41213, 9)

In [313]:
cash_trans_data['Count_Cash_Transactions'] = 1

In [314]:
grp_cash_trans_data = cash_trans_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('Count_Cash_Transactions').reset_index()
grp_cash_trans_data['Primary_Key'] = grp_cash_trans_data['MASK_CREDITCARD'].astype(str) + "_" + grp_cash_trans_data['TRANSACTION_DATE'].astype(str)
grp_cash_trans_data 
print(len(grp_cash_trans_data))

22019


In [315]:
grp_cash_trans_data

,MASK_CREDITCARD,TRANSACTION_DATE,AMOUNT_IN_LCY,Count_Cash_Transactions,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-11-30,10000,2,CUKTNSPPNCCCNNNC_2012-11-30
1,CUKTNSPPNCCCNNNC,2012-12-31,11000,3,CUKTNSPPNCCCNNNC_2012-12-31
2,CUKTNSPPNCCKNUTW,2013-02-28,1809,1,CUKTNSPPNCCKNUTW_2013-02-28
3,CUKTNSPPNCCNNNNN,2013-01-31,5000,1,CUKTNSPPNCCNNNNN_2013-01-31
4,CUKTNSPPNCCNNNNN,2013-02-28,4500,3,CUKTNSPPNCCNNNNN_2013-02-28
...,...,...,...,...,...
22014,FTKUGGWWSFWGNNNF,2012-10-31,6000,3,FTKUGGWWSFWGNNNF_2012-10-31
22015,FTKUGGWWSFWGNNNF,2012-12-31,100,1,FTKUGGWWSFWGNNNF_2012-12-31
22016,FTKUGGWWSFWGNNNF,2013-02-28,200,1,FTKUGGWWSFWGNNNF_2013-02-28
22017,FTKUGGWWUFFSNNNW,2012-03-31,1400,2,FTKUGGWWUFFSNNNW_2012-03-31


In [316]:
grp_cash_trans_data=grp_cash_trans_data.drop(['AMOUNT_IN_LCY'],axis=1)

In [317]:
cash_withdrawal_data = i1_card_transaction[(i1_card_transaction['TRANS_DESCRIPTION'].isin(['WITHDRAWAL']))]

In [318]:
cash_withdrawal_data.shape

(40932, 9)

In [319]:
grp_cash_withdrawal_data  = cash_withdrawal_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_cash_withdrawal_data['Primary_Key'] = grp_cash_withdrawal_data['MASK_CREDITCARD'].astype(str) + "_" + grp_cash_withdrawal_data['TRANSACTION_DATE'].astype(str)
grp_cash_withdrawal_data
print(len(grp_cash_withdrawal_data))

21816


In [320]:
grp_cash_withdrawal_data

,MASK_CREDITCARD,TRANSACTION_DATE,AMOUNT_IN_LCY,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-11-30,10000,CUKTNSPPNCCCNNNC_2012-11-30
1,CUKTNSPPNCCCNNNC,2012-12-31,11000,CUKTNSPPNCCCNNNC_2012-12-31
2,CUKTNSPPNCCKNUTW,2013-02-28,1809,CUKTNSPPNCCKNUTW_2013-02-28
3,CUKTNSPPNCCNNNNN,2013-01-31,5000,CUKTNSPPNCCNNNNN_2013-01-31
4,CUKTNSPPNCCNNNNN,2013-02-28,4500,CUKTNSPPNCCNNNNN_2013-02-28
...,...,...,...,...
21811,FTKUGGWWSFWGNNNF,2012-10-31,6000,FTKUGGWWSFWGNNNF_2012-10-31
21812,FTKUGGWWSFWGNNNF,2012-12-31,100,FTKUGGWWSFWGNNNF_2012-12-31
21813,FTKUGGWWSFWGNNNF,2013-02-28,200,FTKUGGWWSFWGNNNF_2013-02-28
21814,FTKUGGWWUFFSNNNW,2012-03-31,1400,FTKUGGWWUFFSNNNW_2012-03-31


In [321]:
grp_cash_withdrawal_data.rename(columns = {"AMOUNT_IN_LCY": "Withdrawal_Amt"}, 
                           inplace = True)

In [322]:
# Merchant Related Variables 

In [323]:
#FINANCIAL INSTITUTIONS

In [324]:
FI_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['FINANCIAL INSTITUTIONS']))]

In [325]:
FI_spend_data.shape

(360764, 9)

In [326]:
grp_FI_spend_data  = FI_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_FI_spend_data['Primary_Key'] = grp_FI_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_FI_spend_data['TRANSACTION_DATE'].astype(str)
grp_FI_spend_data
print(len(grp_FI_spend_data))

141321


In [327]:
grp_FI_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,AMOUNT_IN_LCY,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-11-30,550,CUKTNSPPNCCCNNNC_2012-11-30
1,CUKTNSPPNCCCNNNC,2012-12-31,14208,CUKTNSPPNCCCNNNC_2012-12-31
2,CUKTNSPPNCCCNNNC,2013-01-31,1632,CUKTNSPPNCCCNNNC_2013-01-31
3,CUKTNSPPNCCCNNNC,2013-02-28,4550,CUKTNSPPNCCCNNNC_2013-02-28
4,CUKTNSPPNCCCNNNC,2013-03-31,6628,CUKTNSPPNCCCNNNC_2013-03-31
...,...,...,...,...
141316,FTKUGGWWUNGPNNNT,2013-02-28,652,FTKUGGWWUNGPNNNT_2013-02-28
141317,FTKUGGWWUNGPNNNT,2013-03-31,1144,FTKUGGWWUNGPNNNT_2013-03-31
141318,FTKUGGWWUNGPNNNT,2013-04-30,10592,FTKUGGWWUNGPNNNT_2013-04-30
141319,FTKUGGWWUNGPNNNT,2013-05-31,671,FTKUGGWWUNGPNNNT_2013-05-31


In [328]:
grp_FI_spend_data.rename(columns = {"AMOUNT_IN_LCY": "FI_Spend_Amt"}, 
                           inplace = True)

In [329]:
# Grocery and Supermarkets 

In [330]:
Grocery_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['GROCERY STORES, SUPERMARKETS','MISCELLANEOUS FOOD STORES - CONVENIENCE STORES AND SPECIALTY MARKETS',
                                                                                        'CANDY, NUT, AND CONFECTIONERY STORES','BAKERIES','FREEZER AND LOCKER MEAT PROVISIONERS','DAIRY PRODUCTS STORES']))]

In [331]:
Grocery_spend_data.shape

(73675, 9)

In [332]:
grp_Grocery_spend_data  = Grocery_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_Grocery_spend_data['Primary_Key'] = grp_Grocery_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_Grocery_spend_data['TRANSACTION_DATE'].astype(str)
grp_Grocery_spend_data
print(len(grp_Grocery_spend_data))

28609


In [333]:
grp_Grocery_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Grocery_Spend_Amt"}, 
                           inplace = True)

In [334]:
grp_Grocery_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,Grocery_Spend_Amt,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-11-30,1925,CUKTNSPPNCCCNNNC_2012-11-30
1,CUKTNSPPNCCCNNNC,2012-12-31,2911,CUKTNSPPNCCCNNNC_2012-12-31
2,CUKTNSPPNCCCNNNC,2013-01-31,1342,CUKTNSPPNCCCNNNC_2013-01-31
3,CUKTNSPPNCCCNNNC,2013-02-28,903,CUKTNSPPNCCCNNNC_2013-02-28
4,CUKTNSPPNCCCNNNC,2013-03-31,654,CUKTNSPPNCCCNNNC_2013-03-31
...,...,...,...,...
28604,FTKUGGWWSFWGNNNF,2012-11-30,518,FTKUGGWWSFWGNNNF_2012-11-30
28605,FTKUGGWWSFWGNNNF,2012-12-31,36,FTKUGGWWSFWGNNNF_2012-12-31
28606,FTKUGGWWSFWGNNNF,2013-02-28,120,FTKUGGWWSFWGNNNF_2013-02-28
28607,FTKUGGWWUFFSNNNW,2012-03-31,124,FTKUGGWWUFFSNNNW_2012-03-31


In [335]:
# AUTOMATED CASH DISBURSE

In [336]:
AutoCashDisb_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['AUTOMATED CASH DISBURSE']))]

In [337]:
AutoCashDisb_spend_data.shape

(40924, 9)

In [338]:
grp_AutoCashDisb_spend_data  = AutoCashDisb_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_AutoCashDisb_spend_data['Primary_Key'] = grp_AutoCashDisb_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_AutoCashDisb_spend_data['TRANSACTION_DATE'].astype(str)
grp_AutoCashDisb_spend_data
print(len(grp_AutoCashDisb_spend_data))

21811


In [339]:
grp_AutoCashDisb_spend_data.rename(columns = {"AMOUNT_IN_LCY": "AutoCashDisb_Spend_Amt"}, 
                           inplace = True)

In [340]:
grp_AutoCashDisb_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,AutoCashDisb_Spend_Amt,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-11-30,10000,CUKTNSPPNCCCNNNC_2012-11-30
1,CUKTNSPPNCCCNNNC,2012-12-31,11000,CUKTNSPPNCCCNNNC_2012-12-31
2,CUKTNSPPNCCKNUTW,2013-02-28,1809,CUKTNSPPNCCKNUTW_2013-02-28
3,CUKTNSPPNCCNNNNN,2013-01-31,5000,CUKTNSPPNCCNNNNN_2013-01-31
4,CUKTNSPPNCCNNNNN,2013-02-28,4500,CUKTNSPPNCCNNNNN_2013-02-28
...,...,...,...,...
21806,FTKUGGWWSFWGNNNF,2012-10-31,6000,FTKUGGWWSFWGNNNF_2012-10-31
21807,FTKUGGWWSFWGNNNF,2012-12-31,100,FTKUGGWWSFWGNNNF_2012-12-31
21808,FTKUGGWWSFWGNNNF,2013-02-28,200,FTKUGGWWSFWGNNNF_2013-02-28
21809,FTKUGGWWUFFSNNNW,2012-03-31,1400,FTKUGGWWUFFSNNNW_2012-03-31


In [341]:
# SERVICE STATIONS

In [342]:
ServiceStations_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['SERVICE STATIONS','AUTOMATED FUEL DISPENSERS']))]

In [343]:
ServiceStations_spend_data.shape

(38305, 9)

In [344]:
grp_ServiceStations_spend_data  = ServiceStations_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_ServiceStations_spend_data['Primary_Key'] = grp_ServiceStations_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_ServiceStations_spend_data['TRANSACTION_DATE'].astype(str)
grp_ServiceStations_spend_data
print(len(grp_ServiceStations_spend_data))

15031


In [345]:
grp_ServiceStations_spend_data.rename(columns = {"AMOUNT_IN_LCY": "ServiceStations_Spend_Amt"}, 
                           inplace = True)

In [346]:
# Restaurants

In [347]:
Restaurant_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['EATING PLACES, RESTAURANTS','FAST FOOD RESTAURANTS','DRINKING PLACES']))]

In [348]:
Restaurant_spend_data.shape

(28993, 9)

In [349]:
grp_Restaurant_spend_data  = Restaurant_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_Restaurant_spend_data['Primary_Key'] = grp_Restaurant_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_Restaurant_spend_data['TRANSACTION_DATE'].astype(str)
grp_Restaurant_spend_data
print(len(grp_Restaurant_spend_data))

14779


In [350]:
grp_Restaurant_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Restaurant_Spend_Amt"}, 
                           inplace = True)

In [351]:
# Clothing

In [352]:
Clothing_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['FAMILY CLOTHING STORES','SHOE STORES','MEN’S, WOMEN’S CLOTHING STORES','CHILDREN’S AND INFANT’S WEAR STORES',
                                                                                          'WOMEN’S READY-TO-WEAR STORES','MISCELLANEOUS APPAREL AND ACCESSORY SHOPS','SPORTS AND RIDING APPAREL STORES',
                                                                                          'TAILORS, ALTERATIONS','WOMEN’S ACCESSORY AND SPECIALTY SHOPS','MEN’S AND BOY’S CLOTHING AND ACCESSORIES STORES',
                                                                                          'FURRIERS AND FUR SHOPS','WIG AND TOUPEE STORES','UNIFORMS, COMMERCIAL CLOTHING','CLOTHING RENTAL',
                                                                                          'COMMERCIAL FOOTWEAR']))]

In [353]:
Clothing_spend_data.shape

(42607, 9)

In [354]:
grp_Clothing_spend_data  = Clothing_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_Clothing_spend_data['Primary_Key'] = grp_Clothing_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_Clothing_spend_data['TRANSACTION_DATE'].astype(str)
grp_Clothing_spend_data
print(len(grp_Clothing_spend_data))

20525


In [355]:
grp_Clothing_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Clothing_Spend_Amt"}, 
                           inplace = True)

In [356]:
# Hotels

In [357]:
Hotels_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['HOTELS, MOTELS, AND RESORTS','HOTELS/MOTELS/INNS/RESORTS']))]

In [358]:
Hotels_spend_data.shape

(21529, 9)

In [359]:
grp_Hotels_spend_data  = Hotels_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_Hotels_spend_data['Primary_Key'] = grp_Hotels_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_Hotels_spend_data['TRANSACTION_DATE'].astype(str)
grp_Hotels_spend_data
print(len(grp_Hotels_spend_data))

12457


In [360]:
grp_Hotels_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Hotels_Spend_Amt"}, 
                           inplace = True)

In [361]:
# E- Channel Payments

In [362]:
i1_card_transaction.TRANS_DESCRIPTION.value_counts()

PURCHASE                           294240
CARD PAYMENT                        95432
PERIODIC FIXED FEE                  94142
PURCHASE REVERSAL                   83897
WITHDRAWAL                          40932
CASH FEE                            37702
LATE PAYMENT CHARGES                35562
PROFIT EARNED(SPECIAL SAV A\C)      23044
REBATE CONTRIBUTED BY BANK          20236
E-CHANNEL PAYMENTS                  14823
MONTHLY MURABAHA INSTALMENT         11281
MONTHLY MURABAHA INSTALLMENT         7410
UNIQUE TRANSACTION                   6017
MONTLHLY MURABAHA INSTALLMENT        5097
UTILITY BILL PAYMENT                 4396
MIGRATION TRANSACTION DEBIT          4315
PAYMENT                              2425
CREDIT VOUCHER                       1997
MIGRATION TRANSACTION CREDIT         1370
DEBIT ADJUSTMENTS                    1087
CARD REPLACEMENT FEE                  714
CARD FEE                              602
CASH ADVANCE                          281
PERIODIC FIXED FEE 14-10-11 REV   

In [363]:
Echannel_payment_data = i1_card_transaction[(i1_card_transaction['TRANS_DESCRIPTION'].isin(['E-CHANNEL PAYMENTS']))] 
Echannel_payment_data
Echannel_payment_data['Count_Echannel_Payment'] = 1
Echannel_payment_data.shape

(14823, 10)

In [364]:
Echannel_payment_data

,MASK_CREDITCARD,TRANS_DESCRIPTION,POST_DATE_AND_TIME,REVERSAL_FLAG,AMOUNT_IN_LCY,TRANSACTION_DATE,PREV_TRANSACTION_DATE,Merchant_Category,Merchant_Category_Description,Count_Echannel_Payment
38,CUKTNSPPNCCCNNNC,E-CHANNEL PAYMENTS,2013-03-10,N,5000,2013-03-31,2013-03-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
43,CUKTNSPPNCCCNNNC,E-CHANNEL PAYMENTS,2013-04-04,N,5000,2013-04-30,2013-04-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
51,CUKTNSPPNCCCNNNC,E-CHANNEL PAYMENTS,2013-06-02,N,5000,2013-06-30,2013-06-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
69,CUKTNSPPNCCFNNNS,E-CHANNEL PAYMENTS,2013-04-13,N,3000,2013-04-30,2013-04-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
70,CUKTNSPPNCCFNNNS,E-CHANNEL PAYMENTS,2013-04-13,N,150,2013-04-30,2013-04-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
...,...,...,...,...,...,...,...,...,...,...
787990,FTKUGGWWUNGPNNNT,E-CHANNEL PAYMENTS,2012-07-30,N,10000,2012-07-31,2012-07-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
788006,FTKUGGWWUNGPNNNT,E-CHANNEL PAYMENTS,2012-09-15,N,10000,2012-09-30,2012-09-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
788024,FTKUGGWWUNGPNNNT,E-CHANNEL PAYMENTS,2012-11-19,N,11632,2012-11-30,2012-11-30,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1
788041,FTKUGGWWUNGPNNNT,E-CHANNEL PAYMENTS,2013-01-20,N,11500,2013-01-31,2013-01-31,FINANCIAL INSTITUTIONS,FINANCIAL INSTITUTIONS – MERCHANDISE AND SERVICES,1


In [365]:
grp_Echannel_payment_data  = Echannel_payment_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('Count_Echannel_Payment').reset_index()
grp_Echannel_payment_data['Primary_Key'] = grp_Echannel_payment_data['MASK_CREDITCARD'].astype(str) + "_" + grp_Echannel_payment_data['TRANSACTION_DATE'].astype(str)
grp_Echannel_payment_data
print(len(grp_Echannel_payment_data))

9598


In [366]:
grp_Echannel_payment_data

,MASK_CREDITCARD,TRANSACTION_DATE,AMOUNT_IN_LCY,Count_Echannel_Payment,Primary_Key
0,CUKTNSPPNCCCNNNC,2013-03-31,5000,1,CUKTNSPPNCCCNNNC_2013-03-31
1,CUKTNSPPNCCCNNNC,2013-04-30,5000,1,CUKTNSPPNCCCNNNC_2013-04-30
2,CUKTNSPPNCCCNNNC,2013-06-30,5000,1,CUKTNSPPNCCCNNNC_2013-06-30
3,CUKTNSPPNCCFNNNS,2013-04-30,3150,2,CUKTNSPPNCCFNNNS_2013-04-30
4,CUKTNSPPNCCFNNNS,2013-05-31,3350,2,CUKTNSPPNCCFNNNS_2013-05-31
...,...,...,...,...,...
9593,FTKUGGWWUNGPNNNT,2012-07-31,10000,1,FTKUGGWWUNGPNNNT_2012-07-31
9594,FTKUGGWWUNGPNNNT,2012-09-30,10000,1,FTKUGGWWUNGPNNNT_2012-09-30
9595,FTKUGGWWUNGPNNNT,2012-11-30,11632,1,FTKUGGWWUNGPNNNT_2012-11-30
9596,FTKUGGWWUNGPNNNT,2013-01-31,11500,1,FTKUGGWWUNGPNNNT_2013-01-31


In [367]:
grp_Echannel_payment_data=grp_Echannel_payment_data.drop(['AMOUNT_IN_LCY'],axis=1)

In [368]:
# UTILITIES

In [369]:
utility_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['UTILITIES','TELECOMMUNICATION SERVICES',
                                                                                        'COMPUTER NETWORK SERVICES','CABLE, SATELLITE, AND OTHER PAY TELEVISION AND RADIO']))]
utility_spend_data.shape

(13270, 9)

In [370]:
grp_utility_spend_data  = utility_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_utility_spend_data['Primary_Key'] = grp_utility_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_utility_spend_data['TRANSACTION_DATE'].astype(str)
grp_utility_spend_data
print(len(grp_utility_spend_data))

8688


In [371]:
grp_utility_spend_data.rename(columns = {"AMOUNT_IN_LCY": "utility_Spend_Amt"}, 
                           inplace = True)

In [372]:
grp_utility_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,utility_Spend_Amt,Primary_Key
0,CUKTNSPPNCGCNNNN,2013-01-31,125,CUKTNSPPNCGCNNNN_2013-01-31
1,CUKTNSPPNCGNNNNC,2012-11-30,420,CUKTNSPPNCGNNNNC_2012-11-30
2,CUKTNSPPNCGNNNNC,2012-12-31,49,CUKTNSPPNCGNNNNC_2012-12-31
3,CUKTNSPPNCGNNNNC,2013-01-31,49,CUKTNSPPNCGNNNNC_2013-01-31
4,CUKTNSPPNCGNNNNC,2013-02-28,49,CUKTNSPPNCGNNNNC_2013-02-28
...,...,...,...,...
8683,FTKUGGWPWTSPNNNT,2013-06-30,100,FTKUGGWPWTSPNNNT_2013-06-30
8684,FTKUGGWPWUGKNNNS,2012-03-31,500,FTKUGGWPWUGKNNNS_2012-03-31
8685,FTKUGGWPWUKFNNNP,2013-02-28,715,FTKUGGWPWUKFNNNP_2013-02-28
8686,FTKUGGWPWUPGNNNT,2012-10-31,1373,FTKUGGWPWUPGNNNT_2012-10-31


In [373]:
# JEWELERY

In [374]:
jewel_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['JEWELRY STORES, WATCHES, CLOCKS, AND SILVERWARE STORES','WATCH/JEWELRY REPAIR',
                                                                                       'PRECIOUS STONES AND METALS, WATCHES AND JEWELRY','GLASSWARE, CRYSTAL STORES']))]
jewel_spend_data.shape

(4978, 9)

In [375]:
grp_jewel_spend_data  = jewel_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_jewel_spend_data['Primary_Key'] = grp_jewel_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_jewel_spend_data['TRANSACTION_DATE'].astype(str)
grp_jewel_spend_data
print(len(grp_jewel_spend_data))

4245


In [376]:
grp_jewel_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Jewelry_Spend_Amt"}, 
                           inplace = True)

In [377]:
# Airline

In [378]:
airline_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['AIRLINES','AIRLINES, AIR CARRIERS',
                                                                                        'AIRPORTS, FLYING FIELDS']))]
airline_spend_data.shape

(8360, 9)

In [379]:
grp_airline_spend_data  = airline_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_airline_spend_data['Primary_Key'] = grp_airline_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_airline_spend_data['TRANSACTION_DATE'].astype(str)
grp_airline_spend_data
print(len(grp_airline_spend_data))

5877


In [380]:
grp_airline_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Airline_Spend_Amt"}, 
                           inplace = True)

In [381]:
# Education 

In [382]:
Education_spend_data = i1_card_transaction[(i1_card_transaction['Merchant_Category'].isin(['EDUCATIONAL SERVICES','COLLEGES, UNIVERSITIES','ELEMENTARY, SECONDARY SCHOOLS',
                                                                                           'BUSINESS/SECRETARIAL SCHOOLS','CORRESPONDENCE SCHOOLS','VOCATIONAL/TRADE SCHOOLS',
                                                                                           'DANCE HALL, STUDIOS, SCHOOLS','STATIONERY STORES, OFFICE, AND SCHOOL SUPPLY STORES']))]
Education_spend_data.shape

(2068, 9)

In [383]:
grp_Education_spend_data  = Education_spend_data.groupby(['MASK_CREDITCARD','TRANSACTION_DATE']).sum('AMOUNT_IN_LCY').reset_index()
grp_Education_spend_data['Primary_Key'] = grp_Education_spend_data['MASK_CREDITCARD'].astype(str) + "_" + grp_Education_spend_data['TRANSACTION_DATE'].astype(str)
grp_Education_spend_data
print(len(grp_Education_spend_data))

1732


In [384]:
grp_Education_spend_data.rename(columns = {"AMOUNT_IN_LCY": "Education_Spend_Amt"}, 
                           inplace = True)

In [385]:
#grp_late_payment_data  = late_payment_data.groupby(['MASK_CREDITCARD','PREV_TRANSACTION_DATE']).MASK_CREDITCARD.transform('count')

In [386]:
#df2 = df.groupby(['Courses','Duration']).Courses.transform('count')

In [387]:
#grp_late_payment_data.info()

In [388]:
#type(grp_late_payment_data)

In [389]:
#grp_late_payment_data =grp_late_payment_data.to_frame() 

In [390]:
#grp_late_payment_data

In [391]:
#df2 = df.groupby(['Courses','Duration'])['Fee'].agg('count').reset_index()

In [392]:
#late_payment_data

In [393]:
# Creating Snapshot data 

In [394]:
i8_cards.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109142 entries, 0 to 109141
Data columns (total 50 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   MASKED_CUSTOMER_ID    109142 non-null  object        
 1   MASK_CREDITCARD       109142 non-null  object        
 2   CARD_LIMIT            109142 non-null  int64         
 3   CASH_LIMIT            109142 non-null  int64         
 4   OS_BALANCE            106684 non-null  float64       
 5   OPENING_BALANCE       106684 non-null  float64       
 6   CLOSING_BALANCE       106684 non-null  float64       
 7   CLOSING_BALANCE_CASH  106684 non-null  float64       
 8   CLOSING_BALANCE_FEES  106684 non-null  float64       
 9   NB_DAYS_DELINQ        109142 non-null  float64       
 10  Customer_Occupation   60635 non-null   object        
 11  A_Gender              82870 non-null   object        
 12  A_Marital_Status      75171 non-null   object        
 13 

In [395]:
min_snapshot_date = pd.to_datetime('2010-01-31')
max_snapshot_date = pd.to_datetime('2011-12-31')

In [396]:
Snapshot_Data = i8_cards[(i8_cards['report_date'] >= min_snapshot_date) & (i8_cards['report_date']<= max_snapshot_date)]

In [397]:
Snapshot_Data.shape

(51504, 50)

In [398]:
Snapshot_Data["MASK_CREDITCARD"].nunique()

2832

In [399]:
Snapshot_Data_Excl_MOB_less6 = Snapshot_Data[(Snapshot_Data['Exclusion_MOB_LT6'] == 1)]

In [400]:
Snapshot_Data_Excl_MOB_less6.shape

(7108, 50)

In [401]:
Snapshot_Data_Excl_MOB_less6["MASK_CREDITCARD"].nunique()

1546

In [402]:
Snapshot_Data_Excl_DPD_GE90 = Snapshot_Data[(Snapshot_Data['Exclusion_DPD_GE4'] == 1)]

In [403]:
Snapshot_Data_Excl_DPD_GE90.shape

(8072, 50)

In [404]:
Snapshot_Data_Excl_DPD_GE90["MASK_CREDITCARD"].nunique()

472

In [405]:
Snapshot_Modeling_Data = Snapshot_Data[(Snapshot_Data['Modeling_Data'] ==1)]

In [406]:
Snapshot_Modeling_Data.shape

(36324, 50)

In [407]:
Snapshot_Modeling_Data["MASK_CREDITCARD"].nunique()

2179

In [408]:
list(Snapshot_Modeling_Data.columns)

['MASKED_CUSTOMER_ID',
 'MASK_CREDITCARD',
 'CARD_LIMIT',
 'CASH_LIMIT',
 'OS_BALANCE',
 'OPENING_BALANCE',
 'CLOSING_BALANCE',
 'CLOSING_BALANCE_CASH',
 'CLOSING_BALANCE_FEES',
 'NB_DAYS_DELINQ',
 'Customer_Occupation',
 'A_Gender',
 'A_Marital_Status',
 'A_Industry',
 'A_Emp_Sts',
 'A_DOB',
 'A_Emp_Start',
 'A_No_Depend',
 'A_salary',
 'A_Sector',
 'min_period',
 'report_date',
 'min_report_date',
 'max_report_date',
 'issue_date',
 'mob',
 'min_mob',
 'max_mob',
 'cum_count',
 'min_cum_count',
 'max_cum_count',
 'valid_vintage',
 'bad_def',
 'issue_date_yr',
 'issue_date_yr_mon',
 'bad_flag',
 'report_date_yr_mon',
 'Exclusion_MOB_LT6',
 'Exclusion_DPD_GE4',
 'Inclusion_MOB_GE6',
 'Inclusion_DPD_LT4',
 'Modeling_Data',
 'Bad',
 'Good',
 'card_expiry_date',
 'customer_join_date',
 'REGION',
 'NATIONALITY',
 'cust_mob',
 'Age']

In [409]:
Snapshot_Modeling_Data

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,Inclusion_DPD_LT4,Modeling_Data,Bad,Good,card_expiry_date,customer_join_date,REGION,NATIONALITY,cust_mob,Age
4766,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,0.00,66228.57,0.00,0.00,0.0,0.0,...,1,1,0,1,2011-11-30,2008-09-30,Abu_Dhabi,Lebanon,16.0,2.0
4767,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,4193.35,0.00,4193.35,0.00,0.0,0.0,...,1,1,0,1,2011-11-30,2008-09-30,Abu_Dhabi,Lebanon,17.0,2.0
4768,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,8193.35,4193.35,8193.35,0.00,0.0,0.0,...,1,1,0,1,2011-11-30,2008-09-30,Abu_Dhabi,Lebanon,18.0,2.0
4769,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,33544.35,8193.35,33544.35,0.00,0.0,0.0,...,1,1,0,1,2011-11-30,2008-09-30,Abu_Dhabi,Lebanon,19.0,3.0
4770,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,21210.63,33544.35,21210.63,0.00,0.0,0.0,...,1,1,0,1,2011-11-30,2008-09-30,Abu_Dhabi,Lebanon,20.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108035,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6137.85,432.85,6137.85,1500.00,480.0,0.0,...,1,1,0,1,2013-07-31,2009-02-28,Others,United Arab Emirates,30.0,50.0
108036,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6237.85,6137.85,6237.85,832.85,380.0,0.0,...,1,1,0,1,2013-07-31,2009-02-28,Others,United Arab Emirates,31.0,50.0
108037,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6014.06,6237.85,6014.06,209.06,480.0,0.0,...,1,1,0,1,2013-07-31,2009-02-28,Others,United Arab Emirates,32.0,50.0
108038,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6161.65,6014.06,6161.65,0.00,300.0,0.0,...,1,1,0,1,2013-07-31,2009-02-28,Others,United Arab Emirates,33.0,50.0


In [410]:
#snap_check

In [411]:
Snapshot_Modeling_Data["MASK_CREDITCARD"].nunique()

2179

In [412]:
Snapshot_Modeling_Data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36324 entries, 4766 to 108039
Data columns (total 50 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MASKED_CUSTOMER_ID    36324 non-null  object        
 1   MASK_CREDITCARD       36324 non-null  object        
 2   CARD_LIMIT            36324 non-null  int64         
 3   CASH_LIMIT            36324 non-null  int64         
 4   OS_BALANCE            36324 non-null  float64       
 5   OPENING_BALANCE       36324 non-null  float64       
 6   CLOSING_BALANCE       36324 non-null  float64       
 7   CLOSING_BALANCE_CASH  36324 non-null  float64       
 8   CLOSING_BALANCE_FEES  36324 non-null  float64       
 9   NB_DAYS_DELINQ        36324 non-null  float64       
 10  Customer_Occupation   19836 non-null  object        
 11  A_Gender              26281 non-null  object        
 12  A_Marital_Status      24939 non-null  object        
 13  A_Industry  

In [413]:
# Base Modeling data 

In [414]:
Base_Modeling_Data = i8_cards[i8_cards['MASK_CREDITCARD'].isin(list(Snapshot_Modeling_Data.MASK_CREDITCARD))]

In [415]:
Base_Modeling_Data.shape

(86316, 50)

In [416]:
list(Base_Modeling_Data.columns)

['MASKED_CUSTOMER_ID',
 'MASK_CREDITCARD',
 'CARD_LIMIT',
 'CASH_LIMIT',
 'OS_BALANCE',
 'OPENING_BALANCE',
 'CLOSING_BALANCE',
 'CLOSING_BALANCE_CASH',
 'CLOSING_BALANCE_FEES',
 'NB_DAYS_DELINQ',
 'Customer_Occupation',
 'A_Gender',
 'A_Marital_Status',
 'A_Industry',
 'A_Emp_Sts',
 'A_DOB',
 'A_Emp_Start',
 'A_No_Depend',
 'A_salary',
 'A_Sector',
 'min_period',
 'report_date',
 'min_report_date',
 'max_report_date',
 'issue_date',
 'mob',
 'min_mob',
 'max_mob',
 'cum_count',
 'min_cum_count',
 'max_cum_count',
 'valid_vintage',
 'bad_def',
 'issue_date_yr',
 'issue_date_yr_mon',
 'bad_flag',
 'report_date_yr_mon',
 'Exclusion_MOB_LT6',
 'Exclusion_DPD_GE4',
 'Inclusion_MOB_GE6',
 'Inclusion_DPD_LT4',
 'Modeling_Data',
 'Bad',
 'Good',
 'card_expiry_date',
 'customer_join_date',
 'REGION',
 'NATIONALITY',
 'cust_mob',
 'Age']

In [417]:
Model_prep= Base_Modeling_Data[['MASKED_CUSTOMER_ID',
 'MASK_CREDITCARD',
 'CARD_LIMIT',
 'CASH_LIMIT',
 'OS_BALANCE',
 'OPENING_BALANCE',
 'CLOSING_BALANCE',
 'CLOSING_BALANCE_CASH',
 'CLOSING_BALANCE_FEES',
 'NB_DAYS_DELINQ',
 'Customer_Occupation',
 'A_Gender',
 'A_Marital_Status',
 'A_Industry',
 'A_Emp_Sts',
 'A_DOB',
 'A_Emp_Start',
 'A_No_Depend',
 'A_salary',
 'A_Sector',
 'report_date',
 'mob',
 'bad_def',
 'bad_flag',
 'card_expiry_date',
 'REGION',
 'NATIONALITY',
 'Age']]

In [418]:
Model_prep.shape

(86316, 28)

In [419]:
Model_prep['Primary_Key'] = Model_prep['MASK_CREDITCARD'].astype(str) + "_" + Model_prep['report_date'].astype(str)

In [420]:
Model_prep

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,A_Sector,report_date,mob,bad_def,bad_flag,card_expiry_date,REGION,NATIONALITY,Age,Primary_Key
4760,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,30913.33,0.00,30913.33,0.0,0.00,0.0,...,NaN,2009-07-31,0,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-07-31
4761,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36867.21,30913.33,36867.21,0.0,300.00,1.0,...,NaN,2009-08-31,1,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-08-31
4762,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36767.21,36867.21,36767.21,0.0,500.00,2.0,...,NaN,2009-09-30,2,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-09-30
4763,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,6510.17,36767.21,6510.17,0.0,0.00,0.0,...,NaN,2009-10-31,3,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-10-31
4764,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,15555.17,6510.17,15555.17,0.0,0.00,1.0,...,NaN,2009-11-30,4,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108054,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7684.60,5805.11,7684.60,2400.0,1850.00,0.0,...,Government,2013-03-31,43,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-03-31
108055,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7216.14,7684.60,7216.14,2400.0,2050.00,0.0,...,Government,2013-04-30,44,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-04-30
108056,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7194.53,7216.14,7194.53,2400.0,2250.00,0.0,...,Government,2013-05-31,45,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-05-31
108057,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6675.08,7194.53,6675.08,2400.0,2450.00,0.0,...,Government,2013-06-30,46,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-06-30


In [421]:
Model_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86316 entries, 4760 to 108058
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   MASKED_CUSTOMER_ID    86316 non-null  object        
 1   MASK_CREDITCARD       86316 non-null  object        
 2   CARD_LIMIT            86316 non-null  int64         
 3   CASH_LIMIT            86316 non-null  int64         
 4   OS_BALANCE            85093 non-null  float64       
 5   OPENING_BALANCE       85093 non-null  float64       
 6   CLOSING_BALANCE       85093 non-null  float64       
 7   CLOSING_BALANCE_CASH  85093 non-null  float64       
 8   CLOSING_BALANCE_FEES  85093 non-null  float64       
 9   NB_DAYS_DELINQ        86316 non-null  float64       
 10  Customer_Occupation   48855 non-null  object        
 11  A_Gender              64229 non-null  object        
 12  A_Marital_Status      60925 non-null  object        
 13  A_Industry  

In [422]:
grp_payment_data

,MASK_CREDITCARD,PREV_TRANSACTION_DATE,AMOUNT_IN_LCY,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-12-31,10000,CUKTNSPPNCCCNNNC_2012-12-31
1,CUKTNSPPNCCCNNNC,2013-02-28,3000,CUKTNSPPNCCCNNNC_2013-02-28
2,CUKTNSPPNCCFNNNS,2013-06-30,3000,CUKTNSPPNCCFNNNS_2013-06-30
3,CUKTNSPPNCCNNNNN,2012-12-31,8,CUKTNSPPNCCNNNNN_2012-12-31
4,CUKTNSPPNCCNNNNN,2013-01-31,193,CUKTNSPPNCCNNNNN_2013-01-31
...,...,...,...,...
83351,FTKUGGWWUNGPNNNT,2012-12-31,106,FTKUGGWWUNGPNNNT_2012-12-31
83352,FTKUGGWWUNGPNNNT,2013-02-28,100,FTKUGGWWUNGPNNNT_2013-02-28
83353,FTKUGGWWUNGPNNNT,2013-03-31,100,FTKUGGWWUNGPNNNT_2013-03-31
83354,FTKUGGWWUNGPNNNT,2013-05-31,141,FTKUGGWWUNGPNNNT_2013-05-31


In [423]:
Model_prep1 =pd.merge(Model_prep,grp_payment_data[['Primary_Key','AMOUNT_IN_LCY',]],how='left', on ='Primary_Key')

In [424]:
Model_prep.shape

(86316, 29)

In [425]:
Model_prep1.shape

(86316, 30)

In [426]:
grp_late_payment_data

,MASK_CREDITCARD,TRANSACTION_DATE,Count_Late_Payment,Primary_Key
0,CUKTNSPPNCCCNNNC,2013-01-31,1,CUKTNSPPNCCCNNNC_2013-01-31
1,CUKTNSPPNCCCNNNC,2013-02-28,1,CUKTNSPPNCCCNNNC_2013-02-28
2,CUKTNSPPNCCCNNNC,2013-03-31,1,CUKTNSPPNCCCNNNC_2013-03-31
3,CUKTNSPPNCCCNNNC,2013-05-31,1,CUKTNSPPNCCCNNNC_2013-05-31
4,CUKTNSPPNCCCNNNC,2013-06-30,1,CUKTNSPPNCCCNNNC_2013-06-30
...,...,...,...,...
35556,FTKUGGWWUFFSNNNW,2012-07-31,1,FTKUGGWWUFFSNNNW_2012-07-31
35557,FTKUGGWWUFFSNNNW,2012-09-30,1,FTKUGGWWUFFSNNNW_2012-09-30
35558,FTKUGGWWUFFSNNNW,2013-01-31,1,FTKUGGWWUFFSNNNW_2013-01-31
35559,FTKUGGWWUFFSNNNW,2013-04-30,1,FTKUGGWWUFFSNNNW_2013-04-30


In [427]:
Model_prep2 =pd.merge(Model_prep1,grp_late_payment_data[['Primary_Key','Count_Late_Payment',]],how='left', on ='Primary_Key')

In [428]:
Model_prep2.shape

(86316, 31)

In [429]:
grp_cash_trans_data

,MASK_CREDITCARD,TRANSACTION_DATE,Count_Cash_Transactions,Primary_Key
0,CUKTNSPPNCCCNNNC,2012-11-30,2,CUKTNSPPNCCCNNNC_2012-11-30
1,CUKTNSPPNCCCNNNC,2012-12-31,3,CUKTNSPPNCCCNNNC_2012-12-31
2,CUKTNSPPNCCKNUTW,2013-02-28,1,CUKTNSPPNCCKNUTW_2013-02-28
3,CUKTNSPPNCCNNNNN,2013-01-31,1,CUKTNSPPNCCNNNNN_2013-01-31
4,CUKTNSPPNCCNNNNN,2013-02-28,3,CUKTNSPPNCCNNNNN_2013-02-28
...,...,...,...,...
22014,FTKUGGWWSFWGNNNF,2012-10-31,3,FTKUGGWWSFWGNNNF_2012-10-31
22015,FTKUGGWWSFWGNNNF,2012-12-31,1,FTKUGGWWSFWGNNNF_2012-12-31
22016,FTKUGGWWSFWGNNNF,2013-02-28,1,FTKUGGWWSFWGNNNF_2013-02-28
22017,FTKUGGWWUFFSNNNW,2012-03-31,2,FTKUGGWWUFFSNNNW_2012-03-31


In [430]:
Model_prep3 =pd.merge(Model_prep2,grp_cash_trans_data[['Primary_Key','Count_Cash_Transactions',]],how='left', on ='Primary_Key')

In [431]:
Model_prep3.shape

(86316, 32)

In [432]:
Model_prep3

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,bad_def,bad_flag,card_expiry_date,REGION,NATIONALITY,Age,Primary_Key,AMOUNT_IN_LCY,Count_Late_Payment,Count_Cash_Transactions
0,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,30913.33,0.00,30913.33,0.0,0.00,0.0,...,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-07-31,NaN,NaN,NaN
1,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36867.21,30913.33,36867.21,0.0,300.00,1.0,...,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-08-31,NaN,1.0,NaN
2,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36767.21,36867.21,36767.21,0.0,500.00,2.0,...,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-09-30,36567.0,NaN,NaN
3,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,6510.17,36767.21,6510.17,0.0,0.00,0.0,...,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-10-31,NaN,NaN,NaN
4,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,15555.17,6510.17,15555.17,0.0,0.00,1.0,...,0,0.0,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-11-30,3228.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86311,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7684.60,5805.11,7684.60,2400.0,1850.00,0.0,...,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-03-31,581.0,NaN,NaN
86312,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7216.14,7684.60,7216.14,2400.0,2050.00,0.0,...,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-04-30,768.0,NaN,NaN
86313,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7194.53,7216.14,7194.53,2400.0,2250.00,0.0,...,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-05-31,722.0,NaN,NaN
86314,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6675.08,7194.53,6675.08,2400.0,2450.00,0.0,...,0,0.0,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-06-30,719.0,NaN,NaN


In [433]:
Model_prep3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86316 entries, 0 to 86315
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   MASKED_CUSTOMER_ID       86316 non-null  object        
 1   MASK_CREDITCARD          86316 non-null  object        
 2   CARD_LIMIT               86316 non-null  int64         
 3   CASH_LIMIT               86316 non-null  int64         
 4   OS_BALANCE               85093 non-null  float64       
 5   OPENING_BALANCE          85093 non-null  float64       
 6   CLOSING_BALANCE          85093 non-null  float64       
 7   CLOSING_BALANCE_CASH     85093 non-null  float64       
 8   CLOSING_BALANCE_FEES     85093 non-null  float64       
 9   NB_DAYS_DELINQ           86316 non-null  float64       
 10  Customer_Occupation      48855 non-null  object        
 11  A_Gender                 64229 non-null  object        
 12  A_Marital_Status         60925 n

In [434]:
Model_prep3.rename(columns = {"AMOUNT_IN_LCY": "Payment"}, 
                           inplace = True)

In [435]:
Model_prep3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86316 entries, 0 to 86315
Data columns (total 32 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   MASKED_CUSTOMER_ID       86316 non-null  object        
 1   MASK_CREDITCARD          86316 non-null  object        
 2   CARD_LIMIT               86316 non-null  int64         
 3   CASH_LIMIT               86316 non-null  int64         
 4   OS_BALANCE               85093 non-null  float64       
 5   OPENING_BALANCE          85093 non-null  float64       
 6   CLOSING_BALANCE          85093 non-null  float64       
 7   CLOSING_BALANCE_CASH     85093 non-null  float64       
 8   CLOSING_BALANCE_FEES     85093 non-null  float64       
 9   NB_DAYS_DELINQ           86316 non-null  float64       
 10  Customer_Occupation      48855 non-null  object        
 11  A_Gender                 64229 non-null  object        
 12  A_Marital_Status         60925 n

In [436]:
#Model_prep3=Model_prep3.drop(['util'],axis=1)

In [437]:
# Utilization 

In [438]:
Model_prep3 = Model_prep3.assign(
util =np.where((Model_prep3['CARD_LIMIT'].isnull())|(Model_prep3['CARD_LIMIT']==0)|
               (Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0),0,Model_prep3['OS_BALANCE']/Model_prep3['CARD_LIMIT']),
min_due = 0.05*Model_prep3['OS_BALANCE'] 
)

In [439]:
Model_prep3

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,card_expiry_date,REGION,NATIONALITY,Age,Primary_Key,Payment,Count_Late_Payment,Count_Cash_Transactions,util,min_due
0,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,30913.33,0.00,30913.33,0.0,0.00,0.0,...,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-07-31,NaN,NaN,NaN,0.088324,1545.6665
1,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36867.21,30913.33,36867.21,0.0,300.00,1.0,...,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-08-31,NaN,1.0,NaN,0.105335,1843.3605
2,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36767.21,36867.21,36767.21,0.0,500.00,2.0,...,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-09-30,36567.0,NaN,NaN,0.105049,1838.3605
3,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,6510.17,36767.21,6510.17,0.0,0.00,0.0,...,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-10-31,NaN,NaN,NaN,0.018600,325.5085
4,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,15555.17,6510.17,15555.17,0.0,0.00,1.0,...,2011-11-30,Abu_Dhabi,Lebanon,2.0,FTKUGFFSTNNNNCCP_2009-11-30,3228.0,NaN,NaN,0.044443,777.7585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86311,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7684.60,5805.11,7684.60,2400.0,1850.00,0.0,...,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-03-31,581.0,NaN,NaN,0.960575,384.2300
86312,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7216.14,7684.60,7216.14,2400.0,2050.00,0.0,...,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-04-30,768.0,NaN,NaN,0.902018,360.8070
86313,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,7194.53,7216.14,7194.53,2400.0,2250.00,0.0,...,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-05-31,722.0,NaN,NaN,0.899316,359.7265
86314,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,6675.08,7194.53,6675.08,2400.0,2450.00,0.0,...,2013-07-31,Others,United Arab Emirates,51.0,FTKUGGNNWWUTCKWK_2013-06-30,719.0,NaN,NaN,0.834385,333.7540


In [440]:
Model_prep3 = Model_prep3.assign(
pay_exp =np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Payment'].isnull())|(Model_prep3['Payment']==0),0,Model_prep3['Payment']/Model_prep3['OS_BALANCE']),
pay_mindue = np.where((Model_prep3['min_due'].isnull())|(Model_prep3['min_due']==0)|
                      (Model_prep3['Payment'].isnull())|(Model_prep3['Payment']==0),0,Model_prep3['Payment']/Model_prep3['min_due'])
)

In [441]:
Model_prep3 =pd.merge(Model_prep3,grp_FI_spend_data[['Primary_Key','FI_Spend_Amt',]],how='left', on ='Primary_Key')

In [442]:
Model_prep3 =pd.merge(Model_prep3,grp_Grocery_spend_data[['Primary_Key','Grocery_Spend_Amt',]],how='left', on ='Primary_Key')

In [443]:
Model_prep3 =pd.merge(Model_prep3,grp_AutoCashDisb_spend_data[['Primary_Key','AutoCashDisb_Spend_Amt',]],how='left', on ='Primary_Key')

In [444]:
Model_prep3 =pd.merge(Model_prep3,grp_ServiceStations_spend_data[['Primary_Key','ServiceStations_Spend_Amt',]],how='left', on ='Primary_Key')

In [445]:
Model_prep3 =pd.merge(Model_prep3,grp_Restaurant_spend_data[['Primary_Key','Restaurant_Spend_Amt',]],how='left', on ='Primary_Key')

In [446]:
Model_prep3 =pd.merge(Model_prep3,grp_Clothing_spend_data[['Primary_Key','Clothing_Spend_Amt',]],how='left', on ='Primary_Key')

In [447]:
Model_prep3 =pd.merge(Model_prep3,grp_Hotels_spend_data[['Primary_Key','Hotels_Spend_Amt',]],how='left', on ='Primary_Key')

In [448]:
Model_prep3 =pd.merge(Model_prep3,grp_cash_withdrawal_data[['Primary_Key','Withdrawal_Amt',]],how='left', on ='Primary_Key')

In [449]:
grp_Echannel_payment_data

,MASK_CREDITCARD,TRANSACTION_DATE,Count_Echannel_Payment,Primary_Key
0,CUKTNSPPNCCCNNNC,2013-03-31,1,CUKTNSPPNCCCNNNC_2013-03-31
1,CUKTNSPPNCCCNNNC,2013-04-30,1,CUKTNSPPNCCCNNNC_2013-04-30
2,CUKTNSPPNCCCNNNC,2013-06-30,1,CUKTNSPPNCCCNNNC_2013-06-30
3,CUKTNSPPNCCFNNNS,2013-04-30,2,CUKTNSPPNCCFNNNS_2013-04-30
4,CUKTNSPPNCCFNNNS,2013-05-31,2,CUKTNSPPNCCFNNNS_2013-05-31
...,...,...,...,...
9593,FTKUGGWWUNGPNNNT,2012-07-31,1,FTKUGGWWUNGPNNNT_2012-07-31
9594,FTKUGGWWUNGPNNNT,2012-09-30,1,FTKUGGWWUNGPNNNT_2012-09-30
9595,FTKUGGWWUNGPNNNT,2012-11-30,1,FTKUGGWWUNGPNNNT_2012-11-30
9596,FTKUGGWWUNGPNNNT,2013-01-31,1,FTKUGGWWUNGPNNNT_2013-01-31


In [450]:
Model_prep3 =pd.merge(Model_prep3,grp_Echannel_payment_data[['Primary_Key','Count_Echannel_Payment',]],how='left', on ='Primary_Key')

In [451]:
grp_utility_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,utility_Spend_Amt,Primary_Key
0,CUKTNSPPNCGCNNNN,2013-01-31,125,CUKTNSPPNCGCNNNN_2013-01-31
1,CUKTNSPPNCGNNNNC,2012-11-30,420,CUKTNSPPNCGNNNNC_2012-11-30
2,CUKTNSPPNCGNNNNC,2012-12-31,49,CUKTNSPPNCGNNNNC_2012-12-31
3,CUKTNSPPNCGNNNNC,2013-01-31,49,CUKTNSPPNCGNNNNC_2013-01-31
4,CUKTNSPPNCGNNNNC,2013-02-28,49,CUKTNSPPNCGNNNNC_2013-02-28
...,...,...,...,...
8683,FTKUGGWPWTSPNNNT,2013-06-30,100,FTKUGGWPWTSPNNNT_2013-06-30
8684,FTKUGGWPWUGKNNNS,2012-03-31,500,FTKUGGWPWUGKNNNS_2012-03-31
8685,FTKUGGWPWUKFNNNP,2013-02-28,715,FTKUGGWPWUKFNNNP_2013-02-28
8686,FTKUGGWPWUPGNNNT,2012-10-31,1373,FTKUGGWPWUPGNNNT_2012-10-31


In [452]:
Model_prep3 =pd.merge(Model_prep3,grp_utility_spend_data[['Primary_Key','utility_Spend_Amt',]],how='left', on ='Primary_Key')

In [453]:
grp_jewel_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,Jewelry_Spend_Amt,Primary_Key
0,CUKTNSPPNCCFNNNS,2013-03-31,62500,CUKTNSPPNCCFNNNS_2013-03-31
1,CUKTNSPPNCCUNNNP,2012-11-30,27400,CUKTNSPPNCCUNNNP_2012-11-30
2,CUKTNSPPNCFKUGFK,2013-04-30,1735,CUKTNSPPNCFKUGFK_2013-04-30
3,CUKTNSPPNCFKUGFK,2013-06-30,270,CUKTNSPPNCFKUGFK_2013-06-30
4,CUKTNSPPNCFPNNNC,2013-03-31,2900,CUKTNSPPNCFPNNNC_2013-03-31
...,...,...,...,...
4240,FTKUGGWPWPTTNNNU,2013-01-31,2400,FTKUGGWPWPTTNNNU_2013-01-31
4241,FTKUGGWPWPTTNNNU,2013-03-31,2000,FTKUGGWPWPTTNNNU_2013-03-31
4242,FTKUGGWPWSPNNNNN,2012-07-31,9499,FTKUGGWPWSPNNNNN_2012-07-31
4243,FTKUGGWPWSPNNNNN,2013-03-31,723,FTKUGGWPWSPNNNNN_2013-03-31


In [454]:
Model_prep3 =pd.merge(Model_prep3,grp_jewel_spend_data[['Primary_Key','Jewelry_Spend_Amt',]],how='left', on ='Primary_Key')

In [455]:
grp_airline_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,Airline_Spend_Amt,Primary_Key
0,CUKTNSPPNCCSNNNF,2013-06-30,6410,CUKTNSPPNCCSNNNF_2013-06-30
1,CUKTNSPPNCFNNNNG,2013-04-30,5795,CUKTNSPPNCFNNNNG_2013-04-30
2,CUKTNSPPNCFSNNNW,2013-02-28,760,CUKTNSPPNCFSNNNW_2013-02-28
3,CUKTNSPPNCFSNNNW,2013-06-30,1794,CUKTNSPPNCFSNNNW_2013-06-30
4,CUKTNSPPNCFTNNNU,2013-02-28,888,CUKTNSPPNCFTNNNU_2013-02-28
...,...,...,...,...
5872,FTKUGGWPWWWWNNNF,2012-08-31,1238,FTKUGGWPWWWWNNNF_2012-08-31
5873,FTKUGGWPWWWWNNNF,2012-12-31,12630,FTKUGGWPWWWWNNNF_2012-12-31
5874,FTKUGGWWGNPPNNNP,2012-06-30,3790,FTKUGGWWGNPPNNNP_2012-06-30
5875,FTKUGGWWGNPPNNNP,2012-10-31,2967,FTKUGGWWGNPPNNNP_2012-10-31


In [456]:
Model_prep3 =pd.merge(Model_prep3,grp_airline_spend_data[['Primary_Key','Airline_Spend_Amt',]],how='left', on ='Primary_Key')

In [457]:
grp_Education_spend_data

,MASK_CREDITCARD,TRANSACTION_DATE,Education_Spend_Amt,Primary_Key
0,CUKTNSPPNCFSSCFT,2013-02-28,421,CUKTNSPPNCFSSCFT_2013-02-28
1,CUKTNSPPNCGFUCUP,2013-02-28,434,CUKTNSPPNCGFUCUP_2013-02-28
2,CUKTNSPPNCKNNNNU,2013-02-28,274,CUKTNSPPNCKNNNNU_2013-02-28
3,CUKTNSPPNCKTNNNT,2013-06-30,225,CUKTNSPPNCKTNNNT_2013-06-30
4,CUKTNSPPNCNKNNNU,2013-03-31,58,CUKTNSPPNCNKNNNU_2013-03-31
...,...,...,...,...
1727,FTKUGGWPWCWTNNNU,2012-11-30,63633,FTKUGGWPWCWTNNNU_2012-11-30
1728,FTKUGGWPWCWTNNNU,2013-05-31,13192,FTKUGGWPWCWTNNNU_2013-05-31
1729,FTKUGGWPWCWTNNNU,2013-06-30,13964,FTKUGGWPWCWTNNNU_2013-06-30
1730,FTKUGGWPWUUCNNNU,2013-06-30,491,FTKUGGWPWUUCNNNU_2013-06-30


In [458]:
Model_prep3 =pd.merge(Model_prep3,grp_Education_spend_data[['Primary_Key','Education_Spend_Amt',]],how='left', on ='Primary_Key')

In [459]:
Model_prep3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86316 entries, 0 to 86315
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   MASKED_CUSTOMER_ID         86316 non-null  object        
 1   MASK_CREDITCARD            86316 non-null  object        
 2   CARD_LIMIT                 86316 non-null  int64         
 3   CASH_LIMIT                 86316 non-null  int64         
 4   OS_BALANCE                 85093 non-null  float64       
 5   OPENING_BALANCE            85093 non-null  float64       
 6   CLOSING_BALANCE            85093 non-null  float64       
 7   CLOSING_BALANCE_CASH       85093 non-null  float64       
 8   CLOSING_BALANCE_FEES       85093 non-null  float64       
 9   NB_DAYS_DELINQ             86316 non-null  float64       
 10  Customer_Occupation        48855 non-null  object        
 11  A_Gender                   64229 non-null  object        
 12  A_Ma

In [460]:
Model_prep3 = Model_prep3.assign(
cash_util =np.where((Model_prep3['CASH_LIMIT'].isnull())|(Model_prep3['CASH_LIMIT']==0)|
               (Model_prep3['Withdrawal_Amt'].isnull())|(Model_prep3['Withdrawal_Amt']==0),0,Model_prep3['Withdrawal_Amt']/Model_prep3['CASH_LIMIT']),
    
hotel_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Hotels_Spend_Amt'].isnull())|(Model_prep3['Hotels_Spend_Amt']==0),0,Model_prep3['Hotels_Spend_Amt']/Model_prep3['OS_BALANCE']),
    
clothing_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Clothing_Spend_Amt'].isnull())|(Model_prep3['Clothing_Spend_Amt']==0),0,Model_prep3['Clothing_Spend_Amt']/Model_prep3['OS_BALANCE']),
    
restaurant_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Restaurant_Spend_Amt'].isnull())|(Model_prep3['Restaurant_Spend_Amt']==0),0,Model_prep3['Restaurant_Spend_Amt']/Model_prep3['OS_BALANCE']),
    
servicestation_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['ServiceStations_Spend_Amt'].isnull())|(Model_prep3['ServiceStations_Spend_Amt']==0),0,Model_prep3['ServiceStations_Spend_Amt']/Model_prep3['OS_BALANCE']),
    
autocashdisb_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['AutoCashDisb_Spend_Amt'].isnull())|(Model_prep3['AutoCashDisb_Spend_Amt']==0),0,Model_prep3['AutoCashDisb_Spend_Amt']/Model_prep3['OS_BALANCE']), 
    
grocery_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Grocery_Spend_Amt'].isnull())|(Model_prep3['Grocery_Spend_Amt']==0),0,Model_prep3['Grocery_Spend_Amt']/Model_prep3['OS_BALANCE']), 
    
FI_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['FI_Spend_Amt'].isnull())|(Model_prep3['FI_Spend_Amt']==0),0,Model_prep3['FI_Spend_Amt']/Model_prep3['OS_BALANCE']) ,
    
utility_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['utility_Spend_Amt'].isnull())|(Model_prep3['utility_Spend_Amt']==0),0,Model_prep3['utility_Spend_Amt']/Model_prep3['OS_BALANCE']) , 
    
Jewelry_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Jewelry_Spend_Amt'].isnull())|(Model_prep3['Jewelry_Spend_Amt']==0),0,Model_prep3['Jewelry_Spend_Amt']/Model_prep3['OS_BALANCE']) ,  
    
Airline_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Airline_Spend_Amt'].isnull())|(Model_prep3['Airline_Spend_Amt']==0),0,Model_prep3['Airline_Spend_Amt']/Model_prep3['OS_BALANCE']) , 
    
Education_spend_exp= np.where((Model_prep3['OS_BALANCE'].isnull())|(Model_prep3['OS_BALANCE']==0)|
                  (Model_prep3['Education_Spend_Amt'].isnull())|(Model_prep3['Education_Spend_Amt']==0),0,Model_prep3['Education_Spend_Amt']/Model_prep3['OS_BALANCE'])  
)

In [461]:
Model_prep3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86316 entries, 0 to 86315
Data columns (total 61 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   MASKED_CUSTOMER_ID         86316 non-null  object        
 1   MASK_CREDITCARD            86316 non-null  object        
 2   CARD_LIMIT                 86316 non-null  int64         
 3   CASH_LIMIT                 86316 non-null  int64         
 4   OS_BALANCE                 85093 non-null  float64       
 5   OPENING_BALANCE            85093 non-null  float64       
 6   CLOSING_BALANCE            85093 non-null  float64       
 7   CLOSING_BALANCE_CASH       85093 non-null  float64       
 8   CLOSING_BALANCE_FEES       85093 non-null  float64       
 9   NB_DAYS_DELINQ             86316 non-null  float64       
 10  Customer_Occupation        48855 non-null  object        
 11  A_Gender                   64229 non-null  object        
 12  A_Ma

In [462]:
#check_snap

In [463]:
# List of Snapshot months 
# 2010-01-31 ,2010-02-28 ,2010-03-31,2010-04-30,2010-05-31,2010-06-30,2010-07-31,2010-08-31,2010-09-30,2010-10-31,2010-11-30,2010-12-31
# 2011-01-31 ,2011-02-28 ,2011-03-31,2011-04-30,2011-05-31,2011-06-30,2011-07-31,2011-08-31,2011-09-30,2011-10-31,2011-11-30,2011-12-31

In [464]:
snapshot_range =['2010-01-31','2010-02-28','2010-03-31','2010-04-30','2010-05-31','2010-06-30','2010-07-31','2010-08-31',
                 '2010-09-30','2010-10-31','2010-11-30','2010-12-31','2011-01-31','2011-02-28','2011-03-31','2011-04-30',
                 '2011-05-31','2011-06-30','2011-07-31','2011-08-31','2011-09-30','2011-10-31','2011-11-30','2011-12-31']

In [465]:
snapshot_range

['2010-01-31',
 '2010-02-28',
 '2010-03-31',
 '2010-04-30',
 '2010-05-31',
 '2010-06-30',
 '2010-07-31',
 '2010-08-31',
 '2010-09-30',
 '2010-10-31',
 '2010-11-30',
 '2010-12-31',
 '2011-01-31',
 '2011-02-28',
 '2011-03-31',
 '2011-04-30',
 '2011-05-31',
 '2011-06-30',
 '2011-07-31',
 '2011-08-31',
 '2011-09-30',
 '2011-10-31',
 '2011-11-30',
 '2011-12-31']

In [466]:
type(snapshot_range)

list

In [467]:
Model_prep3.shape

(86316, 61)

In [468]:
#Model_prep3['cashwithdrawal_limit'].agg(['min', 'max'])
Model_prep3['cash_util'].agg(['min', 'max'])

min    0.000
max    5.908
Name: cash_util, dtype: float64

In [469]:
Model_prep3['hotel_spend_exp'].agg(['min', 'max'])

min        0.0
max    12595.0
Name: hotel_spend_exp, dtype: float64

In [470]:
Model_prep3['clothing_spend_exp'].agg(['min', 'max'])

min       0.000000
max    1163.934426
Name: clothing_spend_exp, dtype: float64

In [471]:
Model_prep3['restaurant_spend_exp'].agg(['min', 'max'])

min       0.0
max    1090.0
Name: restaurant_spend_exp, dtype: float64

In [472]:
Model_prep3['servicestation_spend_exp'].agg(['min', 'max'])

min     0.0
max    15.2
Name: servicestation_spend_exp, dtype: float64

In [473]:
Model_prep3['autocashdisb_spend_exp'].agg(['min', 'max'])

min       0.0
max    1200.0
Name: autocashdisb_spend_exp, dtype: float64

In [474]:
Model_prep3['grocery_spend_exp'].agg(['min', 'max'])

min      0.000000
max    238.235294
Name: grocery_spend_exp, dtype: float64

In [475]:
Model_prep3['FI_spend_exp'].agg(['min', 'max'])

min          0.0
max    2013200.0
Name: FI_spend_exp, dtype: float64

In [476]:
snap = Model_prep3[(Model_prep3['report_date'] <= pd.to_datetime('2010-01-31'))]

In [477]:
snap.shape

(9730, 61)

In [478]:
snap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9730 entries, 0 to 86273
Data columns (total 61 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   MASKED_CUSTOMER_ID         9730 non-null   object        
 1   MASK_CREDITCARD            9730 non-null   object        
 2   CARD_LIMIT                 9730 non-null   int64         
 3   CASH_LIMIT                 9730 non-null   int64         
 4   OS_BALANCE                 8507 non-null   float64       
 5   OPENING_BALANCE            8507 non-null   float64       
 6   CLOSING_BALANCE            8507 non-null   float64       
 7   CLOSING_BALANCE_CASH       8507 non-null   float64       
 8   CLOSING_BALANCE_FEES       8507 non-null   float64       
 9   NB_DAYS_DELINQ             9730 non-null   float64       
 10  Customer_Occupation        4950 non-null   object        
 11  A_Gender                   6628 non-null   object        
 12  A_Mar

In [479]:
snap

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,clothing_spend_exp,restaurant_spend_exp,servicestation_spend_exp,autocashdisb_spend_exp,grocery_spend_exp,FI_spend_exp,utility_spend_exp,Jewelry_spend_exp,Airline_spend_exp,Education_spend_exp
0,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,30913.33,0.00,30913.33,0.0,0.0,0.0,...,0.0,0.027593,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
1,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36867.21,30913.33,36867.21,0.0,300.0,1.0,...,0.0,0.000000,0.0,0.0,0.0,0.008137,0.000000,0.0,0.000000,0.0
2,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36767.21,36867.21,36767.21,0.0,500.0,2.0,...,0.0,0.087796,0.0,0.0,0.0,0.999994,0.000000,0.0,0.000000,0.0
3,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,6510.17,36767.21,6510.17,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0
4,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,15555.17,6510.17,15555.17,0.0,0.0,1.0,...,0.0,0.000000,0.0,0.0,0.0,0.207519,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86269,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,100.00,100.00,100.00,0.0,100.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,32.000000,34.890000,0.0,0.000000,0.0
86270,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,3154.78,100.00,3154.78,0.0,100.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.031698,0.433945,0.0,0.390835,0.0
86271,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,5540.83,3154.78,5540.83,0.0,200.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.074899,0.000000,0.0,0.000000,0.0
86272,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,5936.75,5540.83,5936.75,0.0,300.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.110161,0.337053,0.0,0.000000,0.0


In [480]:
snap['snapshot_date']=pd.to_datetime("2010-01-31")

In [481]:
snap['app_diff']=((snap['snapshot_date']-snap['report_date'])/np.timedelta64(1,'M'))
snap['app_diff']=snap['app_diff'].round()

In [482]:
snap

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,servicestation_spend_exp,autocashdisb_spend_exp,grocery_spend_exp,FI_spend_exp,utility_spend_exp,Jewelry_spend_exp,Airline_spend_exp,Education_spend_exp,snapshot_date,app_diff
0,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,30913.33,0.00,30913.33,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,2010-01-31,6.0
1,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36867.21,30913.33,36867.21,0.0,300.0,1.0,...,0.0,0.0,0.0,0.008137,0.000000,0.0,0.000000,0.0,2010-01-31,5.0
2,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36767.21,36867.21,36767.21,0.0,500.0,2.0,...,0.0,0.0,0.0,0.999994,0.000000,0.0,0.000000,0.0,2010-01-31,4.0
3,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,6510.17,36767.21,6510.17,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,2010-01-31,3.0
4,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,15555.17,6510.17,15555.17,0.0,0.0,1.0,...,0.0,0.0,0.0,0.207519,0.000000,0.0,0.000000,0.0,2010-01-31,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86269,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,100.00,100.00,100.00,0.0,100.0,0.0,...,0.0,0.0,0.0,32.000000,34.890000,0.0,0.000000,0.0,2010-01-31,4.0
86270,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,3154.78,100.00,3154.78,0.0,100.0,0.0,...,0.0,0.0,0.0,0.031698,0.433945,0.0,0.390835,0.0,2010-01-31,3.0
86271,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,5540.83,3154.78,5540.83,0.0,200.0,0.0,...,0.0,0.0,0.0,0.074899,0.000000,0.0,0.000000,0.0,2010-01-31,2.0
86272,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,5936.75,5540.83,5936.75,0.0,300.0,0.0,...,0.0,0.0,0.0,0.110161,0.337053,0.0,0.000000,0.0,2010-01-31,1.0


In [483]:
#snap =pd.merge(snap,Model_prep3[['MASK_CREDITCARD','report_date','mob','bad_def']],how='left',left_on=['MASK_CREDITCARD','snapshot_date'],right_on= ['MASK_CREDITCARD','report_date'] )

In [484]:
snap_excl= snap[snap['app_diff']==0][['MASK_CREDITCARD','report_date','mob','bad_def']]

In [485]:
snap_excl

,MASK_CREDITCARD,report_date,mob,bad_def
6,FTKUGFFSTNNNNCCP,2010-01-31,6,0
36,FTKUGFFSTNNNNCKN,2010-01-31,7,0
71,FTKUGFFSTNNNNCNK,2010-01-31,7,0
121,FTKUGFFSTNNNNCSS,2010-01-31,7,0
171,FTKUGFFSTNNNNCTW,2010-01-31,7,0
...,...,...,...,...
85773,FTKUGGNNWTNNPUFP,2010-01-31,6,0
85853,FTKUGGNNWTSTUSCN,2010-01-31,1,0
85961,FTKUGGNNWUFSCPUW,2010-01-31,1,0
86197,FTKUGGNNWWKWCNWW,2010-01-31,5,0


In [486]:
snap_excl.rename(columns = {"report_date": "snapshot_date","mob": "snapshot_mob","bad_def": "snapshot_bad_def"}, 
           inplace = True)

In [487]:
snap =pd.merge(snap,snap_excl,how='left',on=['MASK_CREDITCARD','snapshot_date'])

In [488]:
snap.shape

(9730, 65)

In [489]:
snap

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,grocery_spend_exp,FI_spend_exp,utility_spend_exp,Jewelry_spend_exp,Airline_spend_exp,Education_spend_exp,snapshot_date,app_diff,snapshot_mob,snapshot_bad_def
0,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,30913.33,0.00,30913.33,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,2010-01-31,6.0,6,0
1,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36867.21,30913.33,36867.21,0.0,300.0,1.0,...,0.0,0.008137,0.000000,0.0,0.000000,0.0,2010-01-31,5.0,6,0
2,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,36767.21,36867.21,36767.21,0.0,500.0,2.0,...,0.0,0.999994,0.000000,0.0,0.000000,0.0,2010-01-31,4.0,6,0
3,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,6510.17,36767.21,6510.17,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,2010-01-31,3.0,6,0
4,SEEEEEF,FTKUGFFSTNNNNCCP,350000,210000,15555.17,6510.17,15555.17,0.0,0.0,1.0,...,0.0,0.207519,0.000000,0.0,0.000000,0.0,2010-01-31,2.0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9725,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,100.00,100.00,100.00,0.0,100.0,0.0,...,0.0,32.000000,34.890000,0.0,0.000000,0.0,2010-01-31,4.0,5,0
9726,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,3154.78,100.00,3154.78,0.0,100.0,0.0,...,0.0,0.031698,0.433945,0.0,0.390835,0.0,2010-01-31,3.0,5,0
9727,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,5540.83,3154.78,5540.83,0.0,200.0,0.0,...,0.0,0.074899,0.000000,0.0,0.000000,0.0,2010-01-31,2.0,5,0
9728,PREFTTT,FTKUGGNNWWUTCKWK,8000,4800,5936.75,5540.83,5936.75,0.0,300.0,0.0,...,0.0,0.110161,0.337053,0.0,0.000000,0.0,2010-01-31,1.0,5,0


In [490]:
snap = snap[snap['snapshot_mob']>=6]
snap = snap[snap['snapshot_bad_def']==0]

In [491]:
snap.shape

(8012, 65)

In [492]:
snap.MASK_CREDITCARD.nunique()

993

In [493]:
#snap_uniq = pd.merge(snap_uniq,,  how='left', left_on='[A_c1,c2]', right_on = '[B_c1,c2]')

In [494]:
snap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8012 entries, 0 to 9713
Data columns (total 65 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   MASKED_CUSTOMER_ID         8012 non-null   object        
 1   MASK_CREDITCARD            8012 non-null   object        
 2   CARD_LIMIT                 8012 non-null   int64         
 3   CASH_LIMIT                 8012 non-null   int64         
 4   OS_BALANCE                 6951 non-null   float64       
 5   OPENING_BALANCE            6951 non-null   float64       
 6   CLOSING_BALANCE            6951 non-null   float64       
 7   CLOSING_BALANCE_CASH       6951 non-null   float64       
 8   CLOSING_BALANCE_FEES       6951 non-null   float64       
 9   NB_DAYS_DELINQ             8012 non-null   float64       
 10  Customer_Occupation        4010 non-null   object        
 11  A_Gender                   5365 non-null   object        
 12  A_Mari

In [495]:
snap.Primary_Key.nunique()

8012

In [496]:
snap.MASK_CREDITCARD.nunique()

993

In [497]:
snap_uniq = snap[['MASK_CREDITCARD','snapshot_date']]

In [498]:
snap_uniq = snap_uniq.drop_duplicates()

In [499]:
snap_uniq.shape

(993, 2)

In [500]:
snap_uniq['Snapshot_Key'] = snap_uniq['MASK_CREDITCARD'].astype(str) + "_" + snap_uniq['snapshot_date'].astype(str)

In [501]:
snap_uniq

,MASK_CREDITCARD,snapshot_date,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,FTKUGFFSTNNNNCCP_2010-01-31
7,FTKUGFFSTNNNNCKN,2010-01-31,FTKUGFFSTNNNNCKN_2010-01-31
15,FTKUGFFSTNNNNCNK,2010-01-31,FTKUGFFSTNNNNCNK_2010-01-31
23,FTKUGFFSTNNNNCSS,2010-01-31,FTKUGFFSTNNNNCSS_2010-01-31
31,FTKUGFFSTNNNNCTW,2010-01-31,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...
9565,FTKUGGNNSWPWFFCF,2010-01-31,FTKUGGNNSWPWFFCF_2010-01-31
9587,FTKUGGNNTKFKPTSU,2010-01-31,FTKUGGNNTKFKPTSU_2010-01-31
9626,FTKUGGNNTWCKNWNC,2010-01-31,FTKUGGNNTWCKNWNC_2010-01-31
9648,FTKUGGNNUFPSGTFC,2010-01-31,FTKUGGNNUFPSGTFC_2010-01-31


In [502]:
# Factor Creation

In [503]:
def nan_sum(series):
    if (series.isnull().all() == True):
        return np.nan
    else:
        return np.sum(series)

In [504]:
factor_0 = snap.assign(
          
           # Late fees
           nbr_late_fee_3m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Count_Late_Payment'],np.nan),
           nbr_late_fee_3m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_late_fee_6m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Count_Late_Payment'],np.nan),
           nbr_late_fee_6m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_late_fee_12m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Count_Late_Payment'],np.nan),
           nbr_late_fee_12m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           # Cash Transactions
           nbr_cash_trans_3m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Count_Cash_Transactions'],np.nan),
           nbr_cash_trans_3m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_cash_trans_6m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Count_Cash_Transactions'],np.nan),
           nbr_cash_trans_6m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_cash_trans_12m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Count_Cash_Transactions'],np.nan),
           nbr_cash_trans_12m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           # Month Since DPD 
           month_since_0=np.where(snap['NB_DAYS_DELINQ'] > 0,snap['app_diff'],np.nan),
           month_since_30=np.where(snap['NB_DAYS_DELINQ']> 1,snap['app_diff'],np.nan),
           month_since_60=np.where(snap['NB_DAYS_DELINQ'] > 2,snap['app_diff'],np.nan),
           month_since_90=np.where(snap['NB_DAYS_DELINQ'] > 3,snap['app_diff'],np.nan),
           
           # Number of times account went above a certain dpd bucket
           nbr_0_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 0),1,0),
           nbr_0_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_0_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 0),1,0),
           nbr_0_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_0_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 0),1,0),
           nbr_0_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           nbr_30_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 1),1,0),
           nbr_30_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_30_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 1),1,0),
           nbr_30_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_30_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 1),1,0),
           nbr_30_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           nbr_60_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 2),1,0),
           nbr_60_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_60_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 2),1,0),
           nbr_60_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_60_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 2),1,0),
           nbr_60_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           nbr_90_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 3),1,0),
           nbr_90_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_90_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 3),1,0),
           nbr_90_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_90_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 3),1,0),
           nbr_90_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           # Max DPD 
           max_dpd_3m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['NB_DAYS_DELINQ'],0),
           max_dpd_6m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['NB_DAYS_DELINQ'],0),
           max_dpd_12m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['NB_DAYS_DELINQ'],0),
     
           # Average Utilization
           avg_util_3m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['util'],np.nan),
           avg_util_6m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['util'],np.nan),
           avg_util_12m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['util'],np.nan),
    
           # Payment/Min Due
           nbr_pay_gt_mindue_3m_num = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['pay_mindue'] > 1),1,0),
           nbr_pay_gt_mindue_3m_den = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_pay_gt_mindue_6m_num = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['pay_mindue'] > 1),1,0),
           nbr_pay_gt_mindue_6m_den = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_pay_gt_mindue_12m_num = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['pay_mindue'] > 1),1,0),
           nbr_pay_gt_mindue_12m_den = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
    
           # Cash withdrawal /Cash Limit or Cash Utilization
           avg_cash_util_3m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['cash_util'],np.nan),
           avg_cash_util_6m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['cash_util'],np.nan),
           avg_cash_util_12m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['cash_util'],np.nan),
    
           # Cash Withdrawal
           avg_amt_cash_withd_3m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Withdrawal_Amt'],np.nan),
           avg_amt_cash_withd_6m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Withdrawal_Amt'],np.nan),
           avg_amt_cash_withd_12m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Withdrawal_Amt'],np.nan),

           # Clothing 
           avg_amt_clothing_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Clothing_Spend_Amt'],np.nan),
           avg_amt_clothing_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Clothing_Spend_Amt'],np.nan),
           avg_amt_clothing_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Clothing_Spend_Amt'],np.nan),
           avg_perc_amt_clothing_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['clothing_spend_exp'],np.nan),
           avg_perc_amt_clothing_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['clothing_spend_exp'],np.nan),
           avg_perc_amt_clothing_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['clothing_spend_exp'],np.nan),
    
           # Utility
           avg_amt_utility_bills_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['utility_Spend_Amt'],np.nan),
           avg_amt_utility_bills_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['utility_Spend_Amt'],np.nan),
           avg_amt_utility_bills_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['utility_Spend_Amt'],np.nan),
           avg_perc_amt_utility_bills_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['utility_spend_exp'],np.nan),
           avg_perc_amt_utility_bills_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['utility_spend_exp'],np.nan),
           avg_perc_amt_utility_bills_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['utility_spend_exp'],np.nan),
    
           # Jewelery
           avg_amt_jewellery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Jewelry_Spend_Amt'],np.nan),
           avg_amt_jewellery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Jewelry_Spend_Amt'],np.nan),
           avg_amt_jewellery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Jewelry_Spend_Amt'],np.nan),
           avg_perc_amt_jewellery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Jewelry_spend_exp'],np.nan),
           avg_perc_amt_jewellery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Jewelry_spend_exp'],np.nan),
           avg_perc_amt_jewellery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Jewelry_spend_exp'],np.nan),
       
           # Airline
           avg_amt_airline_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Airline_Spend_Amt'],np.nan),
           avg_amt_airline_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Airline_Spend_Amt'],np.nan),
           avg_amt_airline_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Airline_Spend_Amt'],np.nan),
           avg_perc_amt_airline_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Airline_spend_exp'],np.nan),
           avg_perc_amt_airline_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Airline_spend_exp'],np.nan),
           avg_perc_amt_airline_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Airline_spend_exp'],np.nan),
    
           # Echannel Transactions 
           avg_echannel_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Count_Echannel_Payment'],np.nan),
           avg_echannel_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Count_Echannel_Payment'],np.nan),
           avg_echannel_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Count_Echannel_Payment'],np.nan),
    
           #Education 
           avg_amt_education_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Education_Spend_Amt'],np.nan),
           avg_amt_education_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Education_Spend_Amt'],np.nan),
           avg_amt_education_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Education_Spend_Amt'],np.nan),
           avg_perc_amt_education_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Education_spend_exp'],np.nan),
           avg_perc_amt_education_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Education_spend_exp'],np.nan),
           avg_perc_amt_education_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Education_spend_exp'],np.nan),
    
           # Hotel 
           avg_amt_hotel_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Hotels_Spend_Amt'],np.nan),
           avg_amt_hotel_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Hotels_Spend_Amt'],np.nan),
           avg_amt_hotel_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Hotels_Spend_Amt'],np.nan),
           avg_perc_amt_hotel_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['hotel_spend_exp'],np.nan),
           avg_perc_amt_hotel_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['hotel_spend_exp'],np.nan),
           avg_perc_amt_hotel_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['hotel_spend_exp'],np.nan),
    

           # Restaurant 
           avg_amt_restaurant_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Restaurant_Spend_Amt'],np.nan),
           avg_amt_restaurant_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Restaurant_Spend_Amt'],np.nan),
           avg_amt_restaurant_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Restaurant_Spend_Amt'],np.nan),
           avg_perc_amt_restaurant_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['restaurant_spend_exp'],np.nan),
           avg_perc_amt_restaurant_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['restaurant_spend_exp'],np.nan),
           avg_perc_amt_restaurant_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['restaurant_spend_exp'],np.nan),

           # Service Station 
           avg_amt_servicestation_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['ServiceStations_Spend_Amt'],np.nan),
           avg_amt_servicestation_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['ServiceStations_Spend_Amt'],np.nan),
           avg_amt_servicestation_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['ServiceStations_Spend_Amt'],np.nan),
           avg_perc_amt_servicestation_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['servicestation_spend_exp'],np.nan),
           avg_perc_amt_servicestation_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['servicestation_spend_exp'],np.nan),
           avg_perc_amt_servicestation_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['servicestation_spend_exp'],np.nan),


           # Automated Cash Disbursement 
           avg_amt_autocashdisb_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['AutoCashDisb_Spend_Amt'],np.nan),
           avg_amt_autocashdisb_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['AutoCashDisb_Spend_Amt'],np.nan),
           avg_amt_autocashdisb_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['AutoCashDisb_Spend_Amt'],np.nan),
           avg_perc_amt_autocashdisb_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['autocashdisb_spend_exp'],np.nan),
           avg_perc_amt_autocashdisb_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['autocashdisb_spend_exp'],np.nan),
           avg_perc_amt_autocashdisb_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['autocashdisb_spend_exp'],np.nan),
  
           # Groceries 
           avg_amt_grocery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Grocery_Spend_Amt'],np.nan),
           avg_amt_grocery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Grocery_Spend_Amt'],np.nan),
           avg_amt_grocery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Grocery_Spend_Amt'],np.nan),
           avg_perc_amt_grocery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['grocery_spend_exp'],np.nan),
           avg_perc_amt_grocery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['grocery_spend_exp'],np.nan),
           avg_perc_amt_grocery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['grocery_spend_exp'],np.nan),   

           # Financial Institutions
           avg_amt_FI_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['FI_Spend_Amt'],np.nan),
           avg_amt_FI_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['FI_Spend_Amt'],np.nan),
           avg_amt_FI_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['FI_Spend_Amt'],np.nan),
           avg_perc_amt_FI_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['FI_spend_exp'],np.nan),
           avg_perc_amt_FI_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['FI_spend_exp'],np.nan),
           avg_perc_amt_FI_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['FI_spend_exp'],np.nan)  

).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                   'nbr_late_fee_3m_num':'sum',
                                                    'nbr_late_fee_3m_den':'sum',
                                                    'nbr_late_fee_6m_num':'sum',
                                                    'nbr_late_fee_6m_den':'sum',
                                                    'nbr_late_fee_12m_num':'sum',
                                                    'nbr_late_fee_12m_den':'sum',
                                                    'nbr_cash_trans_3m_num':'sum',
                                                    'nbr_cash_trans_3m_den':'sum',
                                                    'nbr_cash_trans_6m_num':'sum',
                                                    'nbr_cash_trans_6m_den':'sum',
                                                    'nbr_cash_trans_12m_num':'sum',
                                                    'nbr_cash_trans_12m_den':'sum',
                                                    'month_since_0':'min',
                                                    'month_since_30':'min',
                                                    'month_since_60':'min',
                                                    'month_since_90':'min',
                                                    'nbr_0_3m_num':'sum',
                                                    'nbr_0_3m_den':'sum',
                                                    'nbr_0_6m_num':'sum',
                                                    'nbr_0_6m_den':'sum',
                                                    'nbr_0_12m_num':'sum',
                                                    'nbr_0_12m_den':'sum',
                                                    'nbr_30_3m_num':'sum',
                                                    'nbr_30_3m_den':'sum',
                                                    'nbr_30_6m_num':'sum',
                                                    'nbr_30_6m_den':'sum',
                                                    'nbr_30_12m_num':'sum',
                                                    'nbr_30_12m_den':'sum',
                                                    'nbr_60_3m_num':'sum',
                                                    'nbr_60_3m_den':'sum',
                                                    'nbr_60_6m_num':'sum',
                                                    'nbr_60_6m_den':'sum',
                                                    'nbr_60_12m_num':'sum',
                                                    'nbr_60_12m_den':'sum',
                                                    'nbr_90_3m_num':'sum',
                                                    'nbr_90_3m_den':'sum',
                                                    'nbr_90_6m_num':'sum',
                                                    'nbr_90_6m_den':'sum',
                                                    'nbr_90_12m_num':'sum',
                                                    'nbr_90_12m_den':'sum',
                                                    'max_dpd_3m':'max',
                                                    'max_dpd_6m':'max',
                                                    'max_dpd_12m':'max',
                                                    'avg_util_3m':'mean',
                                                    'avg_util_6m':'mean',
                                                    'avg_util_12m':'mean',  
                                                    'nbr_pay_gt_mindue_3m_num':'sum',
                                                    'nbr_pay_gt_mindue_3m_den':'sum',
                                                    'nbr_pay_gt_mindue_6m_num':'sum',
                                                    'nbr_pay_gt_mindue_6m_den':'sum',
                                                    'nbr_pay_gt_mindue_12m_num':'sum',
                                                    'nbr_pay_gt_mindue_12m_den':'sum',
                                                    'avg_cash_util_3m':'mean',
                                                    'avg_cash_util_6m':'mean',
                                                    'avg_cash_util_12m':'mean', 
                                                    'avg_amt_cash_withd_3m':'mean',
                                                    'avg_amt_cash_withd_6m':'mean',
                                                    'avg_amt_cash_withd_12m':'mean', 
                                                    'avg_amt_clothing_3m':'mean',
                                                    'avg_amt_clothing_6m':'mean',
                                                    'avg_amt_clothing_12m':'mean', 
                                                    'avg_perc_amt_clothing_3m':'mean',
                                                    'avg_perc_amt_clothing_6m':'mean',
                                                    'avg_perc_amt_clothing_12m':'mean',  
                                                    'avg_amt_utility_bills_3m':'mean',
                                                    'avg_amt_utility_bills_6m':'mean',
                                                    'avg_amt_utility_bills_12m':'mean', 
                                                    'avg_perc_amt_utility_bills_3m':'mean',
                                                    'avg_perc_amt_utility_bills_6m':'mean',
                                                    'avg_perc_amt_utility_bills_12m':'mean',  
                                                    'avg_amt_jewellery_3m':'mean',
                                                    'avg_amt_jewellery_6m':'mean',
                                                    'avg_amt_jewellery_12m':'mean', 
                                                    'avg_perc_amt_jewellery_3m':'mean',
                                                    'avg_perc_amt_jewellery_6m':'mean',
                                                    'avg_perc_amt_jewellery_12m':'mean',  
                                                    'avg_amt_airline_3m':'mean',
                                                    'avg_amt_airline_6m':'mean',
                                                    'avg_amt_airline_12m':'mean', 
                                                    'avg_perc_amt_airline_3m':'mean',
                                                    'avg_perc_amt_airline_6m':'mean',
                                                    'avg_perc_amt_airline_12m':'mean', 
                                                    'avg_echannel_3m':'mean',
                                                    'avg_echannel_6m':'mean',
                                                    'avg_echannel_12m':'mean', 
                                                    'avg_amt_education_3m':'mean',
                                                    'avg_amt_education_6m':'mean',
                                                    'avg_amt_education_12m':'mean', 
                                                    'avg_perc_amt_education_3m':'mean',
                                                    'avg_perc_amt_education_6m':'mean',
                                                    'avg_perc_amt_education_12m':'mean',
                                                    'avg_amt_hotel_3m':'mean',
                                                    'avg_amt_hotel_6m':'mean',
                                                    'avg_amt_hotel_12m':'mean', 
                                                    'avg_perc_amt_hotel_3m':'mean',
                                                    'avg_perc_amt_hotel_6m':'mean',
                                                    'avg_perc_amt_hotel_12m':'mean',
                                                    'avg_amt_restaurant_3m':'mean',
                                                    'avg_amt_restaurant_6m':'mean',
                                                    'avg_amt_restaurant_12m':'mean', 
                                                    'avg_perc_amt_restaurant_3m':'mean',
                                                    'avg_perc_amt_restaurant_6m':'mean',
                                                    'avg_perc_amt_restaurant_12m':'mean',
                                                    'avg_amt_servicestation_3m':'mean',
                                                    'avg_amt_servicestation_6m':'mean',
                                                    'avg_amt_servicestation_12m':'mean', 
                                                    'avg_perc_amt_servicestation_3m':'mean',
                                                    'avg_perc_amt_servicestation_6m':'mean',
                                                    'avg_perc_amt_servicestation_12m':'mean',
                                                    'avg_amt_autocashdisb_3m':'mean',
                                                    'avg_amt_autocashdisb_6m':'mean',
                                                    'avg_amt_autocashdisb_12m':'mean', 
                                                    'avg_perc_amt_autocashdisb_3m':'mean',
                                                    'avg_perc_amt_autocashdisb_6m':'mean',
                                                    'avg_perc_amt_autocashdisb_12m':'mean',
                                                    'avg_amt_grocery_3m':'mean',
                                                    'avg_amt_grocery_6m':'mean',
                                                    'avg_amt_grocery_12m':'mean', 
                                                    'avg_perc_amt_grocery_3m':'mean',
                                                    'avg_perc_amt_grocery_6m':'mean',
                                                    'avg_perc_amt_grocery_12m':'mean',
                                                    'avg_amt_FI_3m':'mean',
                                                    'avg_amt_FI_6m':'mean',
                                                    'avg_amt_FI_12m':'mean', 
                                                    'avg_perc_amt_FI_3m':'mean',
                                                    'avg_perc_amt_FI_6m':'mean',
                                                    'avg_perc_amt_FI_12m':'mean'
                                                  })

In [505]:
factor_0 = factor_0.reset_index()

In [506]:
factor_0

,MASK_CREDITCARD,snapshot_date,nbr_late_fee_3m_num,nbr_late_fee_3m_den,nbr_late_fee_6m_num,nbr_late_fee_6m_den,nbr_late_fee_12m_num,nbr_late_fee_12m_den,nbr_cash_trans_3m_num,nbr_cash_trans_3m_den,...,avg_amt_grocery_12m,avg_perc_amt_grocery_3m,avg_perc_amt_grocery_6m,avg_perc_amt_grocery_12m,avg_amt_FI_3m,avg_amt_FI_6m,avg_amt_FI_12m,avg_perc_amt_FI_3m,avg_perc_amt_FI_6m,avg_perc_amt_FI_12m
0,FTKUGFFSTNNNNCCP,2010-01-31,0.0,3,1.0,6,1.0,7,0.0,3,...,NaN,0.000000,0.000000,0.000000,44538.500000,31536.000000,31536.000000,0.501258,0.418651,0.358843
1,FTKUGFFSTNNNNCKN,2010-01-31,0.0,3,0.0,6,1.0,8,0.0,3,...,1293.500000,0.001338,0.062860,0.047145,7046.333333,6267.500000,4739.375000,47.684400,50.956298,38.473474
2,FTKUGFFSTNNNNCNK,2010-01-31,0.0,3,1.0,6,2.0,8,0.0,3,...,142.000000,0.000000,0.001425,0.001069,3100.000000,2519.833333,3555.250000,0.125244,0.119309,0.095247
3,FTKUGFFSTNNNNCSS,2010-01-31,1.0,3,1.0,6,1.0,8,3.0,3,...,588.333333,0.000000,0.011932,0.008949,2027.333333,4637.333333,9623.875000,0.115984,0.210493,0.241431
4,FTKUGFFSTNNNNCTW,2010-01-31,2.0,3,5.0,6,5.0,8,0.0,3,...,663.500000,0.000958,0.000479,0.007130,1266.666667,1200.000000,4270.750000,0.057790,0.054229,0.056819
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,2.0,3,3.0,6,3.0,7,1.0,3,...,479.000000,0.008375,0.012677,0.010866,1513.333333,1858.333333,1618.571429,0.075294,0.092470,0.079260
989,FTKUGGNNTKFKPTSU,2010-01-31,2.0,3,4.0,6,4.0,7,0.0,3,...,NaN,0.000000,0.000000,0.000000,649.000000,527.833333,465.285714,0.142914,0.114253,0.164549
990,FTKUGGNNTWCKNWNC,2010-01-31,1.0,3,1.0,6,1.0,7,0.0,3,...,335.000000,0.020010,0.010005,0.008576,668.000000,575.333333,518.857143,0.140551,0.121264,0.103941
991,FTKUGGNNUFPSGTFC,2010-01-31,0.0,3,2.0,6,2.0,7,1.0,3,...,799.666667,1.081798,0.540899,0.463628,241.000000,178.833333,178.833333,0.063727,0.394959,0.338536


In [507]:
factor_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 993 entries, 0 to 992
Columns: 129 entries, MASK_CREDITCARD to avg_perc_amt_FI_12m
dtypes: datetime64[ns](1), float64(91), int32(36), object(1)
memory usage: 861.2+ KB


In [508]:
factor_0['nbr_late_fee_3m']=np.where(factor_0['nbr_late_fee_3m_den'] > 0,factor_0['nbr_late_fee_3m_num']/factor_0['nbr_late_fee_3m_den'],np.nan)
factor_0['nbr_late_fee_6m']=np.where(factor_0['nbr_late_fee_6m_den'] > 0,factor_0['nbr_late_fee_6m_num']/factor_0['nbr_late_fee_6m_den'],np.nan)
factor_0['nbr_late_fee_12m']=np.where(factor_0['nbr_late_fee_12m_den'] > 0,factor_0['nbr_late_fee_12m_num']/factor_0['nbr_late_fee_12m_den'],np.nan)

factor_0['nbr_cash_trans_3m']=np.where(factor_0['nbr_cash_trans_3m_den'] > 0,factor_0['nbr_cash_trans_3m_num']/factor_0['nbr_cash_trans_3m_den'],np.nan)
factor_0['nbr_cash_trans_6m']=np.where(factor_0['nbr_cash_trans_6m_den'] > 0,factor_0['nbr_cash_trans_6m_num']/factor_0['nbr_cash_trans_6m_den'],np.nan)
factor_0['nbr_cash_trans_12m']=np.where(factor_0['nbr_cash_trans_12m_den'] > 0,factor_0['nbr_cash_trans_12m_num']/factor_0['nbr_cash_trans_12m_den'],np.nan)

factor_0['nbr_0_3m']=np.where(factor_0['nbr_0_3m_den'] > 0,factor_0['nbr_0_3m_num']/factor_0['nbr_0_3m_den'],np.nan)
factor_0['nbr_0_6m']=np.where(factor_0['nbr_0_6m_den'] > 0,factor_0['nbr_0_6m_num']/factor_0['nbr_0_6m_den'],np.nan)
factor_0['nbr_0_12m']=np.where(factor_0['nbr_0_12m_den'] > 0,factor_0['nbr_0_12m_num']/factor_0['nbr_0_12m_den'],np.nan)

factor_0['nbr_30_3m']=np.where(factor_0['nbr_30_3m_den'] > 0,factor_0['nbr_30_3m_num']/factor_0['nbr_30_3m_den'],np.nan)
factor_0['nbr_30_6m']=np.where(factor_0['nbr_30_6m_den'] > 0,factor_0['nbr_30_6m_num']/factor_0['nbr_30_6m_den'],np.nan)
factor_0['nbr_30_12m']=np.where(factor_0['nbr_30_12m_den'] > 0,factor_0['nbr_30_12m_num']/factor_0['nbr_30_12m_den'],np.nan)

factor_0['nbr_60_3m']=np.where(factor_0['nbr_60_3m_den'] > 0,factor_0['nbr_60_3m_num']/factor_0['nbr_60_3m_den'],np.nan)
factor_0['nbr_60_6m']=np.where(factor_0['nbr_60_6m_den'] > 0,factor_0['nbr_60_6m_num']/factor_0['nbr_60_6m_den'],np.nan)
factor_0['nbr_60_12m']=np.where(factor_0['nbr_60_12m_den'] > 0,factor_0['nbr_60_12m_num']/factor_0['nbr_60_12m_den'],np.nan)

factor_0['nbr_90_3m']=np.where(factor_0['nbr_90_3m_den'] > 0,factor_0['nbr_90_3m_num']/factor_0['nbr_90_3m_den'],np.nan)
factor_0['nbr_90_6m']=np.where(factor_0['nbr_90_6m_den'] > 0,factor_0['nbr_90_6m_num']/factor_0['nbr_90_6m_den'],np.nan)
factor_0['nbr_90_12m']=np.where(factor_0['nbr_90_12m_den'] > 0,factor_0['nbr_90_12m_num']/factor_0['nbr_90_12m_den'],np.nan)

factor_0['nbr_pay_gt_mindue_3m']=np.where(factor_0['nbr_pay_gt_mindue_3m_den'] > 0,factor_0['nbr_pay_gt_mindue_3m_num']/factor_0['nbr_pay_gt_mindue_3m_den'],np.nan)
factor_0['nbr_pay_gt_mindue_6m']=np.where(factor_0['nbr_pay_gt_mindue_6m_den'] > 0,factor_0['nbr_pay_gt_mindue_6m_num']/factor_0['nbr_pay_gt_mindue_6m_den'],np.nan)
factor_0['nbr_pay_gt_mindue_12m']=np.where(factor_0['nbr_pay_gt_mindue_12m_den'] > 0,factor_0['nbr_pay_gt_mindue_12m_num']/factor_0['nbr_pay_gt_mindue_12m_den'],np.nan)


In [509]:
factor_0=factor_0.drop([ 'nbr_late_fee_3m_num','nbr_late_fee_3m_den','nbr_late_fee_6m_num','nbr_late_fee_6m_den',
                         'nbr_late_fee_12m_num','nbr_late_fee_12m_den','nbr_cash_trans_3m_num','nbr_cash_trans_3m_den',
                         'nbr_cash_trans_6m_num','nbr_cash_trans_6m_den','nbr_cash_trans_12m_num','nbr_cash_trans_12m_den',
                         'nbr_0_3m_num','nbr_0_3m_den','nbr_0_6m_num','nbr_0_6m_den','nbr_0_12m_num','nbr_0_12m_den',
                         'nbr_30_3m_num','nbr_30_3m_den','nbr_30_6m_num','nbr_30_6m_den','nbr_30_12m_num','nbr_30_12m_den',
                         'nbr_60_3m_num','nbr_60_3m_den','nbr_60_6m_num','nbr_60_6m_den','nbr_60_12m_num','nbr_60_12m_den',
                         'nbr_90_3m_num','nbr_90_3m_den','nbr_90_6m_num','nbr_90_6m_den','nbr_90_12m_num','nbr_90_12m_den',
                         'nbr_pay_gt_mindue_3m_num' ,'nbr_pay_gt_mindue_3m_den','nbr_pay_gt_mindue_6m_num','nbr_pay_gt_mindue_6m_den',
                          'nbr_pay_gt_mindue_12m_num','nbr_pay_gt_mindue_12m_den'],axis=1)

In [510]:
type(factor_0)

pandas.core.frame.DataFrame

In [511]:
factor_0.shape

(993, 108)

In [512]:
factor_0['Snapshot_Key'] = factor_0['MASK_CREDITCARD'].astype(str) + "_" + factor_0['snapshot_date'].astype(str)

In [513]:
#check1

In [514]:
#snap.sort_values(by=['Primary_Key','MASK_CREDITCARD','report_date','snapshot_date','app_diff'],ascending= True,inplace=True)

In [515]:
#snap_check =snap

In [516]:
#snap_check.sort_values(by=['MASK_CREDITCARD','snapshot_date','app_diff'],ascending= False,inplace=True)

In [517]:
#snap_check

In [518]:
#snap

In [519]:
#cons_incr_decr

In [520]:
# Creating Function 

In [521]:
def cons_incr_dcr_delinq(snap_data):
    # Sorting the data is descending order 
    snap_data.sort_values(by=['MASK_CREDITCARD','snapshot_date','app_diff'],ascending= False,inplace=True)
    
    #L12M 
    snap_data_l12m = snap_data[(snap_data['app_diff'] <= 11)]
    #L6M 
    snap_data_l6m = snap_data[(snap_data['app_diff'] <= 5)]
    #L3M 
    snap_data_l3m = snap_data[(snap_data['app_diff'] <= 2)]
    
    # Delinquency Increase
    snap_data_l12m['del_incr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()>0,1,0)
    snap_data_l6m['del_incr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()>0,1,0)
    snap_data_l3m['del_incr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()>0,1,0)
    
    snap_data_l12m['grp_ids_del_incr_flag'] = snap_data_l12m['del_incr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_del_incr_flag'] = snap_data_l6m['del_incr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_del_incr_flag'] = snap_data_l3m['del_incr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["del_incr_flag_cnt"] = snap_data_l12m['del_incr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_del_incr_flag']]).cumsum()
    snap_data_l6m["del_incr_flag_cnt"] = snap_data_l6m['del_incr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_del_incr_flag']]).cumsum()
    snap_data_l3m["del_incr_flag_cnt"] = snap_data_l3m['del_incr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_del_incr_flag']]).cumsum()

    # Delinquency decrease
    snap_data_l12m['del_dcr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()<0,1,0)
    snap_data_l6m['del_dcr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()<0,1,0)
    snap_data_l3m['del_dcr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()<0,1,0)
    
    snap_data_l12m['grp_ids_del_dcr_flag'] = snap_data_l12m['del_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_del_dcr_flag'] = snap_data_l6m['del_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_del_dcr_flag'] = snap_data_l3m['del_dcr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["del_dcr_flag_cnt"] = snap_data_l12m['del_dcr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_del_dcr_flag']]).cumsum()
    snap_data_l6m["del_dcr_flag_cnt"] = snap_data_l6m['del_dcr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_del_dcr_flag']]).cumsum()
    snap_data_l3m["del_dcr_flag_cnt"] = snap_data_l3m['del_dcr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_del_dcr_flag']]).cumsum()
    
    factor_l12m = snap_data_l12m.assign(
                    Cons_inc_delinq_12m = snap_data_l12m["del_incr_flag_cnt"],
                    Cons_dec_delinq_12m = snap_data_l12m["del_dcr_flag_cnt"] ,
                    Tot_Cons_inc_delinq_12m = snap_data_l12m["del_incr_flag"],
                    Tot_Cons_dec_delinq_12m = snap_data_l12m["del_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_delinq_12m':'max',
                                                    'Cons_dec_delinq_12m':'max',
                                                    'Tot_Cons_inc_delinq_12m':'sum',
                                                    'Tot_Cons_dec_delinq_12m':'sum'
                                                  })
    factor_l12m = factor_l12m.reset_index()
    factor_l12m['Snapshot_Key'] = factor_l12m['MASK_CREDITCARD'].astype(str) + "_" + factor_l12m['snapshot_date'].astype(str)
    
    factor_l6m = snap_data_l6m.assign(
                    Cons_inc_delinq_6m = snap_data_l6m["del_incr_flag_cnt"],
                    Cons_dec_delinq_6m = snap_data_l6m["del_dcr_flag_cnt"] ,
                    Tot_Cons_inc_delinq_6m = snap_data_l6m["del_incr_flag"],
                    Tot_Cons_dec_delinq_6m = snap_data_l6m["del_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_delinq_6m':'max',
                                                    'Cons_dec_delinq_6m':'max',
                                                    'Tot_Cons_inc_delinq_6m':'sum',
                                                    'Tot_Cons_dec_delinq_6m':'sum'
                                                  })
    factor_l6m = factor_l6m.reset_index()
    factor_l6m['Snapshot_Key'] = factor_l6m['MASK_CREDITCARD'].astype(str) + "_" + factor_l6m['snapshot_date'].astype(str)
    
    factor_l3m = snap_data_l3m.assign(
                    Cons_inc_delinq_3m = snap_data_l3m["del_incr_flag_cnt"],
                    Cons_dec_delinq_3m = snap_data_l3m["del_dcr_flag_cnt"] ,
                    Tot_Cons_inc_delinq_3m = snap_data_l3m["del_incr_flag"],
                    Tot_Cons_dec_delinq_3m = snap_data_l3m["del_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_delinq_3m':'max',
                                                    'Cons_dec_delinq_3m':'max',
                                                    'Tot_Cons_inc_delinq_3m':'sum',
                                                    'Tot_Cons_dec_delinq_3m':'sum'
                                                  })
    factor_l3m = factor_l3m.reset_index()   
    factor_l3m['Snapshot_Key'] = factor_l3m['MASK_CREDITCARD'].astype(str) + "_" + factor_l3m['snapshot_date'].astype(str)
    
    return factor_l12m, factor_l6m,factor_l3m
    

In [522]:
l12m_inc_dec_delinq,l6m_inc_dec_delinq,l3m_inc_dec_delinq = cons_incr_dcr_delinq(snap)

In [523]:
l12m_inc_dec_delinq

,MASK_CREDITCARD,snapshot_date,Cons_inc_delinq_12m,Cons_dec_delinq_12m,Tot_Cons_inc_delinq_12m,Tot_Cons_dec_delinq_12m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,0,0,0,0,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,0,0,0,0,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,2,1,3,1,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,1,2,1,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,1,3,2,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,1,1,1,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,1,2,1,FTKUGGNNUFPSGTFC_2010-01-31


In [524]:
l6m_inc_dec_delinq

,MASK_CREDITCARD,snapshot_date,Cons_inc_delinq_6m,Cons_dec_delinq_6m,Tot_Cons_inc_delinq_6m,Tot_Cons_dec_delinq_6m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,1,1,2,2,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,0,0,0,0,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,0,0,0,0,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,2,1,2,1,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,1,2,1,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,1,2,2,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,1,1,1,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,1,2,1,FTKUGGNNUFPSGTFC_2010-01-31


In [525]:
l3m_inc_dec_delinq

,MASK_CREDITCARD,snapshot_date,Cons_inc_delinq_3m,Cons_dec_delinq_3m,Tot_Cons_inc_delinq_3m,Tot_Cons_dec_delinq_3m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,0,1,0,1,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,0,0,0,0,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,0,0,0,0,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,0,1,0,1,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,2,0,2,0,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,0,1,0,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,0,1,0,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,1,1,1,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,0,0,0,0,FTKUGGNNUFPSGTFC_2010-01-31


In [526]:
def cons_incr_dcr_util(snap_data):
    # Sorting the data is descending order 
    snap_data.sort_values(by=['MASK_CREDITCARD','snapshot_date','app_diff'],ascending= False,inplace=True)
    
    #L12M 
    snap_data_l12m = snap_data[(snap_data['app_diff'] <= 11)]
    #L6M 
    snap_data_l6m = snap_data[(snap_data['app_diff'] <= 5)]
    #L3M 
    snap_data_l3m = snap_data[(snap_data['app_diff'] <= 2)]
    

    
    # Utilization Increase
    snap_data_l12m['util_incr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['util'].diff()>0,1,0)
    snap_data_l6m['util_incr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['util'].diff()>0,1,0)
    snap_data_l3m['util_incr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['util'].diff()>0,1,0)
    
    snap_data_l12m['grp_ids_util_incr_flag'] = snap_data_l12m['util_incr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_util_incr_flag'] = snap_data_l6m['util_incr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_util_incr_flag'] = snap_data_l3m['util_incr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["util_incr_flag_cnt"] = snap_data_l12m['util_incr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_util_incr_flag']]).cumsum()
    snap_data_l6m["util_incr_flag_cnt"] = snap_data_l6m['util_incr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_util_incr_flag']]).cumsum()
    snap_data_l3m["util_incr_flag_cnt"] = snap_data_l3m['util_incr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_util_incr_flag']]).cumsum()

    # Utilization decrease
    snap_data_l12m['util_dcr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['util'].diff()<0,1,0)
    snap_data_l6m['util_dcr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['util'].diff()<0,1,0)
    snap_data_l3m['util_dcr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['util'].diff()<0,1,0)
    
    snap_data_l12m['grp_ids_util_dcr_flag'] = snap_data_l12m['util_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_util_dcr_flag'] = snap_data_l6m['util_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_util_dcr_flag'] = snap_data_l3m['util_dcr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["util_dcr_flag_cnt"] = snap_data_l12m['util_dcr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_util_dcr_flag']]).cumsum()
    snap_data_l6m["util_dcr_flag_cnt"] = snap_data_l6m['util_dcr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_util_dcr_flag']]).cumsum()
    snap_data_l3m["util_dcr_flag_cnt"] = snap_data_l3m['util_dcr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_util_dcr_flag']]).cumsum()
    
    factor_l12m = snap_data_l12m.assign(
                    Cons_inc_util_12m = snap_data_l12m["util_incr_flag_cnt"],
                    Cons_dec_util_12m = snap_data_l12m["util_dcr_flag_cnt"] ,
                    Tot_Cons_inc_util_12m = snap_data_l12m["util_incr_flag"],
                    Tot_Cons_dec_util_12m = snap_data_l12m["util_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_util_12m':'max',
                                                    'Cons_dec_util_12m':'max',
                                                    'Tot_Cons_inc_util_12m':'sum',
                                                    'Tot_Cons_dec_util_12m':'sum'
                                                  })
    factor_l12m = factor_l12m.reset_index()
    factor_l12m['Snapshot_Key'] = factor_l12m['MASK_CREDITCARD'].astype(str) + "_" + factor_l12m['snapshot_date'].astype(str)
    
    factor_l6m = snap_data_l6m.assign(
                    Cons_inc_util_6m = snap_data_l6m["util_incr_flag_cnt"],
                    Cons_dec_util_6m = snap_data_l6m["util_dcr_flag_cnt"] ,
                    Tot_Cons_inc_util_6m = snap_data_l6m["util_incr_flag"],
                    Tot_Cons_dec_util_6m = snap_data_l6m["util_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_util_6m':'max',
                                                    'Cons_dec_util_6m':'max',
                                                    'Tot_Cons_inc_util_6m':'sum',
                                                    'Tot_Cons_dec_util_6m':'sum'
                                                  })
    factor_l6m = factor_l6m.reset_index()
    factor_l6m['Snapshot_Key'] = factor_l6m['MASK_CREDITCARD'].astype(str) + "_" + factor_l6m['snapshot_date'].astype(str)
    
    factor_l3m = snap_data_l3m.assign(
                    Cons_inc_util_3m = snap_data_l3m["util_incr_flag_cnt"],
                    Cons_dec_util_3m = snap_data_l3m["util_dcr_flag_cnt"] ,
                    Tot_Cons_inc_util_3m = snap_data_l3m["util_incr_flag"],
                    Tot_Cons_dec_util_3m = snap_data_l3m["util_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_util_3m':'max',
                                                    'Cons_dec_util_3m':'max',
                                                    'Tot_Cons_inc_util_3m':'sum',
                                                    'Tot_Cons_dec_util_3m':'sum'
                                                  })
    factor_l3m = factor_l3m.reset_index()   
    factor_l3m['Snapshot_Key'] = factor_l3m['MASK_CREDITCARD'].astype(str) + "_" + factor_l3m['snapshot_date'].astype(str)
    
    return factor_l12m, factor_l6m,factor_l3m 

In [527]:
l12m_inc_dec_util,l6m_inc_dec_util,l3m_inc_dec_util = cons_incr_dcr_util(snap)

In [528]:
l12m_inc_dec_util

,MASK_CREDITCARD,snapshot_date,Cons_inc_util_12m,Cons_dec_util_12m,Tot_Cons_inc_util_12m,Tot_Cons_dec_util_12m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,2,3,3,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,1,1,3,2,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,3,2,4,3,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,4,3,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,2,3,4,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,2,1,4,2,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,2,1,4,2,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,1,1,3,3,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,6,0,6,0,FTKUGGNNUFPSGTFC_2010-01-31


In [529]:
l6m_inc_dec_util

,MASK_CREDITCARD,snapshot_date,Cons_inc_util_6m,Cons_dec_util_6m,Tot_Cons_inc_util_6m,Tot_Cons_dec_util_6m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,2,2,3,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,1,1,2,2,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,3,1,3,2,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,3,2,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,2,2,3,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,1,3,2,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,2,1,3,2,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,1,1,2,3,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,5,0,5,0,FTKUGGNNUFPSGTFC_2010-01-31


In [530]:
l3m_inc_dec_util

,MASK_CREDITCARD,snapshot_date,Cons_inc_util_3m,Cons_dec_util_3m,Tot_Cons_inc_util_3m,Tot_Cons_dec_util_3m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,0,1,0,1,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,1,1,1,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,2,0,2,0,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,1,1,1,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [531]:
def cons_incr_dcr_pay_exp(snap_data):
    # Sorting the data is descending order 
    snap_data.sort_values(by=['MASK_CREDITCARD','snapshot_date','app_diff'],ascending= False,inplace=True)
    
    #L12M 
    snap_data_l12m = snap_data[(snap_data['app_diff'] <= 11)]
    #L6M 
    snap_data_l6m = snap_data[(snap_data['app_diff'] <= 5)]
    #L3M 
    snap_data_l3m = snap_data[(snap_data['app_diff'] <= 2)]
    

    
    # Utilization Increase
    snap_data_l12m['pay_exp_incr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['pay_exp'].diff()>0,1,0)
    snap_data_l6m['pay_exp_incr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['pay_exp'].diff()>0,1,0)
    snap_data_l3m['pay_exp_incr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['pay_exp'].diff()>0,1,0)
    
    snap_data_l12m['grp_ids_pay_exp_incr_flag'] = snap_data_l12m['pay_exp_incr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_pay_exp_incr_flag'] = snap_data_l6m['pay_exp_incr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_pay_exp_incr_flag'] = snap_data_l3m['pay_exp_incr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["pay_exp_incr_flag_cnt"] = snap_data_l12m['pay_exp_incr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_pay_exp_incr_flag']]).cumsum()
    snap_data_l6m["pay_exp_incr_flag_cnt"] = snap_data_l6m['pay_exp_incr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_pay_exp_incr_flag']]).cumsum()
    snap_data_l3m["pay_exp_incr_flag_cnt"] = snap_data_l3m['pay_exp_incr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_pay_exp_incr_flag']]).cumsum()

    # Utilization decrease
    snap_data_l12m['pay_exp_dcr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['pay_exp'].diff()<0,1,0)
    snap_data_l6m['pay_exp_dcr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['pay_exp'].diff()<0,1,0)
    snap_data_l3m['pay_exp_dcr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['pay_exp'].diff()<0,1,0)
    
    snap_data_l12m['grp_ids_pay_exp_dcr_flag'] = snap_data_l12m['pay_exp_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_pay_exp_dcr_flag'] = snap_data_l6m['pay_exp_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_pay_exp_dcr_flag'] = snap_data_l3m['pay_exp_dcr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["pay_exp_dcr_flag_cnt"] = snap_data_l12m['pay_exp_dcr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_pay_exp_dcr_flag']]).cumsum()
    snap_data_l6m["pay_exp_dcr_flag_cnt"] = snap_data_l6m['pay_exp_dcr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_pay_exp_dcr_flag']]).cumsum()
    snap_data_l3m["pay_exp_dcr_flag_cnt"] = snap_data_l3m['pay_exp_dcr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_pay_exp_dcr_flag']]).cumsum()
    
    factor_l12m = snap_data_l12m.assign(
                    Cons_inc_pay_exp_12m = snap_data_l12m["pay_exp_incr_flag_cnt"],
                    Cons_dec_pay_exp_12m = snap_data_l12m["pay_exp_dcr_flag_cnt"] ,
                    Tot_Cons_inc_pay_exp_12m = snap_data_l12m["pay_exp_incr_flag"],
                    Tot_Cons_dec_pay_exp_12m = snap_data_l12m["pay_exp_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_pay_exp_12m':'max',
                                                    'Cons_dec_pay_exp_12m':'max',
                                                    'Tot_Cons_inc_pay_exp_12m':'sum',
                                                    'Tot_Cons_dec_pay_exp_12m':'sum'
                                                  })
    factor_l12m = factor_l12m.reset_index()
    factor_l12m['Snapshot_Key'] = factor_l12m['MASK_CREDITCARD'].astype(str) + "_" + factor_l12m['snapshot_date'].astype(str)
    
    factor_l6m = snap_data_l6m.assign(
                    Cons_inc_pay_exp_6m = snap_data_l6m["pay_exp_incr_flag_cnt"],
                    Cons_dec_pay_exp_6m = snap_data_l6m["pay_exp_dcr_flag_cnt"] ,
                    Tot_Cons_inc_pay_exp_6m = snap_data_l6m["pay_exp_incr_flag"],
                    Tot_Cons_dec_pay_exp_6m = snap_data_l6m["pay_exp_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_pay_exp_6m':'max',
                                                    'Cons_dec_pay_exp_6m':'max',
                                                    'Tot_Cons_inc_pay_exp_6m':'sum',
                                                    'Tot_Cons_dec_pay_exp_6m':'sum'
                                                  })
    factor_l6m = factor_l6m.reset_index()
    factor_l6m['Snapshot_Key'] = factor_l6m['MASK_CREDITCARD'].astype(str) + "_" + factor_l6m['snapshot_date'].astype(str)
    
    factor_l3m = snap_data_l3m.assign(
                    Cons_inc_pay_exp_3m = snap_data_l3m["pay_exp_incr_flag_cnt"],
                    Cons_dec_pay_exp_3m = snap_data_l3m["pay_exp_dcr_flag_cnt"] ,
                    Tot_Cons_inc_pay_exp_3m = snap_data_l3m["pay_exp_incr_flag"],
                    Tot_Cons_dec_pay_exp_3m = snap_data_l3m["pay_exp_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_pay_exp_3m':'max',
                                                    'Cons_dec_pay_exp_3m':'max',
                                                    'Tot_Cons_inc_pay_exp_3m':'sum',
                                                    'Tot_Cons_dec_pay_exp_3m':'sum'
                                                  })
    factor_l3m = factor_l3m.reset_index()
    factor_l3m['Snapshot_Key'] = factor_l3m['MASK_CREDITCARD'].astype(str) + "_" + factor_l3m['snapshot_date'].astype(str)
    
    return factor_l12m, factor_l6m,factor_l3m 

In [532]:
l12m_inc_dec_pay_exp,l6m_inc_dec_pay_exp,l3m_inc_dec_pay_exp = cons_incr_dcr_pay_exp(snap)

In [533]:
l12m_inc_dec_pay_exp

,MASK_CREDITCARD,snapshot_date,Cons_inc_pay_exp_12m,Cons_dec_pay_exp_12m,Tot_Cons_inc_pay_exp_12m,Tot_Cons_dec_pay_exp_12m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,2,1,4,2,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,4,1,5,2,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,2,2,4,3,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,3,2,5,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,2,2,3,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,2,2,3,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,3,1,4,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [534]:
l6m_inc_dec_pay_exp

,MASK_CREDITCARD,snapshot_date,Cons_inc_pay_exp_6m,Cons_dec_pay_exp_6m,Tot_Cons_inc_pay_exp_6m,Tot_Cons_dec_pay_exp_6m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,2,1,2,2,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,2,2,2,3,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,2,1,4,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,2,2,3,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,2,2,3,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,3,1,4,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [535]:
l3m_inc_dec_pay_exp

,MASK_CREDITCARD,snapshot_date,Cons_inc_pay_exp_3m,Cons_dec_pay_exp_3m,Tot_Cons_inc_pay_exp_3m,Tot_Cons_dec_pay_exp_3m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,0,2,0,2,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,0,2,0,2,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,0,2,0,2,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,2,0,2,0,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [536]:
def cons_incr_dcr_pay_mindue(snap_data):
    # Sorting the data is descending order 
    snap_data.sort_values(by=['MASK_CREDITCARD','snapshot_date','app_diff'],ascending= False,inplace=True)
    
    #L12M 
    snap_data_l12m = snap_data[(snap_data['app_diff'] <= 11)]
    #L6M 
    snap_data_l6m = snap_data[(snap_data['app_diff'] <= 5)]
    #L3M 
    snap_data_l3m = snap_data[(snap_data['app_diff'] <= 2)]
    

    
    # Utilization Increase
    snap_data_l12m['pay_mindue_incr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['pay_mindue'].diff()>0,1,0)
    snap_data_l6m['pay_mindue_incr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['pay_mindue'].diff()>0,1,0)
    snap_data_l3m['pay_mindue_incr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['pay_mindue'].diff()>0,1,0)
    
    snap_data_l12m['grp_ids_pay_mindue_incr_flag'] = snap_data_l12m['pay_mindue_incr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_pay_mindue_incr_flag'] = snap_data_l6m['pay_mindue_incr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_pay_mindue_incr_flag'] = snap_data_l3m['pay_mindue_incr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["pay_mindue_incr_flag_cnt"] = snap_data_l12m['pay_mindue_incr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_pay_mindue_incr_flag']]).cumsum()
    snap_data_l6m["pay_mindue_incr_flag_cnt"] = snap_data_l6m['pay_mindue_incr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_pay_mindue_incr_flag']]).cumsum()
    snap_data_l3m["pay_mindue_incr_flag_cnt"] = snap_data_l3m['pay_mindue_incr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_pay_mindue_incr_flag']]).cumsum()

    # Utilization decrease
    snap_data_l12m['pay_mindue_dcr_flag']= np.where(snap_data_l12m.groupby('MASK_CREDITCARD')['pay_mindue'].diff()<0,1,0)
    snap_data_l6m['pay_mindue_dcr_flag'] = np.where(snap_data_l6m.groupby('MASK_CREDITCARD')['pay_mindue'].diff()<0,1,0)
    snap_data_l3m['pay_mindue_dcr_flag'] = np.where(snap_data_l3m.groupby('MASK_CREDITCARD')['pay_mindue'].diff()<0,1,0)
    
    snap_data_l12m['grp_ids_pay_mindue_dcr_flag'] = snap_data_l12m['pay_mindue_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l6m['grp_ids_pay_mindue_dcr_flag'] = snap_data_l6m['pay_mindue_dcr_flag'].diff().ne(0).cumsum()
    snap_data_l3m['grp_ids_pay_mindue_dcr_flag'] = snap_data_l3m['pay_mindue_dcr_flag'].diff().ne(0).cumsum()
    
    snap_data_l12m["pay_mindue_dcr_flag_cnt"] = snap_data_l12m['pay_mindue_dcr_flag'].groupby([snap_data_l12m['MASK_CREDITCARD'],snap_data_l12m['grp_ids_pay_mindue_dcr_flag']]).cumsum()
    snap_data_l6m["pay_mindue_dcr_flag_cnt"] = snap_data_l6m['pay_mindue_dcr_flag'].groupby([snap_data_l6m['MASK_CREDITCARD'],snap_data_l6m['grp_ids_pay_mindue_dcr_flag']]).cumsum()
    snap_data_l3m["pay_mindue_dcr_flag_cnt"] = snap_data_l3m['pay_mindue_dcr_flag'].groupby([snap_data_l3m['MASK_CREDITCARD'],snap_data_l3m['grp_ids_pay_mindue_dcr_flag']]).cumsum()
    
    factor_l12m = snap_data_l12m.assign(
                    Cons_inc_pay_mindue_12m = snap_data_l12m["pay_mindue_incr_flag_cnt"],
                    Cons_dec_pay_mindue_12m = snap_data_l12m["pay_mindue_dcr_flag_cnt"] ,
                    Tot_Cons_inc_pay_mindue_12m = snap_data_l12m["pay_mindue_incr_flag"],
                    Tot_Cons_dec_pay_mindue_12m = snap_data_l12m["pay_mindue_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_pay_mindue_12m':'max',
                                                    'Cons_dec_pay_mindue_12m':'max',
                                                    'Tot_Cons_inc_pay_mindue_12m':'sum',
                                                    'Tot_Cons_dec_pay_mindue_12m':'sum'
                                                  })
    factor_l12m = factor_l12m.reset_index()
    factor_l12m['Snapshot_Key'] = factor_l12m['MASK_CREDITCARD'].astype(str) + "_" + factor_l12m['snapshot_date'].astype(str)
    
    factor_l6m = snap_data_l6m.assign(
                    Cons_inc_pay_mindue_6m = snap_data_l6m["pay_mindue_incr_flag_cnt"],
                    Cons_dec_pay_mindue_6m = snap_data_l6m["pay_mindue_dcr_flag_cnt"] ,
                    Tot_Cons_inc_pay_mindue_6m = snap_data_l6m["pay_mindue_incr_flag"],
                    Tot_Cons_dec_pay_mindue_6m = snap_data_l6m["pay_mindue_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_pay_mindue_6m':'max',
                                                    'Cons_dec_pay_mindue_6m':'max',
                                                    'Tot_Cons_inc_pay_mindue_6m':'sum',
                                                    'Tot_Cons_dec_pay_mindue_6m':'sum'
                                                  })
    factor_l6m = factor_l6m.reset_index()
    factor_l6m['Snapshot_Key'] = factor_l6m['MASK_CREDITCARD'].astype(str) + "_" + factor_l6m['snapshot_date'].astype(str)
    
    factor_l3m = snap_data_l3m.assign(
                    Cons_inc_pay_mindue_3m = snap_data_l3m["pay_mindue_incr_flag_cnt"],
                    Cons_dec_pay_mindue_3m = snap_data_l3m["pay_mindue_dcr_flag_cnt"] ,
                    Tot_Cons_inc_pay_mindue_3m = snap_data_l3m["pay_mindue_incr_flag"],
                    Tot_Cons_dec_pay_mindue_3m = snap_data_l3m["pay_mindue_dcr_flag"]
                    ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                    'Cons_inc_pay_mindue_3m':'max',
                                                    'Cons_dec_pay_mindue_3m':'max',
                                                    'Tot_Cons_inc_pay_mindue_3m':'sum',
                                                    'Tot_Cons_dec_pay_mindue_3m':'sum'
                                                  })
    factor_l3m = factor_l3m.reset_index()   
    factor_l3m['Snapshot_Key'] = factor_l3m['MASK_CREDITCARD'].astype(str) + "_" + factor_l3m['snapshot_date'].astype(str)
    
    return factor_l12m, factor_l6m,factor_l3m 

In [537]:
l12m_inc_dec_pay_mindue,l6m_inc_dec_pay_mindue,l3m_inc_dec_pay_mindue = cons_incr_dcr_pay_mindue(snap)

In [538]:
l12m_inc_dec_pay_mindue

,MASK_CREDITCARD,snapshot_date,Cons_inc_pay_mindue_12m,Cons_dec_pay_mindue_12m,Tot_Cons_inc_pay_mindue_12m,Tot_Cons_dec_pay_mindue_12m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,2,1,4,2,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,4,1,5,2,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,2,2,4,3,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,3,2,5,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,2,2,3,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,2,2,3,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,3,1,4,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [539]:
l6m_inc_dec_pay_mindue

,MASK_CREDITCARD,snapshot_date,Cons_inc_pay_mindue_6m,Cons_dec_pay_mindue_6m,Tot_Cons_inc_pay_mindue_6m,Tot_Cons_dec_pay_mindue_6m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,2,1,2,2,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,2,1,3,2,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,2,2,2,3,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,1,2,1,4,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,1,2,2,3,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,1,2,2,3,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,3,1,4,1,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [540]:
l3m_inc_dec_pay_mindue

,MASK_CREDITCARD,snapshot_date,Cons_inc_pay_mindue_3m,Cons_dec_pay_mindue_3m,Tot_Cons_inc_pay_mindue_3m,Tot_Cons_dec_pay_mindue_3m,Snapshot_Key
0,FTKUGFFSTNNNNCCP,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCCP_2010-01-31
1,FTKUGFFSTNNNNCKN,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCKN_2010-01-31
2,FTKUGFFSTNNNNCNK,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCNK_2010-01-31
3,FTKUGFFSTNNNNCSS,2010-01-31,1,1,1,1,FTKUGFFSTNNNNCSS_2010-01-31
4,FTKUGFFSTNNNNCTW,2010-01-31,0,2,0,2,FTKUGFFSTNNNNCTW_2010-01-31
...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,0,2,0,2,FTKUGGNNSWPWFFCF_2010-01-31
989,FTKUGGNNTKFKPTSU,2010-01-31,0,2,0,2,FTKUGGNNTKFKPTSU_2010-01-31
990,FTKUGGNNTWCKNWNC,2010-01-31,2,0,2,0,FTKUGGNNTWCKNWNC_2010-01-31
991,FTKUGGNNUFPSGTFC,2010-01-31,2,0,2,0,FTKUGGNNUFPSGTFC_2010-01-31


In [541]:
#snap_l12m = snap[(snap['app_diff'] <= 11)]

In [542]:
# Joining the calculated variables for a particular snapshot 

In [543]:
#snap_uniq = pd.merge(snap_uniq,,  how='left', left_on='[A_c1,c2]', right_on = '[B_c1,c2]')

In [544]:
snap_uniq =pd.merge(snap_uniq,factor_0,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [545]:
snap_uniq

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,nbr_30_12m,nbr_60_3m,nbr_60_6m,nbr_60_12m,nbr_90_3m,nbr_90_6m,nbr_90_12m,nbr_pay_gt_mindue_3m,nbr_pay_gt_mindue_6m,nbr_pay_gt_mindue_12m
0,FTKUGFFSTNNNNCCP,2010-01-31,FTKUGFFSTNNNNCCP_2010-01-31,2.0,4.0,NaN,NaN,1.0,2.0,2.0,...,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.500000,0.428571
1,FTKUGFFSTNNNNCKN,2010-01-31,FTKUGFFSTNNNNCKN_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.666667,0.500000
2,FTKUGFFSTNNNNCNK,2010-01-31,FTKUGFFSTNNNNCNK_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.750000
3,FTKUGFFSTNNNNCSS,2010-01-31,FTKUGFFSTNNNNCSS_2010-01-31,2.0,NaN,NaN,NaN,1.0,1.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.666667,0.625000
4,FTKUGFFSTNNNNCTW,2010-01-31,FTKUGFFSTNNNNCTW_2010-01-31,0.0,0.0,NaN,NaN,2.0,2.0,2.0,...,0.125000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.166667,0.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,FTKUGGNNSWPWFFCF_2010-01-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.333333,0.285714
989,FTKUGGNNTKFKPTSU,2010-01-31,FTKUGGNNTKFKPTSU_2010-01-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.666667,0.500000,0.428571
990,FTKUGGNNTWCKNWNC,2010-01-31,FTKUGGNNTWCKNWNC_2010-01-31,1.0,NaN,NaN,NaN,1.0,1.0,1.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,1.000000,0.833333,0.714286
991,FTKUGGNNUFPSGTFC,2010-01-31,FTKUGGNNUFPSGTFC_2010-01-31,3.0,3.0,NaN,NaN,0.0,2.0,2.0,...,0.142857,0.0,0.0,0.0,0.0,0.0,0.0,0.333333,0.166667,0.142857


In [546]:
# Joining Delinquency Increase Decrease variables 

In [547]:
snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_delinq,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [548]:
snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_delinq,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [549]:
snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_delinq,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [550]:
# Joining Utilization Increase Decrease variables 

In [551]:
snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_util,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [552]:
snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_util,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [553]:
snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_util,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [554]:
# Joining Payment by Exposure Increase Decrease variables 

In [555]:
snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_pay_exp,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [556]:
snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_pay_exp,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [557]:
snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_pay_exp,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [558]:
# Joining Payment by min due Increase Decrease variables 

In [559]:
snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_pay_mindue,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [560]:
snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_pay_mindue,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [561]:
snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_pay_mindue,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [562]:
snap_uniq

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,Tot_Cons_inc_pay_mindue_12m,Tot_Cons_dec_pay_mindue_12m,Cons_inc_pay_mindue_6m,Cons_dec_pay_mindue_6m,Tot_Cons_inc_pay_mindue_6m,Tot_Cons_dec_pay_mindue_6m,Cons_inc_pay_mindue_3m,Cons_dec_pay_mindue_3m,Tot_Cons_inc_pay_mindue_3m,Tot_Cons_dec_pay_mindue_3m
0,FTKUGFFSTNNNNCCP,2010-01-31,FTKUGFFSTNNNNCCP_2010-01-31,2.0,4.0,NaN,NaN,1.0,2.0,2.0,...,3,2,2,1,3,2,1,1,1,1
1,FTKUGFFSTNNNNCKN,2010-01-31,FTKUGFFSTNNNNCKN_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,4,2,2,1,2,2,1,1,1,1
2,FTKUGFFSTNNNNCNK,2010-01-31,FTKUGFFSTNNNNCNK_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,5,2,2,1,3,2,1,1,1,1
3,FTKUGFFSTNNNNCSS,2010-01-31,FTKUGFFSTNNNNCSS_2010-01-31,2.0,NaN,NaN,NaN,1.0,1.0,1.0,...,4,3,2,2,2,3,1,1,1,1
4,FTKUGFFSTNNNNCTW,2010-01-31,FTKUGFFSTNNNNCTW_2010-01-31,0.0,0.0,NaN,NaN,2.0,2.0,2.0,...,2,5,1,2,1,4,0,2,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,FTKUGGNNSWPWFFCF_2010-01-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,2,3,1,2,2,3,0,2,0,2
989,FTKUGGNNTKFKPTSU,2010-01-31,FTKUGGNNTKFKPTSU_2010-01-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,2,3,1,2,2,3,0,2,0,2
990,FTKUGGNNTWCKNWNC,2010-01-31,FTKUGGNNTWCKNWNC_2010-01-31,1.0,NaN,NaN,NaN,1.0,1.0,1.0,...,4,1,3,1,4,1,2,0,2,0
991,FTKUGGNNUFPSGTFC,2010-01-31,FTKUGGNNUFPSGTFC_2010-01-31,3.0,3.0,NaN,NaN,0.0,2.0,2.0,...,2,0,2,0,2,0,2,0,2,0


In [563]:
snap.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8012 entries, 9707 to 6
Data columns (total 65 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   MASKED_CUSTOMER_ID         8012 non-null   object        
 1   MASK_CREDITCARD            8012 non-null   object        
 2   CARD_LIMIT                 8012 non-null   int64         
 3   CASH_LIMIT                 8012 non-null   int64         
 4   OS_BALANCE                 6951 non-null   float64       
 5   OPENING_BALANCE            6951 non-null   float64       
 6   CLOSING_BALANCE            6951 non-null   float64       
 7   CLOSING_BALANCE_CASH       6951 non-null   float64       
 8   CLOSING_BALANCE_FEES       6951 non-null   float64       
 9   NB_DAYS_DELINQ             8012 non-null   float64       
 10  Customer_Occupation        4010 non-null   object        
 11  A_Gender                   5365 non-null   object        
 12  A_Mari

In [564]:
# Selecting static variables 

In [565]:
snap.snapshot_date.value_counts()

2010-01-31    8012
Name: snapshot_date, dtype: int64

In [566]:
snap_static = snap[(snap['report_date'] ==pd.to_datetime('2010-01-31'))]

In [567]:
snap_static

,MASKED_CUSTOMER_ID,MASK_CREDITCARD,CARD_LIMIT,CASH_LIMIT,OS_BALANCE,OPENING_BALANCE,CLOSING_BALANCE,CLOSING_BALANCE_CASH,CLOSING_BALANCE_FEES,NB_DAYS_DELINQ,...,grocery_spend_exp,FI_spend_exp,utility_spend_exp,Jewelry_spend_exp,Airline_spend_exp,Education_spend_exp,snapshot_date,app_diff,snapshot_mob,snapshot_bad_def
9713,PPHHQFS,FTKUGGNNWTNNPUFP,14000,8400,7809.97,6250.00,7809.97,500.00,290.0,0.0,...,0.105122,1.373245,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,6,0
9654,QSEPPTE,FTKUGGNNUFPSGTFC,20000,12000,8614.94,4776.78,8614.94,0.00,100.0,0.0,...,0.047360,0.066512,0.058039,0.000000,0.000000,0.000000,2010-01-31,0.0,6,0
9632,QTHSHSH,FTKUGGNNTWCKNWNC,5000,3000,4514.30,5165.55,4514.30,1021.92,970.0,0.0,...,0.060031,0.147753,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,6,0
9593,FTDDDSR,FTKUGGNNTKFKPTSU,5000,3000,5197.04,5187.54,5197.04,1026.07,1090.0,1.0,...,0.000000,0.038483,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,6,0
9571,EEEEEEHT,FTKUGGNNSWPWFFCF,20000,12000,20241.71,19501.71,20241.71,4000.00,1350.0,1.0,...,0.000000,0.009881,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,GFDEEED,FTKUGFFSTNNNNCTW,24000,14400,23834.43,24018.97,23834.43,0.00,1700.0,2.0,...,0.000000,0.012587,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,7,0
30,GFGPPSS,FTKUGFFSTNNNNCSS,35000,21000,26075.25,9794.60,26075.25,8000.00,1250.0,0.0,...,0.000000,0.022627,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,7,0
22,FGPGDTS,FTKUGFFSTNNNNCNK,40000,24000,25354.76,25686.18,25354.76,0.00,1000.0,0.0,...,0.000000,0.082825,0.000000,0.022481,0.068823,0.032657,2010-01-31,0.0,7,0
14,DFGEGSE,FTKUGFFSTNNNNCKN,100000,60000,100.00,100.00,100.00,0.00,100.0,0.0,...,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2010-01-31,0.0,7,0


In [568]:
snap_static.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 993 entries, 9713 to 6
Data columns (total 65 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   MASKED_CUSTOMER_ID         993 non-null    object        
 1   MASK_CREDITCARD            993 non-null    object        
 2   CARD_LIMIT                 993 non-null    int64         
 3   CASH_LIMIT                 993 non-null    int64         
 4   OS_BALANCE                 993 non-null    float64       
 5   OPENING_BALANCE            993 non-null    float64       
 6   CLOSING_BALANCE            993 non-null    float64       
 7   CLOSING_BALANCE_CASH       993 non-null    float64       
 8   CLOSING_BALANCE_FEES       993 non-null    float64       
 9   NB_DAYS_DELINQ             993 non-null    float64       
 10  Customer_Occupation        495 non-null    object        
 11  A_Gender                   662 non-null    object        
 12  A_Marit

In [569]:
#snap = Model_prep3[(Model_prep3['report_date'] <= pd.to_datetime('2010-01-31'))]

In [570]:
snap.shape

(8012, 65)

In [571]:
snap_static.shape

(993, 65)

In [572]:
snap_static = snap_static[['MASK_CREDITCARD','snapshot_date','CARD_LIMIT','OS_BALANCE','NB_DAYS_DELINQ',
                           'Customer_Occupation','A_Gender','A_Marital_Status','A_Industry','A_Emp_Sts',
                           'A_No_Depend','A_salary','A_Sector','mob','bad_def','bad_flag','REGION',
                           'NATIONALITY','Age']]

In [573]:
snap_static = snap_static.drop_duplicates()

In [574]:
snap_static

,MASK_CREDITCARD,snapshot_date,CARD_LIMIT,OS_BALANCE,NB_DAYS_DELINQ,Customer_Occupation,A_Gender,A_Marital_Status,A_Industry,A_Emp_Sts,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age
9713,FTKUGGNNWTNNPUFP,2010-01-31,14000,7809.97,0.0,EMPLOYEE,FEMALE,SINGLE,PERSONAL,EMPLOYED,NaN,36331.0,Government,6,0,0.0,Abu_Dhabi,United Arab Emirates,34.0
9654,FTKUGGNNUFPSGTFC,2010-01-31,20000,8614.94,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,0,0.0,Abu_Dhabi,Egypt,NaN
9632,FTKUGGNNTWCKNWNC,2010-01-31,5000,4514.30,0.0,OFFICER,MALE,MARRIED,PERSONAL,EMPLOYED,2.0,15600.0,Government,6,0,0.0,Abu_Dhabi,United Arab Emirates,27.0
9593,FTKUGGNNTKFKPTSU,2010-01-31,5000,5197.04,1.0,NaN,FEMALE,SINGLE,PERSONAL,NaN,NaN,17975.0,NaN,6,0,0.0,Abu_Dhabi,United Arab Emirates,25.0
9571,FTKUGGNNSWPWFFCF,2010-01-31,20000,20241.71,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,0,1.0,Abu_Dhabi,United Arab Emirates,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38,FTKUGFFSTNNNNCTW,2010-01-31,24000,23834.43,2.0,BEING OBTAINED,MALE,MARRIED,PERSONAL,EMPLOYED,NaN,NaN,NaN,7,0,1.0,Others,United Arab Emirates,40.0
30,FTKUGFFSTNNNNCSS,2010-01-31,35000,26075.25,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,United Arab Emirates,NaN
22,FTKUGFFSTNNNNCNK,2010-01-31,40000,25354.76,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,Greece,NaN
14,FTKUGFFSTNNNNCKN,2010-01-31,100000,100.00,0.0,SVP HEAR OF RISK MANAGEMENT,MALE,MARRIED,PERSONAL,EMPLOYED,3.0,NaN,Semi Government,7,0,0.0,Abu_Dhabi,Pakistan,50.0


In [575]:
snap_static['Snapshot_Key'] = snap_static['MASK_CREDITCARD'].astype(str) + "_" + snap_static['snapshot_date'].astype(str)

In [576]:
# Joining to get static variables 

In [577]:
snap_uniq =pd.merge(snap_uniq,snap_static,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

In [578]:
snap_uniq

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,A_Emp_Sts,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age
0,FTKUGFFSTNNNNCCP,2010-01-31,FTKUGFFSTNNNNCCP_2010-01-31,2.0,4.0,NaN,NaN,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,6,0,0.0,Abu_Dhabi,Lebanon,2.0
1,FTKUGFFSTNNNNCKN,2010-01-31,FTKUGFFSTNNNNCKN_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,3.0,NaN,Semi Government,7,0,0.0,Abu_Dhabi,Pakistan,50.0
2,FTKUGFFSTNNNNCNK,2010-01-31,FTKUGFFSTNNNNCNK_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,Greece,NaN
3,FTKUGFFSTNNNNCSS,2010-01-31,FTKUGFFSTNNNNCSS_2010-01-31,2.0,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,United Arab Emirates,NaN
4,FTKUGFFSTNNNNCTW,2010-01-31,FTKUGFFSTNNNNCTW_2010-01-31,0.0,0.0,NaN,NaN,2.0,2.0,2.0,...,EMPLOYED,NaN,NaN,NaN,7,0,1.0,Others,United Arab Emirates,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
988,FTKUGGNNSWPWFFCF,2010-01-31,FTKUGGNNSWPWFFCF_2010-01-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,6,0,1.0,Abu_Dhabi,United Arab Emirates,NaN
989,FTKUGGNNTKFKPTSU,2010-01-31,FTKUGGNNTKFKPTSU_2010-01-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,17975.0,NaN,6,0,0.0,Abu_Dhabi,United Arab Emirates,25.0
990,FTKUGGNNTWCKNWNC,2010-01-31,FTKUGGNNTWCKNWNC_2010-01-31,1.0,NaN,NaN,NaN,1.0,1.0,1.0,...,EMPLOYED,2.0,15600.0,Government,6,0,0.0,Abu_Dhabi,United Arab Emirates,27.0
991,FTKUGGNNUFPSGTFC,2010-01-31,FTKUGGNNUFPSGTFC_2010-01-31,3.0,3.0,NaN,NaN,0.0,2.0,2.0,...,NaN,NaN,NaN,NaN,6,0,0.0,Abu_Dhabi,Egypt,NaN


In [579]:
snap_uniq.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 993 entries, 0 to 992
Columns: 174 entries, MASK_CREDITCARD to Age
dtypes: datetime64[ns](1), float64(112), int32(50), int64(1), object(10)
memory usage: 1.1+ MB


In [580]:
list(snap_uniq.columns)

['MASK_CREDITCARD',
 'snapshot_date',
 'Snapshot_Key',
 'month_since_0',
 'month_since_30',
 'month_since_60',
 'month_since_90',
 'max_dpd_3m',
 'max_dpd_6m',
 'max_dpd_12m',
 'avg_util_3m',
 'avg_util_6m',
 'avg_util_12m',
 'avg_cash_util_3m',
 'avg_cash_util_6m',
 'avg_cash_util_12m',
 'avg_amt_cash_withd_3m',
 'avg_amt_cash_withd_6m',
 'avg_amt_cash_withd_12m',
 'avg_amt_clothing_3m',
 'avg_amt_clothing_6m',
 'avg_amt_clothing_12m',
 'avg_perc_amt_clothing_3m',
 'avg_perc_amt_clothing_6m',
 'avg_perc_amt_clothing_12m',
 'avg_amt_utility_bills_3m',
 'avg_amt_utility_bills_6m',
 'avg_amt_utility_bills_12m',
 'avg_perc_amt_utility_bills_3m',
 'avg_perc_amt_utility_bills_6m',
 'avg_perc_amt_utility_bills_12m',
 'avg_amt_jewellery_3m',
 'avg_amt_jewellery_6m',
 'avg_amt_jewellery_12m',
 'avg_perc_amt_jewellery_3m',
 'avg_perc_amt_jewellery_6m',
 'avg_perc_amt_jewellery_12m',
 'avg_amt_airline_3m',
 'avg_amt_airline_6m',
 'avg_amt_airline_12m',
 'avg_perc_amt_airline_3m',
 'avg_perc_amt_

In [581]:
snapshot_range =['2010-01-31','2010-02-28','2010-03-31','2010-04-30','2010-05-31','2010-06-30','2010-07-31','2010-08-31',
                 '2010-09-30','2010-10-31','2010-11-30','2010-12-31','2011-01-31','2011-02-28','2011-03-31','2011-04-30',
                 '2011-05-31','2011-06-30','2011-07-31','2011-08-31','2011-09-30','2011-10-31','2011-11-30','2011-12-31']

In [582]:
snapshot_range[0]

'2010-01-31'

In [583]:
snapshot_range[1]

'2010-02-28'

In [584]:
for i in snapshot_range :
    print(i)
    snap = Model_prep3[(Model_prep3['report_date'] <= pd.to_datetime(i))]
    break

2010-01-31


In [585]:
model_data_prep = pd.DataFrame()

for i in snapshot_range :
        snap = Model_prep3[(Model_prep3['report_date'] <= pd.to_datetime(i))]
        snap['snapshot_date']=pd.to_datetime(i)
        snap['app_diff']=((snap['snapshot_date']-snap['report_date'])/np.timedelta64(1,'M'))
        snap['app_diff']= snap['app_diff'].round()
        snap_excl= snap[snap['app_diff']==0][['MASK_CREDITCARD','report_date','mob','bad_def']]
        snap_excl.rename(columns = {"report_date": "snapshot_date","mob": "snapshot_mob","bad_def": "snapshot_bad_def"},inplace = True)
        snap =pd.merge(snap,snap_excl,how='left',on=['MASK_CREDITCARD','snapshot_date'])
        
        # Applying exclusion
        snap = snap[snap['snapshot_mob']>=6]
        snap = snap[snap['snapshot_bad_def']==0]
        
        snap_uniq = snap[['MASK_CREDITCARD','snapshot_date']]
        snap_uniq = snap_uniq.drop_duplicates()
        snap_uniq['Snapshot_Key'] = snap_uniq['MASK_CREDITCARD'].astype(str) + "_" + snap_uniq['snapshot_date'].astype(str)
        factor_0 = snap.assign(
          
           # Late fees
           nbr_late_fee_3m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Count_Late_Payment'],np.nan),
           nbr_late_fee_3m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_late_fee_6m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Count_Late_Payment'],np.nan),
           nbr_late_fee_6m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_late_fee_12m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Count_Late_Payment'],np.nan),
           nbr_late_fee_12m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           # Cash Transactions
           nbr_cash_trans_3m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Count_Cash_Transactions'],np.nan),
           nbr_cash_trans_3m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_cash_trans_6m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Count_Cash_Transactions'],np.nan),
           nbr_cash_trans_6m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_cash_trans_12m_num =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Count_Cash_Transactions'],np.nan),
           nbr_cash_trans_12m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           # Month Since DPD 
           month_since_0=np.where(snap['NB_DAYS_DELINQ'] > 0,snap['app_diff'],np.nan),
           month_since_30=np.where(snap['NB_DAYS_DELINQ']> 1,snap['app_diff'],np.nan),
           month_since_60=np.where(snap['NB_DAYS_DELINQ'] > 2,snap['app_diff'],np.nan),
           month_since_90=np.where(snap['NB_DAYS_DELINQ'] > 3,snap['app_diff'],np.nan),
           
           # Number of times account went above a certain dpd bucket
           nbr_0_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 0),1,0),
           nbr_0_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_0_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 0),1,0),
           nbr_0_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_0_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 0),1,0),
           nbr_0_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           nbr_30_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 1),1,0),
           nbr_30_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_30_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 1),1,0),
           nbr_30_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_30_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 1),1,0),
           nbr_30_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           nbr_60_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 2),1,0),
           nbr_60_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_60_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 2),1,0),
           nbr_60_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_60_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 2),1,0),
           nbr_60_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           nbr_90_3m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['NB_DAYS_DELINQ'] > 3),1,0),
           nbr_90_3m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           nbr_90_6m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['NB_DAYS_DELINQ'] > 3),1,0),
           nbr_90_6m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           nbr_90_12m_num=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['NB_DAYS_DELINQ'] > 3),1,0),
           nbr_90_12m_den=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
           
           # Max DPD 
           max_dpd_3m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['NB_DAYS_DELINQ'],0),
           max_dpd_6m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['NB_DAYS_DELINQ'],0),
           max_dpd_12m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['NB_DAYS_DELINQ'],0),
     
           # Average Utilization
           avg_util_3m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['util'],np.nan),
           #avg_util_3m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
           avg_util_6m=np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['util'],np.nan),
           #avg_util_6m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
           avg_util_12m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['util'],np.nan),
           #avg_util_12m_den =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
    
          # Payment/Min Due
          nbr_pay_gt_mindue_3m_num = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2) & (snap['pay_mindue'] > 1),1,0),
          nbr_pay_gt_mindue_3m_den = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),1,0),
          nbr_pay_gt_mindue_6m_num = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5) & (snap['pay_mindue'] > 1),1,0),
          nbr_pay_gt_mindue_6m_den = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),1,0),
          nbr_pay_gt_mindue_12m_num = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11) & (snap['pay_mindue'] > 1),1,0),
          nbr_pay_gt_mindue_12m_den = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),1,0),
          
          # Cash withdrawal /Cash Limit or Cash Utilization
          avg_cash_util_3m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['cash_util'],np.nan),
          avg_cash_util_6m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['cash_util'],np.nan),
          avg_cash_util_12m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['cash_util'],np.nan),
            
           # Cash Withdrawal
           avg_amt_cash_withd_3m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Withdrawal_Amt'],np.nan),
           avg_amt_cash_withd_6m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Withdrawal_Amt'],np.nan),
           avg_amt_cash_withd_12m =np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Withdrawal_Amt'],np.nan),
            
           # Clothing 
           avg_amt_clothing_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Clothing_Spend_Amt'],np.nan),
           avg_amt_clothing_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Clothing_Spend_Amt'],np.nan),
           avg_amt_clothing_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Clothing_Spend_Amt'],np.nan),
           avg_perc_amt_clothing_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['clothing_spend_exp'],np.nan),
           avg_perc_amt_clothing_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['clothing_spend_exp'],np.nan),
           avg_perc_amt_clothing_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['clothing_spend_exp'],np.nan),
            
           # Utility
           avg_amt_utility_bills_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['utility_Spend_Amt'],np.nan),
           avg_amt_utility_bills_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['utility_Spend_Amt'],np.nan),
           avg_amt_utility_bills_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['utility_Spend_Amt'],np.nan),
           avg_perc_amt_utility_bills_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['utility_spend_exp'],np.nan),
           avg_perc_amt_utility_bills_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['utility_spend_exp'],np.nan),
           avg_perc_amt_utility_bills_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['utility_spend_exp'],np.nan),
    

           # Jewelery
           avg_amt_jewellery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Jewelry_Spend_Amt'],np.nan),
           avg_amt_jewellery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Jewelry_Spend_Amt'],np.nan),
           avg_amt_jewellery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Jewelry_Spend_Amt'],np.nan),
           avg_perc_amt_jewellery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Jewelry_spend_exp'],np.nan),
           avg_perc_amt_jewellery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Jewelry_spend_exp'],np.nan),
           avg_perc_amt_jewellery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Jewelry_spend_exp'],np.nan),
            

           # Airline
           avg_amt_airline_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Airline_Spend_Amt'],np.nan),
           avg_amt_airline_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Airline_Spend_Amt'],np.nan),
           avg_amt_airline_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Airline_Spend_Amt'],np.nan),
           avg_perc_amt_airline_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Airline_spend_exp'],np.nan),
           avg_perc_amt_airline_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Airline_spend_exp'],np.nan),
           avg_perc_amt_airline_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Airline_spend_exp'],np.nan),
            
           # Echannel Transactions 
           avg_echannel_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Count_Echannel_Payment'],np.nan),
           avg_echannel_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Count_Echannel_Payment'],np.nan),
           avg_echannel_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Count_Echannel_Payment'],np.nan),
    
           #Education 
           avg_amt_education_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Education_Spend_Amt'],np.nan),
           avg_amt_education_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Education_Spend_Amt'],np.nan),
           avg_amt_education_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Education_Spend_Amt'],np.nan),
           avg_perc_amt_education_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Education_spend_exp'],np.nan),
           avg_perc_amt_education_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Education_spend_exp'],np.nan),
           avg_perc_amt_education_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Education_spend_exp'],np.nan),
    
           # Hotel 
           avg_amt_hotel_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Hotels_Spend_Amt'],np.nan),
           avg_amt_hotel_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Hotels_Spend_Amt'],np.nan),
           avg_amt_hotel_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Hotels_Spend_Amt'],np.nan),
           avg_perc_amt_hotel_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['hotel_spend_exp'],np.nan),
           avg_perc_amt_hotel_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['hotel_spend_exp'],np.nan),
           avg_perc_amt_hotel_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['hotel_spend_exp'],np.nan),
    

           # Restaurant 
           avg_amt_restaurant_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Restaurant_Spend_Amt'],np.nan),
           avg_amt_restaurant_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Restaurant_Spend_Amt'],np.nan),
           avg_amt_restaurant_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Restaurant_Spend_Amt'],np.nan),
           avg_perc_amt_restaurant_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['restaurant_spend_exp'],np.nan),
           avg_perc_amt_restaurant_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['restaurant_spend_exp'],np.nan),
           avg_perc_amt_restaurant_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['restaurant_spend_exp'],np.nan),

           # Service Station 
           avg_amt_servicestation_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['ServiceStations_Spend_Amt'],np.nan),
           avg_amt_servicestation_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['ServiceStations_Spend_Amt'],np.nan),
           avg_amt_servicestation_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['ServiceStations_Spend_Amt'],np.nan),
           avg_perc_amt_servicestation_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['servicestation_spend_exp'],np.nan),
           avg_perc_amt_servicestation_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['servicestation_spend_exp'],np.nan),
           avg_perc_amt_servicestation_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['servicestation_spend_exp'],np.nan),


           # Automated Cash Disbursement 
           avg_amt_autocashdisb_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['AutoCashDisb_Spend_Amt'],np.nan),
           avg_amt_autocashdisb_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['AutoCashDisb_Spend_Amt'],np.nan),
           avg_amt_autocashdisb_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['AutoCashDisb_Spend_Amt'],np.nan),
           avg_perc_amt_autocashdisb_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['autocashdisb_spend_exp'],np.nan),
           avg_perc_amt_autocashdisb_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['autocashdisb_spend_exp'],np.nan),
           avg_perc_amt_autocashdisb_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['autocashdisb_spend_exp'],np.nan),
  
           # Groceries 
           avg_amt_grocery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['Grocery_Spend_Amt'],np.nan),
           avg_amt_grocery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['Grocery_Spend_Amt'],np.nan),
           avg_amt_grocery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['Grocery_Spend_Amt'],np.nan),
           avg_perc_amt_grocery_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['grocery_spend_exp'],np.nan),
           avg_perc_amt_grocery_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['grocery_spend_exp'],np.nan),
           avg_perc_amt_grocery_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['grocery_spend_exp'],np.nan),   

           # Financial Institutions
           avg_amt_FI_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['FI_Spend_Amt'],np.nan),
           avg_amt_FI_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['FI_Spend_Amt'],np.nan),
           avg_amt_FI_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['FI_Spend_Amt'],np.nan),
           avg_perc_amt_FI_3m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=2),snap['FI_spend_exp'],np.nan),
           avg_perc_amt_FI_6m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=5),snap['FI_spend_exp'],np.nan),
           avg_perc_amt_FI_12m = np.where((snap['app_diff'] >=0) & (snap['app_diff'] <=11),snap['FI_spend_exp'],np.nan)

          ).groupby(['MASK_CREDITCARD','snapshot_date']).agg({
                                                   'nbr_late_fee_3m_num':'sum',
                                                    'nbr_late_fee_3m_den':'sum',
                                                    'nbr_late_fee_6m_num':'sum',
                                                    'nbr_late_fee_6m_den':'sum',
                                                    'nbr_late_fee_12m_num':'sum',
                                                    'nbr_late_fee_12m_den':'sum',
                                                    'nbr_cash_trans_3m_num':'sum',
                                                    'nbr_cash_trans_3m_den':'sum',
                                                    'nbr_cash_trans_6m_num':'sum',
                                                    'nbr_cash_trans_6m_den':'sum',
                                                    'nbr_cash_trans_12m_num':'sum',
                                                    'nbr_cash_trans_12m_den':'sum',
                                                    'month_since_0':'min',
                                                    'month_since_30':'min',
                                                    'month_since_60':'min',
                                                    'month_since_90':'min',
                                                    'nbr_0_3m_num':'sum',
                                                    'nbr_0_3m_den':'sum',
                                                    'nbr_0_6m_num':'sum',
                                                    'nbr_0_6m_den':'sum',
                                                    'nbr_0_12m_num':'sum',
                                                    'nbr_0_12m_den':'sum',
                                                    'nbr_30_3m_num':'sum',
                                                    'nbr_30_3m_den':'sum',
                                                    'nbr_30_6m_num':'sum',
                                                    'nbr_30_6m_den':'sum',
                                                    'nbr_30_12m_num':'sum',
                                                    'nbr_30_12m_den':'sum',
                                                    'nbr_60_3m_num':'sum',
                                                    'nbr_60_3m_den':'sum',
                                                    'nbr_60_6m_num':'sum',
                                                    'nbr_60_6m_den':'sum',
                                                    'nbr_60_12m_num':'sum',
                                                    'nbr_60_12m_den':'sum',
                                                    'nbr_90_3m_num':'sum',
                                                    'nbr_90_3m_den':'sum',
                                                    'nbr_90_6m_num':'sum',
                                                    'nbr_90_6m_den':'sum',
                                                    'nbr_90_12m_num':'sum',
                                                    'nbr_90_12m_den':'sum',
                                                    'max_dpd_3m':'max',
                                                    'max_dpd_6m':'max',
                                                    'max_dpd_12m':'max',
                                                    'avg_util_3m':'mean',
                                                    'avg_util_6m':'mean',
                                                    'avg_util_12m':'mean',  
                                                    'nbr_pay_gt_mindue_3m_num':'sum',
                                                    'nbr_pay_gt_mindue_3m_den':'sum',
                                                    'nbr_pay_gt_mindue_6m_num':'sum',
                                                    'nbr_pay_gt_mindue_6m_den':'sum',
                                                    'nbr_pay_gt_mindue_12m_num':'sum',
                                                    'nbr_pay_gt_mindue_12m_den':'sum',
                                                    'avg_cash_util_3m':'mean',
                                                    'avg_cash_util_6m':'mean',
                                                    'avg_cash_util_12m':'mean', 
                                                    'avg_amt_cash_withd_3m':'mean',
                                                    'avg_amt_cash_withd_6m':'mean',
                                                    'avg_amt_cash_withd_12m':'mean', 
                                                    'avg_amt_clothing_3m':'mean',
                                                    'avg_amt_clothing_6m':'mean',
                                                    'avg_amt_clothing_12m':'mean', 
                                                    'avg_perc_amt_clothing_3m':'mean',
                                                    'avg_perc_amt_clothing_6m':'mean',
                                                    'avg_perc_amt_clothing_12m':'mean',  
                                                    'avg_amt_utility_bills_3m':'mean',
                                                    'avg_amt_utility_bills_6m':'mean',
                                                    'avg_amt_utility_bills_12m':'mean', 
                                                    'avg_perc_amt_utility_bills_3m':'mean',
                                                    'avg_perc_amt_utility_bills_6m':'mean',
                                                    'avg_perc_amt_utility_bills_12m':'mean',  
                                                    'avg_amt_jewellery_3m':'mean',
                                                    'avg_amt_jewellery_6m':'mean',
                                                    'avg_amt_jewellery_12m':'mean', 
                                                    'avg_perc_amt_jewellery_3m':'mean',
                                                    'avg_perc_amt_jewellery_6m':'mean',
                                                    'avg_perc_amt_jewellery_12m':'mean',  
                                                    'avg_amt_airline_3m':'mean',
                                                    'avg_amt_airline_6m':'mean',
                                                    'avg_amt_airline_12m':'mean', 
                                                    'avg_perc_amt_airline_3m':'mean',
                                                    'avg_perc_amt_airline_6m':'mean',
                                                    'avg_perc_amt_airline_12m':'mean', 
                                                    'avg_echannel_3m':'mean',
                                                    'avg_echannel_6m':'mean',
                                                    'avg_echannel_12m':'mean', 
                                                    'avg_amt_education_3m':'mean',
                                                    'avg_amt_education_6m':'mean',
                                                    'avg_amt_education_12m':'mean', 
                                                    'avg_perc_amt_education_3m':'mean',
                                                    'avg_perc_amt_education_6m':'mean',
                                                    'avg_perc_amt_education_12m':'mean',
                                                    'avg_amt_hotel_3m':'mean',
                                                    'avg_amt_hotel_6m':'mean',
                                                    'avg_amt_hotel_12m':'mean', 
                                                    'avg_perc_amt_hotel_3m':'mean',
                                                    'avg_perc_amt_hotel_6m':'mean',
                                                    'avg_perc_amt_hotel_12m':'mean',
                                                    'avg_amt_restaurant_3m':'mean',
                                                    'avg_amt_restaurant_6m':'mean',
                                                    'avg_amt_restaurant_12m':'mean', 
                                                    'avg_perc_amt_restaurant_3m':'mean',
                                                    'avg_perc_amt_restaurant_6m':'mean',
                                                    'avg_perc_amt_restaurant_12m':'mean',
                                                    'avg_amt_servicestation_3m':'mean',
                                                    'avg_amt_servicestation_6m':'mean',
                                                    'avg_amt_servicestation_12m':'mean', 
                                                    'avg_perc_amt_servicestation_3m':'mean',
                                                    'avg_perc_amt_servicestation_6m':'mean',
                                                    'avg_perc_amt_servicestation_12m':'mean',
                                                    'avg_amt_autocashdisb_3m':'mean',
                                                    'avg_amt_autocashdisb_6m':'mean',
                                                    'avg_amt_autocashdisb_12m':'mean', 
                                                    'avg_perc_amt_autocashdisb_3m':'mean',
                                                    'avg_perc_amt_autocashdisb_6m':'mean',
                                                    'avg_perc_amt_autocashdisb_12m':'mean',
                                                    'avg_amt_grocery_3m':'mean',
                                                    'avg_amt_grocery_6m':'mean',
                                                    'avg_amt_grocery_12m':'mean', 
                                                    'avg_perc_amt_grocery_3m':'mean',
                                                    'avg_perc_amt_grocery_6m':'mean',
                                                    'avg_perc_amt_grocery_12m':'mean',
                                                    'avg_amt_FI_3m':'mean',
                                                    'avg_amt_FI_6m':'mean',
                                                    'avg_amt_FI_12m':'mean', 
                                                    'avg_perc_amt_FI_3m':'mean',
                                                    'avg_perc_amt_FI_6m':'mean',
                                                    'avg_perc_amt_FI_12m':'mean'
                                                  })
        factor_0 = factor_0.reset_index()
        factor_0['nbr_late_fee_3m']=np.where(factor_0['nbr_late_fee_3m_den'] > 0,factor_0['nbr_late_fee_3m_num']/factor_0['nbr_late_fee_3m_den'],np.nan)
        factor_0['nbr_late_fee_6m']=np.where(factor_0['nbr_late_fee_6m_den'] > 0,factor_0['nbr_late_fee_6m_num']/factor_0['nbr_late_fee_6m_den'],np.nan)
        factor_0['nbr_late_fee_12m']=np.where(factor_0['nbr_late_fee_12m_den'] > 0,factor_0['nbr_late_fee_12m_num']/factor_0['nbr_late_fee_12m_den'],np.nan)

        factor_0['nbr_cash_trans_3m']=np.where(factor_0['nbr_cash_trans_3m_den'] > 0,factor_0['nbr_cash_trans_3m_num']/factor_0['nbr_cash_trans_3m_den'],np.nan)
        factor_0['nbr_cash_trans_6m']=np.where(factor_0['nbr_cash_trans_6m_den'] > 0,factor_0['nbr_cash_trans_6m_num']/factor_0['nbr_cash_trans_6m_den'],np.nan)
        factor_0['nbr_cash_trans_12m']=np.where(factor_0['nbr_cash_trans_12m_den'] > 0,factor_0['nbr_cash_trans_12m_num']/factor_0['nbr_cash_trans_12m_den'],np.nan)

        factor_0['nbr_0_3m']=np.where(factor_0['nbr_0_3m_den'] > 0,factor_0['nbr_0_3m_num']/factor_0['nbr_0_3m_den'],np.nan)
        factor_0['nbr_0_6m']=np.where(factor_0['nbr_0_6m_den'] > 0,factor_0['nbr_0_6m_num']/factor_0['nbr_0_6m_den'],np.nan)
        factor_0['nbr_0_12m']=np.where(factor_0['nbr_0_12m_den'] > 0,factor_0['nbr_0_12m_num']/factor_0['nbr_0_12m_den'],np.nan)

        factor_0['nbr_30_3m']=np.where(factor_0['nbr_30_3m_den'] > 0,factor_0['nbr_30_3m_num']/factor_0['nbr_30_3m_den'],np.nan)
        factor_0['nbr_30_6m']=np.where(factor_0['nbr_30_6m_den'] > 0,factor_0['nbr_30_6m_num']/factor_0['nbr_30_6m_den'],np.nan)
        factor_0['nbr_30_12m']=np.where(factor_0['nbr_30_12m_den'] > 0,factor_0['nbr_30_12m_num']/factor_0['nbr_30_12m_den'],np.nan)

        factor_0['nbr_60_3m']=np.where(factor_0['nbr_60_3m_den'] > 0,factor_0['nbr_60_3m_num']/factor_0['nbr_60_3m_den'],np.nan)
        factor_0['nbr_60_6m']=np.where(factor_0['nbr_60_6m_den'] > 0,factor_0['nbr_60_6m_num']/factor_0['nbr_60_6m_den'],np.nan)
        factor_0['nbr_60_12m']=np.where(factor_0['nbr_60_12m_den'] > 0,factor_0['nbr_60_12m_num']/factor_0['nbr_60_12m_den'],np.nan)

        factor_0['nbr_90_3m']=np.where(factor_0['nbr_90_3m_den'] > 0,factor_0['nbr_90_3m_num']/factor_0['nbr_90_3m_den'],np.nan)
        factor_0['nbr_90_6m']=np.where(factor_0['nbr_90_6m_den'] > 0,factor_0['nbr_90_6m_num']/factor_0['nbr_90_6m_den'],np.nan)
        factor_0['nbr_90_12m']=np.where(factor_0['nbr_90_12m_den'] > 0,factor_0['nbr_90_12m_num']/factor_0['nbr_90_12m_den'],np.nan)

        factor_0['nbr_pay_gt_mindue_3m']=np.where(factor_0['nbr_pay_gt_mindue_3m_den'] > 0,factor_0['nbr_pay_gt_mindue_3m_num']/factor_0['nbr_pay_gt_mindue_3m_den'],np.nan)
        factor_0['nbr_pay_gt_mindue_6m']=np.where(factor_0['nbr_pay_gt_mindue_6m_den'] > 0,factor_0['nbr_pay_gt_mindue_6m_num']/factor_0['nbr_pay_gt_mindue_6m_den'],np.nan)
        factor_0['nbr_pay_gt_mindue_12m']=np.where(factor_0['nbr_pay_gt_mindue_12m_den'] > 0,factor_0['nbr_pay_gt_mindue_12m_num']/factor_0['nbr_pay_gt_mindue_12m_den'],np.nan)
        factor_0=factor_0.drop([ 'nbr_late_fee_3m_num','nbr_late_fee_3m_den','nbr_late_fee_6m_num','nbr_late_fee_6m_den',
                         'nbr_late_fee_12m_num','nbr_late_fee_12m_den','nbr_cash_trans_3m_num','nbr_cash_trans_3m_den',
                         'nbr_cash_trans_6m_num','nbr_cash_trans_6m_den','nbr_cash_trans_12m_num','nbr_cash_trans_12m_den',
                         'nbr_0_3m_num','nbr_0_3m_den','nbr_0_6m_num','nbr_0_6m_den','nbr_0_12m_num','nbr_0_12m_den',
                         'nbr_30_3m_num','nbr_30_3m_den','nbr_30_6m_num','nbr_30_6m_den','nbr_30_12m_num','nbr_30_12m_den',
                         'nbr_60_3m_num','nbr_60_3m_den','nbr_60_6m_num','nbr_60_6m_den','nbr_60_12m_num','nbr_60_12m_den',
                         'nbr_90_3m_num','nbr_90_3m_den','nbr_90_6m_num','nbr_90_6m_den','nbr_90_12m_num','nbr_90_12m_den',
                         'nbr_pay_gt_mindue_3m_num' ,'nbr_pay_gt_mindue_3m_den','nbr_pay_gt_mindue_6m_num','nbr_pay_gt_mindue_6m_den',
                          'nbr_pay_gt_mindue_12m_num','nbr_pay_gt_mindue_12m_den'],axis=1)
        factor_0['Snapshot_Key'] = factor_0['MASK_CREDITCARD'].astype(str) + "_" + factor_0['snapshot_date'].astype(str)
        l12m_inc_dec_delinq,l6m_inc_dec_delinq,l3m_inc_dec_delinq = cons_incr_dcr_delinq(snap)
        l12m_inc_dec_util,l6m_inc_dec_util,l3m_inc_dec_util = cons_incr_dcr_util(snap)
        l12m_inc_dec_pay_exp,l6m_inc_dec_pay_exp,l3m_inc_dec_pay_exp = cons_incr_dcr_pay_exp(snap)
        l12m_inc_dec_pay_mindue,l6m_inc_dec_pay_mindue,l3m_inc_dec_pay_mindue = cons_incr_dcr_pay_mindue(snap)

        # Joining the calculated variables for a particular snapshot 
        snap_uniq =pd.merge(snap_uniq,factor_0,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

        # Joining Delinquency Increase Decrease variables 
        snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_delinq,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_delinq,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_delinq,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
 
        # Joining Utilization Increase Decrease variables 
        snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_util,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_util,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_util,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

        # Joining Payment by Exposure Increase Decrease variables 
        snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_pay_exp,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_pay_exp,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_pay_exp,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

        # Joining Payment by min due Increase Decrease variables 
        snap_uniq =pd.merge(snap_uniq,l12m_inc_dec_pay_mindue,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l6m_inc_dec_pay_mindue,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        snap_uniq =pd.merge(snap_uniq,l3m_inc_dec_pay_mindue,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])

        # Selecting static variables 
        snap_static = snap[(snap['report_date'] == pd.to_datetime(i))]
        snap_static = snap_static[['MASK_CREDITCARD','snapshot_date','CARD_LIMIT','OS_BALANCE','NB_DAYS_DELINQ',
                                   'Customer_Occupation','A_Gender','A_Marital_Status','A_Industry','A_Emp_Sts',
                                   'A_No_Depend','A_salary','A_Sector','mob','bad_def','bad_flag','REGION',
                                   'NATIONALITY','Age']]
        snap_static = snap_static.drop_duplicates()
        snap_static['Snapshot_Key'] = snap_static['MASK_CREDITCARD'].astype(str) + "_" + snap_static['snapshot_date'].astype(str)
        snap_uniq =pd.merge(snap_uniq,snap_static,how='left',on=['MASK_CREDITCARD','snapshot_date','Snapshot_Key'])
        model_data_prep =model_data_prep.append(snap_uniq)

In [586]:
model_data_prep.shape

(36324, 174)

In [587]:
model_data_prep

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,A_Emp_Sts,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age
0,FTKUGFFSTNNNNCCP,2010-01-31,FTKUGFFSTNNNNCCP_2010-01-31,2.0,4.0,NaN,NaN,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,6,0,0.0,Abu_Dhabi,Lebanon,2.0
1,FTKUGFFSTNNNNCKN,2010-01-31,FTKUGFFSTNNNNCKN_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,3.0,NaN,Semi Government,7,0,0.0,Abu_Dhabi,Pakistan,50.0
2,FTKUGFFSTNNNNCNK,2010-01-31,FTKUGFFSTNNNNCNK_2010-01-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,Greece,NaN
3,FTKUGFFSTNNNNCSS,2010-01-31,FTKUGFFSTNNNNCSS_2010-01-31,2.0,NaN,NaN,NaN,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,United Arab Emirates,NaN
4,FTKUGFFSTNNNNCTW,2010-01-31,FTKUGFFSTNNNNCTW_2010-01-31,0.0,0.0,NaN,NaN,2.0,2.0,2.0,...,EMPLOYED,NaN,NaN,NaN,7,0,1.0,Others,United Arab Emirates,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,FTKUGGNNWUWUUNUK,2011-12-31,FTKUGGNNWUWUUNUK_2011-12-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,4.0,NaN,NaN,6,0,0.0,Abu_Dhabi,United Arab Emirates,34.0
1841,FTKUGGNNWWCCFCSW,2011-12-31,FTKUGGNNWWCCFCSW_2011-12-31,3.0,NaN,NaN,NaN,0.0,1.0,1.0,...,EMPLOYED,NaN,14775.0,Government,12,0,0.0,Abu_Dhabi,United Arab Emirates,27.0
1842,FTKUGGNNWWKWCNWW,2011-12-31,FTKUGGNNWWKWCNWW_2011-12-31,0.0,17.0,NaN,NaN,1.0,1.0,1.0,...,EMPLOYED,NaN,23980.0,Government,28,0,0.0,Others,United Arab Emirates,34.0
1843,FTKUGGNNWWSFGTUT,2011-12-31,FTKUGGNNWWSFGTUT_2011-12-31,0.0,NaN,NaN,NaN,1.0,1.0,1.0,...,EMPLOYED,NaN,14992.0,Government,16,0,1.0,Abu_Dhabi,United Arab Emirates,24.0


In [588]:
model_data_prep.snapshot_date.value_counts()

2011-12-31    1845
2011-11-30    1840
2011-10-31    1834
2011-09-30    1824
2011-08-31    1818
2011-07-31    1812
2011-06-30    1784
2011-05-31    1750
2011-04-30    1711
2011-03-31    1672
2011-02-28    1621
2011-01-31    1574
2010-12-31    1506
2010-11-30    1457
2010-10-31    1425
2010-09-30    1369
2010-08-31    1345
2010-07-31    1288
2010-06-30    1254
2010-05-31    1217
2010-04-30    1171
2010-03-31    1136
2010-02-28    1078
2010-01-31     993
Name: snapshot_date, dtype: int64

In [589]:
#model_data_prep.to_csv('data_prep_model.csv',index= False)

In [590]:
model_data_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36324 entries, 0 to 1844
Columns: 174 entries, MASK_CREDITCARD to Age
dtypes: datetime64[ns](1), float64(112), int32(50), int64(1), object(10)
memory usage: 41.6+ MB


In [591]:
model_data_prep.describe()

,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,avg_util_3m,avg_util_6m,avg_util_12m,...,Tot_Cons_dec_pay_mindue_3m,CARD_LIMIT,OS_BALANCE,NB_DAYS_DELINQ,A_No_Depend,A_salary,mob,bad_def,bad_flag,Age
count,23504.000000,9154.000000,3587.000000,1436.000000,36324.000000,36324.000000,36324.000000,36324.000000,36324.000000,36324.000000,...,36324.000000,3.632400e+04,36324.000000,36324.000000,12398.000000,15759.000000,36324.000000,36324.0,36324.000000,27225.000000
mean,3.448860,7.462858,8.112071,9.149721,0.537303,0.716028,0.911656,0.789808,0.787213,0.764672,...,0.887925,3.748430e+04,23726.758150,0.272134,3.069124,32614.353855,16.095612,0.0,0.056106,36.091680
std,5.025038,6.985805,6.876225,6.800943,0.832146,0.997586,1.123754,0.264116,0.255316,0.245389,...,0.600744,8.207641e+04,23827.301905,0.572352,1.885993,24899.923173,6.718497,0.0,0.230130,9.584838
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000e+00,0.000000,0.000000,1.000000,2565.000000,6.000000,0.0,0.000000,2.000000
25%,0.000000,1.000000,2.000000,3.000000,0.000000,0.000000,0.000000,0.713569,0.714530,0.684568,...,1.000000,2.000000e+04,12500.745000,0.000000,2.000000,18337.500000,10.000000,0.0,0.000000,30.000000
50%,1.000000,6.000000,7.000000,7.000000,0.000000,1.000000,1.000000,0.906709,0.897786,0.853275,...,1.000000,3.000000e+04,20180.645000,0.000000,3.000000,24778.000000,15.000000,0.0,0.000000,35.000000
75%,5.000000,12.000000,12.000000,14.000000,1.000000,1.000000,1.000000,0.962297,0.957200,0.939490,...,1.000000,3.500000e+04,32289.200000,0.000000,4.000000,38400.000000,21.000000,0.0,0.000000,42.000000
max,29.000000,29.000000,29.000000,28.000000,9.000000,9.000000,9.000000,2.204283,1.454720,1.484272,...,2.000000,1.100000e+06,690975.310000,3.000000,12.000000,293736.000000,33.000000,0.0,1.000000,69.000000


In [592]:
stats = model_data_prep.describe()

In [593]:
type(stats)

pandas.core.frame.DataFrame

In [594]:
stats.to_csv('stats.csv')

In [595]:
missing_cnt = model_data_prep.isnull().sum()

In [596]:
type(missing_cnt)

pandas.core.series.Series

In [597]:
missing_cnt =missing_cnt.to_frame() 

In [598]:
missing_cnt.to_csv('missing_cnt.csv')

In [599]:
#ttv

In [600]:
#MASK_CREDITCARD	snapshot_date	Snapshot_Key

TTV split - Data Prep for Binning 

In [601]:
#Checking bad_flag counts 

In [602]:
model_data_prep = model_data_prep.round(10)

In [603]:
model_data_prep.bad_flag.value_counts()

0.0    34286
1.0     2038
Name: bad_flag, dtype: int64

In [604]:
# Sort the data on KEY ( i.e. Snapshot_Key) in ascending order

In [605]:
model_data_prep.sort_values(by=['Snapshot_Key'],ascending= True,inplace=True)

In [606]:
# Training Test Split 

In [607]:
model_data_prep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36324 entries, 0 to 1844
Columns: 174 entries, MASK_CREDITCARD to Age
dtypes: datetime64[ns](1), float64(112), int32(50), int64(1), object(10)
memory usage: 41.6+ MB


In [608]:
model_data_prep

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,A_Emp_Sts,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age
0,FTKUGFFSTNNNNCCP,2010-01-31,FTKUGFFSTNNNNCCP_2010-01-31,2.0,4.0,NaN,NaN,1.0,2.0,2.0,...,NaN,NaN,NaN,NaN,6,0,0.0,Abu_Dhabi,Lebanon,2.0
0,FTKUGFFSTNNNNCCP,2010-02-28,FTKUGFFSTNNNNCCP_2010-02-28,3.0,5.0,NaN,NaN,0.0,2.0,2.0,...,NaN,NaN,NaN,NaN,7,0,0.0,Abu_Dhabi,Lebanon,2.0
0,FTKUGFFSTNNNNCCP,2010-03-31,FTKUGFFSTNNNNCCP_2010-03-31,4.0,6.0,NaN,NaN,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,8,0,0.0,Abu_Dhabi,Lebanon,2.0
0,FTKUGFFSTNNNNCCP,2010-04-30,FTKUGFFSTNNNNCCP_2010-04-30,5.0,7.0,NaN,NaN,0.0,1.0,2.0,...,NaN,NaN,NaN,NaN,9,0,0.0,Abu_Dhabi,Lebanon,3.0
0,FTKUGFFSTNNNNCCP,2010-05-31,FTKUGFFSTNNNNCCP_2010-05-31,6.0,8.0,NaN,NaN,0.0,0.0,2.0,...,NaN,NaN,NaN,NaN,10,0,0.0,Abu_Dhabi,Lebanon,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1817,FTKUGGNNWWUTCKWK,2011-08-31,FTKUGGNNWWUTCKWK_2011-08-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,4.0,29553.0,Government,24,0,0.0,Others,United Arab Emirates,50.0
1823,FTKUGGNNWWUTCKWK,2011-09-30,FTKUGGNNWWUTCKWK_2011-09-30,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,4.0,29553.0,Government,25,0,0.0,Others,United Arab Emirates,50.0
1833,FTKUGGNNWWUTCKWK,2011-10-31,FTKUGGNNWWUTCKWK_2011-10-31,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,4.0,29553.0,Government,26,0,0.0,Others,United Arab Emirates,50.0
1839,FTKUGGNNWWUTCKWK,2011-11-30,FTKUGGNNWWUTCKWK_2011-11-30,NaN,NaN,NaN,NaN,0.0,0.0,0.0,...,EMPLOYED,4.0,29553.0,Government,27,0,0.0,Others,United Arab Emirates,50.0


In [609]:
list(model_data_prep.columns)

['MASK_CREDITCARD',
 'snapshot_date',
 'Snapshot_Key',
 'month_since_0',
 'month_since_30',
 'month_since_60',
 'month_since_90',
 'max_dpd_3m',
 'max_dpd_6m',
 'max_dpd_12m',
 'avg_util_3m',
 'avg_util_6m',
 'avg_util_12m',
 'avg_cash_util_3m',
 'avg_cash_util_6m',
 'avg_cash_util_12m',
 'avg_amt_cash_withd_3m',
 'avg_amt_cash_withd_6m',
 'avg_amt_cash_withd_12m',
 'avg_amt_clothing_3m',
 'avg_amt_clothing_6m',
 'avg_amt_clothing_12m',
 'avg_perc_amt_clothing_3m',
 'avg_perc_amt_clothing_6m',
 'avg_perc_amt_clothing_12m',
 'avg_amt_utility_bills_3m',
 'avg_amt_utility_bills_6m',
 'avg_amt_utility_bills_12m',
 'avg_perc_amt_utility_bills_3m',
 'avg_perc_amt_utility_bills_6m',
 'avg_perc_amt_utility_bills_12m',
 'avg_amt_jewellery_3m',
 'avg_amt_jewellery_6m',
 'avg_amt_jewellery_12m',
 'avg_perc_amt_jewellery_3m',
 'avg_perc_amt_jewellery_6m',
 'avg_perc_amt_jewellery_12m',
 'avg_amt_airline_3m',
 'avg_amt_airline_6m',
 'avg_amt_airline_12m',
 'avg_perc_amt_airline_3m',
 'avg_perc_amt_

In [610]:
#To replace all na & all special values with '-555555'
def fill_na(df, value):
    df.fillna(value, inplace=True)
    df.replace([np.inf,-np.inf],value ,inplace=True)

fill_na(model_data_prep ,-555555 )

In [611]:
model_data_prep.isnull().sum().sort_values(ascending = False)

MASK_CREDITCARD                    0
Tot_Cons_inc_delinq_3m             0
Tot_Cons_inc_delinq_12m            0
Tot_Cons_dec_delinq_12m            0
Cons_inc_delinq_6m                 0
Cons_dec_delinq_6m                 0
Tot_Cons_inc_delinq_6m             0
Tot_Cons_dec_delinq_6m             0
Cons_inc_delinq_3m                 0
Cons_dec_delinq_3m                 0
Tot_Cons_dec_delinq_3m             0
Cons_inc_delinq_12m                0
Cons_inc_util_12m                  0
Cons_dec_util_12m                  0
Tot_Cons_inc_util_12m              0
Tot_Cons_dec_util_12m              0
Cons_inc_util_6m                   0
Cons_dec_util_6m                   0
Tot_Cons_inc_util_6m               0
Tot_Cons_dec_util_6m               0
Cons_dec_delinq_12m                0
nbr_pay_gt_mindue_12m              0
Cons_dec_util_3m                   0
nbr_30_3m                          0
nbr_late_fee_6m                    0
nbr_late_fee_12m                   0
nbr_cash_trans_3m                  0
n

In [612]:
# from sklearn.model_selection import train_test_split 
train, test = train_test_split(model_data_prep, test_size=0.30,random_state=2023, stratify=model_data_prep['bad_flag']) 

In [613]:
train.shape

(25426, 174)

In [614]:
train["MASK_CREDITCARD"].nunique()

2165

In [615]:
train_good = train[(train['bad_flag'] ==0)]

In [616]:
train_good.shape

(23999, 174)

In [617]:
train_good["MASK_CREDITCARD"].nunique()

2031

In [618]:
train_bad = train[(train['bad_flag'] ==1)]

In [619]:
train_bad.shape

(1427, 174)

In [620]:
train_bad["MASK_CREDITCARD"].nunique()

293

In [621]:
test.shape

(10898, 174)

In [622]:
test_good = test[(test['bad_flag'] ==0)]

In [623]:
test_good.shape

(10287, 174)

In [624]:
test_good["MASK_CREDITCARD"].nunique()

1960

In [625]:
test_bad = test[(test['bad_flag'] ==1)]

In [626]:
test_bad.shape

(611, 174)

In [627]:
test_bad["MASK_CREDITCARD"].nunique()

240

In [628]:
train.bad_flag.value_counts()

0.0    23999
1.0     1427
Name: bad_flag, dtype: int64

In [629]:
test.bad_flag.value_counts()

0.0    10287
1.0      611
Name: bad_flag, dtype: int64

In [630]:
train["F_build"] = 1
test["F_build"] = 0

model_data = train.append(test)

In [631]:
model_data["F_build"].value_counts()

1    25426
0    10898
Name: F_build, dtype: int64

In [632]:
model_data.shape

(36324, 175)

In [633]:
model_data

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age,F_build
1489,FTKUGGNNNSUNNNNT,2011-11-30,FTKUGGNNNSUNNNNT_2011-11-30,1.0,-555555.0,-555555.0,-555555.0,1.0,1.0,1.0,...,-555555.0,-555555.00,-555555,29,0,1.0,Dubai,Iran,39.0,1
1829,FTKUGGNNWSSWWNTG,2011-12-31,FTKUGGNNWSSWWNTG_2011-12-31,1.0,1.0,-555555.0,-555555.0,2.0,2.0,2.0,...,1.0,24600.00,Government,26,0,0.0,Others,United Arab Emirates,33.0,1
859,FTKUGFNNTFTNPSWF,2011-02-28,FTKUGFNNTFTNPSWF_2011-02-28,-555555.0,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,...,-555555.0,29350.00,Government,16,0,0.0,Abu_Dhabi,United Arab Emirates,25.0,1
1290,FTKUGGNNKWKCGKNS,2011-09-30,FTKUGGNNKWKCGKNS_2011-09-30,6.0,-555555.0,-555555.0,-555555.0,0.0,0.0,1.0,...,-555555.0,19870.00,Government,19,0,0.0,Others,United Arab Emirates,33.0,1
1213,FTKUGGNNNFWCNNNW,2011-03-31,FTKUGGNNNFWCNNNW_2011-03-31,0.0,0.0,18.0,-555555.0,2.0,2.0,2.0,...,-555555.0,-555555.00,-555555,21,0,1.0,Dubai,Iran,61.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899,FTKUGGNNNPUUNNNW,2010-03-31,FTKUGGNNNPUUNNNW_2010-03-31,-555555.0,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,...,-555555.0,-555555.00,-555555,9,0,0.0,Abu_Dhabi,India,-555555.0,0
312,FTKUGFNNNPCTNNNN,2010-03-31,FTKUGFNNNPCTNNNN_2010-03-31,7.0,-555555.0,-555555.0,-555555.0,0.0,0.0,1.0,...,2.0,13827.00,Government,9,0,0.0,Abu_Dhabi,United Arab Emirates,40.0,0
1,FTKUGFFSTNNNNCKN,2011-08-31,FTKUGFFSTNNNNCKN_2011-08-31,-555555.0,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,...,3.0,-555555.00,Semi Government,26,0,0.0,Abu_Dhabi,Pakistan,51.0,0
1280,FTKUGGNNNCUTGFWS,2011-06-30,FTKUGGNNNCUTGFWS_2011-06-30,2.0,-555555.0,-555555.0,-555555.0,1.0,1.0,1.0,...,-555555.0,10396.00,Government,19,0,0.0,Dubai,United Arab Emirates,40.0,0


In [634]:
def cal_badrate(col):
    bad_rate = str((col.sum() / col.count())*100)
    print('Bad rate is : ' + bad_rate )
    print(col.value_counts())

In [635]:
cal_badrate(train.bad_flag)

Bad rate is : 5.612365295366947
0.0    23999
1.0     1427
Name: bad_flag, dtype: int64


In [636]:
cal_badrate(test.bad_flag)

Bad rate is : 5.6065333088640115
0.0    10287
1.0      611
Name: bad_flag, dtype: int64


In [637]:
cal_badrate(model_data_prep.bad_flag)

Bad rate is : 5.610615570972359
0.0    34286
1.0     2038
Name: bad_flag, dtype: int64


In [638]:
train.shape

(25426, 175)

In [639]:
train.head()

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age,F_build
1489,FTKUGGNNNSUNNNNT,2011-11-30,FTKUGGNNNSUNNNNT_2011-11-30,1.0,-555555.0,-555555.0,-555555.0,1.0,1.0,1.0,...,-555555.0,-555555.0,-555555,29,0,1.0,Dubai,Iran,39.0,1
1829,FTKUGGNNWSSWWNTG,2011-12-31,FTKUGGNNWSSWWNTG_2011-12-31,1.0,1.0,-555555.0,-555555.0,2.0,2.0,2.0,...,1.0,24600.0,Government,26,0,0.0,Others,United Arab Emirates,33.0,1
859,FTKUGFNNTFTNPSWF,2011-02-28,FTKUGFNNTFTNPSWF_2011-02-28,-555555.0,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,...,-555555.0,29350.0,Government,16,0,0.0,Abu_Dhabi,United Arab Emirates,25.0,1
1290,FTKUGGNNKWKCGKNS,2011-09-30,FTKUGGNNKWKCGKNS_2011-09-30,6.0,-555555.0,-555555.0,-555555.0,0.0,0.0,1.0,...,-555555.0,19870.0,Government,19,0,0.0,Others,United Arab Emirates,33.0,1
1213,FTKUGGNNNFWCNNNW,2011-03-31,FTKUGGNNNFWCNNNW_2011-03-31,0.0,0.0,18.0,-555555.0,2.0,2.0,2.0,...,-555555.0,-555555.0,-555555,21,0,1.0,Dubai,Iran,61.0,1


In [640]:
train["MASK_CREDITCARD"].nunique()

2165

In [641]:
train["Snapshot_Key"].nunique()

25426

In [642]:
test.shape

(10898, 175)

In [643]:
test.head()

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,...,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age,F_build
824,FTKUGFNNPWKTPFCF,2011-04-30,FTKUGFNNPWKTPFCF_2011-04-30,-555555.0,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,...,-555555.0,230000.0,-555555,6,0,0.0,Abu_Dhabi,United Arab Emirates,47.0,0
1032,FTKUGGNNNFCSNNNN,2010-11-30,FTKUGGNNNFCSNNNN_2010-11-30,2.0,-555555.0,-555555.0,-555555.0,1.0,1.0,1.0,...,-555555.0,-555555.0,-555555,17,0,0.0,Abu_Dhabi,India,42.0,0
1045,FTKUGGNNGFTFKTKS,2011-01-31,FTKUGGNNGFTFKTKS_2011-01-31,8.0,-555555.0,-555555.0,-555555.0,0.0,0.0,1.0,...,-555555.0,30177.0,-555555,10,0,0.0,Abu_Dhabi,United Arab Emirates,32.0,0
650,FTKUGFNNNSUSNNNF,2011-10-31,FTKUGFNNNSUSNNNF_2011-10-31,-555555.0,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,...,-555555.0,-555555.0,-555555,28,0,0.0,Abu_Dhabi,Palestine,-555555.0,0
643,FTKUGFNNNUSCNNNN,2010-12-31,FTKUGFNNNUSCNNNN_2010-12-31,1.0,1.0,13.0,13.0,2.0,2.0,2.0,...,-555555.0,-555555.0,Government,20,0,0.0,Abu_Dhabi,United Arab Emirates,36.0,0


In [644]:
test["MASK_CREDITCARD"].nunique()

2080

In [645]:
test["Snapshot_Key"].nunique()

10898

In [646]:
# check_train_test

In [647]:
# Creating list of categorical , constant and ID cols not required for binning of numerical cols 

In [648]:
train.dtypes

MASK_CREDITCARD                            object
snapshot_date                      datetime64[ns]
Snapshot_Key                               object
month_since_0                             float64
month_since_30                            float64
month_since_60                            float64
month_since_90                            float64
max_dpd_3m                                float64
max_dpd_6m                                float64
max_dpd_12m                               float64
avg_util_3m                               float64
avg_util_6m                               float64
avg_util_12m                              float64
avg_cash_util_3m                          float64
avg_cash_util_6m                          float64
avg_cash_util_12m                         float64
avg_amt_cash_withd_3m                     float64
avg_amt_cash_withd_6m                     float64
avg_amt_cash_withd_12m                    float64
avg_amt_clothing_3m                       float64


In [649]:
list_notreq_num_bin = ['snapshot_date','Snapshot_Key','CARD_LIMIT','OS_BALANCE','NB_DAYS_DELINQ','Customer_Occupation',
                      'A_Gender','A_Marital_Status','A_Industry','A_Emp_Sts','A_Sector','bad_def','REGION','NATIONALITY','F_build']

In [650]:
len(list_notreq_num_bin)

15

In [651]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25426 entries, 1489 to 669
Columns: 175 entries, MASK_CREDITCARD to F_build
dtypes: datetime64[ns](1), float64(112), int32(50), int64(2), object(10)
memory usage: 29.3+ MB


In [652]:
# train['month_since_0'].dtype.kind

In [653]:
# train_dtypes = np.array(train.dtypes)
# train_numericDtypes= [x.kind in 'bifc' for x in train_dtypes]

In [654]:
#train_numericDtypes

In [655]:
# df2 = df1[df1.columns[df1.columns.isin(my_list)]]

In [656]:
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#                                                     stratify=y, 
#                                                     test_size=0.25)

In [657]:
# def missing(df ,cut_off):
#     pct_null =df.isnull().sum() / len(df)*100
#     missing_features = pct_null[pct_null > cut_off].index
#     #missing_features.shape
#     df.drop(missing_features, axis=1, inplace=True) 
    
# #Remove above 95% missing columns
# missing(iv_input, 95) #67 variables are dropped

In [658]:
# #To replace all na & all special values with '-555555'
# def fill_na(df, value):
#     df.fillna(value, inplace=True)
#     df.replace([np.inf,-np.inf],value ,inplace=True)

# fill_na(iv_input ,-555555 )

In [659]:
# iv_input.isnull().sum().sort_values(ascending = False)

In [660]:
# def cal_badrate(col):
#     bad_rate = str((col.sum() / col.count())*100)
#     print('Bad rate is : ' + bad_rate )
#     print(col.value_counts())
    
# cal_badrate(iv_input.bad_flag)

Information Value

In [661]:
def iv_woe(data, target, IDcol,exclcol, bins=40, show_woe=True):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    # Excluding columns not required for numerical binning 
    data = data[data.columns[~data.columns.isin(exclcol)]]
    
    #Extract Column Names
    cols = data.columns

    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target,IDcol])]:
        if (data[ivars].dtype.kind in 'bifc'):
            d0 = pd.DataFrame({'x':data[ivars], 'decile': pd.qcut(data[ivars], bins, duplicates='drop') , 'y': data[target]})
              
        else:
            d0 = pd.DataFrame({'x':data[ivars], 'decile': data[ivars], 'y': data[target]})
        
        d0['decile'] = np.where(d0['x'] == -555555, '-555555' ,d0['decile'])
        d = d0.groupby('decile', as_index= False).agg({"y": ["count", "sum"]})
        d3 = d0.assign(Min = d0['x'], Max = d0['x']).groupby('decile', as_index= False).agg({"Min": 'min', "Max":'max'})
        
        d.columns = ['decile', 'N', 'bad']
        d['pop_perc'] = (d['N']/d['N'].sum())*100
        #d['bad_perc'] = np.maximum(d['bad'], 0.005) / d['bad'].sum()
        d['bad_perc'] = d['bad'] / d['bad'].sum()
        d['good'] = d['N'] - d['bad']
        #d['good_perc'] = np.maximum(d['good'], 0.005) / d['good'].sum()
        d['good_perc'] = d['good']/ d['good'].sum()
        d['bad_rate'] =  (d['bad']/d['N'])*100
        d['woe'] = np.log(d['good_perc']/d['bad_perc'])
        d['diff_good_bad_perc'] = d['good_perc'] - d['bad_perc']
        d['contribution'] = d['woe'] * d['diff_good_bad_perc']
        d['IV'] = d['contribution'].sum()
        
        d.insert(loc=0, column='Variable', value=ivars)
        d4 = pd.merge(left=d, right=d3,on = 'decile', how='left')
        
        #print("Information value of " + ivars + " is " + str(round(d4['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d4['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d4], axis=0)
        woeDF.sort_values(['Variable','Max'], inplace = True)
        #woeDF = woeDF[['Variable','N','decile','bad','good','bad_perc','bad_perc1','good_perc','good_perc1','diff_good_bad_perc','Min','Max','pop_perc','bad_rate','woe','contribution','IV']]
        woeDF = woeDF[['Variable','decile','bad','good','Min','Max','pop_perc','bad_rate','woe','contribution','IV']]


        #Show WOE Table
        if show_woe == True:
            print(d4)
    return newDF, woeDF

In [662]:
iv, woe = iv_woe(data =train , target ='bad_flag', IDcol='MASK_CREDITCARD',exclcol = list_notreq_num_bin , bins=40, show_woe = False)

In [663]:
iv

,Variable,IV
0,month_since_0,17.202128
0,month_since_30,19.770336
0,month_since_60,6.111314
0,month_since_90,1.362891
0,max_dpd_3m,5.478813
0,max_dpd_6m,6.002194
0,max_dpd_12m,7.690968
0,avg_util_3m,16.647240
0,avg_util_6m,12.322406
0,avg_util_12m,7.689259


In [664]:
woe

,Variable,decile,bad,good,Min,Max,pop_perc,bad_rate,woe,contribution,IV
7,A_No_Depend,-555555,999.0,15693.0,-555555.000000,-555555.000000,65.649335,5.984903,-0.068223,0.003150,0.029075
0,A_No_Depend,"(-555555.001, 1.0]",107.0,1831.0,1.000000,1.000000,7.622119,5.521156,0.017351,0.000023,0.029075
1,A_No_Depend,"(1.0, 2.0]",96.0,2132.0,2.000000,2.000000,8.762684,4.308797,0.278030,0.005995,0.029075
2,A_No_Depend,"(2.0, 3.0]",89.0,1605.0,3.000000,3.000000,6.662471,5.253837,0.069805,0.000315,0.029075
3,A_No_Depend,"(3.0, 4.0]",37.0,1075.0,4.000000,4.000000,4.373476,3.327338,0.546720,0.010314,0.029075
...,...,...,...,...,...,...,...,...,...,...,...
1,nbr_pay_gt_mindue_6m,"(0.167, 0.333]",284.0,2298.0,0.333333,0.333333,10.154959,10.999225,-0.731618,0.075550,0.363225
2,nbr_pay_gt_mindue_6m,"(0.333, 0.5]",374.0,5137.0,0.500000,0.500000,21.674664,6.786427,-0.202469,0.009726,0.363225
3,nbr_pay_gt_mindue_6m,"(0.5, 0.667]",267.0,5273.0,0.666667,0.666667,21.788720,4.819495,0.160668,0.005240,0.363225
4,nbr_pay_gt_mindue_6m,"(0.667, 0.833]",120.0,4481.0,0.833333,0.833333,18.095650,2.608129,0.797672,0.081860,0.363225


In [665]:
iv.shape

(158, 2)

In [666]:
train_num_bin = train[train.columns[~train.columns.isin(list_notreq_num_bin)]]

In [667]:
train_num_bin.shape

(25426, 160)

In [668]:
train_num_bin.dtypes

MASK_CREDITCARD                     object
month_since_0                      float64
month_since_30                     float64
month_since_60                     float64
month_since_90                     float64
max_dpd_3m                         float64
max_dpd_6m                         float64
max_dpd_12m                        float64
avg_util_3m                        float64
avg_util_6m                        float64
avg_util_12m                       float64
avg_cash_util_3m                   float64
avg_cash_util_6m                   float64
avg_cash_util_12m                  float64
avg_amt_cash_withd_3m              float64
avg_amt_cash_withd_6m              float64
avg_amt_cash_withd_12m             float64
avg_amt_clothing_3m                float64
avg_amt_clothing_6m                float64
avg_amt_clothing_12m               float64
avg_perc_amt_clothing_3m           float64
avg_perc_amt_clothing_6m           float64
avg_perc_amt_clothing_12m          float64
avg_amt_uti

In [669]:

train_num_bin[["Cons_inc_delinq_3m", "bad_flag"]].value_counts()

Cons_inc_delinq_3m  bad_flag
0                   0.0         17293
1                   0.0          6471
                    1.0           706
0                   1.0           464
2                   1.0           257
                    0.0           235
dtype: int64

In [670]:
train_num_bin[["Cons_inc_delinq_3m"]].value_counts()

Cons_inc_delinq_3m
0                     17757
1                      7177
2                       492
dtype: int64

In [671]:
train_num_bin[["bad_flag"]].value_counts()

bad_flag
0.0         23999
1.0          1427
dtype: int64

In [672]:
granular = train_num_bin.groupby(['Cons_inc_delinq_6m','bad_flag'])['MASK_CREDITCARD'].count()

In [673]:
# result = df.groupby(['Courses','Fee']).count()

In [674]:
granular =granular.to_frame(name ='count') 

In [675]:
granular.reset_index(inplace=True)

In [676]:
granular

,Cons_inc_delinq_6m,bad_flag,count
0,0,0.0,13285
1,0,1.0,203
2,1,0.0,9691
3,1,1.0,700
4,2,0.0,886
5,2,1.0,371
6,3,0.0,119
7,3,1.0,142
8,4,0.0,18
9,4,1.0,11


In [677]:
type(granular)

pandas.core.frame.DataFrame

In [678]:
iv

,Variable,IV
0,month_since_0,17.202128
0,month_since_30,19.770336
0,month_since_60,6.111314
0,month_since_90,1.362891
0,max_dpd_3m,5.478813
0,max_dpd_6m,6.002194
0,max_dpd_12m,7.690968
0,avg_util_3m,16.647240
0,avg_util_6m,12.322406
0,avg_util_12m,7.689259


In [679]:
iv.sort_values(by = 'IV',ascending=False,inplace=True)
iv = iv.set_index("Variable")
iv

,IV
Variable,
avg_perc_amt_hotel_12m,inf
avg_amt_FI_12m,inf
A_salary,inf
Tot_Cons_inc_pay_mindue_6m,inf
Tot_Cons_inc_pay_exp_6m,inf
Tot_Cons_dec_delinq_6m,inf
avg_amt_autocashdisb_6m,inf
avg_amt_cash_withd_6m,inf
avg_amt_education_12m,inf


In [680]:
iv.reset_index(inplace=True)

In [681]:
iv

,Variable,IV
0,avg_perc_amt_hotel_12m,inf
1,avg_amt_FI_12m,inf
2,A_salary,inf
3,Tot_Cons_inc_pay_mindue_6m,inf
4,Tot_Cons_inc_pay_exp_6m,inf
5,Tot_Cons_dec_delinq_6m,inf
6,avg_amt_autocashdisb_6m,inf
7,avg_amt_cash_withd_6m,inf
8,avg_amt_education_12m,inf
9,avg_perc_amt_FI_6m,inf


In [682]:
iv.shape

(158, 2)

In [683]:
#Dropping variables based on IV < 0.02
weak_pred = iv[iv['IV']< 0.02]
print(weak_pred.shape) #6 variables
weak = list(weak_pred['Variable'])

(6, 2)


In [684]:
weak

['avg_perc_amt_education_3m',
 'nbr_90_3m',
 'avg_echannel_12m',
 'avg_echannel_6m',
 'nbr_90_6m',
 'avg_echannel_3m']

In [685]:
train_num_bin.drop(weak ,axis=1, inplace = True)

In [686]:
train_num_bin.shape

(25426, 154)

In [687]:
train_num_bin.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25426 entries, 1489 to 669
Columns: 154 entries, MASK_CREDITCARD to Age
dtypes: float64(104), int32(49), object(1)
memory usage: 25.3+ MB


In [688]:
list(train_num_bin.columns)

['MASK_CREDITCARD',
 'month_since_0',
 'month_since_30',
 'month_since_60',
 'month_since_90',
 'max_dpd_3m',
 'max_dpd_6m',
 'max_dpd_12m',
 'avg_util_3m',
 'avg_util_6m',
 'avg_util_12m',
 'avg_cash_util_3m',
 'avg_cash_util_6m',
 'avg_cash_util_12m',
 'avg_amt_cash_withd_3m',
 'avg_amt_cash_withd_6m',
 'avg_amt_cash_withd_12m',
 'avg_amt_clothing_3m',
 'avg_amt_clothing_6m',
 'avg_amt_clothing_12m',
 'avg_perc_amt_clothing_3m',
 'avg_perc_amt_clothing_6m',
 'avg_perc_amt_clothing_12m',
 'avg_amt_utility_bills_3m',
 'avg_amt_utility_bills_6m',
 'avg_amt_utility_bills_12m',
 'avg_perc_amt_utility_bills_3m',
 'avg_perc_amt_utility_bills_6m',
 'avg_perc_amt_utility_bills_12m',
 'avg_amt_jewellery_3m',
 'avg_amt_jewellery_6m',
 'avg_amt_jewellery_12m',
 'avg_perc_amt_jewellery_3m',
 'avg_perc_amt_jewellery_6m',
 'avg_perc_amt_jewellery_12m',
 'avg_amt_airline_3m',
 'avg_amt_airline_6m',
 'avg_amt_airline_12m',
 'avg_perc_amt_airline_3m',
 'avg_perc_amt_airline_6m',
 'avg_perc_amt_airline

In [689]:
# Removing features with missing percentage >95%

In [690]:
def missing(df ,cut_off):
    #pct_null =df.isnull().sum() / len(df)*100
    pct_null =(train_num_bin.isin([-555555]).sum()/len(train_num_bin))*100
    missing_features = pct_null[pct_null > cut_off].index
    #missing_features.shape
    df.drop(missing_features, axis=1, inplace=True) 

In [691]:
# #Remove above 95% missing columns
missing(train_num_bin, 95) #3 variables are dropped

In [692]:
# Left columns after dropping IV <0.02 and missing% >95%

In [693]:
train_num_bin.shape

(25426, 151)

In [694]:
list(train_num_bin.columns)

['MASK_CREDITCARD',
 'month_since_0',
 'month_since_30',
 'month_since_60',
 'max_dpd_3m',
 'max_dpd_6m',
 'max_dpd_12m',
 'avg_util_3m',
 'avg_util_6m',
 'avg_util_12m',
 'avg_cash_util_3m',
 'avg_cash_util_6m',
 'avg_cash_util_12m',
 'avg_amt_cash_withd_3m',
 'avg_amt_cash_withd_6m',
 'avg_amt_cash_withd_12m',
 'avg_amt_clothing_3m',
 'avg_amt_clothing_6m',
 'avg_amt_clothing_12m',
 'avg_perc_amt_clothing_3m',
 'avg_perc_amt_clothing_6m',
 'avg_perc_amt_clothing_12m',
 'avg_amt_utility_bills_3m',
 'avg_amt_utility_bills_6m',
 'avg_amt_utility_bills_12m',
 'avg_perc_amt_utility_bills_3m',
 'avg_perc_amt_utility_bills_6m',
 'avg_perc_amt_utility_bills_12m',
 'avg_amt_jewellery_3m',
 'avg_amt_jewellery_6m',
 'avg_amt_jewellery_12m',
 'avg_perc_amt_jewellery_3m',
 'avg_perc_amt_jewellery_6m',
 'avg_perc_amt_jewellery_12m',
 'avg_amt_airline_3m',
 'avg_amt_airline_6m',
 'avg_amt_airline_12m',
 'avg_perc_amt_airline_3m',
 'avg_perc_amt_airline_6m',
 'avg_perc_amt_airline_12m',
 'avg_amt_ed

In [695]:
iv = iv[iv['Variable'].isin(train_num_bin.columns)]

In [696]:
iv

,Variable,IV
0,avg_perc_amt_hotel_12m,inf
1,avg_amt_FI_12m,inf
2,A_salary,inf
3,Tot_Cons_inc_pay_mindue_6m,inf
4,Tot_Cons_inc_pay_exp_6m,inf
5,Tot_Cons_dec_delinq_6m,inf
6,avg_amt_autocashdisb_6m,inf
7,avg_amt_cash_withd_6m,inf
8,avg_amt_education_12m,inf
9,avg_perc_amt_FI_6m,inf


In [697]:
iv.shape

(149, 2)

In [698]:
iv_raw = woe[woe['Variable'].isin(train_num_bin.columns)]

In [699]:
iv_raw.shape

(1546, 11)

In [700]:
iv_raw['Variable'].nunique()

149

In [701]:
iv_raw.tail(101)

,Variable,decile,bad,good,Min,Max,pop_perc,bad_rate,woe,contribution,IV
0,nbr_30_3m,"(-0.001, 0.333]",1076.0,23629.0,0.000000,0.333333,97.164320,4.355394,0.266786,0.061508,0.700144
1,nbr_30_3m,"(0.333, 0.667]",267.0,285.0,0.666667,0.666667,2.171006,48.369565,-2.757197,0.483145,0.700144
2,nbr_30_3m,"(0.667, 1.0]",84.0,85.0,1.000000,1.000000,0.664674,49.704142,-2.810603,0.155491,0.700144
0,nbr_30_6m,"(-0.001, 0.167]",814.0,23011.0,0.000000,0.166667,93.703296,3.416579,0.519329,0.201710,1.142691
1,nbr_30_6m,"(0.167, 0.333]",228.0,525.0,0.333333,0.333333,2.961535,30.278884,-1.988385,0.274198,1.142691
2,nbr_30_6m,"(0.333, 0.5]",168.0,249.0,0.500000,0.500000,1.640053,40.287770,-2.428949,0.260758,1.142691
3,nbr_30_6m,"(0.5, 1.0]",217.0,214.0,0.666667,1.000000,1.695115,50.348028,-2.836359,0.406025,1.142691
0,nbr_60_12m,"(-0.001, 0.0833]",994.0,23302.0,0.000000,0.083333,95.555730,4.091208,0.332119,0.091131,0.742249
1,nbr_60_12m,"(0.0833, 0.182]",168.0,343.0,0.090909,0.181818,2.009754,32.876712,-2.108671,0.218115,0.742249
2,nbr_60_12m,"(0.182, 1.0]",265.0,354.0,0.200000,1.000000,2.434516,42.810985,-2.532871,0.433004,0.742249


In [702]:
iv_raw.loc[iv_raw["Variable"]=="nbr_30_3m"].to_dict()

{'Variable': {0: 'nbr_30_3m', 1: 'nbr_30_3m', 2: 'nbr_30_3m'},
 'decile': {0: Interval(-0.001, 0.333, closed='right'),
  1: Interval(0.333, 0.667, closed='right'),
  2: Interval(0.667, 1.0, closed='right')},
 'bad': {0: 1076.0, 1: 267.0, 2: 84.0},
 'good': {0: 23629.0, 1: 285.0, 2: 85.0},
 'Min': {0: 0.0, 1: 0.6666666667, 2: 1.0},
 'Max': {0: 0.3333333333, 1: 0.6666666667, 2: 1.0},
 'pop_perc': {0: 97.16431998741446,
  1: 2.1710060567922596,
  2: 0.6646739557932825},
 'bad_rate': {0: 4.355393645011131,
  1: 48.369565217391305,
  2: 49.70414201183432},
 'woe': {0: 0.2667864851593384, 1: -2.757197302450099, 2: -2.810603366671497},
 'contribution': {0: 0.061508493495880494,
  1: 0.48314456998370425,
  2: 0.15549083294163532},
 'IV': {0: 0.7001438964212201, 1: 0.7001438964212201, 2: 0.7001438964212201}}

In [703]:
pd.set_option('display.max_columns', None)

In [704]:
iv_raw.loc[iv_raw["Variable"]=="nbr_30_3m"]

,Variable,decile,bad,good,Min,Max,pop_perc,bad_rate,woe,contribution,IV
0,nbr_30_3m,"(-0.001, 0.333]",1076.0,23629.0,0.000000,0.333333,97.164320,4.355394,0.266786,0.061508,0.700144
1,nbr_30_3m,"(0.333, 0.667]",267.0,285.0,0.666667,0.666667,2.171006,48.369565,-2.757197,0.483145,0.700144
2,nbr_30_3m,"(0.667, 1.0]",84.0,85.0,1.000000,1.000000,0.664674,49.704142,-2.810603,0.155491,0.700144


In [705]:
#Exporting data to the excel sheet
iv_raw.to_excel("Iv_raw_sheet.xlsx",sheet_name='IV_sheet')

In [706]:
# Variables where granular level iv and woe is required 

In [707]:
list_granular_iv_woe = ['MASK_CREDITCARD','bad_flag','Cons_inc_delinq_6m','Cons_inc_delinq_3m','Cons_inc_delinq_12m',
                        'Cons_dec_util_3m','Cons_inc_pay_exp_3m','Cons_inc_pay_mindue_3m','Cons_inc_util_3m',
                        'Tot_Cons_inc_delinq_3m','Tot_Cons_inc_pay_exp_3m','Tot_Cons_inc_pay_mindue_3m','Tot_Cons_inc_util_3m',
                        'max_dpd_12m','max_dpd_3m','max_dpd_6m','nbr_0_3m','nbr_30_12m','nbr_30_3m','nbr_30_6m',
                        'nbr_60_12m','nbr_60_3m','nbr_60_6m','nbr_90_12m','month_since_0','month_since_30']

In [708]:
def iv_woe1(data, target, IDcol,inclcol, show_woe=True):
    
    #Empty Dataframe
    newDF,woeDF = pd.DataFrame(), pd.DataFrame()
    
    # Excluding columns not required for numerical binning 
    data = data[data.columns[data.columns.isin(inclcol)]]
    
    #Extract Column Names
    cols = data.columns

    
    #Run WOE and IV on all the independent variables
    for ivars in cols[~cols.isin([target,IDcol])]:
#         if (data[ivars].dtype.kind in 'bifc'):
#             d0 = pd.DataFrame({'x':data[ivars], 'decile': pd.qcut(data[ivars], bins, duplicates='drop') , 'y': data[target]})
              
#         else:
        d0 = pd.DataFrame({'x':data[ivars], 'decile': data[ivars], 'y': data[target]})
        
        d0['decile'] = np.where(d0['x'] == -555555, '-555555' ,d0['decile'])
        d = d0.groupby('decile', as_index= False).agg({"y": ["count", "sum"]})
        d3 = d0.assign(Min = d0['x'], Max = d0['x']).groupby('decile', as_index= False).agg({"Min": 'min', "Max":'max'})
        
        d.columns = ['decile', 'N', 'bad']
        d['pop_perc'] = (d['N']/d['N'].sum())*100
        #d['bad_perc'] = np.maximum(d['bad'], 0.005) / d['bad'].sum()
        d['bad_perc'] = d['bad'] / d['bad'].sum()
        d['good'] = d['N'] - d['bad']
        #d['good_perc'] = np.maximum(d['good'], 0.005) / d['good'].sum()
        d['good_perc'] = d['good']/ d['good'].sum()
        d['bad_rate'] =  (d['bad']/d['N'])*100
        d['woe'] = np.log(d['good_perc']/d['bad_perc'])
        d['diff_good_bad_perc'] = d['good_perc'] - d['bad_perc']
        d['contribution'] = d['woe'] * d['diff_good_bad_perc']
        d['IV'] = d['contribution'].sum()
        
        d.insert(loc=0, column='Variable', value=ivars)
        d4 = pd.merge(left=d, right=d3,on = 'decile', how='left')
        
        #print("Information value of " + ivars + " is " + str(round(d4['IV'].sum(),6)))
        temp =pd.DataFrame({"Variable" : [ivars], "IV" : [d4['IV'].sum()]}, columns = ["Variable", "IV"])
        newDF=pd.concat([newDF,temp], axis=0)
        woeDF=pd.concat([woeDF,d4], axis=0)
        woeDF.sort_values(['Variable','Max'], inplace = True)
        #woeDF = woeDF[['Variable','N','decile','bad','good','bad_perc','bad_perc1','good_perc','good_perc1','diff_good_bad_perc','Min','Max','pop_perc','bad_rate','woe','contribution','IV']]
        woeDF = woeDF[['Variable','decile','bad','good','Min','Max','pop_perc','bad_rate','woe','contribution','IV']]


        #Show WOE Table
        if show_woe == True:
            print(d4)
    return newDF, woeDF

In [709]:
iv1, woe1 = iv_woe1(data = train_num_bin , target ='bad_flag', IDcol='MASK_CREDITCARD',inclcol = list_granular_iv_woe , show_woe = False)

In [710]:
iv1

,Variable,IV
0,month_since_0,inf
0,month_since_30,inf
0,max_dpd_3m,17.848752
0,max_dpd_6m,18.059361
0,max_dpd_12m,17.577890
0,nbr_0_3m,6.173656
0,nbr_30_3m,5.406254
0,nbr_30_6m,10.652586
0,nbr_30_12m,inf
0,nbr_60_3m,2.789512


In [711]:
woe1

,Variable,decile,bad,good,Min,Max,pop_perc,bad_rate,woe,contribution,IV
0,Cons_dec_util_3m,0,478.0,4761.0,0.000000,0.000000,20.604893,9.123879,-0.523836,0.071548,0.142863
1,Cons_dec_util_3m,1,783.0,13879.0,1.000000,1.000000,57.665382,5.340336,0.052562,0.001556,0.142863
2,Cons_dec_util_3m,2,166.0,5359.0,2.000000,2.000000,21.729725,3.004525,0.652107,0.069758,0.142863
0,Cons_inc_delinq_12m,0,119.0,10475.0,0.000000,0.000000,41.666011,1.123277,1.655185,0.584421,1.470915
1,Cons_inc_delinq_12m,1,525.0,11145.0,1.000000,1.000000,45.897900,4.498715,0.232910,0.022473,1.470915
2,Cons_inc_delinq_12m,2,430.0,1845.0,2.000000,2.000000,8.947534,18.901099,-1.365988,0.306601,1.470915
3,Cons_inc_delinq_12m,3,248.0,399.0,3.000000,3.000000,2.544639,38.330757,-2.346905,0.368852,1.470915
4,Cons_inc_delinq_12m,4,47.0,73.0,4.000000,4.000000,0.471958,39.166667,-2.382126,0.071212,1.470915
5,Cons_inc_delinq_12m,5,30.0,43.0,5.000000,5.000000,0.287108,41.095890,-2.462435,0.047356,1.470915
6,Cons_inc_delinq_12m,6,8.0,14.0,6.000000,6.000000,0.086526,36.363636,-2.262822,0.011366,1.470915


In [712]:
woe1.to_excel("Iv_raw_sheet_granular.xlsx",sheet_name='IV_sheet')

In [713]:
#check1_break

In [714]:
train_num_bin['MASK_CREDITCARD'].nunique()

2165

In [715]:
list_req_cat_bin = ['MASK_CREDITCARD','bad_flag','Customer_Occupation','A_Gender','A_Marital_Status','A_Industry','A_Emp_Sts',
                     'A_Sector','REGION','NATIONALITY']

In [716]:
iv2, woe2 = iv_woe1(data = train , target ='bad_flag', IDcol='MASK_CREDITCARD',inclcol = list_req_cat_bin , show_woe = False)

In [717]:
iv2

,Variable,IV
0,Customer_Occupation,inf
0,A_Gender,0.072816
0,A_Marital_Status,inf
0,A_Industry,inf
0,A_Emp_Sts,inf
0,A_Sector,inf
0,REGION,0.305656
0,NATIONALITY,inf


In [718]:
woe2

,Variable,decile,bad,good,Min,Max,pop_perc,bad_rate,woe,contribution,IV
0,A_Emp_Sts,-555555,513.0,8840.0,-555555,-555555,36.785181,5.484871,0.024328,0.000215,inf
1,A_Emp_Sts,EMPLOYED,849.0,13947.0,EMPLOYED,EMPLOYED,58.192401,5.738037,-0.023477,0.000324,inf
2,A_Emp_Sts,OTHER,36.0,370.0,OTHER,OTHER,1.596791,8.866995,-0.492454,0.004831,inf
3,A_Emp_Sts,RETIRED,0.0,323.0,RETIRED,RETIRED,1.270353,0.000000,inf,inf,inf
4,A_Emp_Sts,SELF-EMPLOYED,2.0,24.0,SELF-EMPLOYED,SELF-EMPLOYED,0.102258,7.692308,-0.337531,0.000136,inf
...,...,...,...,...,...,...,...,...,...,...,...
33,NATIONALITY,United Kingdom,0.0,46.0,United Kingdom,United Kingdom,0.180917,0.000000,inf,inf,inf
34,NATIONALITY,United States,2.0,53.0,United States,United States,0.216314,3.636364,0.454707,0.000367,inf
0,REGION,Abu_Dhabi,805.0,17046.0,Abu_Dhabi,Abu_Dhabi,70.207661,4.509551,0.230391,0.033674,0.101885
1,REGION,Dubai,402.0,3974.0,Dubai,Dubai,17.210729,9.186472,-0.531361,0.061702,0.101885


In [719]:
woe2.to_excel("Iv_raw_sheet_cat.xlsx",sheet_name='IV_sheet')

In [720]:
train_num_bin.shape

(25426, 151)

In [721]:
train_num_bin

,MASK_CREDITCARD,month_since_0,month_since_30,month_since_60,max_dpd_3m,max_dpd_6m,max_dpd_12m,avg_util_3m,avg_util_6m,avg_util_12m,avg_cash_util_3m,avg_cash_util_6m,avg_cash_util_12m,avg_amt_cash_withd_3m,avg_amt_cash_withd_6m,avg_amt_cash_withd_12m,avg_amt_clothing_3m,avg_amt_clothing_6m,avg_amt_clothing_12m,avg_perc_amt_clothing_3m,avg_perc_amt_clothing_6m,avg_perc_amt_clothing_12m,avg_amt_utility_bills_3m,avg_amt_utility_bills_6m,avg_amt_utility_bills_12m,avg_perc_amt_utility_bills_3m,avg_perc_amt_utility_bills_6m,avg_perc_amt_utility_bills_12m,avg_amt_jewellery_3m,avg_amt_jewellery_6m,avg_amt_jewellery_12m,avg_perc_amt_jewellery_3m,avg_perc_amt_jewellery_6m,avg_perc_amt_jewellery_12m,avg_amt_airline_3m,avg_amt_airline_6m,avg_amt_airline_12m,avg_perc_amt_airline_3m,avg_perc_amt_airline_6m,avg_perc_amt_airline_12m,avg_amt_education_12m,avg_perc_amt_education_6m,avg_perc_amt_education_12m,avg_amt_hotel_3m,avg_amt_hotel_6m,avg_amt_hotel_12m,avg_perc_amt_hotel_3m,avg_perc_amt_hotel_6m,avg_perc_amt_hotel_12m,avg_amt_restaurant_3m,avg_amt_restaurant_6m,avg_amt_restaurant_12m,avg_perc_amt_restaurant_3m,avg_perc_amt_restaurant_6m,avg_perc_amt_restaurant_12m,avg_amt_servicestation_3m,avg_amt_servicestation_6m,avg_amt_servicestation_12m,avg_perc_amt_servicestation_3m,avg_perc_amt_servicestation_6m,avg_perc_amt_servicestation_12m,avg_amt_autocashdisb_3m,avg_amt_autocashdisb_6m,avg_amt_autocashdisb_12m,avg_perc_amt_autocashdisb_3m,avg_perc_amt_autocashdisb_6m,avg_perc_amt_autocashdisb_12m,avg_amt_grocery_3m,avg_amt_grocery_6m,avg_amt_grocery_12m,avg_perc_amt_grocery_3m,avg_perc_amt_grocery_6m,avg_perc_amt_grocery_12m,avg_amt_FI_3m,avg_amt_FI_6m,avg_amt_FI_12m,avg_perc_amt_FI_3m,avg_perc_amt_FI_6m,avg_perc_amt_FI_12m,nbr_late_fee_3m,nbr_late_fee_6m,nbr_late_fee_12m,nbr_cash_trans_3m,nbr_cash_trans_6m,nbr_cash_trans_12m,nbr_0_3m,nbr_0_6m,nbr_0_12m,nbr_30_3m,nbr_30_6m,nbr_30_12m,nbr_60_3m,nbr_60_6m,nbr_60_12m,nbr_90_12m,nbr_pay_gt_mindue_3m,nbr_pay_gt_mindue_6m,nbr_pay_gt_mindue_12m,Cons_inc_delinq_12m,Cons_dec_delinq_12m,Tot_Cons_inc_delinq_12m,Tot_Cons_dec_delinq_12m,Cons_inc_delinq_6m,Cons_dec_delinq_6m,Tot_Cons_inc_delinq_6m,Tot_Cons_dec_delinq_6m,Cons_inc_delinq_3m,Cons_dec_delinq_3m,Tot_Cons_inc_delinq_3m,Tot_Cons_dec_delinq_3m,Cons_inc_util_12m,Cons_dec_util_12m,Tot_Cons_inc_util_12m,Tot_Cons_dec_util_12m,Cons_inc_util_6m,Cons_dec_util_6m,Tot_Cons_inc_util_6m,Tot_Cons_dec_util_6m,Cons_inc_util_3m,Cons_dec_util_3m,Tot_Cons_inc_util_3m,Tot_Cons_dec_util_3m,Cons_inc_pay_exp_12m,Cons_dec_pay_exp_12m,Tot_Cons_inc_pay_exp_12m,Tot_Cons_dec_pay_exp_12m,Cons_inc_pay_exp_6m,Cons_dec_pay_exp_6m,Tot_Cons_inc_pay_exp_6m,Tot_Cons_dec_pay_exp_6m,Cons_inc_pay_exp_3m,Cons_dec_pay_exp_3m,Tot_Cons_inc_pay_exp_3m,Tot_Cons_dec_pay_exp_3m,Cons_inc_pay_mindue_12m,Cons_dec_pay_mindue_12m,Tot_Cons_inc_pay_mindue_12m,Tot_Cons_dec_pay_mindue_12m,Cons_inc_pay_mindue_6m,Cons_dec_pay_mindue_6m,Tot_Cons_inc_pay_mindue_6m,Tot_Cons_dec_pay_mindue_6m,Cons_inc_pay_mindue_3m,Cons_dec_pay_mindue_3m,Tot_Cons_inc_pay_mindue_3m,Tot_Cons_dec_pay_mindue_3m,A_No_Depend,A_salary,mob,bad_flag,Age
1489,FTKUGGNNNSUNNNNT,1.0,-555555.0,-555555.0,1.0,1.0,1.0,0.855200,0.781783,0.800867,0.111111,0.138889,0.069444,2000.0,2500.000000,2500.000000,-555555.0,-555555.0,-555555.00,0.000000,0.000000,0.000000,-555555.0,-555555.0,-555555.0,0.0,0.0,0.0,-555555.0,-555555.0,3000.0,0.000000,0.000000,0.038011,-555555.0,-555555.0,-555555.000000,0.000000,0.000000,0.000000,-555555.0,0.0,0.000000,-555555.0,-555555.0,-555555.0,0.000000,0.000000,0.000000,-555555.0,-555555.00,-555555.000000,0.000000,0.000000,0.000000,-555555.0,-555555.0,-555555.0,0.000000,0.000000,0.000000,2000.0,2500.000000,2500.000000,0.089222,0.122106,0.061053,-555555.0,-555555.0,-555555.000000,0.000000,0.000000,0.000000,986.666667,947.500000,933.750000,0.124723,0.125141,0.119480,0.333333,0.166667,0.166667,0.333333,0.333333,0.166667,0.333333,0.166667,0.083333,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.666667,0.666667,0.833333,1,1,1

In [722]:
list(train_num_bin.columns)

['MASK_CREDITCARD',
 'month_since_0',
 'month_since_30',
 'month_since_60',
 'max_dpd_3m',
 'max_dpd_6m',
 'max_dpd_12m',
 'avg_util_3m',
 'avg_util_6m',
 'avg_util_12m',
 'avg_cash_util_3m',
 'avg_cash_util_6m',
 'avg_cash_util_12m',
 'avg_amt_cash_withd_3m',
 'avg_amt_cash_withd_6m',
 'avg_amt_cash_withd_12m',
 'avg_amt_clothing_3m',
 'avg_amt_clothing_6m',
 'avg_amt_clothing_12m',
 'avg_perc_amt_clothing_3m',
 'avg_perc_amt_clothing_6m',
 'avg_perc_amt_clothing_12m',
 'avg_amt_utility_bills_3m',
 'avg_amt_utility_bills_6m',
 'avg_amt_utility_bills_12m',
 'avg_perc_amt_utility_bills_3m',
 'avg_perc_amt_utility_bills_6m',
 'avg_perc_amt_utility_bills_12m',
 'avg_amt_jewellery_3m',
 'avg_amt_jewellery_6m',
 'avg_amt_jewellery_12m',
 'avg_perc_amt_jewellery_3m',
 'avg_perc_amt_jewellery_6m',
 'avg_perc_amt_jewellery_12m',
 'avg_amt_airline_3m',
 'avg_amt_airline_6m',
 'avg_amt_airline_12m',
 'avg_perc_amt_airline_3m',
 'avg_perc_amt_airline_6m',
 'avg_perc_amt_airline_12m',
 'avg_amt_ed

In [723]:
# IV after removing missing and weak predictors 

In [724]:
#iv, woe = iv_woe(data =train_num_bin , target ='bad_flag', IDcol='MASK_CREDITCARD',exclcol = list_notreq_num_bin , bins=40, show_woe = False)

In [725]:
iv.shape

(149, 2)

In [726]:
iv

,Variable,IV
0,avg_perc_amt_hotel_12m,inf
1,avg_amt_FI_12m,inf
2,A_salary,inf
3,Tot_Cons_inc_pay_mindue_6m,inf
4,Tot_Cons_inc_pay_exp_6m,inf
5,Tot_Cons_dec_delinq_6m,inf
6,avg_amt_autocashdisb_6m,inf
7,avg_amt_cash_withd_6m,inf
8,avg_amt_education_12m,inf
9,avg_perc_amt_FI_6m,inf


In [727]:
iv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149 entries, 0 to 151
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Variable  149 non-null    object 
 1   IV        149 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.5+ KB


In [728]:
iv["w_Variable"] = "w_"+ iv["Variable"]

In [729]:
iv

,Variable,IV,w_Variable
0,avg_perc_amt_hotel_12m,inf,w_avg_perc_amt_hotel_12m
1,avg_amt_FI_12m,inf,w_avg_amt_FI_12m
2,A_salary,inf,w_A_salary
3,Tot_Cons_inc_pay_mindue_6m,inf,w_Tot_Cons_inc_pay_mindue_6m
4,Tot_Cons_inc_pay_exp_6m,inf,w_Tot_Cons_inc_pay_exp_6m
5,Tot_Cons_dec_delinq_6m,inf,w_Tot_Cons_dec_delinq_6m
6,avg_amt_autocashdisb_6m,inf,w_avg_amt_autocashdisb_6m
7,avg_amt_cash_withd_6m,inf,w_avg_amt_cash_withd_6m
8,avg_amt_education_12m,inf,w_avg_amt_education_12m
9,avg_perc_amt_FI_6m,inf,w_avg_perc_amt_FI_6m


In [730]:
# Categorical IV

In [731]:
iv2["w_Variable"] = "w_"+ iv2["Variable"]

In [732]:
iv2

,Variable,IV,w_Variable
0,Customer_Occupation,inf,w_Customer_Occupation
0,A_Gender,0.072816,w_A_Gender
0,A_Marital_Status,inf,w_A_Marital_Status
0,A_Industry,inf,w_A_Industry
0,A_Emp_Sts,inf,w_A_Emp_Sts
0,A_Sector,inf,w_A_Sector
0,REGION,0.305656,w_REGION
0,NATIONALITY,inf,w_NATIONALITY


In [733]:
iv_num_cat = iv.append(iv2)

In [734]:
iv_num_cat

,Variable,IV,w_Variable
0,avg_perc_amt_hotel_12m,inf,w_avg_perc_amt_hotel_12m
1,avg_amt_FI_12m,inf,w_avg_amt_FI_12m
2,A_salary,inf,w_A_salary
3,Tot_Cons_inc_pay_mindue_6m,inf,w_Tot_Cons_inc_pay_mindue_6m
4,Tot_Cons_inc_pay_exp_6m,inf,w_Tot_Cons_inc_pay_exp_6m
5,Tot_Cons_dec_delinq_6m,inf,w_Tot_Cons_dec_delinq_6m
6,avg_amt_autocashdisb_6m,inf,w_avg_amt_autocashdisb_6m
7,avg_amt_cash_withd_6m,inf,w_avg_amt_cash_withd_6m
8,avg_amt_education_12m,inf,w_avg_amt_education_12m
9,avg_perc_amt_FI_6m,inf,w_avg_perc_amt_FI_6m


In [735]:
iv_num_cat.sort_values(by = 'IV',ascending=False,inplace=True)
# iv = iv.set_index("Variable")
# iv

In [736]:
iv_num_cat

,Variable,IV,w_Variable
0,avg_perc_amt_hotel_12m,inf,w_avg_perc_amt_hotel_12m
9,avg_perc_amt_FI_6m,inf,w_avg_perc_amt_FI_6m
0,A_Sector,inf,w_A_Sector
0,A_Emp_Sts,inf,w_A_Emp_Sts
0,A_Industry,inf,w_A_Industry
0,A_Marital_Status,inf,w_A_Marital_Status
0,Customer_Occupation,inf,w_Customer_Occupation
1,avg_amt_FI_12m,inf,w_avg_amt_FI_12m
10,avg_amt_FI_6m,inf,w_avg_amt_FI_6m
0,NATIONALITY,inf,w_NATIONALITY


In [737]:
iv_num_cat.shape

(157, 3)

In [738]:
# iv_check

In [739]:
model_data.shape

(36324, 175)

In [740]:
#before_binning

In [741]:
# Adding woe transformed variables 

In [742]:
model_data['b_nbr_60_12m']=np.select([(model_data['nbr_60_12m']< 0.0833333333) ,
( 0.0833333333 <= model_data['nbr_60_12m'])],['1. Less than 0.08','2. GT0.08'])

model_data['w_nbr_60_12m']=np.select([(model_data['nbr_60_12m']< 0.0833333333) ,
( 0.0833333333 <= model_data['nbr_60_12m'])],[0.504176175139819,-2.16877498309144])

model_data['b_nbr_30_6m']=np.select([(model_data['nbr_30_6m']< 0.1666666667) ,
( 0.1666666667 <= model_data['nbr_30_6m']) & (model_data['nbr_30_6m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_30_6m'])],['1. Less than 0.17','2. GE  0.17 and LT 0.33','3. GT0.33'])

model_data['w_nbr_30_6m']=np.select([(model_data['nbr_30_6m']< 0.1666666667) ,
( 0.1666666667 <= model_data['nbr_30_6m']) & (model_data['nbr_30_6m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_30_6m'])],[0.838593281455951,-1.19442352631146,-2.34512006250684])

model_data['b_nbr_30_3m']=np.select([(model_data['nbr_30_3m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_30_3m'])],['1. Less than 0.33','2. GT0.33'])

model_data['w_nbr_30_3m']=np.select([(model_data['nbr_30_3m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_30_3m'])],[0.653454952881835,-2.21105659845512])

model_data['b_nbr_30_12m']=np.select([(model_data['nbr_30_12m']< 0.0833333333) ,
( 0.0833333333 <= model_data['nbr_30_12m']) & (model_data['nbr_30_12m']< 0.2857142857) ,
( 0.2857142857 <= model_data['nbr_30_12m'])],['1. Less than 0.08','2. GE  0.08 and LT 0.29','3. GT0.29'])

model_data['w_nbr_30_12m']=np.select([(model_data['nbr_30_12m']< 0.0833333333) ,
( 0.0833333333 <= model_data['nbr_30_12m']) & (model_data['nbr_30_12m']< 0.2857142857) ,
( 0.2857142857 <= model_data['nbr_30_12m'])],[1.02446892014789,-1.06841868307329,-2.3397142330732])

model_data['b_nbr_0_3m']=np.select([(model_data['nbr_0_3m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_0_3m']) & (model_data['nbr_0_3m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_0_3m'])],['1. Less than 0.33','2. GE  0.33 and LT 0.67','3. GT0.67'])

model_data['w_nbr_0_3m']=np.select([(model_data['nbr_0_3m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_0_3m']) & (model_data['nbr_0_3m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_0_3m'])],[1.45355384940322,0.127456911020025,-1.41539419614257])

model_data['b_max_dpd_6m']=np.select([(model_data['max_dpd_6m']< 1) ,
( 1 <= model_data['max_dpd_6m']) & (model_data['max_dpd_6m']< 2) ,
( 2 <= model_data['max_dpd_6m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_max_dpd_6m']=np.select([(model_data['max_dpd_6m']< 1) ,
( 1 <= model_data['max_dpd_6m']) & (model_data['max_dpd_6m']< 2) ,
( 2 <= model_data['max_dpd_6m'])],[1.56887992950538,0.310901682739938,-1.85260087174926])

model_data['b_max_dpd_3m']=np.select([(model_data['max_dpd_3m']< 1) ,
( 1 <= model_data['max_dpd_3m']) & (model_data['max_dpd_3m']< 2) ,
( 2 <= model_data['max_dpd_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_max_dpd_3m']=np.select([(model_data['max_dpd_3m']< 1) ,
( 1 <= model_data['max_dpd_3m']) & (model_data['max_dpd_3m']< 2) ,
( 2 <= model_data['max_dpd_3m'])],[1.45355384940322,-0.060129032698468,-2.21105659845512])

model_data['b_max_dpd_12m']=np.select([(model_data['max_dpd_12m']< 1) ,
( 1 <= model_data['max_dpd_12m']) & (model_data['max_dpd_12m']< 2) ,
( 2 <= model_data['max_dpd_12m']) & (model_data['max_dpd_12m']< 3) ,
( 3 <= model_data['max_dpd_12m'])],['1. Less than 1','2. GE  1 and LT 2','3. GE  2 and LT 3','4. GT3'])

model_data['w_max_dpd_12m']=np.select([(model_data['max_dpd_12m']< 1) ,
( 1 <= model_data['max_dpd_12m']) & (model_data['max_dpd_12m']< 2) ,
( 2 <= model_data['max_dpd_12m']) & (model_data['max_dpd_12m']< 3) ,
( 3 <= model_data['max_dpd_12m'])],[1.68704711153518,0.628633625644431,-0.954229182927039,-2.16877498309144])

model_data['b_Tot_Cons_inc_util_3m']=np.select([(model_data['Tot_Cons_inc_util_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_util_3m']) & (model_data['Tot_Cons_inc_util_3m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_util_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Tot_Cons_inc_util_3m']=np.select([(model_data['Tot_Cons_inc_util_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_util_3m']) & (model_data['Tot_Cons_inc_util_3m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_util_3m'])],[0.6011865931311,0.0611006036252319,-0.608373782431807])

model_data['b_Tot_Cons_inc_pay_mindue_3m']=np.select([(model_data['Tot_Cons_inc_pay_mindue_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_pay_mindue_3m']) & (model_data['Tot_Cons_inc_pay_mindue_3m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_mindue_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Tot_Cons_inc_pay_mindue_3m']=np.select([(model_data['Tot_Cons_inc_pay_mindue_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_pay_mindue_3m']) & (model_data['Tot_Cons_inc_pay_mindue_3m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_mindue_3m'])],[-0.494581534595944,0.180545723119494,0.630052851731206])

model_data['b_Tot_Cons_inc_pay_exp_3m']=np.select([(model_data['Tot_Cons_inc_pay_exp_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_pay_exp_3m']) & (model_data['Tot_Cons_inc_pay_exp_3m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_exp_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Tot_Cons_inc_pay_exp_3m']=np.select([(model_data['Tot_Cons_inc_pay_exp_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_pay_exp_3m']) & (model_data['Tot_Cons_inc_pay_exp_3m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_exp_3m'])],[-0.494581534595944,0.180545723119494,0.630052851731206])

model_data['b_Tot_Cons_inc_delinq_3m']=np.select([(model_data['Tot_Cons_inc_delinq_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_delinq_3m'])],['1. Less than 1','2. GT1'])

model_data['w_Tot_Cons_inc_delinq_3m']=np.select([(model_data['Tot_Cons_inc_delinq_3m']< 1) ,
( 1 <= model_data['Tot_Cons_inc_delinq_3m'])],[0.795734697781167,-0.881733309090452])

model_data['b_Cons_inc_util_3m']=np.select([(model_data['Cons_inc_util_3m']< 1) ,
( 1 <= model_data['Cons_inc_util_3m']) & (model_data['Cons_inc_util_3m']< 2) ,
( 2 <= model_data['Cons_inc_util_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Cons_inc_util_3m']=np.select([(model_data['Cons_inc_util_3m']< 1) ,
( 1 <= model_data['Cons_inc_util_3m']) & (model_data['Cons_inc_util_3m']< 2) ,
( 2 <= model_data['Cons_inc_util_3m'])],[0.6011865931311,0.0611006036252319,-0.608373782431807])

model_data['b_Cons_inc_pay_mindue_3m']=np.select([(model_data['Cons_inc_pay_mindue_3m']< 1) ,
( 1 <= model_data['Cons_inc_pay_mindue_3m']) & (model_data['Cons_inc_pay_mindue_3m']< 2) ,
( 2 <= model_data['Cons_inc_pay_mindue_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Cons_inc_pay_mindue_3m']=np.select([(model_data['Cons_inc_pay_mindue_3m']< 1) ,
( 1 <= model_data['Cons_inc_pay_mindue_3m']) & (model_data['Cons_inc_pay_mindue_3m']< 2) ,
( 2 <= model_data['Cons_inc_pay_mindue_3m'])],[-0.494581534595944,0.180545723119494,0.630052851731206])

model_data['b_Cons_inc_pay_exp_3m']=np.select([(model_data['Cons_inc_pay_exp_3m']< 1) ,
( 1 <= model_data['Cons_inc_pay_exp_3m']) & (model_data['Cons_inc_pay_exp_3m']< 2) ,
( 2 <= model_data['Cons_inc_pay_exp_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Cons_inc_pay_exp_3m']=np.select([(model_data['Cons_inc_pay_exp_3m']< 1) ,
( 1 <= model_data['Cons_inc_pay_exp_3m']) & (model_data['Cons_inc_pay_exp_3m']< 2) ,
( 2 <= model_data['Cons_inc_pay_exp_3m'])],[-0.494581534595944,0.180545723119494,0.630052851731206])

model_data['b_Cons_inc_delinq_6m']=np.select([(model_data['Cons_inc_delinq_6m']< 1) ,
( 1 <= model_data['Cons_inc_delinq_6m']) & (model_data['Cons_inc_delinq_6m']< 2) ,
( 2 <= model_data['Cons_inc_delinq_6m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Cons_inc_delinq_6m']=np.select([(model_data['Cons_inc_delinq_6m']< 1) ,
( 1 <= model_data['Cons_inc_delinq_6m']) & (model_data['Cons_inc_delinq_6m']< 2) ,
( 2 <= model_data['Cons_inc_delinq_6m'])],[1.35874705483363,-0.194565260627355,-2.15343474268792])

model_data['b_Cons_inc_delinq_3m']=np.select([(model_data['Cons_inc_delinq_3m']< 1) ,
( 1 <= model_data['Cons_inc_delinq_3m'])],['1. Less than 1','2. GT1'])

model_data['w_Cons_inc_delinq_3m']=np.select([(model_data['Cons_inc_delinq_3m']< 1) ,
( 1 <= model_data['Cons_inc_delinq_3m'])],[0.795734697781167,-0.881733309090452])

model_data['b_Cons_inc_delinq_12m']=np.select([(model_data['Cons_inc_delinq_12m']< 1) ,
( 1 <= model_data['Cons_inc_delinq_12m']) & (model_data['Cons_inc_delinq_12m']< 2) ,
( 2 <= model_data['Cons_inc_delinq_12m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Cons_inc_delinq_12m']=np.select([(model_data['Cons_inc_delinq_12m']< 1) ,
( 1 <= model_data['Cons_inc_delinq_12m']) & (model_data['Cons_inc_delinq_12m']< 2) ,
( 2 <= model_data['Cons_inc_delinq_12m'])],[1.65518542736031,0.232910158909846,-1.71113501000635])

model_data['b_Cons_dec_util_3m']=np.select([(model_data['Cons_dec_util_3m']< 1) ,
( 1 <= model_data['Cons_dec_util_3m']) & (model_data['Cons_dec_util_3m']< 2) ,
( 2 <= model_data['Cons_dec_util_3m'])],['1. Less than 1','2. GE  1 and LT 2','3. GT2'])

model_data['w_Cons_dec_util_3m']=np.select([(model_data['Cons_dec_util_3m']< 1) ,
( 1 <= model_data['Cons_dec_util_3m']) & (model_data['Cons_dec_util_3m']< 2) ,
( 2 <= model_data['Cons_dec_util_3m'])],[-0.523835547615437,0.0525616650465369,0.652107056819807])

In [743]:
model_data['b_nbr_pay_gt_mindue_6m']=np.select([(model_data['nbr_pay_gt_mindue_6m']< 0.5) ,
( 0.5 <= model_data['nbr_pay_gt_mindue_6m']) & (model_data['nbr_pay_gt_mindue_6m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_pay_gt_mindue_6m']) & (model_data['nbr_pay_gt_mindue_6m']< 0.8333333333) ,
( 0.8333333333 <= model_data['nbr_pay_gt_mindue_6m'])],['1. Less than 0.5','2. GE  0.5 and LT 0.67','3. GE  0.67 and LT 0.83','4. GT0.83'])

model_data['w_nbr_pay_gt_mindue_6m']=np.select([(model_data['nbr_pay_gt_mindue_6m']< 0.5) ,
( 0.5 <= model_data['nbr_pay_gt_mindue_6m']) & (model_data['nbr_pay_gt_mindue_6m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_pay_gt_mindue_6m']) & (model_data['nbr_pay_gt_mindue_6m']< 0.8333333333) ,
( 0.8333333333 <= model_data['nbr_pay_gt_mindue_6m'])],[-0.691423337167794,-0.202469091265764,0.16066825681172,0.925055623331475])

model_data['b_nbr_pay_gt_mindue_3m']=np.select([(model_data['nbr_pay_gt_mindue_3m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_pay_gt_mindue_3m']) & (model_data['nbr_pay_gt_mindue_3m']< 1) ,
( 1 <= model_data['nbr_pay_gt_mindue_3m'])],['1. Less than 0.67','2. GE  0.67 and LT 1','3. GT1'])

model_data['w_nbr_pay_gt_mindue_3m']=np.select([(model_data['nbr_pay_gt_mindue_3m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_pay_gt_mindue_3m']) & (model_data['nbr_pay_gt_mindue_3m']< 1) ,
( 1 <= model_data['nbr_pay_gt_mindue_3m'])],[-0.565940457181956,0.179336763691454,1.00187338949113])

model_data['b_nbr_pay_gt_mindue_12m']=np.select([(model_data['nbr_pay_gt_mindue_12m']< 0.2727272727) ,
( 0.2727272727 <= model_data['nbr_pay_gt_mindue_12m']) & (model_data['nbr_pay_gt_mindue_12m']< 0.4285714286) ,
( 0.4285714286 <= model_data['nbr_pay_gt_mindue_12m']) & (model_data['nbr_pay_gt_mindue_12m']< 0.5454545455) ,
( 0.5454545455 <= model_data['nbr_pay_gt_mindue_12m']) & (model_data['nbr_pay_gt_mindue_12m']< 0.8571428571) ,
( 0.8571428571 <= model_data['nbr_pay_gt_mindue_12m'])],['1. Less than 0.27','2. GE  0.27 and LT 0.43','3. GE  0.43 and LT 0.55','4. GE  0.55 and LT 0.86','5. GT0.86'])

model_data['w_nbr_pay_gt_mindue_12m']=np.select([(model_data['nbr_pay_gt_mindue_12m']< 0.2727272727) ,
( 0.2727272727 <= model_data['nbr_pay_gt_mindue_12m']) & (model_data['nbr_pay_gt_mindue_12m']< 0.4285714286) ,
( 0.4285714286 <= model_data['nbr_pay_gt_mindue_12m']) & (model_data['nbr_pay_gt_mindue_12m']< 0.5454545455) ,
( 0.5454545455 <= model_data['nbr_pay_gt_mindue_12m']) & (model_data['nbr_pay_gt_mindue_12m']< 0.8571428571) ,
( 0.8571428571 <= model_data['nbr_pay_gt_mindue_12m'])],[-0.908316215537997,-0.39382172109966,-0.180504450155348,0.308183615320474,1.12361492969596])

model_data['b_nbr_late_fee_6m']=np.select([(model_data['nbr_late_fee_6m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_late_fee_6m']) & (model_data['nbr_late_fee_6m']< 0.5) ,
( 0.5 <= model_data['nbr_late_fee_6m']) & (model_data['nbr_late_fee_6m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_late_fee_6m'])],['1. Less than 0.33','2. GE  0.33 and LT 0.5','3. GE  0.5 and LT 0.67','4. GT0.67'])

model_data['w_nbr_late_fee_6m']=np.select([(model_data['nbr_late_fee_6m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_late_fee_6m']) & (model_data['nbr_late_fee_6m']< 0.5) ,
( 0.5 <= model_data['nbr_late_fee_6m']) & (model_data['nbr_late_fee_6m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_late_fee_6m'])],[1.43796573179048,0.501262289989716,-0.323304053240368,-1.66501728380947])

model_data['b_nbr_late_fee_3m']=np.select([(model_data['nbr_late_fee_3m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_late_fee_3m']) & (model_data['nbr_late_fee_3m']< 1) ,
( 1 <= model_data['nbr_late_fee_3m'])],['1. Less than 0.67','2. GE  0.67 and LT 1','3. GT1'])

model_data['w_nbr_late_fee_3m']=np.select([(model_data['nbr_late_fee_3m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_late_fee_3m']) & (model_data['nbr_late_fee_3m']< 1) ,
( 1 <= model_data['nbr_late_fee_3m'])],[1.07320170185912,-0.916591088294381,-2.2314815846551])

model_data['b_nbr_late_fee_12m']=np.select([(model_data['nbr_late_fee_12m']< 0.1) ,
( 0.1 <= model_data['nbr_late_fee_12m']) & (model_data['nbr_late_fee_12m']< 0.1818181818) ,
( 0.1818181818 <= model_data['nbr_late_fee_12m']) & (model_data['nbr_late_fee_12m']< 0.4285714286) ,
( 0.4285714286 <= model_data['nbr_late_fee_12m'])],['1. Less than 0.1','2. GE  0.1 and LT 0.18','3. GE  0.18 and LT 0.43','4. GT0.43'])

model_data['w_nbr_late_fee_12m']=np.select([(model_data['nbr_late_fee_12m']< 0.1) ,
( 0.1 <= model_data['nbr_late_fee_12m']) & (model_data['nbr_late_fee_12m']< 0.1818181818) ,
( 0.1818181818 <= model_data['nbr_late_fee_12m']) & (model_data['nbr_late_fee_12m']< 0.4285714286) ,
( 0.4285714286 <= model_data['nbr_late_fee_12m'])],[1.55389830800947,0.943788398998064,0.393389881490364,-1.29925326569158])

model_data['b_nbr_0_6m']=np.select([(model_data['nbr_0_6m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_0_6m']) & (model_data['nbr_0_6m']< 0.5) ,
( 0.5 <= model_data['nbr_0_6m']) & (model_data['nbr_0_6m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_0_6m'])],['1. Less than 0.33','2. GE  0.33 and LT 0.5','3. GE  0.5 and LT 0.67','4. GT0.67'])

model_data['w_nbr_0_6m']=np.select([(model_data['nbr_0_6m']< 0.3333333333) ,
( 0.3333333333 <= model_data['nbr_0_6m']) & (model_data['nbr_0_6m']< 0.5) ,
( 0.5 <= model_data['nbr_0_6m']) & (model_data['nbr_0_6m']< 0.6666666667) ,
( 0.6666666667 <= model_data['nbr_0_6m'])],[1.36697052125479,0.234008873934659,-0.448835244089174,-1.76682002722325])

model_data['b_nbr_0_12m']=np.select([(model_data['nbr_0_12m']< 0.0909090909) ,
( 0.0909090909 <= model_data['nbr_0_12m']) & (model_data['nbr_0_12m']< 0.1818181818) ,
( 0.1818181818 <= model_data['nbr_0_12m']) & (model_data['nbr_0_12m']< 0.3636363636) ,
( 0.3636363636 <= model_data['nbr_0_12m']) & (model_data['nbr_0_12m']< 0.5454545455) ,
( 0.5454545455 <= model_data['nbr_0_12m'])],['1. Less than 0.09','2. GE  0.09 and LT 0.18','3. GE  0.18 and LT 0.36','4. GE  0.36 and LT 0.55','5. GT0.55'])

model_data['w_nbr_0_12m']=np.select([(model_data['nbr_0_12m']< 0.0909090909) ,
( 0.0909090909 <= model_data['nbr_0_12m']) & (model_data['nbr_0_12m']< 0.1818181818) ,
( 0.1818181818 <= model_data['nbr_0_12m']) & (model_data['nbr_0_12m']< 0.3636363636) ,
( 0.3636363636 <= model_data['nbr_0_12m']) & (model_data['nbr_0_12m']< 0.5454545455) ,
( 0.5454545455 <= model_data['nbr_0_12m'])],[1.47278882751092,0.844462723076661,0.397649387735998,-0.312485142604412,-1.69315194985186])

model_data['b_mob']=np.select([(model_data['mob']< 13) ,
( 13 <= model_data['mob']) & (model_data['mob']< 19) ,
( 19 <= model_data['mob'])],['1. Less than 13','2. GE  13 and LT 19','3. GT19'])

model_data['w_mob']=np.select([(model_data['mob']< 13) ,
( 13 <= model_data['mob']) & (model_data['mob']< 19) ,
( 19 <= model_data['mob'])],[-0.0785961818847511,0.0269997351059413,0.0638491989491141])

model_data['b_avg_util_6m']=np.select([(model_data['avg_util_6m']< 0.3791496667) ,
( 0.3791496667 <= model_data['avg_util_6m']) & (model_data['avg_util_6m']< 0.9430231667) ,
( 0.9430231667 <= model_data['avg_util_6m']) & (model_data['avg_util_6m']< 0.9917385) ,
( 0.9917385 <= model_data['avg_util_6m'])],['1. Less than 0.38','2. GE  0.38 and LT 0.94','3. GE  0.94 and LT 0.99','4. GT0.99'])

model_data['w_avg_util_6m']=np.select([(model_data['avg_util_6m']< 0.3791496667) ,
( 0.3791496667 <= model_data['avg_util_6m']) & (model_data['avg_util_6m']< 0.9430231667) ,
( 0.9430231667 <= model_data['avg_util_6m']) & (model_data['avg_util_6m']< 0.9917385) ,
( 0.9917385 <= model_data['avg_util_6m'])],[0.471361966220752,0.219722323033318,-0.134842314912111,-0.998497360292348])

model_data['b_avg_util_3m']=np.select([(model_data['avg_util_3m']< 0.9816488333) ,
( 0.9816488333 <= model_data['avg_util_3m']) & (model_data['avg_util_3m']< 1.0065208333) ,
( 1.0065208333 <= model_data['avg_util_3m'])],['1. Less than 0.98','2. GE  0.98 and LT 1.01','3. GT1.01'])

model_data['w_avg_util_3m']=np.select([(model_data['avg_util_3m']< 0.9816488333) ,
( 0.9816488333 <= model_data['avg_util_3m']) & (model_data['avg_util_3m']< 1.0065208333) ,
( 1.0065208333 <= model_data['avg_util_3m'])],[0.256051257348918,-0.415063807565586,-1.49219678244051])

model_data['b_avg_util_12m']=np.select([(model_data['avg_util_12m']< 0.292144619) ,
( 0.292144619 <= model_data['avg_util_12m']) & (model_data['avg_util_12m']< 0.9752249286) ,
( 0.9752249286 <= model_data['avg_util_12m']) & (model_data['avg_util_12m']< 0.9897059762) ,
( 0.9897059762 <= model_data['avg_util_12m'])],['1. Less than 0.29','2. GE  0.29 and LT 0.98','3. GE  0.98 and LT 0.99','4. GT0.99'])

model_data['w_avg_util_12m']=np.select([(model_data['avg_util_12m']< 0.292144619) ,
( 0.292144619 <= model_data['avg_util_12m']) & (model_data['avg_util_12m']< 0.9752249286) ,
( 0.9752249286 <= model_data['avg_util_12m']) & (model_data['avg_util_12m']< 0.9897059762) ,
( 0.9897059762 <= model_data['avg_util_12m'])],[1.21688507701339,0.0489731819351951,-0.561543158378663,-0.810205032332114])

model_data['b_avg_perc_amt_utility_bills_6m']=np.select([(model_data['avg_perc_amt_utility_bills_6m']< 0.0003468161) ,
( 0.0003468161 <= model_data['avg_perc_amt_utility_bills_6m']) & (model_data['avg_perc_amt_utility_bills_6m']< 0.0188466381) ,
( 0.0188466381 <= model_data['avg_perc_amt_utility_bills_6m'])],['1. Less than 0','2. GE  0 and LT 0.02','3. GT0.02'])

model_data['w_avg_perc_amt_utility_bills_6m']=np.select([(model_data['avg_perc_amt_utility_bills_6m']< 0.0003468161) ,
( 0.0003468161 <= model_data['avg_perc_amt_utility_bills_6m']) & (model_data['avg_perc_amt_utility_bills_6m']< 0.0188466381) ,
( 0.0188466381 <= model_data['avg_perc_amt_utility_bills_6m'])],[-0.0813694058788498,0.308797580914853,1.17136550868172])

model_data['b_avg_perc_amt_utility_bills_3m']=np.select([(model_data['avg_perc_amt_utility_bills_3m']< 0.0039070136) ,
( 0.0039070136 <= model_data['avg_perc_amt_utility_bills_3m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_utility_bills_3m']=np.select([(model_data['avg_perc_amt_utility_bills_3m']< 0.0039070136) ,
( 0.0039070136 <= model_data['avg_perc_amt_utility_bills_3m'])],[-0.057669052624394,0.727665117217766])

model_data['b_avg_perc_amt_utility_bills_12m']=np.select([(model_data['avg_perc_amt_utility_bills_12m']< 0.0049096407) ,
( 0.0049096407 <= model_data['avg_perc_amt_utility_bills_12m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_utility_bills_12m']=np.select([(model_data['avg_perc_amt_utility_bills_12m']< 0.0049096407) ,
( 0.0049096407 <= model_data['avg_perc_amt_utility_bills_12m'])],[-0.084082273720961,0.656911139234075])

model_data['b_avg_perc_amt_servicestation_6m']=np.select([(model_data['avg_perc_amt_servicestation_6m']< 0.0034416943) ,
( 0.0034416943 <= model_data['avg_perc_amt_servicestation_6m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_servicestation_6m']=np.select([(model_data['avg_perc_amt_servicestation_6m']< 0.0034416943) ,
( 0.0034416943 <= model_data['avg_perc_amt_servicestation_6m'])],[-0.0386507767686891,0.426303347523631])

model_data['b_avg_perc_amt_servicestation_3m']=np.select([(model_data['avg_perc_amt_servicestation_3m']< 0.0028966762) ,
( 0.0028966762 <= model_data['avg_perc_amt_servicestation_3m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_servicestation_3m']=np.select([(model_data['avg_perc_amt_servicestation_3m']< 0.0028966762) ,
( 0.0028966762 <= model_data['avg_perc_amt_servicestation_3m'])],[-0.0402903600511576,0.448805986067017])

model_data['b_avg_perc_amt_servicestation_12m']=np.select([(model_data['avg_perc_amt_servicestation_12m']< 0.0005850735) ,
( 0.0005850735 <= model_data['avg_perc_amt_servicestation_12m']) & (model_data['avg_perc_amt_servicestation_12m']< 0.0042191472) ,
( 0.0042191472 <= model_data['avg_perc_amt_servicestation_12m'])],['1. Less than 0','2. GE  0 and LT 0','3. GT0'])

model_data['w_avg_perc_amt_servicestation_12m']=np.select([(model_data['avg_perc_amt_servicestation_12m']< 0.0005850735) ,
( 0.0005850735 <= model_data['avg_perc_amt_servicestation_12m']) & (model_data['avg_perc_amt_servicestation_12m']< 0.0042191472) ,
( 0.0042191472 <= model_data['avg_perc_amt_servicestation_12m'])],[-0.0901347597665944,0.249871897194874,0.447171998775362])

model_data['b_avg_perc_amt_restaurant_6m']=np.select([(model_data['avg_perc_amt_restaurant_6m']< 0.000511853) ,
( 0.000511853 <= model_data['avg_perc_amt_restaurant_6m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_restaurant_6m']=np.select([(model_data['avg_perc_amt_restaurant_6m']< 0.000511853) ,
( 0.000511853 <= model_data['avg_perc_amt_restaurant_6m'])],[-0.157472898665921,0.679107531177937])

model_data['b_avg_perc_amt_restaurant_3m']=np.select([(model_data['avg_perc_amt_restaurant_3m']< 0.0001060418) ,
( 0.0001060418 <= model_data['avg_perc_amt_restaurant_3m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_restaurant_3m']=np.select([(model_data['avg_perc_amt_restaurant_3m']< 0.0001060418) ,
( 0.0001060418 <= model_data['avg_perc_amt_restaurant_3m'])],[-0.11438762321629,0.79821816667144])

model_data['b_avg_perc_amt_restaurant_12m']=np.select([(model_data['avg_perc_amt_restaurant_12m']< 0.000165417) ,
( 0.000165417 <= model_data['avg_perc_amt_restaurant_12m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_restaurant_12m']=np.select([(model_data['avg_perc_amt_restaurant_12m']< 0.000165417) ,
( 0.000165417 <= model_data['avg_perc_amt_restaurant_12m'])],[-0.240012426484067,0.570337090968959])

model_data['b_avg_perc_amt_jewellery_6m']=np.select([(model_data['avg_perc_amt_jewellery_6m']< 0.0095066508) ,
( 0.0095066508 <= model_data['avg_perc_amt_jewellery_6m'])],['1. Less than 0.01','2. GT0.01'])

model_data['w_avg_perc_amt_jewellery_6m']=np.select([(model_data['avg_perc_amt_jewellery_6m']< 0.0095066508) ,
( 0.0095066508 <= model_data['avg_perc_amt_jewellery_6m'])],[0.0501256096903239,-0.486487037480654])

model_data['b_avg_perc_amt_jewellery_3m']=np.select([(model_data['avg_perc_amt_jewellery_3m']< 0.0125851735) ,
( 0.0125851735 <= model_data['avg_perc_amt_jewellery_3m'])],['1. Less than 0.01','2. GT0.01'])

model_data['w_avg_perc_amt_jewellery_3m']=np.select([(model_data['avg_perc_amt_jewellery_3m']< 0.0125851735) ,
( 0.0125851735 <= model_data['avg_perc_amt_jewellery_3m'])],[0.0306315681998124,-0.465881208413025])

model_data['b_avg_perc_amt_jewellery_12m']=np.select([(model_data['avg_perc_amt_jewellery_12m']< 0.0203942301) ,
( 0.0203942301 <= model_data['avg_perc_amt_jewellery_12m'])],['1. Less than 0.02','2. GT0.02'])

model_data['w_avg_perc_amt_jewellery_12m']=np.select([(model_data['avg_perc_amt_jewellery_12m']< 0.0203942301) ,
( 0.0203942301 <= model_data['avg_perc_amt_jewellery_12m'])],[0.0559256648259136,-0.530191709119633])

model_data['b_avg_perc_amt_hotel_6m']=np.select([(model_data['avg_perc_amt_hotel_6m']< 0.0018294476) ,
( 0.0018294476 <= model_data['avg_perc_amt_hotel_6m']) & (model_data['avg_perc_amt_hotel_6m']< 0.0056944346) ,
( 0.0056944346 <= model_data['avg_perc_amt_hotel_6m']) & (model_data['avg_perc_amt_hotel_6m']< 0.0333761867) ,
( 0.0333761867 <= model_data['avg_perc_amt_hotel_6m'])],['1. Less than 0','2. GE  0 and LT 0.01','3. GE  0.01 and LT 0.03','4. GT0.03'])

model_data['w_avg_perc_amt_hotel_6m']=np.select([(model_data['avg_perc_amt_hotel_6m']< 0.0018294476) ,
( 0.0018294476 <= model_data['avg_perc_amt_hotel_6m']) & (model_data['avg_perc_amt_hotel_6m']< 0.0056944346) ,
( 0.0056944346 <= model_data['avg_perc_amt_hotel_6m']) & (model_data['avg_perc_amt_hotel_6m']< 0.0333761867) ,
( 0.0333761867 <= model_data['avg_perc_amt_hotel_6m'])],[-0.152917371799417,0.233712651400042,0.657992220420447,1.88815169912679])

model_data['b_avg_perc_amt_hotel_3m']=np.select([(model_data['avg_perc_amt_hotel_3m']< 0.0014426007) ,
( 0.0014426007 <= model_data['avg_perc_amt_hotel_3m']) & (model_data['avg_perc_amt_hotel_3m']< 0.0118576611) ,
( 0.0118576611 <= model_data['avg_perc_amt_hotel_3m'])],['1. Less than 0','2. GE  0 and LT 0.01','3. GT0.01'])

model_data['w_avg_perc_amt_hotel_3m']=np.select([(model_data['avg_perc_amt_hotel_3m']< 0.0014426007) ,
( 0.0014426007 <= model_data['avg_perc_amt_hotel_3m']) & (model_data['avg_perc_amt_hotel_3m']< 0.0118576611) ,
( 0.0118576611 <= model_data['avg_perc_amt_hotel_3m'])],[-0.105070939541479,0.293543577387068,1.47844255281815])

model_data['b_avg_perc_amt_hotel_12m']=np.select([(model_data['avg_perc_amt_hotel_12m']< 0.0120495946) ,
( 0.0120495946 <= model_data['avg_perc_amt_hotel_12m'])],['1. Less than 0.01','2. GT0.01'])

model_data['w_avg_perc_amt_hotel_12m']=np.select([(model_data['avg_perc_amt_hotel_12m']< 0.0120495946) ,
( 0.0120495946 <= model_data['avg_perc_amt_hotel_12m'])],[-0.140131917942989,1.12834791394808])

model_data['b_avg_perc_amt_grocery_6m']=np.select([(model_data['avg_perc_amt_grocery_6m']< 0.0004173783) ,
( 0.0004173783 <= model_data['avg_perc_amt_grocery_6m']) & (model_data['avg_perc_amt_grocery_6m']< 0.0126805501) ,
( 0.0126805501 <= model_data['avg_perc_amt_grocery_6m'])],['1. Less than 0','2. GE  0 and LT 0.01','3. GT0.01'])

model_data['w_avg_perc_amt_grocery_6m']=np.select([(model_data['avg_perc_amt_grocery_6m']< 0.0004173783) ,
( 0.0004173783 <= model_data['avg_perc_amt_grocery_6m']) & (model_data['avg_perc_amt_grocery_6m']< 0.0126805501) ,
( 0.0126805501 <= model_data['avg_perc_amt_grocery_6m'])],[-0.259254991679664,0.269921284924563,0.900382863223469])

model_data['b_avg_perc_amt_grocery_3m']=np.select([(model_data['avg_perc_amt_grocery_3m']< 0.0012671484) ,
( 0.0012671484 <= model_data['avg_perc_amt_grocery_3m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_grocery_3m']=np.select([(model_data['avg_perc_amt_grocery_3m']< 0.0012671484) ,
( 0.0012671484 <= model_data['avg_perc_amt_grocery_3m'])],[-0.20260381962877,0.698941202281656])

model_data['b_avg_perc_amt_grocery_12m']=np.select([(model_data['avg_perc_amt_grocery_12m']< 0.0004187608) ,
( 0.0004187608 <= model_data['avg_perc_amt_grocery_12m']) & (model_data['avg_perc_amt_grocery_12m']< 0.0230063218) ,
( 0.0230063218 <= model_data['avg_perc_amt_grocery_12m'])],['1. Less than 0','2. GE  0 and LT 0.02','3. GT0.02'])

model_data['w_avg_perc_amt_grocery_12m']=np.select([(model_data['avg_perc_amt_grocery_12m']< 0.0004187608) ,
( 0.0004187608 <= model_data['avg_perc_amt_grocery_12m']) & (model_data['avg_perc_amt_grocery_12m']< 0.0230063218) ,
( 0.0230063218 <= model_data['avg_perc_amt_grocery_12m'])],[-0.318352112346218,0.202950334143002,1.09505407040375])

model_data['b_avg_perc_amt_education_12m']=np.select([(model_data['avg_perc_amt_education_12m']< 0.0016779461) ,
( 0.0016779461 <= model_data['avg_perc_amt_education_12m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_education_12m']=np.select([(model_data['avg_perc_amt_education_12m']< 0.0016779461) ,
( 0.0016779461 <= model_data['avg_perc_amt_education_12m'])],[-0.0318921142162431,0.900843056512769])

model_data['b_avg_perc_amt_clothing_6m']=np.select([(model_data['avg_perc_amt_clothing_6m']< 0.000701097) ,
( 0.000701097 <= model_data['avg_perc_amt_clothing_6m']) & (model_data['avg_perc_amt_clothing_6m']< 0.0032036907) ,
( 0.0032036907 <= model_data['avg_perc_amt_clothing_6m'])],['1. Less than 0','2. GE  0 and LT 0','3. GT0'])

model_data['w_avg_perc_amt_clothing_6m']=np.select([(model_data['avg_perc_amt_clothing_6m']< 0.000701097) ,
( 0.000701097 <= model_data['avg_perc_amt_clothing_6m']) & (model_data['avg_perc_amt_clothing_6m']< 0.0032036907) ,
( 0.0032036907 <= model_data['avg_perc_amt_clothing_6m'])],[-0.29108879563335,0.148252313203326,1.00179268484803])

model_data['b_avg_perc_amt_clothing_3m']=np.select([(model_data['avg_perc_amt_clothing_3m']< 0.0011990494) ,
( 0.0011990494 <= model_data['avg_perc_amt_clothing_3m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_clothing_3m']=np.select([(model_data['avg_perc_amt_clothing_3m']< 0.0011990494) ,
( 0.0011990494 <= model_data['avg_perc_amt_clothing_3m'])],[-0.194841137265265,0.956295208033232])

model_data['b_avg_perc_amt_clothing_12m']=np.select([(model_data['avg_perc_amt_clothing_12m']< 0.0004394496) ,
( 0.0004394496 <= model_data['avg_perc_amt_clothing_12m']) & (model_data['avg_perc_amt_clothing_12m']< 0.0042916544) ,
( 0.0042916544 <= model_data['avg_perc_amt_clothing_12m'])],['1. Less than 0','2. GE  0 and LT 0','3. GT0'])

model_data['w_avg_perc_amt_clothing_12m']=np.select([(model_data['avg_perc_amt_clothing_12m']< 0.0004394496) ,
( 0.0004394496 <= model_data['avg_perc_amt_clothing_12m']) & (model_data['avg_perc_amt_clothing_12m']< 0.0042916544) ,
( 0.0042916544 <= model_data['avg_perc_amt_clothing_12m'])],[-0.387581124702952,0.105565001795312,0.8157252695957])

model_data['b_avg_perc_amt_autocashdisb_6m']=np.select([(model_data['avg_perc_amt_autocashdisb_6m']< 0.0489598266) ,
( 0.0489598266 <= model_data['avg_perc_amt_autocashdisb_6m'])],['1. Less than 0.05','2. GT0.05'])

model_data['w_avg_perc_amt_autocashdisb_6m']=np.select([(model_data['avg_perc_amt_autocashdisb_6m']< 0.0489598266) ,
( 0.0489598266 <= model_data['avg_perc_amt_autocashdisb_6m'])],[-0.028240067953871,0.292722222104753])

model_data['b_avg_perc_amt_autocashdisb_3m']=np.select([(model_data['avg_perc_amt_autocashdisb_3m']< 0.0564452677) ,
( 0.0564452677 <= model_data['avg_perc_amt_autocashdisb_3m'])],['1. Less than 0.06','2. GT0.06'])

model_data['w_avg_perc_amt_autocashdisb_3m']=np.select([(model_data['avg_perc_amt_autocashdisb_3m']< 0.0564452677) ,
( 0.0564452677 <= model_data['avg_perc_amt_autocashdisb_3m'])],[-0.0269865306115662,0.401357387349519])

model_data['b_avg_perc_amt_autocashdisb_12m']=np.select([(model_data['avg_perc_amt_autocashdisb_12m']< 0.0290891887) ,
( 0.0290891887 <= model_data['avg_perc_amt_autocashdisb_12m'])],['1. Less than 0.03','2. GT0.03'])

model_data['w_avg_perc_amt_autocashdisb_12m']=np.select([(model_data['avg_perc_amt_autocashdisb_12m']< 0.0290891887) ,
( 0.0290891887 <= model_data['avg_perc_amt_autocashdisb_12m'])],[-0.0689109792796169,0.236516314488315])

model_data['b_avg_perc_amt_airline_6m']=np.select([(model_data['avg_perc_amt_airline_6m']< 0.0038144752) ,
( 0.0038144752 <= model_data['avg_perc_amt_airline_6m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_airline_6m']=np.select([(model_data['avg_perc_amt_airline_6m']< 0.0038144752) ,
( 0.0038144752 <= model_data['avg_perc_amt_airline_6m'])],[-0.0836772752986007,0.886631179131653])

model_data['b_avg_perc_amt_airline_3m']=np.select([(model_data['avg_perc_amt_airline_3m']< 0.0041825716) ,
( 0.0041825716 <= model_data['avg_perc_amt_airline_3m'])],['1. Less than 0','2. GT0'])

model_data['w_avg_perc_amt_airline_3m']=np.select([(model_data['avg_perc_amt_airline_3m']< 0.0041825716) ,
( 0.0041825716 <= model_data['avg_perc_amt_airline_3m'])],[-0.050210439029353,0.946305430589526])

model_data['b_avg_perc_amt_airline_12m']=np.select([(model_data['avg_perc_amt_airline_12m']< 0.0060716712) ,
( 0.0060716712 <= model_data['avg_perc_amt_airline_12m'])],['1. Less than 0.01','2. GT0.01'])

model_data['w_avg_perc_amt_airline_12m']=np.select([(model_data['avg_perc_amt_airline_12m']< 0.0060716712) ,
( 0.0060716712 <= model_data['avg_perc_amt_airline_12m'])],[-0.115916260807777,1.09900018004425])

model_data['b_avg_perc_amt_FI_6m']=np.select([(model_data['avg_perc_amt_FI_6m']< 0.181005001) ,
( 0.181005001 <= model_data['avg_perc_amt_FI_6m'])],['1. Less than 0.18','2. GT0.18'])

model_data['w_avg_perc_amt_FI_6m']=np.select([(model_data['avg_perc_amt_FI_6m']< 0.181005001) ,
( 0.181005001 <= model_data['avg_perc_amt_FI_6m'])],[-0.0420546765489173,0.273825138217367])

model_data['b_avg_perc_amt_FI_3m']=np.select([(model_data['avg_perc_amt_FI_3m']< 0.0077828794) ,
( 0.0077828794 <= model_data['avg_perc_amt_FI_3m'])],['1. Less than 0.01','2. GT0.01'])

model_data['w_avg_perc_amt_FI_3m']=np.select([(model_data['avg_perc_amt_FI_3m']< 0.0077828794) ,
( 0.0077828794 <= model_data['avg_perc_amt_FI_3m'])],[1.04715933700742,-0.0349270868383801])

model_data['b_avg_perc_amt_FI_12m']=np.select([(model_data['avg_perc_amt_FI_12m']< 0.0743524291) ,
( 0.0743524291 <= model_data['avg_perc_amt_FI_12m'])],['1. Less than 0.07','2. GT0.07'])

model_data['w_avg_perc_amt_FI_12m']=np.select([(model_data['avg_perc_amt_FI_12m']< 0.0743524291) ,
( 0.0743524291 <= model_data['avg_perc_amt_FI_12m'])],[-0.106494922417381,0.0770353268652432])

model_data['b_avg_amt_utility_bills_6m']=np.select([(model_data['avg_amt_utility_bills_6m']< 9) ,
( 9 <= model_data['avg_amt_utility_bills_6m']) & (model_data['avg_amt_utility_bills_6m']< 372.3333333333) ,
( 372.3333333333 <= model_data['avg_amt_utility_bills_6m'])],['1. Less than 9','2. GE  9 and LT 372.33','3. GT372.33'])

model_data['w_avg_amt_utility_bills_6m']=np.select([(model_data['avg_amt_utility_bills_6m']< 9) ,
( 9 <= model_data['avg_amt_utility_bills_6m']) & (model_data['avg_amt_utility_bills_6m']< 372.3333333333) ,
( 372.3333333333 <= model_data['avg_amt_utility_bills_6m'])],[-0.0863334051391333,0.344392543659596,0.594476973239065])

model_data['b_avg_amt_utility_bills_3m']=np.select([(model_data['avg_amt_utility_bills_3m']< 8) ,
( 8 <= model_data['avg_amt_utility_bills_3m']) & (model_data['avg_amt_utility_bills_3m']< 836.3333333333) ,
( 836.3333333333 <= model_data['avg_amt_utility_bills_3m'])],['1. Less than 8','2. GE  8 and LT 836.33','3. GT836.33'])

model_data['w_avg_amt_utility_bills_3m']=np.select([(model_data['avg_amt_utility_bills_3m']< 8) ,
( 8 <= model_data['avg_amt_utility_bills_3m']) & (model_data['avg_amt_utility_bills_3m']< 836.3333333333) ,
( 836.3333333333 <= model_data['avg_amt_utility_bills_3m'])],[-0.0660973340885088,0.441953441211768,1.04715933700742])

model_data['b_avg_amt_utility_bills_12m']=np.select([(model_data['avg_amt_utility_bills_12m']< 9) ,
( 9 <= model_data['avg_amt_utility_bills_12m']) & (model_data['avg_amt_utility_bills_12m']< 1201) ,
( 1201 <= model_data['avg_amt_utility_bills_12m'])],['1. Less than 9','2. GE  9 and LT 1201','3. GT1201'])

model_data['w_avg_amt_utility_bills_12m']=np.select([(model_data['avg_amt_utility_bills_12m']< 9) ,
( 9 <= model_data['avg_amt_utility_bills_12m']) & (model_data['avg_amt_utility_bills_12m']< 1201) ,
( 1201 <= model_data['avg_amt_utility_bills_12m'])],[-0.129325846751127,0.384208832023289,0.79117914529489])

model_data['b_avg_amt_servicestation_6m']=np.select([(model_data['avg_amt_servicestation_6m']< 4) ,
( 4 <= model_data['avg_amt_servicestation_6m']) & (model_data['avg_amt_servicestation_6m']< 101) ,
( 101 <= model_data['avg_amt_servicestation_6m'])],['1. Less than 4','2. GE  4 and LT 101','3. GT101'])

model_data['w_avg_amt_servicestation_6m']=np.select([(model_data['avg_amt_servicestation_6m']< 4) ,
( 4 <= model_data['avg_amt_servicestation_6m']) & (model_data['avg_amt_servicestation_6m']< 101) ,
( 101 <= model_data['avg_amt_servicestation_6m'])],[-0.0495762135519878,-0.22622506623792,0.369963454437587])

model_data['b_avg_amt_servicestation_3m']=np.select([(model_data['avg_amt_servicestation_3m']< 4) ,
( 4 <= model_data['avg_amt_servicestation_3m'])],['1. Less than 4','2. GT4'])

model_data['w_avg_amt_servicestation_3m']=np.select([(model_data['avg_amt_servicestation_3m']< 4) ,
( 4 <= model_data['avg_amt_servicestation_3m'])],[-0.0449719020747556,0.255884300967244])

model_data['b_avg_amt_servicestation_12m']=np.select([(model_data['avg_amt_servicestation_12m']< 4) ,
( 4 <= model_data['avg_amt_servicestation_12m']) & (model_data['avg_amt_servicestation_12m']< 97.3333333333) ,
( 97.3333333333 <= model_data['avg_amt_servicestation_12m'])],['1. Less than 4','2. GE  4 and LT 97.33','3. GT97.33'])

model_data['w_avg_amt_servicestation_12m']=np.select([(model_data['avg_amt_servicestation_12m']< 4) ,
( 4 <= model_data['avg_amt_servicestation_12m']) & (model_data['avg_amt_servicestation_12m']< 97.3333333333) ,
( 97.3333333333 <= model_data['avg_amt_servicestation_12m'])],[-0.0938372590151016,0.0269149141331194,0.298622813172938])

model_data['b_avg_amt_restaurant_6m']=np.select([(model_data['avg_amt_restaurant_6m']< 8) ,
( 8 <= model_data['avg_amt_restaurant_6m']) & (model_data['avg_amt_restaurant_6m']< 309.75) ,
( 309.75 <= model_data['avg_amt_restaurant_6m'])],['1. Less than 8','2. GE  8 and LT 309.75','3. GT309.75'])

model_data['w_avg_amt_restaurant_6m']=np.select([(model_data['avg_amt_restaurant_6m']< 8) ,
( 8 <= model_data['avg_amt_restaurant_6m']) & (model_data['avg_amt_restaurant_6m']< 309.75) ,
( 309.75 <= model_data['avg_amt_restaurant_6m'])],[-0.16435446624294,0.522561066783561,0.834289624357019])

model_data['b_avg_amt_restaurant_3m']=np.select([(model_data['avg_amt_restaurant_3m']< 8) ,
( 8 <= model_data['avg_amt_restaurant_3m'])],['1. Less than 8','2. GT8'])

model_data['w_avg_amt_restaurant_3m']=np.select([(model_data['avg_amt_restaurant_3m']< 8) ,
( 8 <= model_data['avg_amt_restaurant_3m'])],[-0.115099801494433,0.801443232816235])

model_data['b_avg_amt_restaurant_12m']=np.select([(model_data['avg_amt_restaurant_12m']< 12) ,
( 12 <= model_data['avg_amt_restaurant_12m']) & (model_data['avg_amt_restaurant_12m']< 353.3333333333) ,
( 353.3333333333 <= model_data['avg_amt_restaurant_12m'])],['1. Less than 12','2. GE  12 and LT 353.33','3. GT353.33'])

model_data['w_avg_amt_restaurant_12m']=np.select([(model_data['avg_amt_restaurant_12m']< 12) ,
( 12 <= model_data['avg_amt_restaurant_12m']) & (model_data['avg_amt_restaurant_12m']< 353.3333333333) ,
( 353.3333333333 <= model_data['avg_amt_restaurant_12m'])],[-0.248357878419333,0.510549587329402,0.690728623134619])

model_data['b_avg_amt_jewellery_6m']=np.select([(model_data['avg_amt_jewellery_6m']< 35) ,
( 35 <= model_data['avg_amt_jewellery_6m']) & (model_data['avg_amt_jewellery_6m']< 1802) ,
( 1802 <= model_data['avg_amt_jewellery_6m'])],['1. Less than 35','2. GE  35 and LT 1802','3. GT1802'])

model_data['w_avg_amt_jewellery_6m']=np.select([(model_data['avg_amt_jewellery_6m']< 35) ,
( 35 <= model_data['avg_amt_jewellery_6m']) & (model_data['avg_amt_jewellery_6m']< 1802) ,
( 1802 <= model_data['avg_amt_jewellery_6m'])],[0.037029269086347,0.0265813199089039,-0.533622312048211])

model_data['b_avg_amt_jewellery_3m']=np.select([(model_data['avg_amt_jewellery_3m']< 50) ,
( 50 <= model_data['avg_amt_jewellery_3m'])],['1. Less than 50','2. GT50'])

model_data['w_avg_amt_jewellery_3m']=np.select([(model_data['avg_amt_jewellery_3m']< 50) ,
( 50 <= model_data['avg_amt_jewellery_3m'])],[0.0233910095025254,-0.28304298924643])

model_data['b_avg_amt_jewellery_12m']=np.select([(model_data['avg_amt_jewellery_12m']< 35) ,
( 35 <= model_data['avg_amt_jewellery_12m']) & (model_data['avg_amt_jewellery_12m']< 1116.6666666667) ,
( 1116.6666666667 <= model_data['avg_amt_jewellery_12m'])],['1. Less than 35','2. GE  35 and LT 1116.67','3. GT1116.67'])

model_data['w_avg_amt_jewellery_12m']=np.select([(model_data['avg_amt_jewellery_12m']< 35) ,
( 35 <= model_data['avg_amt_jewellery_12m']) & (model_data['avg_amt_jewellery_12m']< 1116.6666666667) ,
( 1116.6666666667 <= model_data['avg_amt_jewellery_12m'])],[0.0105133484334852,0.399976038104412,-0.254090570281559])

model_data['b_avg_amt_hotel_6m']=np.select([(model_data['avg_amt_hotel_6m']< 6) ,
( 6 <= model_data['avg_amt_hotel_6m']) & (model_data['avg_amt_hotel_6m']< 1001.6) ,
( 1001.6 <= model_data['avg_amt_hotel_6m'])],['1. Less than 6','2. GE  6 and LT 1001.6','3. GT1001.6'])

model_data['w_avg_amt_hotel_6m']=np.select([(model_data['avg_amt_hotel_6m']< 6) ,
( 6 <= model_data['avg_amt_hotel_6m']) & (model_data['avg_amt_hotel_6m']< 1001.6) ,
( 1001.6 <= model_data['avg_amt_hotel_6m'])],[-0.169776372364901,0.501050573016339,1.16238688174288])

model_data['b_avg_amt_hotel_3m']=np.select([(model_data['avg_amt_hotel_3m']< 1) ,
( 1 <= model_data['avg_amt_hotel_3m']) & (model_data['avg_amt_hotel_3m']< 1083) ,
( 1083 <= model_data['avg_amt_hotel_3m'])],['1. Less than 1','2. GE  1 and LT 1083','3. GT1083'])

model_data['w_avg_amt_hotel_3m']=np.select([(model_data['avg_amt_hotel_3m']< 1) ,
( 1 <= model_data['avg_amt_hotel_3m']) & (model_data['avg_amt_hotel_3m']< 1083) ,
( 1083 <= model_data['avg_amt_hotel_3m'])],[-0.107112327024327,0.562711068473191,1.42075236550117])

model_data['b_avg_amt_hotel_12m']=np.select([(model_data['avg_amt_hotel_12m']< 6) ,
( 6 <= model_data['avg_amt_hotel_12m']) & (model_data['avg_amt_hotel_12m']< 2384.3333333333) ,
( 2384.3333333333 <= model_data['avg_amt_hotel_12m'])],['1. Less than 6','2. GE  6 and LT 2384.33','3. GT2384.33'])

model_data['w_avg_amt_hotel_12m']=np.select([(model_data['avg_amt_hotel_12m']< 6) ,
( 6 <= model_data['avg_amt_hotel_12m']) & (model_data['avg_amt_hotel_12m']< 2384.3333333333) ,
( 2384.3333333333 <= model_data['avg_amt_hotel_12m'])],[-0.258753573394508,0.545011934706991,1.148656657495])

model_data['b_avg_amt_grocery_6m']=np.select([(model_data['avg_amt_grocery_6m']< 8) ,
( 8 <= model_data['avg_amt_grocery_6m']) & (model_data['avg_amt_grocery_6m']< 544.5) ,
( 544.5 <= model_data['avg_amt_grocery_6m'])],['1. Less than 8','2. GE  8 and LT 544.5','3. GT544.5'])

model_data['w_avg_amt_grocery_6m']=np.select([(model_data['avg_amt_grocery_6m']< 8) ,
( 8 <= model_data['avg_amt_grocery_6m']) & (model_data['avg_amt_grocery_6m']< 544.5) ,
( 544.5 <= model_data['avg_amt_grocery_6m'])],[-0.264043913455709,0.329735273638073,0.684660467502609])

model_data['b_avg_amt_grocery_3m']=np.select([(model_data['avg_amt_grocery_3m']< 6) ,
( 6 <= model_data['avg_amt_grocery_3m']) & (model_data['avg_amt_grocery_3m']< 617.5) ,
( 617.5 <= model_data['avg_amt_grocery_3m'])],['1. Less than 6','2. GE  6 and LT 617.5','3. GT617.5'])

model_data['w_avg_amt_grocery_3m']=np.select([(model_data['avg_amt_grocery_3m']< 6) ,
( 6 <= model_data['avg_amt_grocery_3m']) & (model_data['avg_amt_grocery_3m']< 617.5) ,
( 617.5 <= model_data['avg_amt_grocery_3m'])],[-0.204936748853629,0.51865563327395,0.834370298795367])

model_data['b_avg_amt_grocery_12m']=np.select([(model_data['avg_amt_grocery_12m']< 8) ,
( 8 <= model_data['avg_amt_grocery_12m']) & (model_data['avg_amt_grocery_12m']< 602.5) ,
( 602.5 <= model_data['avg_amt_grocery_12m'])],['1. Less than 8','2. GE  8 and LT 602.5','3. GT602.5'])

model_data['w_avg_amt_grocery_12m']=np.select([(model_data['avg_amt_grocery_12m']< 8) ,
( 8 <= model_data['avg_amt_grocery_12m']) & (model_data['avg_amt_grocery_12m']< 602.5) ,
( 602.5 <= model_data['avg_amt_grocery_12m'])],[-0.334342650505873,0.105031050629479,0.73550427234796])

model_data['b_avg_amt_education_12m']=np.select([(model_data['avg_amt_education_12m']< 13) ,
( 13 <= model_data['avg_amt_education_12m'])],['1. Less than 13','2. GT13'])

model_data['w_avg_amt_education_12m']=np.select([(model_data['avg_amt_education_12m']< 13) ,
( 13 <= model_data['avg_amt_education_12m'])],[-0.0330995148723009,0.495167886435551])

model_data['b_avg_amt_clothing_6m']=np.select([(model_data['avg_amt_clothing_6m']< 10) ,
( 10 <= model_data['avg_amt_clothing_6m']) & (model_data['avg_amt_clothing_6m']< 408.6) ,
( 408.6 <= model_data['avg_amt_clothing_6m'])],['1. Less than 10','2. GE  10 and LT 408.6','3. GT408.6'])

model_data['w_avg_amt_clothing_6m']=np.select([(model_data['avg_amt_clothing_6m']< 10) ,
( 10 <= model_data['avg_amt_clothing_6m']) & (model_data['avg_amt_clothing_6m']< 408.6) ,
( 408.6 <= model_data['avg_amt_clothing_6m'])],[-0.299587778075472,0.378598662131347,1.01428246568358])

model_data['b_avg_amt_clothing_3m']=np.select([(model_data['avg_amt_clothing_3m']< 10) ,
( 10 <= model_data['avg_amt_clothing_3m']) & (model_data['avg_amt_clothing_3m']< 257) ,
( 257 <= model_data['avg_amt_clothing_3m'])],['1. Less than 10','2. GE  10 and LT 257','3. GT257'])

model_data['w_avg_amt_clothing_3m']=np.select([(model_data['avg_amt_clothing_3m']< 10) ,
( 10 <= model_data['avg_amt_clothing_3m']) & (model_data['avg_amt_clothing_3m']< 257) ,
( 257 <= model_data['avg_amt_clothing_3m'])],[-0.198442214232652,0.334224816048128,1.16035425199785])

model_data['b_avg_amt_clothing_12m']=np.select([(model_data['avg_amt_clothing_12m']< 10) ,
( 10 <= model_data['avg_amt_clothing_12m']) & (model_data['avg_amt_clothing_12m']< 389) ,
( 389 <= model_data['avg_amt_clothing_12m'])],['1. Less than 10','2. GE  10 and LT 389','3. GT389'])

model_data['w_avg_amt_clothing_12m']=np.select([(model_data['avg_amt_clothing_12m']< 10) ,
( 10 <= model_data['avg_amt_clothing_12m']) & (model_data['avg_amt_clothing_12m']< 389) ,
( 389 <= model_data['avg_amt_clothing_12m'])],[-0.408385897234867,0.318787066319635,0.628903158094645])

model_data['b_avg_amt_cash_withd_6m']=np.select([(model_data['avg_amt_cash_withd_6m']< 87) ,
( 87 <= model_data['avg_amt_cash_withd_6m']) & (model_data['avg_amt_cash_withd_6m']< 1813.8) ,
( 1813.8 <= model_data['avg_amt_cash_withd_6m'])],['1. Less than 87','2. GE  87 and LT 1813.8','3. GT1813.8'])

model_data['w_avg_amt_cash_withd_6m']=np.select([(model_data['avg_amt_cash_withd_6m']< 87) ,
( 87 <= model_data['avg_amt_cash_withd_6m']) & (model_data['avg_amt_cash_withd_6m']< 1813.8) ,
( 1813.8 <= model_data['avg_amt_cash_withd_6m'])],[-0.0772999502131552,0.387123229275153,0.0396236740753745])

model_data['b_avg_amt_cash_withd_3m']=np.select([(model_data['avg_amt_cash_withd_3m']< 75) ,
( 75 <= model_data['avg_amt_cash_withd_3m']) & (model_data['avg_amt_cash_withd_3m']< 1500.5) ,
( 1500.5 <= model_data['avg_amt_cash_withd_3m'])],['1. Less than 75','2. GE  75 and LT 1500.5','3. GT1500.5'])

model_data['w_avg_amt_cash_withd_3m']=np.select([(model_data['avg_amt_cash_withd_3m']< 75) ,
( 75 <= model_data['avg_amt_cash_withd_3m']) & (model_data['avg_amt_cash_withd_3m']< 1500.5) ,
( 1500.5 <= model_data['avg_amt_cash_withd_3m'])],[-0.0584231253232194,0.413597620832527,0.0773236739275039])

model_data['b_avg_amt_cash_withd_12m']=np.select([(model_data['avg_amt_cash_withd_12m']< 87) ,
( 87 <= model_data['avg_amt_cash_withd_12m']) & (model_data['avg_amt_cash_withd_12m']< 5016.5) ,
( 5016.5 <= model_data['avg_amt_cash_withd_12m'])],['1. Less than 87','2. GE  87 and LT 5016.5','3. GT5016.5'])

model_data['w_avg_amt_cash_withd_12m']=np.select([(model_data['avg_amt_cash_withd_12m']< 87) ,
( 87 <= model_data['avg_amt_cash_withd_12m']) & (model_data['avg_amt_cash_withd_12m']< 5016.5) ,
( 5016.5 <= model_data['avg_amt_cash_withd_12m'])],[-0.137674207184475,0.241023013926401,0.156710321219956])

model_data['b_avg_amt_autocashdisb_6m']=np.select([(model_data['avg_amt_autocashdisb_6m']< 87) ,
( 87 <= model_data['avg_amt_autocashdisb_6m']) & (model_data['avg_amt_autocashdisb_6m']< 2100) ,
( 2100 <= model_data['avg_amt_autocashdisb_6m'])],['1. Less than 87','2. GE  87 and LT 2100','3. GT2100'])

model_data['w_avg_amt_autocashdisb_6m']=np.select([(model_data['avg_amt_autocashdisb_6m']< 87) ,
( 87 <= model_data['avg_amt_autocashdisb_6m']) & (model_data['avg_amt_autocashdisb_6m']< 2100) ,
( 2100 <= model_data['avg_amt_autocashdisb_6m'])],[-0.0771799957956901,0.385637022284303,-0.0155677882676625])

model_data['b_avg_amt_autocashdisb_3m']=np.select([(model_data['avg_amt_autocashdisb_3m']< 75) ,
( 75 <= model_data['avg_amt_autocashdisb_3m']) & (model_data['avg_amt_autocashdisb_3m']< 1500.5) ,
( 1500.5 <= model_data['avg_amt_autocashdisb_3m'])],['1. Less than 75','2. GE  75 and LT 1500.5','3. GT1500.5'])

model_data['w_avg_amt_autocashdisb_3m']=np.select([(model_data['avg_amt_autocashdisb_3m']< 75) ,
( 75 <= model_data['avg_amt_autocashdisb_3m']) & (model_data['avg_amt_autocashdisb_3m']< 1500.5) ,
( 1500.5 <= model_data['avg_amt_autocashdisb_3m'])],[-0.0584231253232194,0.413597620832527,0.0773236739275039])

model_data['b_avg_amt_autocashdisb_12m']=np.select([(model_data['avg_amt_autocashdisb_12m']< 87) ,
( 87 <= model_data['avg_amt_autocashdisb_12m']) & (model_data['avg_amt_autocashdisb_12m']< 5016.5) ,
( 5016.5 <= model_data['avg_amt_autocashdisb_12m'])],['1. Less than 87','2. GE  87 and LT 5016.5','3. GT5016.5'])

model_data['w_avg_amt_autocashdisb_12m']=np.select([(model_data['avg_amt_autocashdisb_12m']< 87) ,
( 87 <= model_data['avg_amt_autocashdisb_12m']) & (model_data['avg_amt_autocashdisb_12m']< 5016.5) ,
( 5016.5 <= model_data['avg_amt_autocashdisb_12m'])],[-0.137315127859995,0.240556813451757,0.156041201087932])

model_data['b_avg_amt_airline_6m']=np.select([(model_data['avg_amt_airline_6m']< 10) ,
( 10 <= model_data['avg_amt_airline_6m'])],['1. Less than 10','2. GT10'])

model_data['w_avg_amt_airline_6m']=np.select([(model_data['avg_amt_airline_6m']< 10) ,
( 10 <= model_data['avg_amt_airline_6m'])],[-0.096022178117652,0.865598577897816])

model_data['b_avg_amt_airline_3m']=np.select([(model_data['avg_amt_airline_3m']< 5) ,
( 5 <= model_data['avg_amt_airline_3m'])],['1. Less than 5','2. GT5'])

model_data['w_avg_amt_airline_3m']=np.select([(model_data['avg_amt_airline_3m']< 5) ,
( 5 <= model_data['avg_amt_airline_3m'])],[-0.0537601534964104,0.892596587272288])

model_data['b_avg_amt_airline_12m']=np.select([(model_data['avg_amt_airline_12m']< 11) ,
( 11 <= model_data['avg_amt_airline_12m']) & (model_data['avg_amt_airline_12m']< 1282) ,
( 1282 <= model_data['avg_amt_airline_12m'])],['1. Less than 11','2. GE  11 and LT 1282','3. GT1282'])

model_data['w_avg_amt_airline_12m']=np.select([(model_data['avg_amt_airline_12m']< 11) ,
( 11 <= model_data['avg_amt_airline_12m']) & (model_data['avg_amt_airline_12m']< 1282) ,
( 1282 <= model_data['avg_amt_airline_12m'])],[-0.161582161848696,0.500280122651948,1.31368827191856])

model_data['b_avg_amt_FI_6m']=np.select([(model_data['avg_amt_FI_6m']< 1561.3333333333) ,
( 1561.3333333333 <= model_data['avg_amt_FI_6m']) & (model_data['avg_amt_FI_6m']< 2196.3333333333) ,
( 2196.3333333333 <= model_data['avg_amt_FI_6m'])],['1. Less than 1561.33','2. GE  1561.33 and LT 2196.33','3. GT2196.33'])

model_data['w_avg_amt_FI_6m']=np.select([(model_data['avg_amt_FI_6m']< 1561.3333333333) ,
( 1561.3333333333 <= model_data['avg_amt_FI_6m']) & (model_data['avg_amt_FI_6m']< 2196.3333333333) ,
( 2196.3333333333 <= model_data['avg_amt_FI_6m'])],[-0.283145667755441,0.191337964429426,0.286568356530375])

model_data['b_avg_amt_FI_3m']=np.select([(model_data['avg_amt_FI_3m']< 50) ,
( 50 <= model_data['avg_amt_FI_3m']) & (model_data['avg_amt_FI_3m']< 938) ,
( 938 <= model_data['avg_amt_FI_3m']) & (model_data['avg_amt_FI_3m']< 3133.3333333333) ,
( 3133.3333333333 <= model_data['avg_amt_FI_3m'])],['1. Less than 50','2. GE  50 and LT 938','3. GE  938 and LT 3133.33','4. GT3133.33'])

model_data['w_avg_amt_FI_3m']=np.select([(model_data['avg_amt_FI_3m']< 50) ,
( 50 <= model_data['avg_amt_FI_3m']) & (model_data['avg_amt_FI_3m']< 938) ,
( 938 <= model_data['avg_amt_FI_3m']) & (model_data['avg_amt_FI_3m']< 3133.3333333333) ,
( 3133.3333333333 <= model_data['avg_amt_FI_3m'])],[1.69935075273054,-0.638407389468797,0.209366466363379,0.43022686982062])

model_data['b_avg_amt_FI_12m']=np.select([(model_data['avg_amt_FI_12m']< 1316.8181818182) ,
( 1316.8181818182 <= model_data['avg_amt_FI_12m']) & (model_data['avg_amt_FI_12m']< 4016.6666666667) ,
( 4016.6666666667 <= model_data['avg_amt_FI_12m'])],['1. Less than 1316.82','2. GE  1316.82 and LT 4016.67','3. GT4016.67'])

model_data['w_avg_amt_FI_12m']=np.select([(model_data['avg_amt_FI_12m']< 1316.8181818182) ,
( 1316.8181818182 <= model_data['avg_amt_FI_12m']) & (model_data['avg_amt_FI_12m']< 4016.6666666667) ,
( 4016.6666666667 <= model_data['avg_amt_FI_12m'])],[-0.0973134192291719,-0.0517279516624263,0.467161733328953])

model_data['b_Tot_Cons_inc_util_6m']=np.select([(model_data['Tot_Cons_inc_util_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_util_6m']) & (model_data['Tot_Cons_inc_util_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_util_6m']) & (model_data['Tot_Cons_inc_util_6m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_util_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

model_data['w_Tot_Cons_inc_util_6m']=np.select([(model_data['Tot_Cons_inc_util_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_util_6m']) & (model_data['Tot_Cons_inc_util_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_util_6m']) & (model_data['Tot_Cons_inc_util_6m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_util_6m'])],[0.747744715140358,0.276452541338117,-0.147017333882326,-0.749747063295])

model_data['b_Tot_Cons_inc_util_12m']=np.select([(model_data['Tot_Cons_inc_util_12m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_util_12m']) & (model_data['Tot_Cons_inc_util_12m']< 6) ,
( 6 <= model_data['Tot_Cons_inc_util_12m']) & (model_data['Tot_Cons_inc_util_12m']< 7) ,
( 7 <= model_data['Tot_Cons_inc_util_12m']) & (model_data['Tot_Cons_inc_util_12m']< 8) ,
( 8 <= model_data['Tot_Cons_inc_util_12m'])],['1. Less than 4','2. GE  4 and LT 6','3. GE  6 and LT 7','4. GE  7 and LT 8','5. GT8'])

model_data['w_Tot_Cons_inc_util_12m']=np.select([(model_data['Tot_Cons_inc_util_12m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_util_12m']) & (model_data['Tot_Cons_inc_util_12m']< 6) ,
( 6 <= model_data['Tot_Cons_inc_util_12m']) & (model_data['Tot_Cons_inc_util_12m']< 7) ,
( 7 <= model_data['Tot_Cons_inc_util_12m']) & (model_data['Tot_Cons_inc_util_12m']< 8) ,
( 8 <= model_data['Tot_Cons_inc_util_12m'])],[0.774517016834438,0.252843938649954,-0.160276241781157,-0.460778399974174,-0.919258883964882])

model_data['b_Tot_Cons_inc_pay_mindue_6m']=np.select([(model_data['Tot_Cons_inc_pay_mindue_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_mindue_6m']) & (model_data['Tot_Cons_inc_pay_mindue_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_mindue_6m']) & (model_data['Tot_Cons_inc_pay_mindue_6m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_pay_mindue_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

model_data['w_Tot_Cons_inc_pay_mindue_6m']=np.select([(model_data['Tot_Cons_inc_pay_mindue_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_mindue_6m']) & (model_data['Tot_Cons_inc_pay_mindue_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_mindue_6m']) & (model_data['Tot_Cons_inc_pay_mindue_6m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_pay_mindue_6m'])],[-0.608151221705559,-0.0282320873945136,0.473629775420125,1.06806495054489])

model_data['b_Tot_Cons_inc_pay_mindue_12m']=np.select([(model_data['Tot_Cons_inc_pay_mindue_12m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_mindue_12m']) & (model_data['Tot_Cons_inc_pay_mindue_12m']< 5) ,
( 5 <= model_data['Tot_Cons_inc_pay_mindue_12m']) & (model_data['Tot_Cons_inc_pay_mindue_12m']< 6) ,
( 6 <= model_data['Tot_Cons_inc_pay_mindue_12m']) & (model_data['Tot_Cons_inc_pay_mindue_12m']< 7) ,
( 7 <= model_data['Tot_Cons_inc_pay_mindue_12m'])],['1. Less than 3','2. GE  3 and LT 5','3. GE  5 and LT 6','4. GE  6 and LT 7','5. GT7'])

model_data['w_Tot_Cons_inc_pay_mindue_12m']=np.select([(model_data['Tot_Cons_inc_pay_mindue_12m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_mindue_12m']) & (model_data['Tot_Cons_inc_pay_mindue_12m']< 5) ,
( 5 <= model_data['Tot_Cons_inc_pay_mindue_12m']) & (model_data['Tot_Cons_inc_pay_mindue_12m']< 6) ,
( 6 <= model_data['Tot_Cons_inc_pay_mindue_12m']) & (model_data['Tot_Cons_inc_pay_mindue_12m']< 7) ,
( 7 <= model_data['Tot_Cons_inc_pay_mindue_12m'])],[-0.687204923010248,-0.261222322288993,0.109769059930673,0.470188766055581,1.06022992957882])

model_data['b_Tot_Cons_inc_pay_exp_6m']=np.select([(model_data['Tot_Cons_inc_pay_exp_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_exp_6m']) & (model_data['Tot_Cons_inc_pay_exp_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_exp_6m']) & (model_data['Tot_Cons_inc_pay_exp_6m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_pay_exp_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

model_data['w_Tot_Cons_inc_pay_exp_6m']=np.select([(model_data['Tot_Cons_inc_pay_exp_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_pay_exp_6m']) & (model_data['Tot_Cons_inc_pay_exp_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_exp_6m']) & (model_data['Tot_Cons_inc_pay_exp_6m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_pay_exp_6m'])],[-0.608151221705559,-0.0282320873945136,0.473629775420125,1.06806495054489])

model_data['b_Tot_Cons_inc_pay_exp_12m']=np.select([(model_data['Tot_Cons_inc_pay_exp_12m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_exp_12m']) & (model_data['Tot_Cons_inc_pay_exp_12m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_pay_exp_12m']) & (model_data['Tot_Cons_inc_pay_exp_12m']< 5) ,
( 5 <= model_data['Tot_Cons_inc_pay_exp_12m']) & (model_data['Tot_Cons_inc_pay_exp_12m']< 6) ,
( 6 <= model_data['Tot_Cons_inc_pay_exp_12m'])],['1. Less than 3','2. GE  3 and LT 4','3. GE  4 and LT 5','4. GE  5 and LT 6','5. GT6'])

model_data['w_Tot_Cons_inc_pay_exp_12m']=np.select([(model_data['Tot_Cons_inc_pay_exp_12m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_pay_exp_12m']) & (model_data['Tot_Cons_inc_pay_exp_12m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_pay_exp_12m']) & (model_data['Tot_Cons_inc_pay_exp_12m']< 5) ,
( 5 <= model_data['Tot_Cons_inc_pay_exp_12m']) & (model_data['Tot_Cons_inc_pay_exp_12m']< 6) ,
( 6 <= model_data['Tot_Cons_inc_pay_exp_12m'])],[-0.687204923010248,-0.387026715624879,-0.191872165841396,0.109769059930673,0.627669964403871])

model_data['b_Tot_Cons_inc_delinq_6m']=np.select([(model_data['Tot_Cons_inc_delinq_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_delinq_6m']) & (model_data['Tot_Cons_inc_delinq_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_delinq_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Tot_Cons_inc_delinq_6m']=np.select([(model_data['Tot_Cons_inc_delinq_6m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_delinq_6m']) & (model_data['Tot_Cons_inc_delinq_6m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_delinq_6m'])],[0.802522445448763,-0.655984905649034,-1.7836788500626])

model_data['b_Tot_Cons_inc_delinq_12m']=np.select([(model_data['Tot_Cons_inc_delinq_12m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_delinq_12m']) & (model_data['Tot_Cons_inc_delinq_12m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_delinq_12m']) & (model_data['Tot_Cons_inc_delinq_12m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_delinq_12m']) & (model_data['Tot_Cons_inc_delinq_12m']< 5) ,
( 5 <= model_data['Tot_Cons_inc_delinq_12m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GE  4 and LT 5','5. GT5'])

model_data['w_Tot_Cons_inc_delinq_12m']=np.select([(model_data['Tot_Cons_inc_delinq_12m']< 2) ,
( 2 <= model_data['Tot_Cons_inc_delinq_12m']) & (model_data['Tot_Cons_inc_delinq_12m']< 3) ,
( 3 <= model_data['Tot_Cons_inc_delinq_12m']) & (model_data['Tot_Cons_inc_delinq_12m']< 4) ,
( 4 <= model_data['Tot_Cons_inc_delinq_12m']) & (model_data['Tot_Cons_inc_delinq_12m']< 5) ,
( 5 <= model_data['Tot_Cons_inc_delinq_12m'])],[1.27384124175409,0.208885394338194,-0.41031177670142,-0.719739420474397,-1.52682339841379])

model_data['b_Tot_Cons_dec_util_6m']=np.select([(model_data['Tot_Cons_dec_util_6m']< 2) ,
( 2 <= model_data['Tot_Cons_dec_util_6m']) & (model_data['Tot_Cons_dec_util_6m']< 3) ,
( 3 <= model_data['Tot_Cons_dec_util_6m']) & (model_data['Tot_Cons_dec_util_6m']< 4) ,
( 4 <= model_data['Tot_Cons_dec_util_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

model_data['w_Tot_Cons_dec_util_6m']=np.select([(model_data['Tot_Cons_dec_util_6m']< 2) ,
( 2 <= model_data['Tot_Cons_dec_util_6m']) & (model_data['Tot_Cons_dec_util_6m']< 3) ,
( 3 <= model_data['Tot_Cons_dec_util_6m']) & (model_data['Tot_Cons_dec_util_6m']< 4) ,
( 4 <= model_data['Tot_Cons_dec_util_6m'])],[-0.610294146218889,-0.14193693173865,0.274655150677884,0.822961859721771])

model_data['b_Tot_Cons_dec_util_3m']=np.select([(model_data['Tot_Cons_dec_util_3m']< 2) ,
( 2 <= model_data['Tot_Cons_dec_util_3m'])],['1. Less than 2','2. GT2'])

model_data['w_Tot_Cons_dec_util_3m']=np.select([(model_data['Tot_Cons_dec_util_3m']< 2) ,
( 2 <= model_data['Tot_Cons_dec_util_3m'])],[-0.129033072043837,0.652107056819807])

model_data['b_Tot_Cons_dec_util_12m']=np.select([(model_data['Tot_Cons_dec_util_12m']< 3) ,
( 3 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 4) ,
( 4 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 5) ,
( 5 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 6) ,
( 6 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 7) ,
( 7 <= model_data['Tot_Cons_dec_util_12m'])],['1. Less than 3','2. GE  3 and LT 4','3. GE  4 and LT 5','4. GE  5 and LT 6','5. GE  6 and LT 7','6. GT7'])

model_data['w_Tot_Cons_dec_util_12m']=np.select([(model_data['Tot_Cons_dec_util_12m']< 3) ,
( 3 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 4) ,
( 4 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 5) ,
( 5 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 6) ,
( 6 <= model_data['Tot_Cons_dec_util_12m']) & (model_data['Tot_Cons_dec_util_12m']< 7) ,
( 7 <= model_data['Tot_Cons_dec_util_12m'])],[-0.664544159044704,-0.310512935165184,-0.179724183500131,-0.00614837221310761,0.388272888431706,0.680527420249582])

model_data['b_Cons_inc_util_6m']=np.select([(model_data['Cons_inc_util_6m']< 2) ,
( 2 <= model_data['Cons_inc_util_6m']) & (model_data['Cons_inc_util_6m']< 3) ,
( 3 <= model_data['Cons_inc_util_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Cons_inc_util_6m']=np.select([(model_data['Cons_inc_util_6m']< 2) ,
( 2 <= model_data['Cons_inc_util_6m']) & (model_data['Cons_inc_util_6m']< 3) ,
( 3 <= model_data['Cons_inc_util_6m'])],[0.332581417382204,-0.0527716565213206,-0.591097146069058])

model_data['b_Cons_inc_util_12m']=np.select([(model_data['Cons_inc_util_12m']< 2) ,
( 2 <= model_data['Cons_inc_util_12m']) & (model_data['Cons_inc_util_12m']< 3) ,
( 3 <= model_data['Cons_inc_util_12m']) & (model_data['Cons_inc_util_12m']< 4) ,
( 4 <= model_data['Cons_inc_util_12m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

model_data['w_Cons_inc_util_12m']=np.select([(model_data['Cons_inc_util_12m']< 2) ,
( 2 <= model_data['Cons_inc_util_12m']) & (model_data['Cons_inc_util_12m']< 3) ,
( 3 <= model_data['Cons_inc_util_12m']) & (model_data['Cons_inc_util_12m']< 4) ,
( 4 <= model_data['Cons_inc_util_12m'])],[0.410165337574565,0.181692818934323,-0.170558648618225,-0.493817789237956])

model_data['b_Cons_inc_pay_mindue_6m']=np.select([(model_data['Cons_inc_pay_mindue_6m']< 2) ,
( 2 <= model_data['Cons_inc_pay_mindue_6m']) & (model_data['Cons_inc_pay_mindue_6m']< 3) ,
( 3 <= model_data['Cons_inc_pay_mindue_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Cons_inc_pay_mindue_6m']=np.select([(model_data['Cons_inc_pay_mindue_6m']< 2) ,
( 2 <= model_data['Cons_inc_pay_mindue_6m']) & (model_data['Cons_inc_pay_mindue_6m']< 3) ,
( 3 <= model_data['Cons_inc_pay_mindue_6m'])],[-0.205689324044534,0.323452429539485,0.962305979744173])

model_data['b_Cons_inc_pay_mindue_12m']=np.select([(model_data['Cons_inc_pay_mindue_12m']< 2) ,
( 2 <= model_data['Cons_inc_pay_mindue_12m']) & (model_data['Cons_inc_pay_mindue_12m']< 3) ,
( 3 <= model_data['Cons_inc_pay_mindue_12m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Cons_inc_pay_mindue_12m']=np.select([(model_data['Cons_inc_pay_mindue_12m']< 2) ,
( 2 <= model_data['Cons_inc_pay_mindue_12m']) & (model_data['Cons_inc_pay_mindue_12m']< 3) ,
( 3 <= model_data['Cons_inc_pay_mindue_12m'])],[-0.379613119964813,0.0831613215316766,0.650304724506751])

model_data['b_Cons_inc_pay_exp_6m']=np.select([(model_data['Cons_inc_pay_exp_6m']< 2) ,
( 2 <= model_data['Cons_inc_pay_exp_6m']) & (model_data['Cons_inc_pay_exp_6m']< 3) ,
( 3 <= model_data['Cons_inc_pay_exp_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Cons_inc_pay_exp_6m']=np.select([(model_data['Cons_inc_pay_exp_6m']< 2) ,
( 2 <= model_data['Cons_inc_pay_exp_6m']) & (model_data['Cons_inc_pay_exp_6m']< 3) ,
( 3 <= model_data['Cons_inc_pay_exp_6m'])],[-0.205689324044534,0.323452429539485,0.962305979744173])

model_data['b_Cons_inc_pay_exp_12m']=np.select([(model_data['Cons_inc_pay_exp_12m']< 2) ,
( 2 <= model_data['Cons_inc_pay_exp_12m']) & (model_data['Cons_inc_pay_exp_12m']< 3) ,
( 3 <= model_data['Cons_inc_pay_exp_12m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Cons_inc_pay_exp_12m']=np.select([(model_data['Cons_inc_pay_exp_12m']< 2) ,
( 2 <= model_data['Cons_inc_pay_exp_12m']) & (model_data['Cons_inc_pay_exp_12m']< 3) ,
( 3 <= model_data['Cons_inc_pay_exp_12m'])],[-0.379613119964813,0.0831613215316766,0.650304724506751])

model_data['b_Cons_dec_util_6m']=np.select([(model_data['Cons_dec_util_6m']< 2) ,
( 2 <= model_data['Cons_dec_util_6m']) & (model_data['Cons_dec_util_6m']< 3) ,
( 3 <= model_data['Cons_dec_util_6m'])],['1. Less than 2','2. GE  2 and LT 3','3. GT3'])

model_data['w_Cons_dec_util_6m']=np.select([(model_data['Cons_dec_util_6m']< 2) ,
( 2 <= model_data['Cons_dec_util_6m']) & (model_data['Cons_dec_util_6m']< 3) ,
( 3 <= model_data['Cons_dec_util_6m'])],[-0.272546324441087,0.185716969234048,0.590358693187007])

model_data['b_Cons_dec_util_12m']=np.select([(model_data['Cons_dec_util_12m']< 2) ,
( 2 <= model_data['Cons_dec_util_12m']) & (model_data['Cons_dec_util_12m']< 3) ,
( 3 <= model_data['Cons_dec_util_12m']) & (model_data['Cons_dec_util_12m']< 4) ,
( 4 <= model_data['Cons_dec_util_12m'])],['1. Less than 2','2. GE  2 and LT 3','3. GE  3 and LT 4','4. GT4'])

model_data['w_Cons_dec_util_12m']=np.select([(model_data['Cons_dec_util_12m']< 2) ,
( 2 <= model_data['Cons_dec_util_12m']) & (model_data['Cons_dec_util_12m']< 3) ,
( 3 <= model_data['Cons_dec_util_12m']) & (model_data['Cons_dec_util_12m']< 4) ,
( 4 <= model_data['Cons_dec_util_12m'])],[-0.416827597750551,-0.0568780577550414,0.319646530289075,0.622788084271731])

model_data['b_Age']=np.select([(model_data['Age']< 2) ,
( 2 <= model_data['Age']) & (model_data['Age']< 31) ,
( 31 <= model_data['Age']) & (model_data['Age']< 41) ,
( 41 <= model_data['Age'])],['Missing','2. GE  2 and LT 31','3. GE  31 and LT 41','4. GT41'])

# model_data['w_Age']=np.select([(model_data['Age']< 2) ,
# ( 2 <= model_data['Age']) & (model_data['Age']< 31) ,
# ( 31 <= model_data['Age']) & (model_data['Age']< 41) ,
# ( 41 <= model_data['Age'])],[0.478768837356711,0.20049320318067,-0.116587844125892,-0.48635751219066])

model_data['w_Age']=np.select([(model_data['Age']< 2) ,
( 2 <= model_data['Age']) & (model_data['Age']< 31) ,
( 31 <= model_data['Age']) & (model_data['Age']< 41) ,
( 41 <= model_data['Age'])],[-0.48635751219066,0.20049320318067,-0.116587844125892,-0.48635751219066])

model_data['b_A_salary']=np.select([(model_data['A_salary']< 2565) ,
( 2565 <= model_data['A_salary']) & (model_data['A_salary']< 21672.85) ,
( 21672.85 <= model_data['A_salary'])],['Missing','2. GE  2565 and LT 21672.85','3. GT21672.85'])

# model_data['w_A_salary']=np.select([(model_data['A_salary']< 2565) ,
# ( 2565 <= model_data['A_salary']) & (model_data['A_salary']< 21672.85) ,
# ( 21672.85 <= model_data['A_salary'])],[-0.371839729686403,0.562722558433838,0.99038665678037])

model_data['w_A_salary']=np.select([(model_data['A_salary']< 2565) ,
( 2565 <= model_data['A_salary']) & (model_data['A_salary']< 21672.85) ,
( 21672.85 <= model_data['A_salary'])],[0.562722558433838,0.562722558433838,0.99038665678037])

model_data['b_A_No_Depend']=np.select([(model_data['A_No_Depend']< 1) ,
( 1 <= model_data['A_No_Depend']) & (model_data['A_No_Depend']< 5) ,
( 5 <= model_data['A_No_Depend'])],['Missing','2. GE  1 and LT 5','3. GT5'])

# model_data['w_A_No_Depend']=np.select([(model_data['A_No_Depend']< 1) ,
# ( 1 <= model_data['A_No_Depend']) & (model_data['A_No_Depend']< 5) ,
# ( 5 <= model_data['A_No_Depend'])],[-0.0682225709283327,0.182823372581369,-0.00117919526016222])

model_data['w_A_No_Depend']=np.select([(model_data['A_No_Depend']< 1) ,
( 1 <= model_data['A_No_Depend']) & (model_data['A_No_Depend']< 5) ,
( 5 <= model_data['A_No_Depend'])],[-0.00117919526016222,0.182823372581369,-0.00117919526016222])

# Replacing Missing for Age , Salary and No of Dependent with WOE of worst or highest bad rate 


In [744]:
model_data['w_A_Emp_Sts']=np.select([(model_data['A_Emp_Sts'].isin([ '-555555'])) ,
(model_data['A_Emp_Sts'].isin([ 'EMPLOYED'])) ,
(model_data['A_Emp_Sts'].isin([ 'OTHER','RETIRED','SELF-EMPLOYED','STUDENT','UNEMPLOYED']))] ,
[0.0243284861424197,-0.0234773002711527,0.103202072415123])

model_data['b_A_Emp_Sts']=np.select([(model_data['A_Emp_Sts'].isin([ '-555555'])) ,
(model_data['A_Emp_Sts'].isin([ 'EMPLOYED'])) ,
(model_data['A_Emp_Sts'].isin([ 'OTHER','RETIRED','SELF-EMPLOYED','STUDENT','UNEMPLOYED']))] ,
['Missing','Salaried','Others'])

In [745]:
# month_since_0 and month_since_30

In [746]:
# model_data['b_month_since_30']=np.select([(model_data['month_since_30']< 3) & (model_data['month_since_30'] >=0),
# ( 3 <= model_data['month_since_30']) & (model_data['month_since_30']< 9) ,
# ( 9 <= model_data['month_since_30'] ) | ( model_data['month_since_30'] == -555555 )],['1. Less than 3','2. GE  3 and LT 9','3. GT 9 or Never'])


# model_data['w_month_since_30']=np.select([(model_data['month_since_30']< 3) & (model_data['month_since_30'] >=0),
# ( 3 <= model_data['month_since_30']) & (model_data['month_since_30']< 9) ,
# ( 9 <= model_data['month_since_30'] ) | ( model_data['month_since_30'] == -555555 )],[-2.21105659845512,-0.774744980953244,0.951875186103849])

# model_data['b_month_since_0']=np.select([(model_data['month_since_0']< 1)  & (model_data['month_since_0'] >=0),
# ( 1 <= model_data['month_since_0']) & (model_data['month_since_0']< 2) ,
# ( 2 <= model_data['month_since_0']) & (model_data['month_since_0']< 4) ,
# ( 4 <= model_data['month_since_0']) & (model_data['month_since_0']< 13) ,
# ( 13 <= model_data['month_since_0']) | ( model_data['month_since_0'] == -555555 )],['1. Less than 1','2. GE  1 and LT 2','3. GE  2 and LT 4','4. GE  4 and LT 13','5. GT 13 or Never'])


# model_data['w_month_since_0']=np.select([(model_data['month_since_0']< 1)  & (model_data['month_since_0'] >=0),
# ( 1 <= model_data['month_since_0']) & (model_data['month_since_0']< 2) ,
# ( 2 <= model_data['month_since_0']) & (model_data['month_since_0']< 4) ,
# ( 4 <= model_data['month_since_0']) & (model_data['month_since_0']< 13) ,
# ( 13 <= model_data['month_since_0']) | ( model_data['month_since_0'] == -555555 )],[-1.23600939238041,-0.161225149569837,0.424494323418099,1.18988945724461,1.69535394485202])

In [747]:
model_data['b_month_since_30']=np.select([(model_data['month_since_30']< 3) & (model_data['month_since_30'] >=0) ,
( 3 <= model_data['month_since_30']) & (model_data['month_since_30']< 11) ,
( 11 <= model_data['month_since_30']) | ( model_data['month_since_30'] == -555555 )],['1. Less than 3','2. GE  3 and LT 11','3. GT 11 or Never'])

model_data['w_month_since_30']=np.select([(model_data['month_since_30']< 3) & (model_data['month_since_30'] >=0) ,
( 3 <= model_data['month_since_30']) & (model_data['month_since_30']< 11) ,
( 11 <= model_data['month_since_30']) | ( model_data['month_since_30'] == -555555 )],[-2.21105659845512,-0.701861836936364,1.00898426523244])


model_data['b_month_since_0']=np.select([(model_data['month_since_0']< 1) & (model_data['month_since_0'] >=0) ,
( 1 <= model_data['month_since_0']) & (model_data['month_since_0']< 2) ,
( 2 <= model_data['month_since_0']) & (model_data['month_since_0']< 9) ,
( 9 <= model_data['month_since_0']) | ( model_data['month_since_0'] == -555555 )],['1. Less than 1','2. GE  1 and LT 2','3. GE  2 and LT 9','4. GT 9 or Never'])

model_data['w_month_since_0']=np.select([(model_data['month_since_0']< 1) & (model_data['month_since_0'] >=0) ,
( 1 <= model_data['month_since_0']) & (model_data['month_since_0']< 2) ,
( 2 <= model_data['month_since_0']) & (model_data['month_since_0']< 9) ,
( 9 <= model_data['month_since_0']) | ( model_data['month_since_0'] == -555555 )],[-1.23600939238041,-0.161225149569837,0.747665702736833,1.64256482524126])

In [748]:
#fetching woe col
woe_col=[]
for x in model_data.columns:
    if "w_" in x:
        woe_col.append(x)

In [749]:
len(woe_col)

121

In [750]:
woe_col

['w_nbr_60_12m',
 'w_nbr_30_6m',
 'w_nbr_30_3m',
 'w_nbr_30_12m',
 'w_nbr_0_3m',
 'w_max_dpd_6m',
 'w_max_dpd_3m',
 'w_max_dpd_12m',
 'w_Tot_Cons_inc_util_3m',
 'w_Tot_Cons_inc_pay_mindue_3m',
 'w_Tot_Cons_inc_pay_exp_3m',
 'w_Tot_Cons_inc_delinq_3m',
 'w_Cons_inc_util_3m',
 'w_Cons_inc_pay_mindue_3m',
 'w_Cons_inc_pay_exp_3m',
 'w_Cons_inc_delinq_6m',
 'w_Cons_inc_delinq_3m',
 'w_Cons_inc_delinq_12m',
 'w_Cons_dec_util_3m',
 'w_nbr_pay_gt_mindue_6m',
 'w_nbr_pay_gt_mindue_3m',
 'w_nbr_pay_gt_mindue_12m',
 'w_nbr_late_fee_6m',
 'w_nbr_late_fee_3m',
 'w_nbr_late_fee_12m',
 'w_nbr_0_6m',
 'w_nbr_0_12m',
 'w_mob',
 'w_avg_util_6m',
 'w_avg_util_3m',
 'w_avg_util_12m',
 'w_avg_perc_amt_utility_bills_6m',
 'w_avg_perc_amt_utility_bills_3m',
 'w_avg_perc_amt_utility_bills_12m',
 'w_avg_perc_amt_servicestation_6m',
 'w_avg_perc_amt_servicestation_3m',
 'w_avg_perc_amt_servicestation_12m',
 'w_avg_perc_amt_restaurant_6m',
 'w_avg_perc_amt_restaurant_3m',
 'w_avg_perc_amt_restaurant_12m',
 'w_a

In [751]:
list(model_data.columns)

['MASK_CREDITCARD',
 'snapshot_date',
 'Snapshot_Key',
 'month_since_0',
 'month_since_30',
 'month_since_60',
 'month_since_90',
 'max_dpd_3m',
 'max_dpd_6m',
 'max_dpd_12m',
 'avg_util_3m',
 'avg_util_6m',
 'avg_util_12m',
 'avg_cash_util_3m',
 'avg_cash_util_6m',
 'avg_cash_util_12m',
 'avg_amt_cash_withd_3m',
 'avg_amt_cash_withd_6m',
 'avg_amt_cash_withd_12m',
 'avg_amt_clothing_3m',
 'avg_amt_clothing_6m',
 'avg_amt_clothing_12m',
 'avg_perc_amt_clothing_3m',
 'avg_perc_amt_clothing_6m',
 'avg_perc_amt_clothing_12m',
 'avg_amt_utility_bills_3m',
 'avg_amt_utility_bills_6m',
 'avg_amt_utility_bills_12m',
 'avg_perc_amt_utility_bills_3m',
 'avg_perc_amt_utility_bills_6m',
 'avg_perc_amt_utility_bills_12m',
 'avg_amt_jewellery_3m',
 'avg_amt_jewellery_6m',
 'avg_amt_jewellery_12m',
 'avg_perc_amt_jewellery_3m',
 'avg_perc_amt_jewellery_6m',
 'avg_perc_amt_jewellery_12m',
 'avg_amt_airline_3m',
 'avg_amt_airline_6m',
 'avg_amt_airline_12m',
 'avg_perc_amt_airline_3m',
 'avg_perc_amt_

In [752]:
model = pd.DataFrame()
model[["MASK_CREDITCARD","snapshot_date","Snapshot_Key","bad_flag","F_build"]] = model_data[["MASK_CREDITCARD","snapshot_date","Snapshot_Key","bad_flag","F_build"]]
model[woe_col] = model_data[woe_col]



In [753]:
model.shape

(36324, 126)

In [754]:
model

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,bad_flag,F_build,w_nbr_60_12m,w_nbr_30_6m,w_nbr_30_3m,w_nbr_30_12m,w_nbr_0_3m,w_max_dpd_6m,w_max_dpd_3m,w_max_dpd_12m,w_Tot_Cons_inc_util_3m,w_Tot_Cons_inc_pay_mindue_3m,w_Tot_Cons_inc_pay_exp_3m,w_Tot_Cons_inc_delinq_3m,w_Cons_inc_util_3m,w_Cons_inc_pay_mindue_3m,w_Cons_inc_pay_exp_3m,w_Cons_inc_delinq_6m,w_Cons_inc_delinq_3m,w_Cons_inc_delinq_12m,w_Cons_dec_util_3m,w_nbr_pay_gt_mindue_6m,w_nbr_pay_gt_mindue_3m,w_nbr_pay_gt_mindue_12m,w_nbr_late_fee_6m,w_nbr_late_fee_3m,w_nbr_late_fee_12m,w_nbr_0_6m,w_nbr_0_12m,w_mob,w_avg_util_6m,w_avg_util_3m,w_avg_util_12m,w_avg_perc_amt_utility_bills_6m,w_avg_perc_amt_utility_bills_3m,w_avg_perc_amt_utility_bills_12m,w_avg_perc_amt_servicestation_6m,w_avg_perc_amt_servicestation_3m,w_avg_perc_amt_servicestation_12m,w_avg_perc_amt_restaurant_6m,w_avg_perc_amt_restaurant_3m,w_avg_perc_amt_restaurant_12m,w_avg_perc_amt_jewellery_6m,w_avg_perc_amt_jewellery_3m,w_avg_perc_amt_jewellery_12m,w_avg_perc_amt_hotel_6m,w_avg_perc_amt_hotel_3m,w_avg_perc_amt_hotel_12m,w_avg_perc_amt_grocery_6m,w_avg_perc_amt_grocery_3m,w_avg_perc_amt_grocery_12m,w_avg_perc_amt_education_12m,w_avg_perc_amt_clothing_6m,w_avg_perc_amt_clothing_3m,w_avg_perc_amt_clothing_12m,w_avg_perc_amt_autocashdisb_6m,w_avg_perc_amt_autocashdisb_3m,w_avg_perc_amt_autocashdisb_12m,w_avg_perc_amt_airline_6m,w_avg_perc_amt_airline_3m,w_avg_perc_amt_airline_12m,w_avg_perc_amt_FI_6m,w_avg_perc_amt_FI_3m,w_avg_perc_amt_FI_12m,w_avg_amt_utility_bills_6m,w_avg_amt_utility_bills_3m,w_avg_amt_utility_bills_12m,w_avg_amt_servicestation_6m,w_avg_amt_servicestation_3m,w_avg_amt_servicestation_12m,w_avg_amt_restaurant_6m,w_avg_amt_restaurant_3m,w_avg_amt_restaurant_12m,w_avg_amt_jewellery_6m,w_avg_amt_jewellery_3m,w_avg_amt_jewellery_12m,w_avg_amt_hotel_6m,w_avg_amt_hotel_3m,w_avg_amt_hotel_12m,w_avg_amt_grocery_6m,w_avg_amt_grocery_3m,w_avg_amt_grocery_12m,w_avg_amt_education_12m,w_avg_amt_clothing_6m,w_avg_amt_clothing_3m,w_avg_amt_clothing_12m,w_avg_amt_cash_withd_6m,w_avg_amt_cash_withd_3m,w_avg_amt_cash_withd_12m,w_avg_amt_autocashdisb_6m,w_avg_amt_autocashdisb_3m,w_avg_amt_autocashdisb_12m,w_avg_amt_airline_6m,w_avg_amt_airline_3m,w_avg_amt_airline_12m,w_avg_amt_FI_6m,w_avg_amt_FI_3m,w_avg_amt_FI_12m,w_Tot_Cons_inc_util_6m,w_Tot_Cons_inc_util_12m,w_Tot_Cons_inc_pay_mindue_6m,w_Tot_Cons_inc_pay_mindue_12m,w_Tot_Cons_inc_pay_exp_6m,w_Tot_Cons_inc_pay_exp_12m,w_Tot_Cons_inc_delinq_6m,w_Tot_Cons_inc_delinq_12m,w_Tot_Cons_dec_util_6m,w_Tot_Cons_dec_util_3m,w_Tot_Cons_dec_util_12m,w_Cons_inc_util_6m,w_Cons_inc_util_12m,w_Cons_inc_pay_mindue_6m,w_Cons_inc_pay_mindue_12m,w_Cons_inc_pay_exp_6m,w_Cons_inc_pay_exp_12m,w_Cons_dec_util_6m,w_Cons_dec_util_12m,w_Age,w_A_salary,w_A_No_Depend,w_A_Emp_Sts,w_month_since_30,w_month_since_0
1489,FTKUGGNNNSUNNNNT,2011-11-30,FTKUGGNNNSUNNNNT_2011-11-30,1.0,1,0.504176,0.838593,0.653455,1.024469,0.127457,0.310902,-0.060129,0.628634,0.061101,0.180546,0.180546,-0.881733,0.061101,0.180546,0.180546,-0.194565,-0.881733,0.232910,0.052562,0.160668,0.179337,0.308184,1.437966,1.073202,0.943788,1.366971,1.472789,0.063849,0.219722,0.256051,0.048973,-0.081369,-0.057669,-0.084082,-0.038651,-0.04029,-0.090135,-0.157473,-0.114388,-0.240012,0.050126,0.030632,-0.530192,-0.152917,-0.105071,-0.140132,-0.259255,-0.202604,-0.318352,-0.031892,-0.291089,-0.194841,-0.387581,0.292722,0.401357,0.236516,-0.083677,-0.050210,-0.115916,-0.042055,-0.034927,0.077035,-0.086333,-0.066097,-0.129326,-0.049576,-0.044972,-0.093837,-0.164354,-0.1151,-0.248358,0.037029,0.023391,-0.254091,-0.169776,-0.107112,-0.258754,-0.264044,-0.204937,-0.334343,-0.0331,-0.299588,-0.198442,-0.408386,0.039624,0.077324,0.241023,-0.015568,0.077324,0.240557,-0.096022,-0.053760,-0.161582,-0.283146,0.209366,-0.097313,0.276453,0.252844,0.473630,0.109769,0.473630,0.109769,0.802522,1.273841,0.274655,-0.129033,0.680527,-0.052772,0.181693,-0.205689,-0.379613,-0.205689,-0.379613,0.185717,0.622788,-0.116588,0.562723,-0.001179,-0.023477,1.00

In [755]:
model.dtypes

MASK_CREDITCARD                              object
snapshot_date                        datetime64[ns]
Snapshot_Key                                 object
bad_flag                                    float64
F_build                                       int64
w_nbr_60_12m                                float64
w_nbr_30_6m                                 float64
w_nbr_30_3m                                 float64
w_nbr_30_12m                                float64
w_nbr_0_3m                                  float64
w_max_dpd_6m                                float64
w_max_dpd_3m                                float64
w_max_dpd_12m                               float64
w_Tot_Cons_inc_util_3m                      float64
w_Tot_Cons_inc_pay_mindue_3m                float64
w_Tot_Cons_inc_pay_exp_3m                   float64
w_Tot_Cons_inc_delinq_3m                    float64
w_Cons_inc_util_3m                          float64
w_Cons_inc_pay_mindue_3m                    float64
w_Cons_inc_p

In [756]:
model_train = model[model['F_build'] ==1]

In [757]:
model_test =  model[model['F_build'] ==0]

In [758]:
model_train.shape

(25426, 126)

In [759]:
model_test.shape

(10898, 126)

In [760]:
#i8_cards= i8_cards[(i8_cards['valid_vintage'] ==1) & (i8_cards['max_mob'] >=18)]

In [761]:
iv_num_cat_filt1 = iv_num_cat[iv_num_cat['w_Variable'].isin(model_train.columns)]

In [762]:
iv_num_cat_filt1.shape

(121, 3)

In [763]:
iv_num_cat_filt1

,Variable,IV,w_Variable
0,avg_perc_amt_hotel_12m,inf,w_avg_perc_amt_hotel_12m
9,avg_perc_amt_FI_6m,inf,w_avg_perc_amt_FI_6m
0,A_Emp_Sts,inf,w_A_Emp_Sts
1,avg_amt_FI_12m,inf,w_avg_amt_FI_12m
10,avg_amt_FI_6m,inf,w_avg_amt_FI_6m
8,avg_amt_education_12m,inf,w_avg_amt_education_12m
6,avg_amt_autocashdisb_6m,inf,w_avg_amt_autocashdisb_6m
4,Tot_Cons_inc_pay_exp_6m,inf,w_Tot_Cons_inc_pay_exp_6m
3,Tot_Cons_inc_pay_mindue_6m,inf,w_Tot_Cons_inc_pay_mindue_6m
2,A_salary,inf,w_A_salary


# Following Template Format 

In [764]:
# new_template

In [765]:
#Required modules
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
pd.options.display.max_columns=None
pd.options.display.max_rows=None
import seaborn as sns
import csv
import warnings 
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
pd.set_option("display.max_columns",None)

from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import add_constant
import statsmodels.api as sm

#### Reading Data (Train+Test with fbuild)

In [766]:
i1_modDt = model_data

In [767]:
i1_modDt.shape

(36324, 417)

#### Subsetting only Train data

In [768]:
log_reg_train_data = i1_modDt[i1_modDt["F_build"]==1]  # Train Data
log_reg_train_data.shape

(25426, 417)

#### Filtering only WOE columns

In [769]:
filter_col=[col for col in log_reg_train_data if col.startswith('w_')]

In [770]:
filter_col

['w_nbr_60_12m',
 'w_nbr_30_6m',
 'w_nbr_30_3m',
 'w_nbr_30_12m',
 'w_nbr_0_3m',
 'w_max_dpd_6m',
 'w_max_dpd_3m',
 'w_max_dpd_12m',
 'w_Tot_Cons_inc_util_3m',
 'w_Tot_Cons_inc_pay_mindue_3m',
 'w_Tot_Cons_inc_pay_exp_3m',
 'w_Tot_Cons_inc_delinq_3m',
 'w_Cons_inc_util_3m',
 'w_Cons_inc_pay_mindue_3m',
 'w_Cons_inc_pay_exp_3m',
 'w_Cons_inc_delinq_6m',
 'w_Cons_inc_delinq_3m',
 'w_Cons_inc_delinq_12m',
 'w_Cons_dec_util_3m',
 'w_nbr_pay_gt_mindue_6m',
 'w_nbr_pay_gt_mindue_3m',
 'w_nbr_pay_gt_mindue_12m',
 'w_nbr_late_fee_6m',
 'w_nbr_late_fee_3m',
 'w_nbr_late_fee_12m',
 'w_nbr_0_6m',
 'w_nbr_0_12m',
 'w_mob',
 'w_avg_util_6m',
 'w_avg_util_3m',
 'w_avg_util_12m',
 'w_avg_perc_amt_utility_bills_6m',
 'w_avg_perc_amt_utility_bills_3m',
 'w_avg_perc_amt_utility_bills_12m',
 'w_avg_perc_amt_servicestation_6m',
 'w_avg_perc_amt_servicestation_3m',
 'w_avg_perc_amt_servicestation_12m',
 'w_avg_perc_amt_restaurant_6m',
 'w_avg_perc_amt_restaurant_3m',
 'w_avg_perc_amt_restaurant_12m',
 'w_a

In [771]:
len(filter_col)

121

In [772]:
#### Selecting variables for model iterations 

In [773]:
# filter_col=['w_nbr_30_12m','w_nbr_late_fee_3m','w_avg_util_12m', 'w_nbr_pay_gt_mindue_3m',
#            'w_Tot_Cons_inc_util_3m','w_Cons_inc_pay_mindue_12m','w_Tot_Cons_inc_delinq_6m','w_Cons_inc_pay_exp_3m']

In [774]:
# filter_col=['w_month_since_0','w_nbr_pay_gt_mindue_12m','w_Tot_Cons_inc_pay_exp_6m', 'w_avg_util_3m',
#            'w_nbr_late_fee_12m','w_max_dpd_3m','w_nbr_60_12m','w_Age','w_A_No_Depend']

In [775]:
# filter_col=['w_month_since_0','w_nbr_pay_gt_mindue_12m','w_Tot_Cons_inc_pay_exp_6m', 'w_avg_util_3m',
#            'w_nbr_late_fee_12m','w_max_dpd_3m','w_nbr_60_12m','w_Age','w_A_No_Depend','w_month_since_30']

In [776]:
# filter_col = ['w_nbr_late_fee_3m',
#  'w_month_since_30',
#  'w_Cons_inc_pay_exp_3m',
#  'w_nbr_60_12m',
#  'w_A_salary',
#  'w_Tot_Cons_inc_delinq_3m',
#  'w_Age',
#  'w_avg_util_3m',
#  'w_nbr_late_fee_12m',
#  'w_Tot_Cons_inc_util_3m',
#  'w_Tot_Cons_inc_delinq_6m',
#  'w_Cons_inc_util_12m',
#  'w_nbr_pay_gt_mindue_12m',
#  'w_nbr_pay_gt_mindue_3m',
#  'w_Tot_Cons_inc_pay_exp_6m',
#  'w_avg_util_12m',
#  'w_Cons_inc_pay_mindue_12m']

In [777]:
# filter_col =  ['w_nbr_late_fee_3m',
#  'w_month_since_30',
#  'w_nbr_60_12m',
#  'w_Cons_inc_pay_exp_3m',
#  'w_A_salary',
#  'w_Tot_Cons_inc_delinq_3m',
#  'w_Age',
#  'w_avg_util_3m',
#  'w_nbr_late_fee_12m',
#  'w_Tot_Cons_inc_util_3m',
#  'w_Tot_Cons_inc_delinq_6m',
#  'w_Cons_inc_util_12m',
#  'w_mob',
#  'w_nbr_pay_gt_mindue_12m',
#  'w_nbr_pay_gt_mindue_3m',
#  'w_Tot_Cons_inc_pay_exp_6m',
#  'w_avg_util_12m',
#  'w_Cons_inc_pay_mindue_12m',
#  'w_Tot_Cons_dec_util_12m']

In [778]:
filter_col = [
         'w_nbr_late_fee_12m',
         'w_max_dpd_3m',
         'w_avg_util_3m',
#        'w_Tot_Cons_inc_delinq_6m',
         'w_nbr_60_12m',
         'w_Age',
         'w_Tot_Cons_inc_pay_exp_6m',
         'w_A_No_Depend'
       ]

In [779]:
tf_data = log_reg_train_data[filter_col]
tf_data.shape

(25426, 7)

In [780]:
tf_data.head()

,w_nbr_late_fee_12m,w_max_dpd_3m,w_avg_util_3m,w_nbr_60_12m,w_Age,w_Tot_Cons_inc_pay_exp_6m,w_A_No_Depend
1489,0.943788,-0.060129,0.256051,0.504176,-0.116588,0.473630,-0.001179
1829,0.393390,-2.211057,0.256051,0.504176,-0.116588,0.473630,0.182823
859,1.553898,1.453554,0.256051,0.504176,0.200493,1.068065,-0.001179
1290,0.943788,1.453554,0.256051,0.504176,-0.116588,0.473630,-0.001179
1213,-1.299253,-2.211057,0.256051,0.504176,-0.486358,-0.608151,-0.001179


Calculating Correlation and removing variables

In [781]:
def corr_var_reduction(df,correlation,sorted_var_list,corr_cutoff):
    df_1=df[sorted_var_list]
    cor_matrix = df_1.corr(correlation).abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
    feat_list = upper_tri.index.tolist()
    drop_list = []
    for i in range(len(feat_list)):
        if feat_list[i] in drop_list :
            continue
        print("row is :",i)
        print("selected feature is : ",feat_list[i])
        for j in range(len(feat_list)):
            if upper_tri.iloc[i][j]>corr_cutoff:
                if (j>i) & (feat_list[j] not in drop_list) :
                    print("correlated feature : ",i,j,feat_list[i],feat_list[j])
                    drop_list.append(feat_list[j])
    final_vars=list(set(feat_list)-set(drop_list))
    return final_vars

In [782]:
iv_num_cat_filt1 = iv_num_cat[iv_num_cat['w_Variable'].isin(filter_col)]

In [783]:
iv_num_cat_filt1.shape

(7, 3)

In [784]:
iv_num_cat_filt1

,Variable,IV,w_Variable
4,Tot_Cons_inc_pay_exp_6m,inf,w_Tot_Cons_inc_pay_exp_6m
11,nbr_late_fee_12m,27.754910,w_nbr_late_fee_12m
16,avg_util_3m,16.647240,w_avg_util_3m
32,max_dpd_3m,5.478813,w_max_dpd_3m
33,Age,5.149994,w_Age
52,nbr_60_12m,2.226748,w_nbr_60_12m
122,A_No_Depend,0.232597,w_A_No_Depend


In [785]:
final_vars=corr_var_reduction(tf_data,"pearson",list(iv_num_cat_filt1["w_Variable"]),0.75)

row is : 0
selected feature is :  w_Tot_Cons_inc_pay_exp_6m
row is : 1
selected feature is :  w_nbr_late_fee_12m
row is : 2
selected feature is :  w_avg_util_3m
row is : 3
selected feature is :  w_max_dpd_3m
row is : 4
selected feature is :  w_Age
row is : 5
selected feature is :  w_nbr_60_12m
row is : 6
selected feature is :  w_A_No_Depend


In [786]:
len(final_vars)

7

In [787]:
final_vars1 = final_vars

In [788]:
tf_data1 = log_reg_train_data[final_vars1]
tf_data1.shape

(25426, 7)

In [789]:
iv_num_cat_filt2 = iv_num_cat[iv_num_cat['w_Variable'].isin(final_vars1)]

In [790]:
iv_num_cat_filt2.shape

(7, 3)

In [791]:
final_vars=corr_var_reduction(tf_data1,"pearson",list(iv_num_cat_filt2["w_Variable"]),0.75)

row is : 0
selected feature is :  w_Tot_Cons_inc_pay_exp_6m
row is : 1
selected feature is :  w_nbr_late_fee_12m
row is : 2
selected feature is :  w_avg_util_3m
row is : 3
selected feature is :  w_max_dpd_3m
row is : 4
selected feature is :  w_Age
row is : 5
selected feature is :  w_nbr_60_12m
row is : 6
selected feature is :  w_A_No_Depend


In [792]:
len(final_vars)

7

In [793]:
final_vars

['w_nbr_60_12m',
 'w_avg_util_3m',
 'w_max_dpd_3m',
 'w_Age',
 'w_nbr_late_fee_12m',
 'w_Tot_Cons_inc_pay_exp_6m',
 'w_A_No_Depend']

In [794]:
# VIF on all independent columns
# vif_data1 = bureau_factor_6[woe_col]
# vif2 = vif(vif_data1).sort_values(by = 'VIF')
# vif2

In [795]:
#sorted_var_list = list(iv["Variable"])

In [796]:
#sorted_var_list


In [797]:
#df_1=train_num_bin[sorted_var_list]

In [798]:
#df_1

In [799]:
#cor_matrix = df_1.corr("pearson").abs()

In [800]:
#cor_matrix

In [801]:
#upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))

In [802]:
#upper_tri

In [803]:
#feat_list = upper_tri.index.tolist()

In [804]:
#feat_list

In [805]:
#len(feat_list)

In [806]:
#check_corr

Correlation

In [807]:
# def corr_var_reduction(df,correlation,sorted_var_list,corr_cutoff):
#     df_1=df[sorted_var_list]
#     cor_matrix = df_1.corr(correlation).abs()
#     upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))
#     feat_list = upper_tri.index.tolist()
#     drop_list = []
#     for i in range(len(feat_list)):
#         if feat_list[i] in drop_list :
#             continue
#         print("row is :",i)
#         print("selected feature is : ",feat_list[i])
#         for j in range(len(feat_list)):
#             if upper_tri.iloc[i][j]>corr_cutoff:
#                 if (j>i) & (feat_list[j] not in drop_list) :
#                     print("correlated feature : ",i,j,feat_list[i],feat_list[j])
#                     drop_list.append(feat_list[j])
#     final_vars=list(set(feat_list)-set(drop_list))
#     return final_vars

In [808]:
#corr_ordered_list = list(iv_corr["Variable"])

In [809]:
#corr_var_reduction(model_train,"pearson",list(iv_num_cat_filt1["w_Variable"]),0.5)

In [810]:
#vars_after_corr =corr_var_reduction(model_train,"pearson",list(iv_num_cat_filt1["w_Variable"]),0.5)

In [811]:
#vars_after_corr

In [812]:
#len(vars_after_corr)

In [813]:

#after_corr_cols= model_train[model_train.columns[model_train.columns.isin(vars_after_corr)]]

In [814]:
#after_corr_cols.shape

In [815]:
#after_corr_cols

In [816]:
# model_train_after_corr = pd.DataFrame()
# model_train_after_corr[["MASK_CREDITCARD","snapshot_date","Snapshot_Key","bad_flag","F_build"]] = model_train[["MASK_CREDITCARD","snapshot_date","Snapshot_Key","bad_flag","F_build"]]
# model_train_after_corr[list(after_corr_cols.columns)] = model_train[list(after_corr_cols.columns)]

In [817]:
# model_train_after_corr.shape

In [818]:
# model_train_after_corr

Calculating VIF

In [819]:
tf_data_2 = tf_data[final_vars]
tf_data_2.shape

(25426, 7)

In [820]:
tf_data_2.head()

,w_nbr_60_12m,w_avg_util_3m,w_max_dpd_3m,w_Age,w_nbr_late_fee_12m,w_Tot_Cons_inc_pay_exp_6m,w_A_No_Depend
1489,0.504176,0.256051,-0.060129,-0.116588,0.943788,0.473630,-0.001179
1829,0.504176,0.256051,-2.211057,-0.116588,0.393390,0.473630,0.182823
859,0.504176,0.256051,1.453554,0.200493,1.553898,1.068065,-0.001179
1290,0.504176,0.256051,1.453554,-0.116588,0.943788,0.473630,-0.001179
1213,0.504176,0.256051,-2.211057,-0.486358,-1.299253,-0.608151,-0.001179


In [821]:
def vif(benchmark_dataset, var_list):
    vif_output = pd.DataFrame()
    temp=var_list
    def calculate_vif(df, var_no):
        temp_vif=pd.DataFrame()
        temp_vif['variable_name']=[var_list[var_no]]
        temp_vif['VIF']=variance_inflation_factor(add_constant(df[var_list]).values, var_no+1)
        return temp_vif
    for i in range(len(var_list)):
        temp_vif = calculate_vif(benchmark_dataset[temp], i)
        vif_output=vif_output.append(temp_vif)
            
    return vif_output.sort_values('VIF').reset_index(drop=True)

In [822]:
vif_output=vif(tf_data_2, var_list=final_vars)
vif_output

,variable_name,VIF
0,w_Tot_Cons_inc_pay_exp_6m,1.017421
1,w_A_No_Depend,1.023082
2,w_Age,1.038899
3,w_avg_util_3m,1.133333
4,w_nbr_60_12m,1.312731
5,w_nbr_late_fee_12m,2.298163
6,w_max_dpd_3m,2.404962


In [823]:
vif_output= vif_output.sort_values(by = 'VIF',ascending=False)

In [824]:
vif_output

,variable_name,VIF
6,w_max_dpd_3m,2.404962
5,w_nbr_late_fee_12m,2.298163
4,w_nbr_60_12m,1.312731
3,w_avg_util_3m,1.133333
2,w_Age,1.038899
1,w_A_No_Depend,1.023082
0,w_Tot_Cons_inc_pay_exp_6m,1.017421


In [825]:
final_vars_2 = vif_output[vif_output['VIF'] <= 5]['variable_name'].to_list()
len(final_vars_2)

7

In [826]:
final_vars_2

['w_max_dpd_3m',
 'w_nbr_late_fee_12m',
 'w_nbr_60_12m',
 'w_avg_util_3m',
 'w_Age',
 'w_A_No_Depend',
 'w_Tot_Cons_inc_pay_exp_6m']

In [827]:
X = log_reg_train_data[final_vars_2]
y = log_reg_train_data[['bad_flag']]
y.shape,X.shape

((25426, 1), (25426, 7))

In [828]:
log_reg_train_data.head()

,MASK_CREDITCARD,snapshot_date,Snapshot_Key,month_since_0,month_since_30,month_since_60,month_since_90,max_dpd_3m,max_dpd_6m,max_dpd_12m,avg_util_3m,avg_util_6m,avg_util_12m,avg_cash_util_3m,avg_cash_util_6m,avg_cash_util_12m,avg_amt_cash_withd_3m,avg_amt_cash_withd_6m,avg_amt_cash_withd_12m,avg_amt_clothing_3m,avg_amt_clothing_6m,avg_amt_clothing_12m,avg_perc_amt_clothing_3m,avg_perc_amt_clothing_6m,avg_perc_amt_clothing_12m,avg_amt_utility_bills_3m,avg_amt_utility_bills_6m,avg_amt_utility_bills_12m,avg_perc_amt_utility_bills_3m,avg_perc_amt_utility_bills_6m,avg_perc_amt_utility_bills_12m,avg_amt_jewellery_3m,avg_amt_jewellery_6m,avg_amt_jewellery_12m,avg_perc_amt_jewellery_3m,avg_perc_amt_jewellery_6m,avg_perc_amt_jewellery_12m,avg_amt_airline_3m,avg_amt_airline_6m,avg_amt_airline_12m,avg_perc_amt_airline_3m,avg_perc_amt_airline_6m,avg_perc_amt_airline_12m,avg_echannel_3m,avg_echannel_6m,avg_echannel_12m,avg_amt_education_3m,avg_amt_education_6m,avg_amt_education_12m,avg_perc_amt_education_3m,avg_perc_amt_education_6m,avg_perc_amt_education_12m,avg_amt_hotel_3m,avg_amt_hotel_6m,avg_amt_hotel_12m,avg_perc_amt_hotel_3m,avg_perc_amt_hotel_6m,avg_perc_amt_hotel_12m,avg_amt_restaurant_3m,avg_amt_restaurant_6m,avg_amt_restaurant_12m,avg_perc_amt_restaurant_3m,avg_perc_amt_restaurant_6m,avg_perc_amt_restaurant_12m,avg_amt_servicestation_3m,avg_amt_servicestation_6m,avg_amt_servicestation_12m,avg_perc_amt_servicestation_3m,avg_perc_amt_servicestation_6m,avg_perc_amt_servicestation_12m,avg_amt_autocashdisb_3m,avg_amt_autocashdisb_6m,avg_amt_autocashdisb_12m,avg_perc_amt_autocashdisb_3m,avg_perc_amt_autocashdisb_6m,avg_perc_amt_autocashdisb_12m,avg_amt_grocery_3m,avg_amt_grocery_6m,avg_amt_grocery_12m,avg_perc_amt_grocery_3m,avg_perc_amt_grocery_6m,avg_perc_amt_grocery_12m,avg_amt_FI_3m,avg_amt_FI_6m,avg_amt_FI_12m,avg_perc_amt_FI_3m,avg_perc_amt_FI_6m,avg_perc_amt_FI_12m,nbr_late_fee_3m,nbr_late_fee_6m,nbr_late_fee_12m,nbr_cash_trans_3m,nbr_cash_trans_6m,nbr_cash_trans_12m,nbr_0_3m,nbr_0_6m,nbr_0_12m,nbr_30_3m,nbr_30_6m,nbr_30_12m,nbr_60_3m,nbr_60_6m,nbr_60_12m,nbr_90_3m,nbr_90_6m,nbr_90_12m,nbr_pay_gt_mindue_3m,nbr_pay_gt_mindue_6m,nbr_pay_gt_mindue_12m,Cons_inc_delinq_12m,Cons_dec_delinq_12m,Tot_Cons_inc_delinq_12m,Tot_Cons_dec_delinq_12m,Cons_inc_delinq_6m,Cons_dec_delinq_6m,Tot_Cons_inc_delinq_6m,Tot_Cons_dec_delinq_6m,Cons_inc_delinq_3m,Cons_dec_delinq_3m,Tot_Cons_inc_delinq_3m,Tot_Cons_dec_delinq_3m,Cons_inc_util_12m,Cons_dec_util_12m,Tot_Cons_inc_util_12m,Tot_Cons_dec_util_12m,Cons_inc_util_6m,Cons_dec_util_6m,Tot_Cons_inc_util_6m,Tot_Cons_dec_util_6m,Cons_inc_util_3m,Cons_dec_util_3m,Tot_Cons_inc_util_3m,Tot_Cons_dec_util_3m,Cons_inc_pay_exp_12m,Cons_dec_pay_exp_12m,Tot_Cons_inc_pay_exp_12m,Tot_Cons_dec_pay_exp_12m,Cons_inc_pay_exp_6m,Cons_dec_pay_exp_6m,Tot_Cons_inc_pay_exp_6m,Tot_Cons_dec_pay_exp_6m,Cons_inc_pay_exp_3m,Cons_dec_pay_exp_3m,Tot_Cons_inc_pay_exp_3m,Tot_Cons_dec_pay_exp_3m,Cons_inc_pay_mindue_12m,Cons_dec_pay_mindue_12m,Tot_Cons_inc_pay_mindue_12m,Tot_Cons_dec_pay_mindue_12m,Cons_inc_pay_mindue_6m,Cons_dec_pay_mindue_6m,Tot_Cons_inc_pay_mindue_6m,Tot_Cons_dec_pay_mindue_6m,Cons_inc_pay_mindue_3m,Cons_dec_pay_mindue_3m,Tot_Cons_inc_pay_mindue_3m,Tot_Cons_dec_pay_mindue_3m,CARD_LIMIT,OS_BALANCE,NB_DAYS_DELINQ,Customer_Occupation,A_Gender,A_Marital_Status,A_Industry,A_Emp_Sts,A_No_Depend,A_salary,A_Sector,mob,bad_def,bad_flag,REGION,NATIONALITY,Age,F_build,b_nbr_60_12m,w_nbr_60_12m,b_nbr_30_6m,w_nbr_30_6m,b_nbr_30_3m,w_nbr_30_3m,b_nbr_30_12m,w_nbr_30_12m,b_nbr_0_3m,w_nbr_0_3m,b_max_dpd_6m,w_max_dpd_6m,b_max_dpd_3m,w_max_dpd_3m,b_max_dpd_12m,w_max_dpd_12m,b_Tot_Cons_inc_util_3m,w_Tot_Cons_inc_util_3m,b_Tot_Cons_inc_pay_mindue_3m,w_Tot_Cons_inc_pay_mindue_3m,b_Tot_Cons_inc_pay_exp_3m,w_Tot_Cons_inc_pay_exp_3m,b_Tot_Cons_inc_delinq_3m,w_Tot_Cons_inc_delinq_3m,b_Cons_inc_util_3m,w_Cons_inc_util_3m,b_Cons_inc_pay_mindue_3m,w_Cons_inc_pay_mindue_3m,b_Cons_inc_pay_exp_3m,w_Cons_inc_pay_exp_3m,b_Cons_inc_delinq_6m,w_Cons_

In [829]:
log_reg_train_data.shape

(25426, 417)

Running Stepwise Logistic regression

In [830]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.05, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit(disp=False)
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.index[new_pval.argmin()]
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit(disp=False)
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.index[pvalues.argmax()]
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [831]:
result = stepwise_selection(X, y,threshold_in=0.05, threshold_out = 0.05)

Add  w_nbr_60_12m                   with p-value 0.0
Add  w_max_dpd_3m                   with p-value 1.24427e-229
Add  w_nbr_late_fee_12m             with p-value 1.06307e-25
Add  w_Tot_Cons_inc_pay_exp_6m      with p-value 2.94873e-18
Add  w_Age                          with p-value 1.09272e-13
Add  w_avg_util_3m                  with p-value 6.26431e-09


In [832]:
result

['w_nbr_60_12m',
 'w_max_dpd_3m',
 'w_nbr_late_fee_12m',
 'w_Tot_Cons_inc_pay_exp_6m',
 'w_Age',
 'w_avg_util_3m']

In [833]:
# feat1 = ['w_nbr_late_fee_12m','w_nbr_60_12m','w_Tot_Cons_inc_util_3m','w_A_salary','w_Age','w_Tot_Cons_inc_pay_exp_6m',
#          'w_A_No_Depend','w_mob']

In [834]:
# feat1 = [
#          'w_nbr_late_fee_12m',
#          'w_max_dpd_3m',
#          'w_avg_util_3m',
#          'w_Tot_Cons_inc_delinq_6m',
#          'w_nbr_60_12m',
#          #'w_Tot_Cons_inc_util_3m',
#          #'w_A_salary',
#          'w_Age',
#          'w_Tot_Cons_inc_pay_exp_6m',
#          'w_A_No_Depend'
#          #'w_mob'
#         ]

In [835]:
# feat1=[
#     'w_nbr_30_12m',
#     'w_nbr_late_fee_3m',
#     'w_avg_util_12m', 
#     'w_nbr_pay_gt_mindue_3m',
#     'w_Tot_Cons_inc_util_3m',
#     'w_Cons_inc_pay_mindue_12m',
#     'w_Tot_Cons_inc_delinq_6m',
#     'w_Cons_inc_pay_exp_3m'
# ]

In [836]:
# feat1= ['w_max_dpd_3m',
#  'w_nbr_60_12m',
#  'w_nbr_late_fee_12m',
#  'w_Tot_Cons_inc_pay_exp_6m',
#  'w_Age',
#  'w_month_since_0',
#  'w_avg_util_3m',
#  'w_nbr_pay_gt_mindue_12m']

In [837]:
# feat1 = ['w_nbr_late_fee_12m',
#  'w_month_since_30',
#  'w_nbr_60_12m',
#  'w_Tot_Cons_inc_pay_exp_6m',
#  'w_Age',
#  'w_avg_util_3m',
#  'w_nbr_pay_gt_mindue_12m']

In [838]:
# feat1 =['w_nbr_late_fee_12m',
#  'w_month_since_30',
#  'w_nbr_late_fee_3m',
#  'w_nbr_60_12m',
#  'w_Cons_inc_pay_exp_3m',
#  'w_A_salary',
#  'w_Age',
#  'w_Tot_Cons_inc_delinq_3m',
#  'w_avg_util_3m',
#  'w_Tot_Cons_inc_util_3m',
#  'w_Cons_inc_util_12m',
#  'w_Tot_Cons_inc_delinq_6m',
#  'w_nbr_pay_gt_mindue_12m',
#  'w_nbr_pay_gt_mindue_3m',
#  'w_Tot_Cons_inc_pay_exp_6m',
#  'w_avg_util_12m']

In [839]:
# feat1 =  ['w_nbr_late_fee_12m',
#  'w_month_since_30',
#  'w_nbr_60_12m',
#  'w_Age',
#  'w_Tot_Cons_inc_delinq_3m',
#  'w_avg_util_3m',
#  'w_nbr_pay_gt_mindue_3m',
#  'w_Tot_Cons_inc_pay_exp_6m']

In [840]:
feat1 = [
        'w_nbr_late_fee_12m',
         'w_max_dpd_3m',
         'w_avg_util_3m',
#        'w_Tot_Cons_inc_delinq_6m',
         'w_nbr_60_12m',
         'w_Age',
         'w_Tot_Cons_inc_pay_exp_6m',
         'w_A_No_Depend'
       ]

In [841]:
iv_num_cat_filt3 = iv_num_cat[iv_num_cat['w_Variable'].isin(X[feat1].columns)]


In [842]:
iv_num_cat_filt3

,Variable,IV,w_Variable
4,Tot_Cons_inc_pay_exp_6m,inf,w_Tot_Cons_inc_pay_exp_6m
11,nbr_late_fee_12m,27.754910,w_nbr_late_fee_12m
16,avg_util_3m,16.647240,w_avg_util_3m
32,max_dpd_3m,5.478813,w_max_dpd_3m
33,Age,5.149994,w_Age
52,nbr_60_12m,2.226748,w_nbr_60_12m
122,A_No_Depend,0.232597,w_A_No_Depend


In [843]:
final_vars3=corr_var_reduction(X[feat1],"pearson",list(iv_num_cat_filt3["w_Variable"]),0.75)

row is : 0
selected feature is :  w_Tot_Cons_inc_pay_exp_6m
row is : 1
selected feature is :  w_nbr_late_fee_12m
row is : 2
selected feature is :  w_avg_util_3m
row is : 3
selected feature is :  w_max_dpd_3m
row is : 4
selected feature is :  w_Age
row is : 5
selected feature is :  w_nbr_60_12m
row is : 6
selected feature is :  w_A_No_Depend


In [844]:
vif_output=vif(X[feat1], var_list=final_vars3)
vif_output

,variable_name,VIF
0,w_Tot_Cons_inc_pay_exp_6m,1.017421
1,w_A_No_Depend,1.023082
2,w_Age,1.038899
3,w_avg_util_3m,1.133333
4,w_nbr_60_12m,1.312731
5,w_nbr_late_fee_12m,2.298163
6,w_max_dpd_3m,2.404962


In [845]:
# vif_feat1 = vif(X[feat1]).sort_values(by = 'VIF',ascending=False)
# vif_feat1

In [846]:
log_reg=sm.Logit(y,sm.add_constant(X[feat1]),disp=True).fit(disp=False)
log_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               bad_flag   No. Observations:                25426
Model:                          Logit   Df Residuals:                    25418
Method:                           MLE   Df Model:                            7
Date:                Tue, 31 Oct 2023   Pseudo R-squ.:                  0.2641
Time:                        15:52:00   Log-Likelihood:                -4044.6
converged:                       True   LL-Null:                       -5496.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.9912      0.049    -60.615      0.000      -3.088      -2.894
w_nbr_late_fee_12m           -0.3865      0.037    -10.516      0.000      -0.459      -0.314
w_max_dpd_3m                 -0.5746      0.031    -18.569      0.000      -0.635      -0.514
w_avg_util_3m                -0.2901      0.050     -5.785      0.000      -0.388      -0.192
w_nbr_60_12m                 -0.3674      0.029    -12.860      0.000      -0.423      -0.311
w_Age                        -0.8839      0.118     -7.521      0.000      -1.114      -0.654
w_Tot_Cons_inc_pay_exp_6m    -0.5739      0.070     -8.220      0.000      -0.711      -0.437
w_A_No_Depend                -0.4988      0.390     -1.279      0.201      -1.263       0.265
=============================================================================================
"""

Removing Variables with positive coefficients and running logistic Regression again

In [847]:
var = pd.DataFrame(log_reg.params)
var = var[var[0]<0].reset_index()
var.columns = ['variable', 'Coef']
var_list = var[var['variable'] != 'const']['variable'].to_list()
var_list

['w_nbr_late_fee_12m',
 'w_max_dpd_3m',
 'w_avg_util_3m',
 'w_nbr_60_12m',
 'w_Age',
 'w_Tot_Cons_inc_pay_exp_6m',
 'w_A_No_Depend']

In [848]:
log_reg_2=sm.Logit(y,sm.add_constant(X[var_list]),disp=True).fit(disp=False)
log_reg_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               bad_flag   No. Observations:                25426
Model:                          Logit   Df Residuals:                    25418
Method:                           MLE   Df Model:                            7
Date:                Tue, 31 Oct 2023   Pseudo R-squ.:                  0.2641
Time:                        15:52:00   Log-Likelihood:                -4044.6
converged:                       True   LL-Null:                       -5496.2
Covariance Type:            nonrobust   LLR p-value:                     0.000
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                        -2.9912      0.049    -60.615      0.000      -3.088      -2.894
w_nbr_late_fee_12m           -0.3865      0.037    -10.516      0.000      -0.459      -0.314
w_max_dpd_3m                 -0.5746      0.031    -18.569      0.000      -0.635      -0.514
w_avg_util_3m                -0.2901      0.050     -5.785      0.000      -0.388      -0.192
w_nbr_60_12m                 -0.3674      0.029    -12.860      0.000      -0.423      -0.311
w_Age                        -0.8839      0.118     -7.521      0.000      -1.114      -0.654
w_Tot_Cons_inc_pay_exp_6m    -0.5739      0.070     -8.220      0.000      -0.711      -0.437
w_A_No_Depend                -0.4988      0.390     -1.279      0.201      -1.263       0.265
=============================================================================================
"""

#### 4.0 Scoring/PD Calculation¶

In [849]:
df=i1_modDt[['Snapshot_Key',
 'bad_flag',
'b_nbr_late_fee_12m',
'b_max_dpd_3m',
'b_avg_util_3m',
'b_nbr_60_12m',
'b_Age',
'b_Tot_Cons_inc_pay_exp_6m',
'b_A_No_Depend',
             
             
'w_nbr_late_fee_12m',
'w_max_dpd_3m',
'w_avg_util_3m',
'w_nbr_60_12m',
'w_Age',
'w_Tot_Cons_inc_pay_exp_6m',
'w_A_No_Depend',
 'F_build'
      ]]

In [850]:
# df=i1_modDt[['Snapshot_Key',
#  'bad_flag',
# 'b_nbr_30_12m',
# 'b_nbr_late_fee_3m',
# 'b_avg_util_12m',
# 'b_nbr_pay_gt_mindue_3m',
# 'b_Tot_Cons_inc_util_3m',
# 'b_Tot_Cons_inc_delinq_6m',
# 'b_Cons_inc_pay_exp_3m',
             
             
# 'w_nbr_30_12m',
# 'w_nbr_late_fee_3m',
# 'w_avg_util_12m',
# 'w_nbr_pay_gt_mindue_3m',
# 'w_Tot_Cons_inc_util_3m',
# 'w_Tot_Cons_inc_delinq_6m',
# 'w_Cons_inc_pay_exp_3m',
#  'F_build'
#       ]]

In [851]:
# df=i1_modDt[['Snapshot_Key',
#  'bad_flag',
# 'b_max_dpd_3m',
# 'b_nbr_60_12m',
# 'b_nbr_late_fee_12m',
# 'b_Tot_Cons_inc_pay_exp_6m',
# 'b_Age',
# 'b_month_since_0',
# 'b_avg_util_3m',
             
             
# 'w_max_dpd_3m',
# 'w_nbr_60_12m',
# 'w_nbr_late_fee_12m',
# 'w_Tot_Cons_inc_pay_exp_6m',
# 'w_Age',
# 'w_month_since_0',
# 'w_avg_util_3m',
#  'F_build'
#       ]]

In [852]:
# df=i1_modDt[['Snapshot_Key',
#  'bad_flag',
# 'b_nbr_late_fee_12m',
# 'b_month_since_30',
# 'b_nbr_60_12m',
# 'b_Age',
# 'b_Tot_Cons_inc_delinq_3m',
# 'b_avg_util_3m',
# 'b_nbr_pay_gt_mindue_3m',
# 'b_Tot_Cons_inc_pay_exp_6m',
             
             
# 'w_nbr_late_fee_12m',
# 'w_month_since_30',
# 'w_nbr_60_12m',
# 'w_Age',
# 'w_Tot_Cons_inc_delinq_3m',
# 'w_avg_util_3m',
# 'w_nbr_pay_gt_mindue_3m',
# 'w_Tot_Cons_inc_pay_exp_6m',
# 'F_build'
#       ]]

In [853]:
df.shape

(36324, 17)

In [854]:
df.head()

,Snapshot_Key,bad_flag,b_nbr_late_fee_12m,b_max_dpd_3m,b_avg_util_3m,b_nbr_60_12m,b_Age,b_Tot_Cons_inc_pay_exp_6m,b_A_No_Depend,w_nbr_late_fee_12m,w_max_dpd_3m,w_avg_util_3m,w_nbr_60_12m,w_Age,w_Tot_Cons_inc_pay_exp_6m,w_A_No_Depend,F_build
1489,FTKUGGNNNSUNNNNT_2011-11-30,1.0,2. GE 0.1 and LT 0.18,2. GE 1 and LT 2,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,Missing,0.943788,-0.060129,0.256051,0.504176,-0.116588,0.473630,-0.001179,1
1829,FTKUGGNNWSSWWNTG_2011-12-31,0.0,3. GE 0.18 and LT 0.43,3. GT2,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,2. GE 1 and LT 5,0.393390,-2.211057,0.256051,0.504176,-0.116588,0.473630,0.182823,1
859,FTKUGFNNTFTNPSWF_2011-02-28,0.0,1. Less than 0.1,1. Less than 1,1. Less than 0.98,1. Less than 0.08,2. GE 2 and LT 31,4. GT4,Missing,1.553898,1.453554,0.256051,0.504176,0.200493,1.068065,-0.001179,1
1290,FTKUGGNNKWKCGKNS_2011-09-30,0.0,2. GE 0.1 and LT 0.18,1. Less than 1,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,Missing,0.943788,1.453554,0.256051,0.504176,-0.116588,0.473630,-0.001179,1
1213,FTKUGGNNNFWCNNNW_2011-03-31,1.0,4. GT0.43,3. GT2,1. Less than 0.98,1. Less than 0.08,4. GT41,1. Less than 2,Missing,-1.299253,-2.211057,0.256051,0.504176,-0.486358,-0.608151,-0.001179,1


In [855]:
const	=	-0.427314286	
x1	=	-0.3865	
x2	=	-0.5746	
x3	=	-0.2901	
x4	=	-0.3674	
x5	=	-0.8839	
x6	=	-0.5739	
x7	=	-0.4988	
x8	=	0	
x9	=	0	
x10	=	0	
x11	=	0	
x12	=	0	
x13	=	0	
x14	=	0	
x15	=	0	
x16	=	0	
x17	=	0	
x18	=	0	
x19	=	0	
x20	=	0	
			
df['S_nbr_late_fee_12m']=	np.round(71.43 - (28.85) * (const+x1*df['w_nbr_late_fee_12m']),0).astype(int)		
df['S_max_dpd_3m']=	np.round(71.43 - (28.85) * (const+x2*df['w_max_dpd_3m']),0).astype(int)		
df['S_avg_util_3m']=	np.round(71.43 - (28.85) * (const+x3*df['w_avg_util_3m']),0).astype(int)		
df['S_nbr_60_12m']=	np.round(71.43 - (28.85) * (const+x4*df['w_nbr_60_12m']),0).astype(int)		
df['S_Age']=	np.round(71.43 - (28.85) * (const+x5*df['w_Age']),0).astype(int)		
df['S_Tot_Cons_inc_pay_exp_6m']=	np.round(71.43 - (28.85) * (const+x6*df['w_Tot_Cons_inc_pay_exp_6m']),0).astype(int)		
df['S_A_No_Depend']=	np.round(71.43 - (28.85) * (const+x7*df['w_A_No_Depend']),0).astype(int)		
			
			
			
			
			
			
			
			
			
			
			
			
			
			
df['ln_odds']=df['w_nbr_late_fee_12m']*x1+df['w_max_dpd_3m']*x2+df['w_avg_util_3m']*x3+df['w_nbr_60_12m']*x4+df['w_Age']*x5+df['w_Tot_Cons_inc_pay_exp_6m']*x6+df['w_A_No_Depend']*x7+(-2.9912)			
			
			
df['Score'] = df['S_nbr_late_fee_12m']+df['S_max_dpd_3m']+df['S_avg_util_3m']+df['S_nbr_60_12m']+df['S_Age']+df['S_Tot_Cons_inc_pay_exp_6m']+df['S_A_No_Depend']			
df['Pred_Prob']= 1/(1+(np.exp((df['Score']-500)/(20/np.log(2)))))			


In [856]:
df.shape

(36324, 27)

In [857]:
df.head()

,Snapshot_Key,bad_flag,b_nbr_late_fee_12m,b_max_dpd_3m,b_avg_util_3m,b_nbr_60_12m,b_Age,b_Tot_Cons_inc_pay_exp_6m,b_A_No_Depend,w_nbr_late_fee_12m,w_max_dpd_3m,w_avg_util_3m,w_nbr_60_12m,w_Age,w_Tot_Cons_inc_pay_exp_6m,w_A_No_Depend,F_build,S_nbr_late_fee_12m,S_max_dpd_3m,S_avg_util_3m,S_nbr_60_12m,S_Age,S_Tot_Cons_inc_pay_exp_6m,S_A_No_Depend,ln_odds,Score,Pred_Prob
1489,FTKUGGNNNSUNNNNT_2011-11-30,1.0,2. GE 0.1 and LT 0.18,2. GE 1 and LT 2,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,Missing,0.943788,-0.060129,0.256051,0.504176,-0.116588,0.473630,-0.001179,1,94,83,86,89,81,92,84,-3.749115,609,0.022365
1829,FTKUGGNNWSSWWNTG_2011-12-31,0.0,3. GE 0.18 and LT 0.43,3. GT2,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,2. GE 1 and LT 5,0.393390,-2.211057,0.256051,0.504176,-0.116588,0.473630,0.182823,1,88,47,86,89,81,92,86,-2.392243,569,0.083834
859,FTKUGFNNTFTNPSWF_2011-02-28,0.0,1. Less than 0.1,1. Less than 1,1. Less than 0.98,1. Less than 0.08,2. GE 2 and LT 31,4. GT4,Missing,1.553898,1.453554,0.256051,0.504176,0.200493,1.068065,-0.001179,1,101,108,86,89,89,101,84,-5.476099,658,0.004169
1290,FTKUGGNNKWKCGKNS_2011-09-30,0.0,2. GE 0.1 and LT 0.18,1. Less than 1,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,Missing,0.943788,1.453554,0.256051,0.504176,-0.116588,0.473630,-0.001179,1,94,108,86,89,81,92,84,-4.618877,634,0.009527
1213,FTKUGGNNNFWCNNNW_2011-03-31,1.0,4. GT0.43,3. GT2,1. Less than 0.98,1. Less than 0.08,4. GT41,1. Less than 2,Missing,-1.299253,-2.211057,0.256051,0.504176,-0.486358,-0.608151,-0.001179,1,69,47,86,89,71,74,84,-0.698583,520,0.333333


In [858]:
df.to_excel("Scored_Data.xlsx")

#### 5.0 Subsetting data based on selected variables

In [859]:
bench_data=df[df['F_build']==1].reset_index(drop=True)
val_data=df[df['F_build']==0].reset_index(drop=True)

In [860]:
bench_data.shape

(25426, 27)

In [861]:
val_data.shape

(10898, 27)

In [862]:
val_data_1=val_data.copy()

In [863]:
bench_data['Flag']='Benchmark_data'
val_data_1['Flag']='OOT'

In [864]:
b_var_list=[col for col in bench_data if col.startswith('b_')]
w_var_list=[col for col in bench_data if col.startswith('w_')]

In [865]:
b_var_list

['b_nbr_late_fee_12m',
 'b_max_dpd_3m',
 'b_avg_util_3m',
 'b_nbr_60_12m',
 'b_Age',
 'b_Tot_Cons_inc_pay_exp_6m',
 'b_A_No_Depend']

In [866]:
w_var_list

['w_nbr_late_fee_12m',
 'w_max_dpd_3m',
 'w_avg_util_3m',
 'w_nbr_60_12m',
 'w_Age',
 'w_Tot_Cons_inc_pay_exp_6m',
 'w_A_No_Depend']

#### 6.0 Score Bands Creation¶

In [867]:
def map_score_band(benchmark_data, final_score, validation_datasets,
                   benchmark_lower_limit=[], benchmark_upper_limit=[], closed_on='right', bins=10):
    '''

    :param benchmark_data: Dataframe.
    :param final_score: String. column name
    :param validation_datasets: List of Dataframes. Validation datasets
    :param benchmark_lower_limit: (Optional) list of scores (integer and float)
    :param benchmark_upper_limit: (Optional) List of scores (integer and float)
    :param closed_on: String. When 'right', upper limit is included. When 'left', lower limit is included
    :param bins: in case upper and lower limits are not specified, the function breaks benchmark dataset in
                 specified number of bins
    :return: 1. a list of mapped datasets stored as dataframes with benchmark dataset indexed at 0
             2. score band mapping
    '''
    if (len(benchmark_lower_limit)==0):
        #   Quantile binning of scores in benchmark data in specified number of bins. Could result in error when
        #   number of bins is large as it will fail to create mutually exclusive bins with unique borders.
        benchmark_score_band = pd.qcut(benchmark_data[final_score], q=bins, precision=0,duplicates = 'drop').unique().sort_values()
        benchmark_data['score_band'] = pd.qcut(benchmark_data[final_score], q=bins, precision=0,duplicates = 'drop')
        score_band_mapping = pd.DataFrame({'score_band': benchmark_score_band,
                                           'lower_limit': np.nan, 'upper_limit': np.nan})
        for i in range(len(benchmark_score_band)):
            score_band_mapping['lower_limit'][i] = int(benchmark_score_band[i].left)
            score_band_mapping['upper_limit'][i] = int(benchmark_score_band[i].right)
            
        score_band_mapping['lower_limit']=score_band_mapping['lower_limit'].astype(int)
        score_band_mapping['upper_limit']=score_band_mapping['upper_limit'].astype(int)

    else:
        score_band_mapping = pd.DataFrame()
        if(closed_on=='right'):
            score_band_mapping['score_band'] = "(" + str(benchmark_lower_limit) + ", " + str(
                benchmark_upper_limit) + "]"
        else:
            score_band_mapping['score_band'] = "[" + str(benchmark_lower_limit) + ", " + str(
                benchmark_upper_limit) + ")"
        score_band_mapping['lower_limit'] = benchmark_lower_limit
        score_band_mapping['upper_limit'] = benchmark_upper_limit

    alpha = 'A'
    order_list = []
    for i in range(0, len(score_band_mapping)):
        order_list.append(alpha)
        alpha = chr(ord(alpha) + 1)

    score_band_mapping['order'] = order_list
    score_band_mapping['ordered_bands'] = (score_band_mapping['order'] + "." +
                                           score_band_mapping['lower_limit'].astype('str') + "-" +
                                           score_band_mapping['upper_limit'].astype('str'))
    score_band_mapping['ordered_bands'][0] = "A.<" + score_band_mapping['upper_limit'][0].astype('str')
    score_band_mapping['ordered_bands'][-1:] = order_list[-1:][0] + ".>" + score_band_mapping['lower_limit'][
                                                                           -1:].astype('str')

    combined_data = [benchmark_data]
    for i in range(len(validation_datasets)):
        combined_data.append(validation_datasets[i])

    for i in range(len(combined_data)):
        combined_data[i]['ordered_bands'] = 'a'
        if (closed_on=='right'):
            for j in range(len(score_band_mapping)):
                if (j > 0 & j < len(score_band_mapping) - 1):
                    combined_data[i].loc[
                        (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]) &
                        (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
                        'ordered_bands'] = score_band_mapping['ordered_bands'][j]
                    combined_data[i].loc[
                        (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]) &
                        (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
                        'bins'] = int(j)
                if (j == 0):
                    combined_data[i].loc[
                        (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
                        'ordered_bands'] = score_band_mapping['ordered_bands'][j]
                    combined_data[i].loc[
                        (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
                        'bins'] = int(j)
                if (j == len(score_band_mapping) - 1):
                    combined_data[i].loc[
                        (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]),
                        'ordered_bands'] = score_band_mapping['ordered_bands'][j]
                    combined_data[i].loc[
                        (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]),
                        'bins'] = int(j)

                                     
        else:
            for j in range(len(score_band_mapping)):
                if (j > 0 & j < len(score_band_mapping) - 1):
                    combined_data[i].loc[
                        (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]) &
                        (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
                        'ordered_bands'] = score_band_mapping['ordered_bands'][j]
                    combined_data[i].loc[
                        (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]) &
                        (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
                        'bins'] = int(j)
                if (j == 0):
                    combined_data[i].loc[
                        (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
                        'ordered_bands'] = score_band_mapping['ordered_bands'][j]
                    combined_data[i].loc[
                        (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
                        'bins'] = int(j)
                if (j == len(score_band_mapping) - 1):
                    combined_data[i].loc[
                        (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]),
                        'ordered_bands'] = j
                    combined_data[i].loc[
                        (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]),
                        'bins'] = int(j)
        (combined_data[i]['bins'])=(combined_data[i]['bins']).astype(int)
    return (combined_data, score_band_mapping)


In [868]:
mapped=(map_score_band(benchmark_data=bench_data, final_score='Score', validation_datasets=[val_data_1], bins=10))
mapped_dataset = mapped[0][1:]
score_band_mapping = mapped[1]

In [869]:
bench_data.head()

,Snapshot_Key,bad_flag,b_nbr_late_fee_12m,b_max_dpd_3m,b_avg_util_3m,b_nbr_60_12m,b_Age,b_Tot_Cons_inc_pay_exp_6m,b_A_No_Depend,w_nbr_late_fee_12m,w_max_dpd_3m,w_avg_util_3m,w_nbr_60_12m,w_Age,w_Tot_Cons_inc_pay_exp_6m,w_A_No_Depend,F_build,S_nbr_late_fee_12m,S_max_dpd_3m,S_avg_util_3m,S_nbr_60_12m,S_Age,S_Tot_Cons_inc_pay_exp_6m,S_A_No_Depend,ln_odds,Score,Pred_Prob,Flag,score_band,ordered_bands,bins
0,FTKUGGNNNSUNNNNT_2011-11-30,1.0,2. GE 0.1 and LT 0.18,2. GE 1 and LT 2,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,Missing,0.943788,-0.060129,0.256051,0.504176,-0.116588,0.473630,-0.001179,1,94,83,86,89,81,92,84,-3.749115,609,0.022365,Benchmark_data,"(608.0, 616.0]",E.608-616,4
1,FTKUGGNNWSSWWNTG_2011-12-31,0.0,3. GE 0.18 and LT 0.43,3. GT2,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,2. GE 1 and LT 5,0.393390,-2.211057,0.256051,0.504176,-0.116588,0.473630,0.182823,1,88,47,86,89,81,92,86,-2.392243,569,0.083834,Benchmark_data,"(556.0, 577.0]",B.556-577,1
2,FTKUGFNNTFTNPSWF_2011-02-28,0.0,1. Less than 0.1,1. Less than 1,1. Less than 0.98,1. Less than 0.08,2. GE 2 and LT 31,4. GT4,Missing,1.553898,1.453554,0.256051,0.504176,0.200493,1.068065,-0.001179,1,101,108,86,89,89,101,84,-5.476099,658,0.004169,Benchmark_data,"(641.0, 660.0]",J.>641,9
3,FTKUGGNNKWKCGKNS_2011-09-30,0.0,2. GE 0.1 and LT 0.18,1. Less than 1,1. Less than 0.98,1. Less than 0.08,3. GE 31 and LT 41,3. GE 3 and LT 4,Missing,0.943788,1.453554,0.256051,0.504176,-0.116588,0.473630,-0.001179,1,94,108,86,89,81,92,84,-4.618877,634,0.009527,Benchmark_data,"(632.0, 641.0]",I.632-641,8
4,FTKUGGNNNFWCNNNW_2011-03-31,1.0,4. GT0.43,3. GT2,1. Less than 0.98,1. Less than 0.08,4. GT41,1. Less than 2,Missing,-1.299253,-2.211057,0.256051,0.504176,-0.486358,-0.608151,-0.001179,1,69,47,86,89,71,74,84,-0.698583,520,0.333333,Benchmark_data,"(476.0, 556.0]",A.<556,0


In [870]:
score_band_mapping

,score_band,lower_limit,upper_limit,order,ordered_bands
0,"(476.0, 556.0]",476,556,A,A.<556
1,"(556.0, 577.0]",556,577,B,B.556-577
2,"(577.0, 594.0]",577,594,C,C.577-594
3,"(594.0, 608.0]",594,608,D,D.594-608
4,"(608.0, 616.0]",608,616,E,E.608-616
5,"(616.0, 622.0]",616,622,F,F.616-622
6,"(622.0, 631.0]",622,631,G,G.622-631
7,"(631.0, 632.0]",631,632,H,H.631-632
8,"(632.0, 641.0]",632,641,I,I.632-641
9,"(641.0, 660.0]",641,660,J,J.>641


#### 7.0 PSI/CSI/HCI¶

In [871]:
def psi_csi_hci(benchmark_data, final_score_band ,snapshot_datasets=[], variable_list=[],
                key='YYYYMM', psi=True, csi=True):
    '''

    :param benchmark_data: Dataframe
    :param final_score_band: String. Column name which consist score bands as per benchmark dataset
    :param snapshot_datasets: List of Dataframes of Validation datasets.
    :param variable_list: List of variables as string, to compute CSI. Can be left blank if CSI=False
    :param key: String. ID to identify Dataset
    :param psi: Boolean. True: function computes PSI. Fasle: does not compute PSI
    :param csi: Boolean. True: function computes CSI for given list of variables
    :return: List of 2 Dataframes. First DF consist of PSI for all Validation Datasets appended. Second DF contains CSI
    '''
    combined_psi_op = pd.DataFrame()
    combined_csi_op = pd.DataFrame()

    # 3.1.1 The function "calculate" computes the psi/csi.
    def calculate(benchmark_data, validation_data, variable, key):
        '''

        :param benchmark_data: Dataframe. Benchmark/Development data for reference
        :param validation_data: Dataframe. Must be binned as per benchmark score bands
        :param variable: String. for PSI, use score bands as per benchmark data. For CSI, provide variable name
        :param key: String. key to identify dataset and label the output
        :return: returns a dataframe with concentration and psi at bin level
        '''
        var_bins = np.sort(np.unique(np.append(benchmark_data[variable].unique(), validation_data[variable].unique())))
        temp_snap_map = pd.DataFrame({
            'value': np.sort(validation_data[variable].unique()),
            'count_snapshot': validation_data.groupby([variable])[key].count().values,
        })
        temp_bench_map = pd.DataFrame({
            'value': np.sort(benchmark_data[variable].unique()),
            'count_benchmark': benchmark_data.groupby([variable])[key].count().values,
        })
        temp_var_map = pd.DataFrame({'value': var_bins})
        temp_var_map = temp_var_map.merge(temp_snap_map, on='value', how='left')
        temp_var_map = temp_var_map.merge(temp_bench_map, on='value', how='left')
        temp_var_map.fillna(0)
        temp_op = pd.DataFrame({
            'key': validation_data[key].unique()[0],
            'variable_name': variable,
            'value': var_bins,
            'count_snapshot': temp_var_map['count_snapshot'].values,
            'count_benchmark': temp_var_map['count_benchmark'].values,
            'concentration_snapshot': temp_var_map['count_snapshot'].values / len(validation_data),
            'concentration_benchmark': temp_var_map['count_benchmark'].values / len(benchmark_data),
        })
        temp_op['difference'] = temp_op['concentration_snapshot'] - temp_op['concentration_benchmark']
        temp_op['log_concentration'] = np.log(temp_op['concentration_snapshot'] / temp_op['concentration_benchmark'])
        temp_op['individual_si'] = temp_op['log_concentration'] * temp_op['difference']

        return (temp_op)

    if (psi==True):
        for i in range(len(snapshot_datasets)):
            temp_psi_op = calculate(benchmark_data=benchmark_data, validation_data=snapshot_datasets[i],
                                    variable=final_score_band, key=key)
            combined_psi_op = combined_psi_op.append(temp_psi_op)


    if (csi==True):
        for j in range(len(variable_list)):
            for i in range(len(snapshot_datasets)):
                temp_csi_op = calculate(benchmark_data=benchmark_data, validation_data=snapshot_datasets[i],
                                        variable=variable_list[j], key=key)
                combined_csi_op = combined_csi_op.append(temp_csi_op)

    return([combined_psi_op, combined_csi_op])


In [872]:
psi_csi_output = psi_csi_hci(benchmark_data=bench_data, final_score_band='ordered_bands',
                             snapshot_datasets=mapped_dataset,variable_list=b_var_list,
                             key='Flag', psi=True, csi=True)
final_psi = psi_csi_output[0]
final_csi = psi_csi_output[1]

In [873]:
csi=final_csi.groupby(['key','variable_name'])[['individual_si']].agg('sum').reset_index()
csi.rename(columns={'individual_si':'csi'},inplace=True)
psi=final_psi.groupby(['key','variable_name'])[['individual_si']].agg('sum').reset_index()
hci=final_psi.groupby(['key','variable_name'])[['concentration_snapshot']].agg('max').reset_index()
psi.rename(columns={'individual_si':'PSI'},inplace=True)
hci.rename(columns={'concentration_snapshot':'HCI'},inplace=True)

In [874]:
psi

,key,variable_name,PSI
0,OOT,ordered_bands,0.001544


In [875]:
hci

,key,variable_name,HCI
0,OOT,ordered_bands,0.163424


In [876]:
csi

,key,variable_name,csi
0,OOT,b_A_No_Depend,0.000318
1,OOT,b_Age,0.000290
2,OOT,b_Tot_Cons_inc_pay_exp_6m,0.000540
3,OOT,b_avg_util_3m,0.000102
4,OOT,b_max_dpd_3m,0.000279
5,OOT,b_nbr_60_12m,0.000112
6,OOT,b_nbr_late_fee_12m,0.000342


#### 8.0 Individual Score Point Distribution

In [877]:
def score_point_distribution(benchmark_dataset, validation_dataset, score, target, key):
    score_point_output=pd.DataFrame()
    
    def calculate_score_point(df, score, target, key):
        temp_score=pd.DataFrame()
        temp_score['Total']=df.groupby([score])[score].agg('count')
        temp_score['Bad']=df.groupby([score])[target].agg('sum')
        temp_score['Good']=temp_score['Total']-temp_score['Bad']
        temp_score['Key']=df[key].max()
        temp_score.reset_index(inplace=True)
        temp_score=temp_score[[score,'Good','Bad','Total','Key']]
        return temp_score
    
    score_point_output=calculate_score_point(benchmark_dataset, score, target, key)
    
    for i in range(len(validation_dataset)):
        temp=calculate_score_point(validation_dataset[i], score, target, key)
        score_point_output=score_point_output.append(temp)
        
    score_point_output['Total']=score_point_output['Good']+score_point_output['Bad']
    
    return score_point_output

In [878]:
score_point_output=score_point_distribution(bench_data, mapped_dataset, 'Score', 'bad_flag', 'Flag')

In [879]:
score_point_output

,Score,Good,Bad,Total,Key
0,477,15.0,40.0,55.0,Benchmark_data
1,479,2.0,6.0,8.0,Benchmark_data
2,486,15.0,28.0,43.0,Benchmark_data
3,487,7.0,23.0,30.0,Benchmark_data
4,488,1.0,0.0,1.0,Benchmark_data
5,489,5.0,6.0,11.0,Benchmark_data
6,492,33.0,67.0,100.0,Benchmark_data
7,494,14.0,5.0,19.0,Benchmark_data
8,495,27.0,18.0,45.0,Benchmark_data
9,496,11.0,16.0,27.0,Benchmark_data


#### 9.0 IV¶

In [880]:
def iv(benchmark_dataset, snapshot_datasets, var_list, key, target):
    
    iv=pd.DataFrame()
    
    def calculate_iv(df,variable,target,key):
        
        temp_iv=pd.DataFrame()

        temp_iv['total']=df.groupby([variable])[target].agg('count')
        temp_iv['bad']=df.groupby([variable])[target].agg('sum')
        temp_iv['good']=temp_iv['total']-temp_iv['bad']

        temp_iv['per_good']=temp_iv['good']/temp_iv['good'].sum()
        temp_iv['per_bad']=temp_iv['bad']/temp_iv['bad'].sum()

        temp_iv['variable_name']=variable
        temp_iv['key']=df[key].max()

        temp_iv.reset_index(inplace=True)
        temp_iv.rename(columns={variable:'var_bins'},inplace=True)

        temp_iv['difference'] = temp_iv['per_good'] - temp_iv['per_bad']
        temp_iv['log_concentration'] = np.log(temp_iv['per_good'] / temp_iv['per_bad'])
        temp_iv['individual_iv'] = temp_iv['log_concentration'] * temp_iv['difference']  
        
        temp_iv.replace({'individual_iv':{np.inf:0,-np.inf:0}}, inplace=True)
        
        return temp_iv
    
    for i in range(len(var_list)):
        temp_iv = calculate_iv(benchmark_dataset, var_list[i], target, key)
        iv=iv.append(temp_iv)
    
    for i in range(len(snapshot_datasets)):
        for j in range(len(var_list)):
            temp_iv = calculate_iv(snapshot_datasets[i], var_list[j], target, key)
            iv=iv.append(temp_iv)
            
    return iv  

In [881]:
iv_output = iv(benchmark_dataset=bench_data, snapshot_datasets=mapped_dataset, var_list=b_var_list, key='Flag', target='bad_flag')

In [882]:
iv_output

,var_bins,total,bad,good,per_good,per_bad,variable_name,key,difference,log_concentration,individual_iv
0,1. Less than 0.1,12243,152.0,12091.0,0.503813,0.106517,b_nbr_late_fee_12m,Benchmark_data,0.397295,1.553898,6.173568e-01
1,2. GE 0.1 and LT 0.18,2653,60.0,2593.0,0.108046,0.042046,b_nbr_late_fee_12m,Benchmark_data,0.066000,0.943788,6.228996e-02
2,3. GE 0.18 and LT 0.43,4770,184.0,4586.0,0.191091,0.128942,b_nbr_late_fee_12m,Benchmark_data,0.062149,0.393390,2.444897e-02
3,4. GT0.43,5760,1031.0,4729.0,0.197050,0.722495,b_nbr_late_fee_12m,Benchmark_data,-0.525445,-1.299253,6.826860e-01
0,1. Less than 1,15028,206.0,14822.0,0.617609,0.144359,b_max_dpd_3m,Benchmark_data,0.473250,1.453554,6.878948e-01
1,2. GE 1 and LT 2,8334,495.0,7839.0,0.326639,0.346882,b_max_dpd_3m,Benchmark_data,-0.020243,-0.060129,1.217190e-03
2,3. GT2,2064,726.0,1338.0,0.055752,0.508760,b_max_dpd_3m,Benchmark_data,-0.453007,-2.211057,1.001625e+00
0,1. Less than 0.98,21612,951.0,20661.0,0.860911,0.666433,b_avg_util_3m,Benchmark_data,0.194478,0.256051,4.979628e-02
1,2. GE 0.98 and LT 1.01,2542,210.0,2332.0,0.097171,0.147162,b_avg_util_3m,Benchmark_data,-0.049991,-0.415064,2.074952e-02
2,3. GT1.01,1272,266.0,1006.0,0.041918,0.186405,b_avg_util_3m,Benchmark_data,-0.144487,-1.492197,2.156025e-01


In [883]:
final_iv=iv_output.groupby(['key','variable_name'])[['individual_iv']].agg('sum').reset_index()
final_iv.rename(columns={'individual_iv':'iv'},inplace=True)

In [884]:
final_iv

,key,variable_name,iv
0,Benchmark_data,b_A_No_Depend,0.011605
1,Benchmark_data,b_Age,0.112819
2,Benchmark_data,b_Tot_Cons_inc_pay_exp_6m,0.210717
3,Benchmark_data,b_avg_util_3m,0.286148
4,Benchmark_data,b_max_dpd_3m,1.690737
5,Benchmark_data,b_nbr_60_12m,1.007010
6,Benchmark_data,b_nbr_late_fee_12m,1.386782
7,OOT,b_A_No_Depend,0.004674
8,OOT,b_Age,0.141390
9,OOT,b_Tot_Cons_inc_pay_exp_6m,0.109849


#### 10.0 VIF

In [885]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.api import add_constant

In [886]:
def vif(benchmark_dataset, snapshot_datasets, var_list, key):
    vif_output = pd.DataFrame()
    temp=var_list+[key]
    def calculate_vif(df, var_no, key):

        temp_vif=pd.DataFrame()
        temp_vif['variable_name']=[var_list[var_no]]
        temp_vif['key']=df[key].max()
        temp_vif['VIF']=variance_inflation_factor(add_constant(df[var_list]).values, var_no+1)
        return temp_vif

    for i in range(len(var_list)):
        temp_vif = calculate_vif(benchmark_dataset[temp], i, key)
        vif_output=vif_output.append(temp_vif)

    for i in range(len(snapshot_datasets)):
        for j in range(len(var_list)):
            temp_vif = calculate_vif(snapshot_datasets[i][temp],j,key)
            vif_output=vif_output.append(temp_vif)

    return vif_output

In [887]:
vif_output=vif(benchmark_dataset=bench_data, snapshot_datasets=mapped_dataset, var_list=w_var_list, key='Flag')

In [888]:
vif_output

,variable_name,key,VIF
0,w_nbr_late_fee_12m,Benchmark_data,2.298163
0,w_max_dpd_3m,Benchmark_data,2.404962
0,w_avg_util_3m,Benchmark_data,1.133333
0,w_nbr_60_12m,Benchmark_data,1.312731
0,w_Age,Benchmark_data,1.038899
0,w_Tot_Cons_inc_pay_exp_6m,Benchmark_data,1.017421
0,w_A_No_Depend,Benchmark_data,1.023082
0,w_nbr_late_fee_12m,OOT,2.340410
0,w_max_dpd_3m,OOT,2.435184
0,w_avg_util_3m,OOT,1.133076


#### 11.0 MAPE

In [889]:
def mape_bandwise(benchmark_dataset, final_ordered_bands, snapshot_datasets, key, target, score):
    mape_output=pd.DataFrame()
    
    def calculate_mape(df, final_ordered_bands, score, target, key):
        temp_mape=pd.DataFrame()
        temp_mape['pred_avg']=df.groupby([final_ordered_bands])[score].agg('mean')
        temp_mape['actual_avg']=df.groupby([final_ordered_bands])[target].agg('mean')
        temp_mape['abs_deviation']=((temp_mape['pred_avg']-temp_mape['actual_avg'])/temp_mape['actual_avg']).abs()
        temp_mape['key']=df[key].max()        
        temp_mape.reset_index(inplace=True)
        return temp_mape
        
    mape_output = calculate_mape(benchmark_dataset, final_ordered_bands, score, target, key)
    
    for i in range(len(snapshot_datasets)):
        temp_mape=calculate_mape(snapshot_datasets[i], final_ordered_bands, score, target, key)
        mape_output=mape_output.append(temp_mape)
        
    return mape_output  

In [890]:
mape_output=mape_bandwise(bench_data, 'ordered_bands', mapped_dataset, 'Flag', 'bad_flag', 'Pred_Prob')

In [891]:
mape_output

,ordered_bands,pred_avg,actual_avg,abs_deviation,key
0,A.<556,0.309555,0.311119,0.005027,Benchmark_data
1,B.556-577,0.084576,0.086542,0.022727,Benchmark_data
2,C.577-594,0.047187,0.044493,0.060543,Benchmark_data
3,D.594-608,0.028786,0.024769,0.162171,Benchmark_data
4,E.608-616,0.019734,0.021807,0.095066,Benchmark_data
5,F.616-622,0.014931,0.012239,0.219915,Benchmark_data
6,G.622-631,0.011663,0.010703,0.089709,Benchmark_data
7,H.631-632,0.010203,0.016187,0.369652,Benchmark_data
8,I.632-641,0.008627,0.010657,0.190454,Benchmark_data
9,J.>641,0.006179,0.007759,0.203649,Benchmark_data


In [892]:
mape=mape_output.groupby(['key'])[['abs_deviation']].agg('mean')
mape.rename(columns={'abs_deviation':'MAPE'},inplace=True)

In [893]:
mape

,MAPE
key,
Benchmark_data,0.141891
OOT,0.290271


In [894]:
def mape_complete(benchmark_dataset, final_ordered_bands, snapshot_datasets, key, target, score):
    mape_output=pd.DataFrame()
    
    def calculate_mape(df, final_ordered_bands, score, target, key):
        temp_mape={}
        temp_mape['key']=df[key].max()
        temp_mape['pred_avg']=df[score].agg('mean')
        temp_mape['actual_avg']=df[target].agg('mean')
        temp_mape['abs_deviation']=abs(((temp_mape['pred_avg']-temp_mape['actual_avg'])/temp_mape['actual_avg']))    
#         temp_mape.reset_index(inplace=True)
#         print(temp_mape)
        return pd.DataFrame([temp_mape]) 
    
    mape_output = calculate_mape(benchmark_dataset, final_ordered_bands, score, target, key)
    
    for i in range(len(snapshot_datasets)):
        temp_mape=calculate_mape(snapshot_datasets[i], final_ordered_bands, score, target, key)
        mape_output=mape_output.append(temp_mape)
        
    return mape_output

In [895]:
mape_complete_output=mape_complete(bench_data, 'ordered_bands', mapped_dataset, 'Flag', 'bad_flag', 'Pred_Prob')

In [896]:
mape_complete_output

,key,pred_avg,actual_avg,abs_deviation
0,Benchmark_data,0.056192,0.056124,0.001225
0,OOT,0.057264,0.056065,0.021382


#### 12.0 Rank Order Analysis

In [897]:
def rank_order(benchmark_dataset, final_ordered_bands, snapshot_datasets, key, target):
    rank_order_output=pd.DataFrame()
    
    def calculate_rank_order(df, final_ordered_bands , target, key):
        temp_rank_order=pd.DataFrame()
        temp_rank_order['Bad']=df.groupby([final_ordered_bands])[target].agg('sum') 
        temp_rank_order['Good']=df.groupby([final_ordered_bands])[target].agg('count')-temp_rank_order['Bad'] 
#         temp_rank_order['bad_rate']=df.groupby([final_ordered_bands])[target].agg('mean') 
#         temp_rank_order['per_population']=df.groupby([final_ordered_bands])[target].agg('count')/len(df.target)
        temp_rank_order['key']=df[key].max() 
        temp_rank_order.reset_index(inplace=True)
        temp_rank_order=temp_rank_order[['ordered_bands','Good','Bad','key']]
        return temp_rank_order
        
    rank_order_output = calculate_rank_order(benchmark_dataset, final_ordered_bands, target, key)
    
    for i in range(len(snapshot_datasets)):
        temp_rank_order=calculate_rank_order(snapshot_datasets[i], final_ordered_bands, target, key)
        rank_order_output=rank_order_output.append(temp_rank_order)
    
    rank_order_output['Total']=rank_order_output['Good']+rank_order_output['Bad']
    
    return rank_order_output

In [898]:
rank_order_output=rank_order(bench_data, 'ordered_bands', mapped_dataset, 'Flag', 'bad_flag')

In [899]:
rank_order_output

,ordered_bands,Good,Bad,key,Total
0,A.<556,1871.0,845.0,Benchmark_data,2716.0
1,B.556-577,2206.0,209.0,Benchmark_data,2415.0
2,C.577-594,2620.0,122.0,Benchmark_data,2742.0
3,D.594-608,2323.0,59.0,Benchmark_data,2382.0
4,E.608-616,2512.0,56.0,Benchmark_data,2568.0
5,F.616-622,2744.0,34.0,Benchmark_data,2778.0
6,G.622-631,4252.0,46.0,Benchmark_data,4298.0
7,H.631-632,547.0,9.0,Benchmark_data,556.0
8,I.632-641,2878.0,31.0,Benchmark_data,2909.0
9,J.>641,2046.0,16.0,Benchmark_data,2062.0


#### 13.0 Variable Trend & Score Contribution

In [900]:
def var_trend(benchmark_dataset, var_list, snapshot_datasets, key, target, score):
    var_trend_output=pd.DataFrame()
    
    def calculate_var_trend(df1,df2, var , target, key, score):
        temp_var_trend=pd.DataFrame()
        temp_var_trend['bench_bad_rate']=df1.groupby([var])[target].agg('mean') 
        temp_var_trend['val_bad_rate']=df2.groupby([var])[target].agg('mean') 
        temp_var_trend['bench_per_population']=df1.groupby([var])[target].agg('count')/len(df1[target])
        temp_var_trend['val_per_population']=df2.groupby([var])[target].agg('count')/len(df2[target])
        temp_var_trend['avg_score']=df2.groupby([var])[score].agg('mean')
        temp_var_trend['Variable']=var
        temp_var_trend['score']=df1.groupby([var])["S_"+var[2:]].agg('max')
        temp_var_trend['min_score']=temp_var_trend.groupby(['Variable'])['score'].transform('min')
        temp_var_trend['max_score']=temp_var_trend.groupby(['Variable'])['score'].transform('max')
        temp_var_trend['exp_score']=temp_var_trend['score']*temp_var_trend['bench_per_population']
        temp_var_trend['factor_exp_score']=temp_var_trend.groupby(['Variable'])['exp_score'].transform('sum')
        temp_var_trend['var_score']=temp_var_trend['bench_per_population']* \
                                    (temp_var_trend['factor_exp_score']-temp_var_trend['score'])* \
                                    (temp_var_trend['factor_exp_score']-temp_var_trend['score'])
        temp_var_trend['var_factor']=temp_var_trend.groupby(['Variable'])['var_score'].transform('sum')
        temp_var_trend['var_dev']=np.sqrt(temp_var_trend['var_factor'])
        temp_var_trend['key']=df2[key].max() 
        temp_var_trend.reset_index(inplace=True)
        temp_var_trend.rename(columns={var:'var_bins'},inplace=True)
        temp_var_trend=temp_var_trend[['Variable','var_bins','score','bench_bad_rate','val_bad_rate',
                                      'bench_per_population','val_per_population','avg_score','min_score','max_score','var_dev','key']]
        return temp_var_trend
    
    for i in range(len(snapshot_datasets)):
        for j in range(len(var_list)):
            temp_var_trend=calculate_var_trend(benchmark_dataset,snapshot_datasets[i], var_list[j], target, key, score)
            var_trend_output=var_trend_output.append(temp_var_trend)
        
    return var_trend_output

In [901]:
var_trend_output=var_trend(bench_data, b_var_list, mapped_dataset, 'Flag', 'bad_flag', 'Score')

In [902]:
var_trend_output

,Variable,var_bins,score,bench_bad_rate,val_bad_rate,bench_per_population,val_per_population,avg_score,min_score,max_score,var_dev,key
0,b_nbr_late_fee_12m,1. Less than 0.1,101,0.012415,0.014074,0.481515,0.482474,628.985165,69,101,12.658953,OOT
1,b_nbr_late_fee_12m,2. GE 0.1 and LT 0.18,94,0.022616,0.026316,0.104342,0.101119,615.909256,69,101,12.658953,OOT
2,b_nbr_late_fee_12m,3. GE 0.18 and LT 0.43,88,0.038574,0.035946,0.187603,0.183795,599.917624,69,101,12.658953,OOT
3,b_nbr_late_fee_12m,4. GT0.43,69,0.178993,0.171992,0.226540,0.232611,553.007890,69,101,12.658953,OOT
0,b_max_dpd_3m,1. Less than 1,108,0.013708,0.014832,0.591049,0.587723,627.982201,47,108,18.278310,OOT
1,b_max_dpd_3m,2. GE 1 and LT 2,83,0.059395,0.060146,0.327775,0.326482,584.549466,47,108,18.278310,OOT
2,b_max_dpd_3m,3. GT2,47,0.351744,0.322995,0.081177,0.085796,521.275936,47,108,18.278310,OOT
0,b_avg_util_3m,1. Less than 0.98,86,0.044003,0.044070,0.849996,0.849514,610.108663,71,86,3.610002,OOT
1,b_avg_util_3m,2. GE 0.98 and LT 1.01,80,0.082612,0.080149,0.099976,0.098458,586.569432,71,86,3.610002,OOT
2,b_avg_util_3m,3. GT1.01,71,0.209119,0.206349,0.050028,0.052028,549.684303,71,86,3.610002,OOT


#### 14.0 Gini (Using ROC_AUC)

In [903]:
import numpy as np
from sklearn import metrics

In [904]:
def gini(benchmark_dataset, snapshot_datasets, target, score, key):
    gini_output=pd.DataFrame()
    
    def calculate_gini(df, target, score, key):
        temp_gini={}
        actual=df[target]
        pred=df[score]
        fpr, tpr, thresholds = metrics.roc_curve(actual, pred)
        auc=metrics.auc(fpr, tpr)
        gini=(2*auc-1)
        temp_gini['key']=df[key].max()
        temp_gini['GINI']=gini
        temp_gini=pd.DataFrame([temp_gini])
        temp_gini.set_index('key',inplace=True)
        return (temp_gini)
    
    gini_output=calculate_gini(benchmark_dataset, target, score, key)
    
    for i in range(len(snapshot_datasets)):
        temp_gini=calculate_gini(snapshot_datasets[i], target, score, key)
        gini_output=gini_output.append(temp_gini)
        
    return gini_output

In [905]:
gini_output=gini(bench_data, mapped_dataset, 'bad_flag', 'Pred_Prob', 'Flag')

In [906]:
gini_output

,GINI
key,
Benchmark_data,0.701231
OOT,0.670736


#### 15.0 Variable Graphs

In [907]:
def var_graphs(benchmark_dataset, val_datasets, var_list, key, target, final_ordered_bands):
    graph=pd.DataFrame()
    
    def calculate_graph(df, var, key, target, final_ordered_bands ):
        temp=pd.DataFrame()
        temp1=pd.DataFrame()
        
        temp['Bad']=df.groupby([var,final_ordered_bands])[target].agg('sum')
        temp['Total']=df.groupby([var,final_ordered_bands])[target].agg('count')
        temp['Good']=temp['Total']-temp['Bad']
        temp['bad_rate']=temp['Bad']/temp['Total']
        temp['per_pop']=temp['Total']/(temp['Total'].sum())
        temp.reset_index(inplace=True)
        temp.rename(columns={var:'var_bins'},inplace=True)

        temp1['var_bins']=df[var].unique()
        temp1['Variable']=var
        temp1.sort_values(['var_bins'],inplace=True)
        temp1['a']=1

        temp2=score_band_mapping[[final_ordered_bands]]
        temp2['bins']=range(10)
        temp2['a']=1

        temp1=temp1.merge(temp2, on=['a'], how='left')
        temp1=temp1.merge(temp, on=[final_ordered_bands,'var_bins'], how='left')
        temp1.fillna(0,inplace=True)
        temp1['key']=df[key].max()
        temp1=temp1[['Variable','var_bins','bins',final_ordered_bands,'Good','Bad','bad_rate','per_pop','key']]
        
        return temp1
    
    for i in range(len(var_list)):
        temp=calculate_graph(benchmark_dataset, var_list[i], key, target, final_ordered_bands)
        graph=graph.append(temp)
        
    for i in range(len(val_datasets)):
        for j in range(len(var_list)):
            temp=calculate_graph(val_datasets[i], var_list[j], key, target, final_ordered_bands)
            graph=graph.append(temp)
    
    return graph

In [908]:
graph_output=var_graphs(bench_data, mapped_dataset, b_var_list, 'Flag','bad_flag', 'ordered_bands')

In [909]:
graph_output

,Variable,var_bins,bins,ordered_bands,Good,Bad,bad_rate,per_pop,key
0,b_nbr_late_fee_12m,1. Less than 0.1,0,A.<556,11.0,1.0,0.083333,0.000472,Benchmark_data
1,b_nbr_late_fee_12m,1. Less than 0.1,1,B.556-577,10.0,0.0,0.000000,0.000393,Benchmark_data
2,b_nbr_late_fee_12m,1. Less than 0.1,2,C.577-594,93.0,9.0,0.088235,0.004012,Benchmark_data
3,b_nbr_late_fee_12m,1. Less than 0.1,3,D.594-608,416.0,11.0,0.025761,0.016794,Benchmark_data
4,b_nbr_late_fee_12m,1. Less than 0.1,4,E.608-616,1513.0,28.0,0.018170,0.060607,Benchmark_data
5,b_nbr_late_fee_12m,1. Less than 0.1,5,F.616-622,1966.0,20.0,0.010070,0.078109,Benchmark_data
6,b_nbr_late_fee_12m,1. Less than 0.1,6,G.622-631,3341.0,36.0,0.010660,0.132817,Benchmark_data
7,b_nbr_late_fee_12m,1. Less than 0.1,7,H.631-632,544.0,9.0,0.016275,0.021749,Benchmark_data
8,b_nbr_late_fee_12m,1. Less than 0.1,8,I.632-641,2354.0,23.0,0.009676,0.093487,Benchmark_data
9,b_nbr_late_fee_12m,1. Less than 0.1,9,J.>641,1843.0,15.0,0.008073,0.073075,Benchmark_data


#### 16.0 Gini/KS Score Point wise

In [910]:
def gini_ks(benchmark_dataset, validation_datasets, score, target, key):
    
    gini_ks_output=pd.DataFrame()
    
    def calculate_gini_ks(data, score, target, key):
        data = data.sort_values([score], ascending=True)
        temp = data[[target, score]]
        Decile  = data[score].unique().tolist()
        temp_data = pd.DataFrame(Decile, columns = ['Probability'])
        temp_data['Bad'] = temp.groupby(score).sum().reset_index()[target]
        temp_data['Good'] = temp.groupby(score).count().reset_index()[target] - temp_data['Bad']
        temp_data['Total'] = temp_data['Bad'] + temp_data['Good']
        temp_data['Bad_Rate'] = temp_data['Bad'] / temp_data['Total']
        temp_data['Perc_Good'] = temp_data['Good'] / temp_data['Good'].sum()
        temp_data['Perc_Bad'] = temp_data['Bad'] / temp_data['Bad'].sum()
        temp_data['Perc_Total'] = temp_data['Total'] / temp_data['Total'].sum()
        # temp_data['Perc_cummm_Good'] = temp_data['Good'] + temp_data['Good'].sum()
        temp_data['Perc_cumm_Good'] = 0.0
        temp_data['Perc_cumm_Bad'] = 0.0
        temp_data['Perc_cumm_pop'] = 0.0
        temp_data['Area'] = 0.0
        for x in temp_data.index:
            if x == 0:
                temp_data['Perc_cumm_Good'][x] = temp_data['Perc_Good'][x]
                temp_data['Perc_cumm_Bad'][x] = temp_data['Perc_Bad'][x]
                temp_data['Perc_cumm_pop'][x] = temp_data['Perc_Total'][x]
                temp_data['Area'][x] = 0.5*temp_data['Perc_cumm_Bad'][x]*temp_data['Perc_cumm_Good'][x]
            else:
                temp_data['Perc_cumm_Good'][x] = temp_data['Perc_Good'][x] + temp_data['Perc_cumm_Good'][x-1]
                temp_data['Perc_cumm_Bad'][x] = temp_data['Perc_Bad'][x] + temp_data['Perc_cumm_Bad'][x-1]
                temp_data['Perc_cumm_pop'][x] = temp_data['Perc_Total'][x] + temp_data['Perc_cumm_pop'][x-1]
                temp_data['Area'][x] = 0.5*(temp_data['Perc_cumm_Good'][x] + temp_data['Perc_cumm_Good'][x-1])*(temp_data['Perc_cumm_Bad'][x] - temp_data['Perc_cumm_Bad'][x-1])
        temp_data['Differentiation'] = temp_data['Perc_cumm_Good'] - temp_data['Perc_cumm_Bad']
#         print(temp_data)
        Gini = 2*temp_data['Area'].sum() - 1
        KS = temp_data['Differentiation'].max()
        result={}
        result['Key']=data[key].max()
        result['Gini']=Gini
        result['KS']=KS
        return pd.DataFrame([result])
    
    gini_ks_output=calculate_gini_ks(benchmark_dataset, score, target, key)
    
    for i in range(len(validation_datasets)):
        temp=calculate_gini_ks(validation_datasets[i], score, target, key)
        gini_ks_output=gini_ks_output.append(temp)
    
    return gini_ks_output

In [911]:
gini_ks_output = gini_ks(bench_data, mapped_dataset, 'Pred_Prob', 'bad_flag', 'Flag')

In [912]:
gini_ks_output

,Key,Gini,KS
0,Benchmark_data,0.701231,0.568730
0,OOT,0.670736,0.535207


#### 17.0 Correlation_matrix

In [913]:
def correlation_matrix(benchmark_dataset, validation_datasets, var_list,key):
    corr_output=pd.DataFrame()
    
    def calculate_corr(df,var_list,key):
        temp=bench_data[var_list].corr()
        temp['key']=df[key].max()
        
        return temp
    
    corr_output=calculate_corr(benchmark_dataset, var_list, key)
    
    for i in range(len(validation_datasets)):
        temp=calculate_corr(validation_datasets[i], var_list, key)
        corr_output=corr_output.append(temp)
        
    return corr_output

In [914]:
corr_output=correlation_matrix(bench_data, mapped_dataset, w_var_list, 'Flag')

In [915]:
corr_output

,w_nbr_late_fee_12m,w_max_dpd_3m,w_avg_util_3m,w_nbr_60_12m,w_Age,w_Tot_Cons_inc_pay_exp_6m,w_A_No_Depend,key
w_nbr_late_fee_12m,1.000000,0.739776,0.318481,0.421344,-0.123520,0.070598,0.020118,Benchmark_data
w_max_dpd_3m,0.739776,1.000000,0.316947,0.467406,-0.103697,0.087410,0.013573,Benchmark_data
w_avg_util_3m,0.318481,0.316947,1.000000,0.146125,-0.064987,0.051553,-0.007995,Benchmark_data
w_nbr_60_12m,0.421344,0.467406,0.146125,1.000000,-0.026889,0.118243,0.024377,Benchmark_data
w_Age,-0.123520,-0.103697,-0.064987,-0.026889,1.000000,-0.002948,0.140367,Benchmark_data
w_Tot_Cons_inc_pay_exp_6m,0.070598,0.087410,0.051553,0.118243,-0.002948,1.000000,0.036364,Benchmark_data
w_A_No_Depend,0.020118,0.013573,-0.007995,0.024377,0.140367,0.036364,1.000000,Benchmark_data
w_nbr_late_fee_12m,1.000000,0.739776,0.318481,0.421344,-0.123520,0.070598,0.020118,OOT
w_max_dpd_3m,0.739776,1.000000,0.316947,0.467406,-0.103697,0.087410,0.013573,OOT
w_avg_util_3m,0.318481,0.316947,1.000000,0.146125,-0.064987,0.051553,-0.007995,OOT


#### 18.0 Creating Single dataframe for Logistic Working Excel

In [916]:
score_band_mapping = mapped[1]
score_band_mapping.reset_index(inplace=True)
score_band_mapping.rename(columns={'index':'decile'},inplace=True)
score_band_mapping['rank']=score_band_mapping['decile']+1
df1=score_band_mapping[['rank','decile','lower_limit','upper_limit']]
df2=rank_order_output[rank_order_output['key']=='Benchmark_data'][['Good','Bad']]
df3=rank_order_output[rank_order_output['key']=='OOS'][['Good','Bad']]
df4=rank_order_output[rank_order_output['key']=='OOT'][['Good','Bad']]
df5=score_point_output[score_point_output['Key']=='Benchmark_data'][['Score','Good','Bad','Total']]
df6=score_point_output[score_point_output['Key']=='OOS'][['Score','Good','Bad','Total']]
df7=score_point_output[score_point_output['Key']=='OOT'][['Score','Good','Bad','Total']]
df8=var_trend_output[var_trend_output['key']=='OOS'].drop(columns=['key']).reset_index(drop=True)
df9=csi[csi['key']=='OOS'][['variable_name','csi']]
df10=var_trend_output[var_trend_output['key']=='OOS'].groupby(['Variable'])['min_score','max_score','var_dev'].agg({'min_score':'min',
                                                                                                                    'max_score':'max',
                                                                                                                    'var_dev':'max'})
df11=var_trend_output[var_trend_output['key']=='OOT'].drop(columns=['key']).reset_index(drop=True)
df12=csi[csi['key']=='OOT'][['variable_name','csi']]
df13=var_trend_output[var_trend_output['key']=='OOT'].groupby(['Variable'])['min_score','max_score','var_dev'].agg({'min_score':'min',
                                                                                                    'max_score':'max',
                                                                                                         'var_dev':'max'})
df14=final_iv.pivot(index='variable_name',columns='key')
df14.columns=df14.columns.droplevel()
df14.columns='IV_'+df14.columns
df14.reset_index(inplace=True)
df15=vif_output.pivot(index='variable_name',columns='key')
df15.columns=df15.columns.droplevel()
df15.columns='VIF_'+df15.columns
df16=gini_ks_output
df17=hci
df18=mape.reset_index()
df19=mape_complete_output
df20=corr_output[corr_output['key']=='Benchmark_data']
df20.columns=range(1,len(df20)+2)
df20=df20.reindex(columns=range(1,17))
# df21=corr_output[corr_output['key']=='OOS']
# df21.columns=range(1,len(df21)+2)
# df21=df21.reindex(columns=range(1,17))
df22=corr_output[corr_output['key']=='OOT']
df22.columns=range(1,len(df22)+2)
df22=df22.reindex(columns=range(1,17))

In [917]:
dfs=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df22]

In [918]:
for i in dfs:
    i.reset_index(inplace=True, drop=True)

In [919]:
final_output=pd.concat(dfs,axis=1)

In [920]:
final_output.to_excel('raw_data_logistic.xlsx',index=False)

In [921]:
graph_output.to_excel('graph_output.xlsx',index=False)

In [922]:
# df=i1_modDt[['FI_ID',
#  'Flag_90_Life',
# 'b_FI_CUST_MAX_DELQ_LAST12M_1',
#  'b_FI_APP_MONTHLY_INCOME_1',
#  'b_FI_CUST_NUM_LOANS_1',
#  'b_FI_CUST_TOT_ASSETS_1',
#  'b_FI_CUST_AGE_OLDEST_REL',
#  'b_Age_1',
#  'b_FI_EMP_STAT',
             
# 'w_FI_CUST_MAX_DELQ_LAST12M_1',
#  'w_FI_APP_MONTHLY_INCOME_1',
#  'w_FI_CUST_NUM_LOANS_1',
#  'w_FI_CUST_TOT_ASSETS_1',
#  'w_FI_CUST_AGE_OLDEST_REL',
#  'w_Age_1',
#  'w_FI_EMP_STAT',
#  'F_build'
#       ]]

In [923]:
#Writing a function to return the dataframe of the vif of the whole woe-transformed data:
# def vif(X):
#     vif = pd.DataFrame()
#     vif['Predictors'] = X.columns  #Here at the place of X, we need a dataframe so that we can access it's columns
#     vif['VIF'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]
#     return vif

# vif_data = df_new_woe[selected_cols]
# vif1 = vif(vif_data).sort_values(by = 'VIF')
# vif1

In [924]:
#     vif = pd.DataFrame()
   # vif['Predictors'] = X.columns  #Here at the place of X, we need a dataframe so that we can access it's columns
    #vif['VIF'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]

In [925]:
# vif_data1 = model_train_after_corr[list(after_corr_cols.columns)]

In [926]:
# vif_data1

In [927]:
    #vif = pd.DataFrame()
#     vif['Predictors'] = vif_data1.columns  #Here at the place of X, we need a dataframe so that we can access it's columns
    #vif['VIF'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]

In [928]:
# vif

In [929]:
#     vif['VIF'] = [ variance_inflation_factor(vif_data1.values, i) for i in range( vif_data1.shape[1]) ]

In [930]:
# vif

In [931]:
    #vif = pd.DataFrame()
    #vif['Predictors'] = X.columns  #Here at the place of X, we need a dataframe so that we can access it's columns
#     vif['VIF'] = [ variance_inflation_factor(X.values, i) for i in range(X.shape[1]) ]

In [932]:
# def vif_1(benchmark_dataset, var_list):
#     vif_output = pd.DataFrame()
#     temp=var_list
#     def calculate_vif(df, var_no):
#         temp_vif=pd.DataFrame()
#         temp_vif['variable_name']=[var_list[var_no]]
#         temp_vif['VIF']=variance_inflation_factor(add_constant(df[var_list]).values, var_no+1)
#         return temp_vif
#     for i in range(len(var_list)):
#         temp_vif = calculate_vif(benchmark_dataset[temp], i)
#         vif_output=vif_output.append(temp_vif)
            
#     return vif_output.sort_values('VIF').reset_index(drop=True)

In [933]:
# vif_test =vif_1(vif_data1,list(after_corr_cols.columns))

In [934]:
# # VIF on all independent columns
# vif_data1 = model_train_after_corr[list(after_corr_cols.columns)]
# vif2 = vif(vif_data1).sort_values(by = 'VIF',ascending=False)
# vif2

In [935]:
# after_vif_cols = model_train_after_corr[vif2['Predictors']]
# after_vif_cols.shape

In [936]:
# model_final = pd.DataFrame(after_vif_cols)
# model_final['target'] = model_train_after_corr['bad_flag']
# model_final.head()

In [937]:
# X1 = model_final.loc[:, model_final.columns != 'target']
# y1 = model_final.loc[:, model_final.columns == 'target']

In [938]:
# y1.shape

In [939]:
# X1.shape

Stepwise Regression 

In [940]:
# ##Step wise Regression

# def stepwise_selection(X, y, 
#                        initial_list=[], 
#                        threshold_in=0.01, 
#                        threshold_out = 0.05, 
#                        verbose=True):
#     """ Perform a forward-backward feature selection 
#     based on p-value from statsmodels.api.OLS
#     Arguments:
#         X - pandas.DataFrame with candidate features
#         y - list-like with the target
#         initial_list - list of features to start with (column names of X)
#         threshold_in - include a feature if its p-value < threshold_in
#         threshold_out - exclude a feature if its p-value > threshold_out
#         verbose - whether to print the sequence of inclusions and exclusions
#     Returns: list of selected features 
#     Always set threshold_in < threshold_out to avoid infinite looping.
#     See https://en.wikipedia.org/wiki/Stepwise_regression for the details
#     """
#     included = list(initial_list)
#     while True:
#         changed=False
#         # forward step
#         excluded = list(set(X.columns)-set(included))
#         new_pval = pd.Series(index=excluded)
#         for new_column in excluded:
#             model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit(disp = False)
#             new_pval[new_column] = model.pvalues[new_column]
#         best_pval = new_pval.min()
#         if best_pval < threshold_in:
#             best_feature = new_pval.index[new_pval.argmin()]
#             #best_feature = new_pval.argmin()
#             included.append(best_feature)
#             changed=True
#             if verbose:
#                 print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

#         # backward step
#         #print(included)
#         model = sm.Logit(y, sm.add_constant(pd.DataFrame(X[included]))).fit(disp = False)
#         # use all coefs except intercept
#         pvalues = model.pvalues.iloc[1:]
#         worst_pval = pvalues.max() # null if pvalues is empty
#         if worst_pval > threshold_out:
#             changed=True
#             worst_feature = pvalues.index[pvalues.argmax()]
#             included.remove(worst_feature)
#             if verbose:
#                 print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
#         if not changed:
#             break
#     return included

In [941]:
# stepwise_selection(X1, y1,  initial_list= X1.columns, threshold_in=0.05, threshold_out = 0.05 )

In [942]:
# feat1 = ['w_nbr_late_fee_12m','w_nbr_30_6m','w_Tot_Cons_inc_util_3m','w_A_salary','w_Age','w_Tot_Cons_inc_pay_exp_6m',
#          'w_A_No_Depend','w_mob']

In [943]:
# X1[feat1]

In [944]:
# iv_num_cat_filt2 = iv_num_cat_filt1[iv_num_cat_filt1['w_Variable'].isin(X1[feat1].columns)]

In [945]:
# iv_num_cat_filt2

In [946]:
# corr_var_reduction(X1[feat1],"pearson",list(iv_num_cat_filt2["w_Variable"]),0.5)

In [947]:
# vif_feat1 = vif(X1[feat1]).sort_values(by = 'VIF',ascending=False)
# vif_feat1

In [948]:
# model_feat1 = sm.Logit(y1,sm.add_constant(X1[feat1])).fit()

In [949]:
# model_feat1.summary2()

In [950]:
# from sklearn.linear_model import LogisticRegression
# classifier=LogisticRegression()

In [951]:
# classifier.fit(X1[feat1],y1)

In [952]:
# classifier

In [953]:
# y_pred_1 = classifier.predict(x_test[feat1])

In [954]:
# y_pred_1 

In [955]:
# def accuracy(y_pred, y_test):
#     return np.sum(y_pred==y_test)/len(y_test)

In [956]:
# acc = accuracy(y_pred_1, y_test)
# print(acc)

In [957]:
# model_test

In [958]:
# y_test= model_test['bad_flag'] #dependent variable is Decision
# x_test= model_test.drop(['bad_flag','MASK_CREDITCARD','snapshot_date','Snapshot_Key','F_build'], axis=1)

In [959]:
# y_test.shape

In [960]:
# x_test.shape

In [961]:
# y_pred= model_feat1.predict(sm.add_constant(x_test[feat1]))

In [962]:
# y_pred

In [963]:
# y_pred1=np.where(y_pred>0.5,1,0)

In [964]:
# ConfusionMatrix = confusion_matrix(y_test, y_pred1)
# print(ConfusionMatrix)

In [965]:
# fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
# auc=metrics.auc(fpr, tpr)
# gini=2*auc-1
# gini,auc

In [966]:
# ## ROC_AUC Measure
# from sklearn.metrics import roc_auc_score
# from sklearn.metrics import roc_curve
# logit_roc_auc = roc_auc_score(y_test,y_pred)
# fpr, tpr, thresholds = roc_curve(y_test, y_pred, pos_label=1)
# plt.figure()
# plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0.0, 1.0])
# plt.ylim([0.0, 1.05])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic')
# plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
# plt.show()

In [967]:
# from sklearn.metrics import accuracy_score,confusion_matrix
# print("Accuracy of Logistic Regression",accuracy_score(y_test,y_pred1))

In [968]:
# from sklearn.metrics import recall_score,precision_score,f1_score

In [969]:
# print("For Logistic regression Model")
# print("-"*50)
# cdf = pd.DataFrame(confusion_matrix(y_test,y_pred1),columns=list(range(0,2)))
# print(cdf)
# print("-"*50)
# print("Precision - ",precision_score(y_test,y_pred1))
# print("Recall - ",recall_score(y_test,y_pred1))
# print("F1 score - ",f1_score(y_test,y_pred1))

In [970]:
# ##KS statistic
# #Python Function for KS Statistic


# def ks(data=None, target=None , prob=None):
#     data['target0'] = 1 - data[target]
#     data['bucket'] = pd.qcut(data[prob], 10)
#     grouped = data.groupby('bucket', as_index = False)
#     kstable = pd.DataFrame()
#     kstable['min_prob'] = grouped.min()[prob]
#     kstable['max_prob'] = grouped.max()[prob]
#     kstable['events']   = grouped.sum()[target]
#     kstable['nonevents'] = grouped.sum()['target0']
#     kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
#     kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2%}'.format)
#     kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2%}'.format)
#     kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
#     kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
#     kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

#     #Formating
#     kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2%}'.format)
#     kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2%}'.format)
#     kstable.index = range(1,11)
#     kstable.index.rename('Decile', inplace=True)
#     pd.set_option('display.max_columns', 9)
#     print(kstable)
    
#     #Display KS
#     from colorama import Fore
#     print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
#     return(kstable)

In [971]:
# prob = pd.DataFrame(y_pred)
# prob.rename(columns = {0:'prob'}, inplace = True)
# target = y_test
# #target.rename(columns = {0:'target'}, inplace = True)
# df = pd.concat([prob,target], axis=1)

In [972]:
# df

In [973]:
# ks_table = ks(data= df, target = 'bad_flag' , prob= 'prob')

In [974]:
# ##DEcile ranking
# ks_table = pd.DataFrame(ks_table)
# #ks_table.reindex()
# ks_table

# Converting into data format for Template in Python

In [975]:
# check_error ## used for break 

In [976]:
# model_data.shape

In [977]:
# model_data

In [978]:
# #Required modules
# import pandas as pd
# import numpy as np
# %matplotlib inline
# import matplotlib.pyplot as plt
# pd.options.display.max_columns=None
# pd.options.display.max_rows=None
# import seaborn as sns
# import csv
# import warnings 
# warnings.filterwarnings('ignore')
# from sklearn.linear_model import LogisticRegression
# pd.set_option("display.max_columns",None)

# from sklearn.model_selection import train_test_split
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.api import add_constant

In [979]:
# i3_modDt = model_data

In [980]:
# i3_modDt.shape

Scoring /PD Calculation

In [981]:
# i3_modDt.head(1)

Selecting columns from final model

In [982]:
# df=i3_modDt[['Snapshot_Key',
#  'bad_flag',
# 'b_Tot_Cons_inc_pay_exp_6m',
# 'b_mob',
# 'b_A_No_Depend',
# 'b_nbr_30_6m',
# 'b_A_salary',
# 'b_nbr_late_fee_12m',
# 'b_Age',
# 'b_Tot_Cons_inc_util_3m',
             
             
# 'w_Tot_Cons_inc_pay_exp_6m',
# 'w_mob',
# 'w_A_No_Depend',
# 'w_nbr_30_6m',
# 'w_A_salary',
# 'w_nbr_late_fee_12m',
# 'w_Age',
# 'w_Tot_Cons_inc_util_3m',
#  'F_build'
#       ]]

In [983]:
# df.shape

In [984]:
# df.head(5)

In [985]:
# const	=	-0.3379875	
# x1	=	-0.6545	
# x2	=	-0.481	
# x3	=	-0.6512	
# x4	=	-0.878	
# x5	=	-1.1494	
# x6	=	-0.5345	
# x7	=	-0.4956	
# x8	=	-0.7601	
# x9	=	0	
# x10	=	0	
# x11	=	0	
# x12	=	0	
# x13	=	0	
# x14	=	0	
# x15	=	0	
# x16	=	0	
# x17	=	0	
# x18	=	0	
# x19	=	0	
# x20	=	0	
			
# df['S_nbr_late_fee_12m']=	np.round(62.5 - (28.85) * (const+x1*df['w_nbr_late_fee_12m']),0).astype(int)		
# df['S_nbr_30_6m']=	np.round(62.5 - (28.85) * (const+x2*df['w_nbr_30_6m']),0).astype(int)		
# df['S_Tot_Cons_inc_util_3m']=	np.round(62.5 - (28.85) * (const+x3*df['w_Tot_Cons_inc_util_3m']),0).astype(int)		
# df['S_A_salary']=	np.round(62.5 - (28.85) * (const+x4*df['w_A_salary']),0).astype(int)		
# df['S_Age']=	np.round(62.5 - (28.85) * (const+x5*df['w_Age']),0).astype(int)		
# df['S_Tot_Cons_inc_pay_exp_6m']=	np.round(62.5 - (28.85) * (const+x6*df['w_Tot_Cons_inc_pay_exp_6m']),0).astype(int)		
# df['S_A_No_Depend']=	np.round(62.5 - (28.85) * (const+x7*df['w_A_No_Depend']),0).astype(int)		
# df['S_mob']=	np.round(62.5 - (28.85) * (const+x8*df['w_mob']),0).astype(int)		
			
			
			
			
			
			
			
			
			
			
			
			
			
# df['ln_odds']=df['w_nbr_late_fee_12m']*x1+df['w_nbr_30_6m']*x2+df['w_Tot_Cons_inc_util_3m']*x3+df['w_A_salary']*x4+df['w_Age']*x5+df['w_Tot_Cons_inc_pay_exp_6m']*x6+df['w_A_No_Depend']*x7+df['w_mob']*x8+(-2.7039)			
# df['Pred_Prob']=(np.exp(df['ln_odds']))/(1+np.exp(df['ln_odds']))			
			
# df['Score'] = df['S_nbr_late_fee_12m']+df['S_nbr_30_6m']+df['S_Tot_Cons_inc_util_3m']+df['S_A_salary']+df['S_Age']+df['S_Tot_Cons_inc_pay_exp_6m']+df['S_A_No_Depend']+df['S_mob']			


In [986]:
# df.shape

In [987]:
# df['Score'].agg(['min', 'max'])

In [988]:
# list(df.columns)

In [989]:
# df.head(50)

Subsetting data based on selected variables

In [990]:
# bench_data=df[df['F_build']==1].reset_index(drop=True)
# val_data=df[df['F_build']==0].reset_index(drop=True)

In [991]:
# bench_data.shape

In [992]:
# val_data.shape

In [993]:
# val_data_1=val_data.copy()

In [994]:
# val_data_1.shape

In [995]:
# bench_data['Flag']='Benchmark_data'
# val_data_1['Flag']='OOT'

In [996]:
# b_var_list=[col for col in bench_data if col.startswith('b_')]
# w_var_list=[col for col in bench_data if col.startswith('w_')]

In [997]:
# b_var_list

In [998]:
# w_var_list

Score Bands Creation

In [999]:
# def map_score_band(benchmark_data, final_score, validation_datasets,
#                    benchmark_lower_limit=[], benchmark_upper_limit=[], closed_on='right', bins=10):
#     '''

#     :param benchmark_data: Dataframe.
#     :param final_score: String. column name
#     :param validation_datasets: List of Dataframes. Validation datasets
#     :param benchmark_lower_limit: (Optional) list of scores (integer and float)
#     :param benchmark_upper_limit: (Optional) List of scores (integer and float)
#     :param closed_on: String. When 'right', upper limit is included. When 'left', lower limit is included
#     :param bins: in case upper and lower limits are not specified, the function breaks benchmark dataset in
#                  specified number of bins
#     :return: 1. a list of mapped datasets stored as dataframes with benchmark dataset indexed at 0
#              2. score band mapping
#     '''
#     if (len(benchmark_lower_limit)==0):
#         #   Quantile binning of scores in benchmark data in specified number of bins. Could result in error when
#         #   number of bins is large as it will fail to create mutually exclusive bins with unique borders.
#         benchmark_score_band = pd.qcut(benchmark_data[final_score], q=bins, precision=0,duplicates = 'drop').unique().sort_values()
#         benchmark_data['score_band'] = pd.qcut(benchmark_data[final_score], q=bins, precision=0,duplicates = 'drop')
#         score_band_mapping = pd.DataFrame({'score_band': benchmark_score_band,
#                                            'lower_limit': np.nan, 'upper_limit': np.nan})
#         for i in range(len(benchmark_score_band)):
#             score_band_mapping['lower_limit'][i] = int(benchmark_score_band[i].left)
#             score_band_mapping['upper_limit'][i] = int(benchmark_score_band[i].right)
            
#         score_band_mapping['lower_limit']=score_band_mapping['lower_limit'].astype(int)
#         score_band_mapping['upper_limit']=score_band_mapping['upper_limit'].astype(int)

#     else:
#         score_band_mapping = pd.DataFrame()
#         if(closed_on=='right'):
#             score_band_mapping['score_band'] = "(" + str(benchmark_lower_limit) + ", " + str(
#                 benchmark_upper_limit) + "]"
#         else:
#             score_band_mapping['score_band'] = "[" + str(benchmark_lower_limit) + ", " + str(
#                 benchmark_upper_limit) + ")"
#         score_band_mapping['lower_limit'] = benchmark_lower_limit
#         score_band_mapping['upper_limit'] = benchmark_upper_limit

#     alpha = 'A'
#     order_list = []
#     for i in range(0, len(score_band_mapping)):
#         order_list.append(alpha)
#         alpha = chr(ord(alpha) + 1)

#     score_band_mapping['order'] = order_list
#     score_band_mapping['ordered_bands'] = (score_band_mapping['order'] + "." +
#                                            score_band_mapping['lower_limit'].astype('str') + "-" +
#                                            score_band_mapping['upper_limit'].astype('str'))
#     score_band_mapping['ordered_bands'][0] = "A.<" + score_band_mapping['upper_limit'][0].astype('str')
#     score_band_mapping['ordered_bands'][-1:] = order_list[-1:][0] + ".>" + score_band_mapping['lower_limit'][
#                                                                            -1:].astype('str')

#     combined_data = [benchmark_data]
#     for i in range(len(validation_datasets)):
#         combined_data.append(validation_datasets[i])

#     for i in range(len(combined_data)):
#         combined_data[i]['ordered_bands'] = 'a'
#         if (closed_on=='right'):
#             for j in range(len(score_band_mapping)):
#                 if (j > 0 & j < len(score_band_mapping) - 1):
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]) &
#                         (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
#                         'ordered_bands'] = score_band_mapping['ordered_bands'][j]
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]) &
#                         (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
#                         'bins'] = int(j)
#                 if (j == 0):
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
#                         'ordered_bands'] = score_band_mapping['ordered_bands'][j]
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] <= score_band_mapping['upper_limit'][j]),
#                         'bins'] = int(j)
#                 if (j == len(score_band_mapping) - 1):
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]),
#                         'ordered_bands'] = score_band_mapping['ordered_bands'][j]
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] > score_band_mapping['lower_limit'][j]),
#                         'bins'] = int(j)

                                     
#         else:
#             for j in range(len(score_band_mapping)):
#                 if (j > 0 & j < len(score_band_mapping) - 1):
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]) &
#                         (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
#                         'ordered_bands'] = score_band_mapping['ordered_bands'][j]
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]) &
#                         (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
#                         'bins'] = int(j)
#                 if (j == 0):
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
#                         'ordered_bands'] = score_band_mapping['ordered_bands'][j]
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] < score_band_mapping['upper_limit'][j]),
#                         'bins'] = int(j)
#                 if (j == len(score_band_mapping) - 1):
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]),
#                         'ordered_bands'] = j
#                     combined_data[i].loc[
#                         (combined_data[i][final_score] >= score_band_mapping['lower_limit'][j]),
#                         'bins'] = int(j)
#         (combined_data[i]['bins'])=(combined_data[i]['bins']).astype(int)
#     return (combined_data, score_band_mapping)

In [1000]:
# mapped=(map_score_band(benchmark_data=bench_data, final_score='Score', validation_datasets=[val_data_1], bins=10))
# mapped_dataset = mapped[0][1:]
# score_band_mapping = mapped[1]

In [1001]:
# bench_data.head()

In [1002]:
# bench_data.loc[bench_data['Score'].idxmax()]

In [1003]:
# score_band_mapping

PSI/CSI/HCI

In [1004]:
# def psi_csi_hci(benchmark_data, final_score_band ,snapshot_datasets=[], variable_list=[],
#                 key='YYYYMM', psi=True, csi=True):
#     '''

#     :param benchmark_data: Dataframe
#     :param final_score_band: String. Column name which consist score bands as per benchmark dataset
#     :param snapshot_datasets: List of Dataframes of Validation datasets.
#     :param variable_list: List of variables as string, to compute CSI. Can be left blank if CSI=False
#     :param key: String. ID to identify Dataset
#     :param psi: Boolean. True: function computes PSI. Fasle: does not compute PSI
#     :param csi: Boolean. True: function computes CSI for given list of variables
#     :return: List of 2 Dataframes. First DF consist of PSI for all Validation Datasets appended. Second DF contains CSI
#     '''
#     combined_psi_op = pd.DataFrame()
#     combined_csi_op = pd.DataFrame()

#     # 3.1.1 The function "calculate" computes the psi/csi.
#     def calculate(benchmark_data, validation_data, variable, key):
#         '''

#         :param benchmark_data: Dataframe. Benchmark/Development data for reference
#         :param validation_data: Dataframe. Must be binned as per benchmark score bands
#         :param variable: String. for PSI, use score bands as per benchmark data. For CSI, provide variable name
#         :param key: String. key to identify dataset and label the output
#         :return: returns a dataframe with concentration and psi at bin level
#         '''
#         var_bins = np.sort(np.unique(np.append(benchmark_data[variable].unique(), validation_data[variable].unique())))
#         temp_snap_map = pd.DataFrame({
#             'value': np.sort(validation_data[variable].unique()),
#             'count_snapshot': validation_data.groupby([variable])[key].count().values,
#         })
#         temp_bench_map = pd.DataFrame({
#             'value': np.sort(benchmark_data[variable].unique()),
#             'count_benchmark': benchmark_data.groupby([variable])[key].count().values,
#         })
#         temp_var_map = pd.DataFrame({'value': var_bins})
#         temp_var_map = temp_var_map.merge(temp_snap_map, on='value', how='left')
#         temp_var_map = temp_var_map.merge(temp_bench_map, on='value', how='left')
#         temp_var_map.fillna(0)
#         temp_op = pd.DataFrame({
#             'key': validation_data[key].unique()[0],
#             'variable_name': variable,
#             'value': var_bins,
#             'count_snapshot': temp_var_map['count_snapshot'].values,
#             'count_benchmark': temp_var_map['count_benchmark'].values,
#             'concentration_snapshot': temp_var_map['count_snapshot'].values / len(validation_data),
#             'concentration_benchmark': temp_var_map['count_benchmark'].values / len(benchmark_data),
#         })
#         temp_op['difference'] = temp_op['concentration_snapshot'] - temp_op['concentration_benchmark']
#         temp_op['log_concentration'] = np.log(temp_op['concentration_snapshot'] / temp_op['concentration_benchmark'])
#         temp_op['individual_si'] = temp_op['log_concentration'] * temp_op['difference']

#         return (temp_op)

#     if (psi==True):
#         for i in range(len(snapshot_datasets)):
#             temp_psi_op = calculate(benchmark_data=benchmark_data, validation_data=snapshot_datasets[i],
#                                     variable=final_score_band, key=key)
#             combined_psi_op = combined_psi_op.append(temp_psi_op)


#     if (csi==True):
#         for j in range(len(variable_list)):
#             for i in range(len(snapshot_datasets)):
#                 temp_csi_op = calculate(benchmark_data=benchmark_data, validation_data=snapshot_datasets[i],
#                                         variable=variable_list[j], key=key)
#                 combined_csi_op = combined_csi_op.append(temp_csi_op)

#     return([combined_psi_op, combined_csi_op])


In [1005]:
# psi_csi_output = psi_csi_hci(benchmark_data=bench_data, final_score_band='ordered_bands',
#                              snapshot_datasets=mapped_dataset,variable_list=b_var_list,
#                              key='Flag', psi=True, csi=True)
# final_psi = psi_csi_output[0]
# final_csi = psi_csi_output[1]

In [1006]:
# csi=final_csi.groupby(['key','variable_name'])[['individual_si']].agg('sum').reset_index()
# csi.rename(columns={'individual_si':'csi'},inplace=True)
# psi=final_psi.groupby(['key','variable_name'])[['individual_si']].agg('sum').reset_index()
# hci=final_psi.groupby(['key','variable_name'])[['concentration_snapshot']].agg('max').reset_index()
# psi.rename(columns={'individual_si':'PSI'},inplace=True)
# hci.rename(columns={'concentration_snapshot':'HCI'},inplace=True)

In [1007]:
# psi

In [1008]:
# hci

In [1009]:
# csi

Individual Score Point Distribution

In [1010]:
# def score_point_distribution(benchmark_dataset, validation_dataset, score, target, key):
#     score_point_output=pd.DataFrame()
    
#     def calculate_score_point(df, score, target, key):
#         temp_score=pd.DataFrame()
#         temp_score['Total']=df.groupby([score])[score].agg('count')
#         temp_score['Bad']=df.groupby([score])[target].agg('sum')
#         temp_score['Good']=temp_score['Total']-temp_score['Bad']
#         temp_score['Key']=df[key].max()
#         temp_score.reset_index(inplace=True)
#         temp_score=temp_score[[score,'Good','Bad','Total','Key']]
#         return temp_score
    
#     score_point_output=calculate_score_point(benchmark_dataset, score, target, key)
    
#     for i in range(len(validation_dataset)):
#         temp=calculate_score_point(validation_dataset[i], score, target, key)
#         score_point_output=score_point_output.append(temp)
        
#     score_point_output['Total']=score_point_output['Good']+score_point_output['Bad']
    
#     return score_point_output

In [1011]:
# score_point_output=score_point_distribution(bench_data, mapped_dataset, 'Score', 'bad_flag', 'Flag')

In [1012]:
# score_point_output

In [1013]:
# score_point_output.shape

IV

In [1014]:
# def iv(benchmark_dataset, snapshot_datasets, var_list, key, target):
    
#     iv=pd.DataFrame()
    
#     def calculate_iv(df,variable,target,key):
        
#         temp_iv=pd.DataFrame()

#         temp_iv['total']=df.groupby([variable])[target].agg('count')
#         temp_iv['bad']=df.groupby([variable])[target].agg('sum')
#         temp_iv['good']=temp_iv['total']-temp_iv['bad']

#         temp_iv['per_good']=temp_iv['good']/temp_iv['good'].sum()
#         temp_iv['per_bad']=temp_iv['bad']/temp_iv['bad'].sum()

#         temp_iv['variable_name']=variable
#         temp_iv['key']=df[key].max()

#         temp_iv.reset_index(inplace=True)
#         temp_iv.rename(columns={variable:'var_bins'},inplace=True)

#         temp_iv['difference'] = temp_iv['per_good'] - temp_iv['per_bad']
#         temp_iv['log_concentration'] = np.log(temp_iv['per_good'] / temp_iv['per_bad'])
#         temp_iv['individual_iv'] = temp_iv['log_concentration'] * temp_iv['difference']  
        
#         temp_iv.replace({'individual_iv':{np.inf:0,-np.inf:0}}, inplace=True)
        
#         return temp_iv
    
#     for i in range(len(var_list)):
#         temp_iv = calculate_iv(benchmark_dataset, var_list[i], target, key)
#         iv=iv.append(temp_iv)
    
#     for i in range(len(snapshot_datasets)):
#         for j in range(len(var_list)):
#             temp_iv = calculate_iv(snapshot_datasets[i], var_list[j], target, key)
#             iv=iv.append(temp_iv)
            
#     return iv   

In [1015]:
# iv_output = iv(benchmark_dataset=bench_data, snapshot_datasets=mapped_dataset, var_list=b_var_list, key='Flag', target='bad_flag')

In [1016]:
# iv_output 

In [1017]:
# final_iv=iv_output.groupby(['key','variable_name'])[['individual_iv']].agg('sum').reset_index()
# final_iv.rename(columns={'individual_iv':'iv'},inplace=True)

In [1018]:
# final_iv

VIF

In [1019]:
# from statsmodels.stats.outliers_influence import variance_inflation_factor
# from statsmodels.api import add_constant

In [1020]:
# def vif(benchmark_dataset, snapshot_datasets, var_list, key):
#     vif_output = pd.DataFrame()
#     temp=var_list+[key]
#     def calculate_vif(df, var_no, key):

#         temp_vif=pd.DataFrame()
#         temp_vif['variable_name']=[var_list[var_no]]
#         temp_vif['key']=df[key].max()
#         temp_vif['VIF']=variance_inflation_factor(add_constant(df[var_list]).values, var_no)
#         return temp_vif

#     for i in range(len(var_list)):
#         temp_vif = calculate_vif(benchmark_dataset[temp], i, key)
#         vif_output=vif_output.append(temp_vif)

#     for i in range(len(snapshot_datasets)):
#         for j in range(len(var_list)):
#             temp_vif = calculate_vif(snapshot_datasets[i][temp],j,key)
#             vif_output=vif_output.append(temp_vif)

#     return vif_output

In [1021]:
# vif_output=vif(benchmark_dataset=bench_data, snapshot_datasets=mapped_dataset, var_list=w_var_list, key='Flag')

In [1022]:
# vif_output

MAPE

In [1023]:
# def mape_bandwise(benchmark_dataset, final_ordered_bands, snapshot_datasets, key, target, score):
#     mape_output=pd.DataFrame()
    
#     def calculate_mape(df, final_ordered_bands, score, target, key):
#         temp_mape=pd.DataFrame()
#         temp_mape['pred_avg']=df.groupby([final_ordered_bands])[score].agg('mean')
#         temp_mape['actual_avg']=df.groupby([final_ordered_bands])[target].agg('mean')
#         temp_mape['abs_deviation']=((temp_mape['pred_avg']-temp_mape['actual_avg'])/temp_mape['actual_avg']).abs()
#         temp_mape['key']=df[key].max()        
#         temp_mape.reset_index(inplace=True)
#         return temp_mape
        
#     mape_output = calculate_mape(benchmark_dataset, final_ordered_bands, score, target, key)
    
#     for i in range(len(snapshot_datasets)):
#         temp_mape=calculate_mape(snapshot_datasets[i], final_ordered_bands, score, target, key)
#         mape_output=mape_output.append(temp_mape)
        
#     return mape_output 

In [1024]:
# mape_output=mape_bandwise(bench_data, 'ordered_bands', mapped_dataset, 'Flag', 'bad_flag', 'Pred_Prob')

In [1025]:
# mape=mape_output.groupby(['key'])[['abs_deviation']].agg('mean')
# mape.rename(columns={'abs_deviation':'MAPE'},inplace=True)

In [1026]:
# mape

In [1027]:
# def mape_complete(benchmark_dataset, final_ordered_bands, snapshot_datasets, key, target, score):
#     mape_output=pd.DataFrame()
    
#     def calculate_mape(df, final_ordered_bands, score, target, key):
#         temp_mape={}
#         temp_mape['key']=df[key].max()
#         temp_mape['pred_avg']=df[score].agg('mean')
#         temp_mape['actual_avg']=df[target].agg('mean')
#         temp_mape['abs_deviation']=abs(((temp_mape['pred_avg']-temp_mape['actual_avg'])/temp_mape['actual_avg']))    
# #         temp_mape.reset_index(inplace=True)
# #         print(temp_mape)
#         return pd.DataFrame([temp_mape]) 
    
#     mape_output = calculate_mape(benchmark_dataset, final_ordered_bands, score, target, key)
    
#     for i in range(len(snapshot_datasets)):
#         temp_mape=calculate_mape(snapshot_datasets[i], final_ordered_bands, score, target, key)
#         mape_output=mape_output.append(temp_mape)
        
#     return mape_output

In [1028]:
# mape_complete_output=mape_complete(bench_data, 'ordered_bands', mapped_dataset, 'Flag', 'bad_flag', 'Pred_Prob')

In [1029]:
# mape_complete_output

Rank Order Analysis

In [1030]:
# def rank_order(benchmark_dataset, final_ordered_bands, snapshot_datasets, key, target):
#     rank_order_output=pd.DataFrame()
    
#     def calculate_rank_order(df, final_ordered_bands , target, key):
#         temp_rank_order=pd.DataFrame()
#         temp_rank_order['Bad']=df.groupby([final_ordered_bands])[target].agg('sum') 
#         temp_rank_order['Good']=df.groupby([final_ordered_bands])[target].agg('count')-temp_rank_order['Bad'] 
# #         temp_rank_order['bad_rate']=df.groupby([final_ordered_bands])[target].agg('mean') 
# #         temp_rank_order['per_population']=df.groupby([final_ordered_bands])[target].agg('count')/len(df.target)
#         temp_rank_order['key']=df[key].max() 
#         temp_rank_order.reset_index(inplace=True)
#         temp_rank_order=temp_rank_order[['ordered_bands','Good','Bad','key']]
#         return temp_rank_order
        
#     rank_order_output = calculate_rank_order(benchmark_dataset, final_ordered_bands, target, key)
    
#     for i in range(len(snapshot_datasets)):
#         temp_rank_order=calculate_rank_order(snapshot_datasets[i], final_ordered_bands, target, key)
#         rank_order_output=rank_order_output.append(temp_rank_order)
    
#     rank_order_output['Total']=rank_order_output['Good']+rank_order_output['Bad']
    
#     return rank_order_output

In [1031]:
# rank_order_output=rank_order(bench_data, 'ordered_bands', mapped_dataset, 'Flag', 'bad_flag')

In [1032]:
# rank_order_output

Variable Trend & Score Contribution

In [1033]:
# def var_trend(benchmark_dataset, var_list, snapshot_datasets, key, target, score):
#     var_trend_output=pd.DataFrame()
    
#     def calculate_var_trend(df1,df2, var , target, key, score):
#         temp_var_trend=pd.DataFrame()
#         temp_var_trend['bench_bad_rate']=df1.groupby([var])[target].agg('mean') 
#         temp_var_trend['val_bad_rate']=df2.groupby([var])[target].agg('mean') 
#         temp_var_trend['bench_per_population']=df1.groupby([var])[target].agg('count')/len(df1[target])
#         temp_var_trend['val_per_population']=df2.groupby([var])[target].agg('count')/len(df2[target])
#         temp_var_trend['avg_score']=df2.groupby([var])[score].agg('mean')
#         temp_var_trend['Variable']=var
#         temp_var_trend['score']=df1.groupby([var])["S_"+var[2:]].agg('max')
#         temp_var_trend['min_score']=temp_var_trend.groupby(['Variable'])['score'].transform('min')
#         temp_var_trend['max_score']=temp_var_trend.groupby(['Variable'])['score'].transform('max')
#         temp_var_trend['exp_score']=temp_var_trend['score']*temp_var_trend['bench_per_population']
#         temp_var_trend['factor_exp_score']=temp_var_trend.groupby(['Variable'])['exp_score'].transform('sum')
#         temp_var_trend['var_score']=temp_var_trend['bench_per_population']* \
#                                     (temp_var_trend['factor_exp_score']-temp_var_trend['score'])* \
#                                     (temp_var_trend['factor_exp_score']-temp_var_trend['score'])
#         temp_var_trend['var_factor']=temp_var_trend.groupby(['Variable'])['var_score'].transform('sum')
#         temp_var_trend['var_dev']=np.sqrt(temp_var_trend['var_factor'])
#         temp_var_trend['key']=df2[key].max() 
#         temp_var_trend.reset_index(inplace=True)
#         temp_var_trend.rename(columns={var:'var_bins'},inplace=True)
#         temp_var_trend=temp_var_trend[['Variable','var_bins','score','bench_bad_rate','val_bad_rate',
#                                       'bench_per_population','val_per_population','avg_score','min_score','max_score','var_dev','key']]
#         return temp_var_trend
    
#     for i in range(len(snapshot_datasets)):
#         for j in range(len(var_list)):
#             temp_var_trend=calculate_var_trend(benchmark_dataset,snapshot_datasets[i], var_list[j], target, key, score)
#             var_trend_output=var_trend_output.append(temp_var_trend)
        
#     return var_trend_output

In [1034]:
# var_trend_output=var_trend(bench_data, b_var_list, mapped_dataset, 'Flag', 'bad_flag', 'Score')

In [1035]:
# var_trend_output

Gini (Using ROC_AUC)

In [1036]:
# import numpy as np
# from sklearn import metrics

In [1037]:
# def gini(benchmark_dataset, snapshot_datasets, target, score, key):
#     gini_output=pd.DataFrame()
    
#     def calculate_gini(df, target, score, key):
#         temp_gini={}
#         actual=df[target]
#         pred=df[score]
#         fpr, tpr, thresholds = metrics.roc_curve(actual, pred)
#         auc=metrics.auc(fpr, tpr)
#         gini=(2*auc-1)
#         temp_gini['key']=df[key].max()
#         temp_gini['GINI']=gini
#         temp_gini=pd.DataFrame([temp_gini])
#         temp_gini.set_index('key',inplace=True)
#         return (temp_gini)
    
#     gini_output=calculate_gini(benchmark_dataset, target, score, key)
    
#     for i in range(len(snapshot_datasets)):
#         temp_gini=calculate_gini(snapshot_datasets[i], target, score, key)
#         gini_output=gini_output.append(temp_gini)
        
#     return gini_output

In [1038]:
# gini_output=gini(bench_data, mapped_dataset, 'bad_flag', 'Pred_Prob', 'Flag')

In [1039]:
# gini_output

Variable Graphs

In [1040]:
# def var_graphs(benchmark_dataset, val_datasets, var_list, key, target, final_ordered_bands):
#     graph=pd.DataFrame()
    
#     def calculate_graph(df, var, key, target, final_ordered_bands ):
#         temp=pd.DataFrame()
#         temp1=pd.DataFrame()
        
#         temp['Bad']=df.groupby([var,final_ordered_bands])[target].agg('sum')
#         temp['Total']=df.groupby([var,final_ordered_bands])[target].agg('count')
#         temp['Good']=temp['Total']-temp['Bad']
#         temp['bad_rate']=temp['Bad']/temp['Total']
#         temp['per_pop']=temp['Total']/(temp['Total'].sum())
#         temp.reset_index(inplace=True)
#         temp.rename(columns={var:'var_bins'},inplace=True)

#         temp1['var_bins']=df[var].unique()
#         temp1['Variable']=var
#         temp1.sort_values(['var_bins'],inplace=True)
#         temp1['a']=1

#         temp2=score_band_mapping[[final_ordered_bands]]
#         temp2['bins']=range(10)
#         temp2['a']=1

#         temp1=temp1.merge(temp2, on=['a'], how='left')
#         temp1=temp1.merge(temp, on=[final_ordered_bands,'var_bins'], how='left')
#         temp1.fillna(0,inplace=True)
#         temp1['key']=df[key].max()
#         temp1=temp1[['Variable','var_bins','bins',final_ordered_bands,'Good','Bad','bad_rate','per_pop','key']]
        
#         return temp1
    
#     for i in range(len(var_list)):
#         temp=calculate_graph(benchmark_dataset, var_list[i], key, target, final_ordered_bands)
#         graph=graph.append(temp)
        
#     for i in range(len(val_datasets)):
#         for j in range(len(var_list)):
#             temp=calculate_graph(val_datasets[i], var_list[j], key, target, final_ordered_bands)
#             graph=graph.append(temp)
    
#     return graph

In [1041]:
# graph_output=var_graphs(bench_data, mapped_dataset, b_var_list, 'Flag','bad_flag', 'ordered_bands')

In [1042]:
# graph_output

Gini/KS Score Point wise

In [1043]:
# def gini_ks(benchmark_dataset, validation_datasets, score, target, key):
    
#     gini_ks_output=pd.DataFrame()
    
#     def calculate_gini_ks(data, score, target, key):
#         data = data.sort_values([score], ascending=True)
#         temp = data[[target, score]]
#         Decile  = data[score].unique().tolist()
#         temp_data = pd.DataFrame(Decile, columns = ['Probability'])
#         temp_data['Bad'] = temp.groupby(score).sum().reset_index()[target]
#         temp_data['Good'] = temp.groupby(score).count().reset_index()[target] - temp_data['Bad']
#         temp_data['Total'] = temp_data['Bad'] + temp_data['Good']
#         temp_data['Bad_Rate'] = temp_data['Bad'] / temp_data['Total']
#         temp_data['Perc_Good'] = temp_data['Good'] / temp_data['Good'].sum()
#         temp_data['Perc_Bad'] = temp_data['Bad'] / temp_data['Bad'].sum()
#         temp_data['Perc_Total'] = temp_data['Total'] / temp_data['Total'].sum()
#         # temp_data['Perc_cummm_Good'] = temp_data['Good'] + temp_data['Good'].sum()
#         temp_data['Perc_cumm_Good'] = 0.0
#         temp_data['Perc_cumm_Bad'] = 0.0
#         temp_data['Perc_cumm_pop'] = 0.0
#         temp_data['Area'] = 0.0
#         for x in temp_data.index:
#             if x == 0:
#                 temp_data['Perc_cumm_Good'][x] = temp_data['Perc_Good'][x]
#                 temp_data['Perc_cumm_Bad'][x] = temp_data['Perc_Bad'][x]
#                 temp_data['Perc_cumm_pop'][x] = temp_data['Perc_Total'][x]
#                 temp_data['Area'][x] = 0.5*temp_data['Perc_cumm_Bad'][x]*temp_data['Perc_cumm_Good'][x]
#             else:
#                 temp_data['Perc_cumm_Good'][x] = temp_data['Perc_Good'][x] + temp_data['Perc_cumm_Good'][x-1]
#                 temp_data['Perc_cumm_Bad'][x] = temp_data['Perc_Bad'][x] + temp_data['Perc_cumm_Bad'][x-1]
#                 temp_data['Perc_cumm_pop'][x] = temp_data['Perc_Total'][x] + temp_data['Perc_cumm_pop'][x-1]
#                 temp_data['Area'][x] = 0.5*(temp_data['Perc_cumm_Good'][x] + temp_data['Perc_cumm_Good'][x-1])*(temp_data['Perc_cumm_Bad'][x] - temp_data['Perc_cumm_Bad'][x-1])
#         temp_data['Differentiation'] = temp_data['Perc_cumm_Good'] - temp_data['Perc_cumm_Bad']
# #         print(temp_data)
#         Gini = 2*temp_data['Area'].sum() - 1
#         KS = temp_data['Differentiation'].max()
#         result={}
#         result['Key']=data[key].max()
#         result['Gini']=Gini
#         result['KS']=KS
#         return pd.DataFrame([result])
    
#     gini_ks_output=calculate_gini_ks(benchmark_dataset, score, target, key)
    
#     for i in range(len(validation_datasets)):
#         temp=calculate_gini_ks(validation_datasets[i], score, target, key)
#         gini_ks_output=gini_ks_output.append(temp)
    
#     return gini_ks_output

In [1044]:
# gini_ks_output = gini_ks(bench_data, mapped_dataset, 'Pred_Prob', 'bad_flag', 'Flag')

In [1045]:
# gini_ks_output

Correlation_matrix

In [1046]:
# def correlation_matrix(benchmark_dataset, validation_datasets, var_list,key):
#     corr_output=pd.DataFrame()
    
#     def calculate_corr(df,var_list,key):
#         temp=bench_data[var_list].corr()
#         temp['key']=df[key].max()
        
#         return temp
    
#     corr_output=calculate_corr(benchmark_dataset, var_list, key)
    
#     for i in range(len(validation_datasets)):
#         temp=calculate_corr(validation_datasets[i], var_list, key)
#         corr_output=corr_output.append(temp)
        
#     return corr_output

In [1047]:
# corr_output=correlation_matrix(bench_data, mapped_dataset, w_var_list, 'Flag')

In [1048]:
# corr_output

Creating Single dataframe for Logistic Working Excel

In [1049]:
# score_band_mapping = mapped[1]
# score_band_mapping.reset_index(inplace=True)
# score_band_mapping.rename(columns={'index':'decile'},inplace=True)
# score_band_mapping['rank']=score_band_mapping['decile']+1
# df1=score_band_mapping[['rank','decile','lower_limit','upper_limit']]
# df2=rank_order_output[rank_order_output['key']=='Benchmark_data'][['Good','Bad']]
# df3=rank_order_output[rank_order_output['key']=='OOS'][['Good','Bad']]
# df4=rank_order_output[rank_order_output['key']=='OOT'][['Good','Bad']]
# df5=score_point_output[score_point_output['Key']=='Benchmark_data'][['Score','Good','Bad','Total']]
# df6=score_point_output[score_point_output['Key']=='OOS'][['Score','Good','Bad','Total']]
# df7=score_point_output[score_point_output['Key']=='OOT'][['Score','Good','Bad','Total']]
# df8=var_trend_output[var_trend_output['key']=='OOS'].drop(columns=['key']).reset_index(drop=True)
# df9=csi[csi['key']=='OOS'][['variable_name','csi']]
# df10=var_trend_output[var_trend_output['key']=='OOS'].groupby(['Variable'])['min_score','max_score','var_dev'].agg({'min_score':'min',
#                                                                                                                     'max_score':'max',
#                                                                                                                     'var_dev':'max'})
# df11=var_trend_output[var_trend_output['key']=='OOT'].drop(columns=['key']).reset_index(drop=True)
# df12=csi[csi['key']=='OOT'][['variable_name','csi']]
# df13=var_trend_output[var_trend_output['key']=='OOT'].groupby(['Variable'])['min_score','max_score','var_dev'].agg({'min_score':'min',
#                                                                                                     'max_score':'max',
#                                                                                                          'var_dev':'max'})
# df14=final_iv.pivot(index='variable_name',columns='key')
# df14.columns=df14.columns.droplevel()
# df14.columns='IV_'+df14.columns
# df14.reset_index(inplace=True)
# df15=vif_output.pivot(index='variable_name',columns='key')
# df15.columns=df15.columns.droplevel()
# df15.columns='VIF_'+df15.columns
# df16=gini_ks_output
# df17=hci
# df18=mape.reset_index()
# df19=mape_complete_output
# df20=corr_output[corr_output['key']=='Benchmark_data']
# df20.columns=range(1,len(df20)+2)
# df20=df20.reindex(columns=range(1,17))
# # df21=corr_output[corr_output['key']=='OOS']
# # df21.columns=range(1,len(df21)+2)
# # df21=df21.reindex(columns=range(1,17))
# df22=corr_output[corr_output['key']=='OOT']
# df22.columns=range(1,len(df22)+2)
# df22=df22.reindex(columns=range(1,17))

In [1050]:
# dfs=[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,df22]

In [1051]:
# for i in dfs:
#     i.reset_index(inplace=True, drop=True)

In [1052]:
#?final_output=pd.concat(dfs,axis=1)

In [1053]:
# final_output.to_excel('raw_data_logistic.xlsx',index=False)

In [1054]:
# graph_output.to_excel('graph_output.xlsx',index=False)

In [1055]:
# pred=model_feat1.predict(sm.add_constant(X1[all_12_feat]))
# pred1=np.where(pred>0.5,1,0)
# fpr, tpr, thresholds = metrics.roc_curve(y1, pred)
# auc=metrics.auc(fpr, tpr)
# gini=2*auc-1
# gini,auc

In [1056]:
#iv_raw = woe1[woe1['Variable'].isin(selected_cols)]

In [1057]:
#Exporting data to the excel sheet
#iv_raw.to_excel("Iv_raw_sheet.xlsx",sheet_name='IV_sheet')

In [1058]:
# #Sorting values based on IV
# pd.options.display.max_rows = None
# iv = pd.DataFrame(iv)
# iv.sort_values(by = 'IV',ascending = False)

In [1059]:
# #Dropping variables based on IV < 0.02
# weak_pred = iv[iv['IV']< 0.02]
# print(weak_pred.shape) #136 variables
# weak = list(weak_pred['Variable'])

# bureau_factor_6.drop(weak ,axis=1, inplace = True)

In [1060]:
# iv1, woe1 = iv_woe(data = bureau_factor_6 , target ='bad_flag', IDcol='cust_id', bins=10, show_woe = False)

In [1061]:
# woe1 = pd.DataFrame(woe1)
# ##To suppress the scientific format ooutput
# pd.set_option('display.float_format', lambda x: '%.6f' % x)
# woe1

In [1062]:
#Creating dataframes to store bins and values for pd.cut
#bin_cut = woe1.groupby('Variable').apply(lambda x: [-np.inf] + list(x.Max.iloc[:-1].values) + [np.inf])
# bin_cut = woe1.groupby('Variable').apply(lambda x: [-np.inf] + list(x.Max.iloc[:-1].values) + [np.inf] )
# WOE_val = woe1.groupby('Variable').apply(lambda x: list(x.woe.values))

In [1063]:
# df_new_woe =pd.DataFrame()
# df_new_woe['cust_id'] = bureau_factor_6['cust_id']

In [1064]:
# # Loop to map binned variables and woe values
# for i in bin_cut.index:
#     df_new_woe[i]=pd.cut(bureau_factor_6[i],bin_cut[i],labels=WOE_val[i],right=True,ordered=False)

In [1065]:
# # To convert all dtype from category to float 
# for i in bin_cut.index:
#     df_new_woe[i] = df_new_woe[i].astype("float")

In [1066]:
# iv1.sort_values(by = 'IV',ascending=False,inplace=True)
# iv1 = iv1.set_index("Variable")
# iv1

In [1067]:
#snap_l12m.shape

In [1068]:
#snap_l12m['del_incr_flag'] = np.where(snap_l12m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()>0,1,0)

In [1069]:
#snap_l12m['grp_ids_del_incr_flag'] = snap_l12m['del_incr_flag'].diff().ne(0).cumsum()

In [1070]:
#snap_l12m.to_csv('snap_l12m.csv',index= False)

In [1071]:
#snap_l12m["del_incr_flag_cnt"] = snap_l12m['del_incr_flag'].groupby([snap_l12m['MASK_CREDITCARD'],snap_l12m['grp_ids_del_incr_flag']]).cumsum()

In [1072]:
#snap_l12m['del_dcr_flag'] = np.where(snap_l12m.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()<0,1,0)

In [1073]:
#snap_l12m['grp_ids_del_dcr_flag'] = snap_l12m['del_dcr_flag'].diff().ne(0).cumsum()

In [1074]:
#snap_l12m["del_dcr_flag_cnt"] = snap_l12m['del_dcr_flag'].groupby([snap_l12m['MASK_CREDITCARD'],snap_l12m['grp_ids_del_dcr_flag']]).cumsum()

In [1075]:
#snap_l12m.to_csv('snap_l12m.csv',index= False)

In [1076]:
#snap['NB_Days_Flag'] = np.where(snap.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()>0,1,0)

In [1077]:
#snap['NB_Days_Flag1'] = snap.groupby('MASK_CREDITCARD')['NB_DAYS_DELINQ'].diff()

In [1078]:
#group_ids = snap['NB_Days_Flag'].diff().ne(0).cumsum()

In [1079]:
#group_ids

In [1080]:
#group_ids
#group_ids.to_csv('group_ids.csv',index= False)

In [1081]:
#snap["Flag"] = snap['NB_Days_Flag'].groupby([snap['MASK_CREDITCARD'], group_ids]).cumsum()

In [1082]:
#i8_cards['min_period'] = i8_cards.groupby('MASK_CREDITCARD')['PERIOD'].transform(min)

In [1083]:
# snap["Max_Flag"] = snap.groupby(['MASK_CREDITCARD'])['Flag'].transform(max)

In [1084]:
# snap['Flag1'] = np.where(snap["Flag"]>1,snap["Flag"],0)

In [1085]:
# snap['tot_cons'] = snap.groupby([snap['MASK_CREDITCARD'], group_ids])['Flag1'].transform(max)

In [1086]:
# snap['tot_cons1'] = np.where(snap["Flag1"]==snap['tot_cons'] ,snap['tot_cons'],0)

In [1087]:
#snap["Flag1"] = snap['Flag'].groupby([snap['MASK_CREDITCARD'], group_ids]).max()

In [1088]:
# snap.to_csv('snap.csv',index= False)

In [1089]:
# factor_0 = base_3[(base_3['monthly_dt'] <= mod_date) & (base_3['monthly_dt']>= mod_date + pd.offsets.MonthEnd(-36))]
#mod_date = pd.to_datetime('2019-04-30')
#last_date = pd.to_datetime('2020-04-30')

In [1090]:
#i1_card_transaction = i1_card_transaction[i1_card_transaction['REVERSAL_FLAG'] =="N"]

In [1091]:
# base_4=base_3.assign(target_90=np.where(base_3['closure_status']<= 0,base_3['def_90'],np.nan),
#                     target_end_90=np.where(base_3['closure_status']<= 0,base_3['def_max_90'],np.nan)
#                     ).groupby('cust_id').agg({'target_90' : 'max',
#                                              'target_end_90' : 'max'})

In [1092]:
#factor_0['credit_limit'] = factor_0.groupby('tgt_key')['credit_limit_amt'].transform(max)

In [1093]:
# i8_cards = i8_cards.assign(
#     Modeling_Data = i8_cards.groupby('report_date_yr_mon')['bad_def'].shift(-1)
# )

In [1094]:
#i8_cards.to_csv('i8_cards_bad_flag.csv',index= False)

In [1095]:
# #To replace all na & all special values with '-555555'
# def fill_na(df, value):
#     df.fillna(value, inplace=True)
#     df.replace([np.inf,-np.inf],value ,inplace=True)

# fill_na(iv_input ,-555555 )

In [1096]:
# merged_data= df1.merge(df2, on=["column1","column2"])

In [1097]:
# Dropping Columns

In [1098]:
#df_transpose=df_transpose.drop(['account_nb','payment_history_grid','asset_class_cd','portfolio_rating_type_cd','val'],axis=1)

In [1099]:
#i8_cards['cust_min_mob'] = i8_cards.groupby('MASKED_CUSTOMER_ID')['mob'].transform(min)
#i8_cards['cust_max_mob'] = i8_cards.groupby('MASKED_CUSTOMER_ID')['mob'].transform(max)

In [1100]:
#i8_cards.cust_max_mob.value_counts()

In [1101]:
#i8_cards['issue_date_yr'] = pd.to_datetime(i8_cards['a_issue_date2']).dt.strftime('%Y')

In [1102]:
#i8_cards.issue_date_yr.value_counts()